# NBA Play‑by‑Play → Turn‑In Package (Lineups & Rim Defense)

This document is the **turn‑in** for the single‑game pipeline. It includes:

1. A table of **every unique 5‑man lineup** per team with possessions and ratings.
2. A table of **every player** with rim defense on/off metrics.
3. A concise, submission‑focused description of the two lineup engines (**Traditional** vs **Lineup Automation**) and how they relate to the deliverables.
4. A clear **process tree** you can follow to reproduce without SQL.

---

## Projects:
Using the data provided (a play-by-play csv file, a box score csv file, and 3 csvs that map columns in the pbp), please transform the data to return two tables:

 

    A table containing every unique 5-man lineup for each team with columns for the following:

    The 5 players in the lineup
    Team
    Offensive possessions played
    Defensive possessions played
    Offensive rating
    Defensive rating
    Net rating

    A table containing every player who played in the game with columns for the following:

    Player ID
    Player Name
    Team
    Offensive possessions played
    Defensive possessions played
    Opponent rim field goal percentage when player in on the court
    Opponent rim field goal percentage when player is off the court
    Opponent rim field goal percentage on/off difference (on-off)

 

We will define a rim shot as any shot that occurred within 4 feet of the basket.

 

You may return the tables in any format you see fit. Please provide the code you used to generate the tables. 


-----------------------------------------------------

### Project 1 — Unique 5‑Man Lineups (per team)

# Final Turn ins: project1_lineups_FINAL.csv (lineup automation) and project2_players_FINAL.csv (player rim defense), with project1_lineups_traditional_with_6th_player.csv (traditional) for reference 
* usually use parquet but used csv for ease of use

Columns: *Team, Player 1…Player 5, Offensive possessions played, Defensive possessions played, Offensive rating, Defensive rating, Net rating*

> **Two views are shown below** for transparency:
>
> * **Traditional (data‑driven)** results **may include 4–6 player lineups** when the raw feed is inconsistent (we still list them in this section for auditability).
> * **Lineup Automation** is the submission‑ready **always‑five** view used for final grading/comparison.

#### Project 1 — Traditional (data‑driven) results *(includes 4–6 player lineups)*: project1_lineups_traditional_with_6th_player.csv

#### Head of substitution‑violation flags (Traditional):

#### Project 1 — Lineup Automation (always‑five; submission‑ready): project1_lineups_FINAL.csv

#### Project 2 — Player Rim Defense (On/Off): project2_players_FINAL.csv

Columns: Player ID, Player Name, Team, Offensive possessions played, Defensive possessions played, Opponent rim FG% (on), Opponent rim FG% (off), On–off difference




## How We Built It (No SQL)

### Two Complementary Lineup Engines (used for turn‑in)

**Traditional (data‑driven) — `run_traditional_data_driven_lineups`**

* Strictly follows raw PBP substitutions (`msgType=8`: `playerId1=IN`, `playerId2=OUT`).
* Lineups may be size **≠ 5** when the feed is inconsistent.
* Period handling: **Q1 & Q3 reset to starters**; **Q2/Q4/OT carry forward** the current five.
* Flags anomalies instead of fixing them: `lineup_size_deviation`, `sub_out_player_not_in_lineup`, `sub_in_player_already_in_lineup`, `action_by_non_lineup_player`.
* **Purpose:** auditing/data‑forensics; exposes feed gaps that can distort analytics.

**Lineup Automation (enhanced) — `run_enhanced_substitution_tracking_with_flags`**

* Adds **intelligent inference** to maintain **exactly five** on court per team.
* Period handling: **Q1 & Q3 reset to starters**; **Q2/Q4/OT carry forward**.
* **First‑Action Auto‑IN:** if a player acts with no prior sub‑in, we inject them (and flag).
* **Inactivity Auto‑OUT:** idle **>120s** → candidate removal (flag and record).
* Emits flags: `missing_sub_in`, `inactivity_periods`, `first_action_events`, `auto_out_events`, `lineup_violations`.
* **Purpose:** submission‑ready analytics (clean, always‑five lineups).

> **Turn‑in rule:** Project 1’s submitted table uses the **Lineup Automation** results (always‑five). The Traditional table is included for transparency and QA.

### End‑to‑End Process Tree

```
Inputs
├─ Play‑by‑Play CSV
├─ Box Score CSV
└─ PBP Lookup CSVs (event, action, option)

Process
├─ Load & Validate
│  ├─ Load box (ACTIVE only) & pbp; drop admin rows
│  ├─ Distance sanity & rim tagging rule (≤4 ft)
│  └─ Join lookup CSVs to enrich event families/types
├─ Dimensions & Maps
│  ├─ Team & player dims (filter out officials; infer team where needed)
│  └─ Enriched PBP view (names, team abbrevs, labeled events)
├─ Lineup Engines (two parallel passes)
│  ├─ Traditional (data‑driven): allow non‑5; generate diagnostic flags
│  └─ Lineup Automation: always‑five; infer missing sub‑ins/auto‑outs + flags
├─ Possession Builder
│  ├─ Start/extend/end per rules (made FG, TO, DREB, FT sequencing, period end)
│  └─ Attribute points to offense lineups on floor
├─ Rim Tagging
│  ├─ Compute shot distance; mark rim attempts/makes (≤4 ft)
│  └─ Accumulate “on‑court” vs “off‑court” opponent rim tallies per player
├─ Aggregations
│  ├─ Project 1: unique 5‑man lineups → off/def possessions & ratings
│  └─ Project 2: player rim on/off → opponent rim FG% on/off & on–off
├─ Validation (no SQL surfaced)
│  ├─ Minutes parity vs box (tolerance window)
│  └─ Consistency checks on lineup sizes & distance outliers
└─ Turn‑In Artifacts
   ├─ **project1_lineups_FINAL.csv** (Lineup Automation, always‑five)
   └─ **project2_players_FINAL.csv** (player rim on/off)


structure:
# NBA Data Engineering Notebook - Process Overview

## Project Overview
NBA data engineering pipeline for analyzing 5-man lineups and player rim defense statistics from play-by-play and box score data.

## Main Processes

### 1. Configuration & Setup
```
├── Project Dependencies
│   ├── Data processing: pandas, numpy, duckdb
│   ├── Orchestration: apache-airflow
│   └── Web framework: FastAPI
│
├── Airflow Configuration
│   ├── Connections, pools, variables setup
│   └── DAG scheduling and dependencies
│
└── Data Schema Definition
    ├── Box score column mappings
    ├── Play-by-play column mappings
    └── Output table specifications
```

### 2. Data Validation Framework
```
└── NBADataValidator
    ├── File structure validation
    ├── Box score data quality checks
    ├── Play-by-play data validation
    └── Coordinate system validation
```

### 3. Data Loading & Processing
```
└── EnhancedNBADataLoader
    ├── Load and validate box score data
    ├── Load and validate play-by-play data
    ├── Create lookup tables and dimensions
    ├── Build enriched data views
    └── Generate data quality reports
```

### 4. Lineup Tracking System
```
├── Traditional Method
│   ├── Standard substitution-based tracking
│   ├── 5-player lineup enforcement
│   └── Basic possession attribution
│
├── Enhanced Method
│   ├── Intelligent inference for missing substitutions
│   ├── First-action event detection
│   ├── Auto-out player identification
│   └── Advanced lineup state management
│
└── PBPProcessor
    ├── Process play-by-play events
    ├── Track lineup changes in real-time
    └── Maintain dual-method state
```

### 5. Possession Analysis Engine
```
└── DualMethodPossessionEngine
    ├── Identify possession boundaries
    ├── Attribute possessions to lineups
    ├── Calculate offensive/defensive ratings
    ├── Track rim defense statistics
    └── Generate comparative analysis
```

### 6. Output Generation
```
├── Lineup Analysis
│   ├── 5-man lineup statistics
│   ├── Offensive/defensive ratings
│   └── Net rating calculations
│
├── Player Statistics
│   ├── Rim defense metrics (on/off court)
│   ├── Possession counts
│   └── Performance validation
│
└── Quality Reports
    ├── Data validation summaries
    ├── Processing statistics
    └── Error logs and diagnostics
```

## Key Features

- **Dual-Method Processing**: Traditional + enhanced lineup tracking
- **Rim Defense Analysis**: 4-foot rim shot tracking and on/off court analysis
- **Possession Attribution**: Accurate possession counting and rating calculations
- **Data Quality Assurance**: Multi-layer validation and error handling
- **Performance Optimization**: DuckDB integration with memory management
- **Production Ready**: Airflow orchestration with proper dependency management

```

### Reproduce (no SQL required, final cell with full pipeline output saved, search run_complete_pipeline.py for more details)

1. Place the provided CSVs in the `data/.../` folder.
2. run this notebook (make folder structure as needed), the notebook will set up codes in order needed
3. us uv sync at repo root
4. cd api/src/airflow_project; astro dev init; astro dev start; 
6. Run the pipeline (run_complete_pipeline.py) (e.g., via Python entrypoint or notebook cells that call the two engines, possession builder, rim tagging, and aggregations).
3. The pipeline writes two CSVs used for turn‑in under an exports/output folder as listed above.

> Notes:
>
> * Rim attempt definition is **fixed at 4 ft**.
> * Traditional results are kept for QA; the submitted `Project 1` table uses the **Lineup Automation** results to ensure 5‑man compliance.

---

## DAG (automation overview)

* Backfilling supported
* Scheduling supported

![Pipeline DAG](api/src/airflow_project/data/mavs_data_engineer_2025/exports/final_submission/dag.png)

---

## Appendix (Quick Reference)

* **Project 1 columns:** Team, Player 1–5, Off/Def possessions, OffRtg, DefRtg, Net.
* **Project 2 columns:** Player ID, Player Name, Team, Off/Def possessions, Opponent rim FG% (on/off), On–off.
* **Lineup engines:** Traditional (audit) vs Lineup Automation (submission).


# Root level Data Engineering needs for testing

In [20]:
%%writefile pyproject.toml
[project]
name = "nba_data_engineering"
version = "0.2.0"
description = "NBA lineup analysis and player rim defense data pipeline"
authors = [
  { name = "Data Engineering Team" },
]
license = "MIT"
readme = "README.md"

requires-python = ">=3.10,<3.13"

dependencies = [
  # Core web framework and API
  "fastapi>=0.104.0",
  "uvicorn[standard]>=0.24.0",
  "python-dotenv>=1.0.0",
  
  # Settings and validation
  "pydantic>=2.0.0",
  "pydantic-settings>=2.0.0",
  
  # Data processing core
  "pandas>=2.1.0,<2.3.0",
  "numpy>=1.24.0,<1.27.0",
  "pyarrow>=12.0.0,<16.0.0",
  
  # High-performance analytics
  "duckdb>=0.10.0,<0.11.0",
  
  # Airflow and orchestration
  "apache-airflow==2.10.4",
  "apache-airflow-providers-postgres>=6.2.1,<7.0.0",
  "apache-airflow-providers-standard>=1.4.1,<2.0.0",
  "apache-airflow-providers-http>=4.5.0,<5.0.0",
  
  # Database connectivity
  "sqlalchemy>=1.4.49,<2.0.0",
  "psycopg2-binary>=2.9.9",
  
  # Data quality and validation
  "tabulate>=0.9.0",
  "scipy>=1.7.0,<1.12.0",
  
  # Development and analysis
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "matplotlib>=3.4.0",
  "ipykernel>=6.25.0",
  
  # Utilities
  "tqdm>=4.67.0",
  "psutil>=5.0.0,<8.0.0",
  "python-jose[cryptography]>=3.3.0",
  "passlib[bcrypt]>=1.7.4",
  "bcrypt==4.0.1",
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "pytest-cov>=4.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
  "pre-commit>=3.0.0",
]

test = [
  "pytest>=7.0.0",
  "pytest-cov>=4.0.0",
  "pytest-mock>=3.10.0",
  "pytest-xdist>=3.0.0",
]

performance = [
  "memory-profiler>=0.60.0",
  "py-spy>=0.3.14",
]

[build-system]
requires = ["setuptools>=64", "wheel"]
build-backend = "setuptools.build_meta"

[tool.black]
line-length = 100
target-version = ['py310', 'py311', 'py312']
include = '\.pyi?$'
extend-exclude = '''
/(
  \.git
  | \.hg
  | \.mypy_cache
  | \.tox
  | \.venv
  | _build
  | buck-out
  | build
  | dist
)/
'''

[tool.isort]
profile = "black"
line_length = 100
multi_line_output = 3

[tool.mypy]
python_version = "3.10"
warn_return_any = true
warn_unused_configs = true
disallow_untyped_defs = true
ignore_missing_imports = true

[tool.pytest.ini_options]
testpaths = ["tests"]
python_files = ["test_*.py", "*_test.py"]
python_classes = ["Test*"]
python_functions = ["test_*"]
addopts = [
    "-v",
    "--strict-markers",
    "--tb=short",
    "--cov=eda",
    "--cov=utils",
    "--cov-report=html",
    "--cov-report=term-missing"
]
markers = [
    "slow: marks tests as slow",
    "integration: marks tests as integration tests",
    "unit: marks tests as unit tests",
]


Overwriting pyproject.toml


# Astro/Airflow Local Dev Quickstart
1. Initialize & Start the Project

# From the directory where you want the project to live
cd api/src
mkdir -p airflow_project && cd airflow_project

astro dev init            # Scaffold Airflow project (dags/, Dockerfile, etc.)
astro dev start           # Build image & start all Airflow services (webserver, scheduler, DB)

    If ports (8080/5432) are busy, set alternates before start:
    astro config set webserver.port 8081 / astro config set postgres.port 5433.
    Astronomer
    Astronomer

2. Everyday Lifecycle Commands

Use these while iterating on DAGs and dependencies:

astro dev stop            # Stop containers, keep project state
astro dev restart         # Stop → rebuild image → start (after reqs/Dockerfile changes)

    These two are your main “apply changes” loop during development.
    Astronomer
    Astronomer

3. Inspect, Logs, & Diagnostics

astro dev status          # Health & ports of services
astro dev logs            # Combined logs (Ctrl+C to quit)
astro dev tail scheduler  # Live-tail a specific service (scheduler/webserver/triggerer)
astro dev ps              # Show running containers for this project
astro dev top             # Process table inside a service container
astro dev stats           # CPU/mem stats per container

    Helpful when debugging start-up issues, stuck tasks, or resource pressure.
    Astronomer
    GitHub

4. Force a DAG Reparse (No Waiting)

astro dev run dags reserialize

    Airflow auto-parses: new files ~5 min, edits ~30 s. This command forces an immediate parse.
    Astronomer
    Astronomer

5. One-off DAG Test Runs

astro run <dag-id>

    Compiles and executes a single DAG in a throwaway worker container—fast feedback without touching the scheduler.
    Astronomer


# Within Airflow project, created files from astro dev init that we adjusted to fit our needs

In [21]:
%%writefile api/src/airflow_project/requirements.txt
# path: api/src/airflow_project/requirements.txt
# Minimal requirements for Airflow worker containers
# Core data processing
pandas==2.2.2
numpy==1.26.4

# High-performance analytics
duckdb==0.10.3

# Data formats and I/O
pyarrow==15.0.2

# Configuration management  
python-dotenv==1.0.1

# Data quality and validation
tabulate==0.9.0
scipy==1.11.4

# Development utilities
tqdm==4.67.1

Overwriting api/src/airflow_project/requirements.txt


In [22]:
%%writefile api/src/airflow_project/airflow_settings.yaml
# This file allows you to configure Airflow Connections, Pools, and Variables in a single place for local development only.
# NOTE: json dicts can be added to the conn_extra field as yaml key value pairs. See the example below.

# For more information, refer to our docs: https://www.astronomer.io/docs/astro/cli/develop-project#configure-airflow_settingsyaml-local-development-only
# For questions, reach out to: https://support.astronomer.io
# For issues create an issue ticket here: https://github.com/astronomer/astro-cli/issues

airflow:
  connections:
    - conn_id:
      conn_type:
      conn_host:
      conn_schema:
      conn_login:
      conn_password:
      conn_port:
      conn_extra:
        example_extra_field: example-value
  pools:
    - pool_name:
      pool_slot:
      pool_description:
  variables:
    - variable_name:
      variable_value:


Overwriting api/src/airflow_project/airflow_settings.yaml


In [23]:
%%writefile api/src/airflow_project/.dockerignore
astro
.git
.env
airflow_settings.yaml
logs/
.venv
airflow.db
airflow.cfg


Overwriting api/src/airflow_project/.dockerignore


In [24]:
%%writefile api/src/airflow_project/.env
#not empty



Overwriting api/src/airflow_project/.env


# Utils + Configs

In [25]:
%%writefile api/src/airflow_project/utils/__init__.py
"""
Utils package for the NBA Player Valuation project.
""" 

Overwriting api/src/airflow_project/utils/__init__.py


In [26]:
%%writefile api/src/airflow_project/utils/config.py
# path: api/src/airflow_project/utils/config.py
from __future__ import annotations
from pathlib import Path
from typing import Dict
import os
from datetime import timedelta


def _find_project_root(anchor: str = "airflow_project") -> Path:
    """Return the project root by locating the given anchor directory."""
    p = Path(__file__).resolve()
    for parent in (p, *p.parents):
        if parent.name == anchor:
            return parent
    return Path.cwd()


# Core project paths
PROJECT_ROOT: Path = _find_project_root()
DATA_DIR: Path = PROJECT_ROOT / "data"
MAVS_DATA_DIR: Path = DATA_DIR / "mavs_data_engineer_2025"
PROCESSED_DIR: Path = MAVS_DATA_DIR / "processed"
EXPORTS_DIR: Path = MAVS_DATA_DIR / "exports"
DUCKDB_DIR: Path = MAVS_DATA_DIR / "duckdb"
LOGS_DIR: Path = PROJECT_ROOT / "logs"

# Ensure directories exist
for directory in (MAVS_DATA_DIR, PROCESSED_DIR, EXPORTS_DIR, DUCKDB_DIR, LOGS_DIR):
    directory.mkdir(parents=True, exist_ok=True)

# Database configuration
DUCKDB_PATH: Path = DUCKDB_DIR / "mavs_enhanced.duckdb"
DUCKDB_CONFIG: Dict[str, str] = {
    "threads": str(min(8, os.cpu_count() or 1)),
    "memory_limit": "6GB",
    "preserve_insertion_order": "false",
    "max_memory": "6GB",
    "temp_directory": str(DUCKDB_DIR / "temp"),
    "checkpoint_threshold": "1GB",
}

# Input data files
BOX_SCORE_FILE: Path = MAVS_DATA_DIR / "box_HOU-DAL.csv"
PBP_FILE: Path = MAVS_DATA_DIR / "pbp_HOU-DAL.csv"
PBP_ACTION_TYPES_FILE: Path = MAVS_DATA_DIR / "pbp_action_types.csv"
PBP_EVENT_MSG_TYPES_FILE: Path = MAVS_DATA_DIR / "pbp_event_msg_types.csv"
PBP_OPTION_TYPES_FILE: Path = MAVS_DATA_DIR / "pbp_option_types.csv"

# === COLUMN SPECIFICATIONS ===

# Box Score Table - Columns used
BOX_SCORE_COLUMNS = {
    "core": [
        "gameId",
        "nbaId",
        "name",
        "nbaTeamId",
        "team",
    ],
    "lineup_tracking": [
        "gs",            # starter flag
        "boxScoreOrder", # sort key for lineup ordering
    ],
    "performance": [
        "secPlayed",
        "pts",
        "reb",
        "ast",
    ],
    "optional": [
        "minDisplay",
        "jerseyNum",
        "startPos",
    ],
}

# Play-by-Play Table - Columns used
PBP_COLUMNS = {
    "core": [
        "gameId",
        "pbpId",
        "period",
        "msgType",
    ],
    "timing": [
        "gameClock",
        "wallClock",
        "wallClockInt",
    ],
    "team_context": [
        "offTeamId",
        "defTeamId",
    ],
    "player_context": [
        "playerId1",
        "playerId2",
        "playerId3",
    ],
    "shot_data": [
        "locX",
        "locY",
        "pts",
    ],
    "event_details": [
        "actionType",
        "option1",
        "option2",
        "option3",
        "description",
    ],
}

# Reference Tables - Full columns needed
PBP_EVENT_TYPES_COLUMNS = {"all": ["EventType", "Description"]}
PBP_ACTION_TYPES_COLUMNS = {"all": ["EventType", "ActionType", "Event", "Description"]}
PBP_OPTION_TYPES_COLUMNS = {
    "all": ["Event", "EventType", "Option1", "Option2", "Option3", "Option4", "Description"]
}

# === OUTPUT TABLE SPECIFICATIONS ===

LINEUPS_OUTPUT_COLUMNS = [
    "Team",
    "Player 1",
    "Player 2",
    "Player 3",
    "Player 4",
    "Player 5",
    "Offensive possessions played",
    "Defensive possessions played",
    "Offensive rating",
    "Defensive rating",
    "Net rating",
]

PLAYERS_OUTPUT_COLUMNS = [
    "Player ID",
    "Player Name",
    "Team",
    "Offensive possessions played",
    "Defensive possessions played",
    "Opponent rim field goal percentage when player is on the court",
    "Opponent rim field goal percentage when player is off the court",
    "Opponent rim field goal percentage on/off difference (on-off)",
]

# === BUSINESS RULES ===

RIM_DISTANCE_FEET: float = 4.0
HOOP_CENTER_X: float = 0.0
HOOP_CENTER_Y: float = 0.0
COORDINATE_SCALE: float = 10.0

MINIMUM_POSSESSIONS_FOR_LINEUP: int = 2
MINIMUM_ATTEMPTS_FOR_RIM_STATS: int = 1
MINIMUM_SECONDS_PLAYED: int = 30

PERFORMANCE_MONITORING: bool = True
MAX_PIPELINE_RUNTIME_SECONDS: int = 120
WARN_IF_SLOWER_THAN_SECONDS: int = 30

EXTREME_DISTANCE_FEET: float = 35.0
TREAT_ZERO_TEAM_AS_ADMIN: bool = True
MAX_CONSECUTIVE_SUBSTITUTION_FAILURES: int = 5

# === NBA SUBSTITUTION CONFIGURATION ===

NBA_SUBSTITUTION_CONFIG = {
    "starter_reset_periods": [1, 3],     # reset to starters at Q1 and Q3
    "lineup_continuity_periods": [2, 4], # continue prior lineups at Q2 and Q4

    "msg_types": {
        "shot_made": 1,
        "shot_missed": 2,
        "rebound": 4,
        "turnover": 5,
        "foul": 6,
        "substitution": 8,
        "start_period": 12,
        "end_period": 13,
    },

    "one_direction": {
        "enabled": True,
        "remove_out_if_present": True,
        "appearance_via_last_name": True,
        "allow_temp_sixth": True,
        "max_lineup_size": 6,
    },

    "validation": {
        "validate_team_membership": True,
        "validate_pre_sub_state": True,
        "min_lineup_size": 5,
        "hard_max_lineup_size": 6,
    },

    "minutes_validation": {
        "enabled": True,
        "tolerance_seconds": 60,
    },

    "recovery": {
        "enable_intelligent_recovery": True,
        "log_recovery_details": True,
        "max_recovery_attempts": 3,
        "prefer_conservative_approach": True,
        "validate_post_recovery": True,
    },

    "debug": {
        "log_all_substitutions": True,
        "log_lineup_state_changes": True,
        "log_period_transitions": True,
        "include_player_context": True,
        "track_recovery_statistics": True,
    },

    "performance": {
        "batch_substitution_processing": False,
        "cache_player_lookups": True,
        "optimize_lineup_comparisons": True,
    },
}

# === AIRFLOW SCHEDULING & SENSOR CONFIG ===

AIRFLOW_OWNER = "nba-analytics"
AIRFLOW_TIMEZONE = "America/New_York"
SCHEDULE_CRON = "0 6 * * *"          # daily at 06:00 NY time
AIRFLOW_RETRIES = 1
AIRFLOW_RETRY_DELAY_MIN = 2

AIRFLOW_FS_CONN_ID = "fs_default"
FILE_SENSOR_POKE_SEC = 30
FILE_SENSOR_TIMEOUT_SEC = 60 * 60     # 1 hour


def airflow_default_args() -> dict:
    """Build default_args with a timezone-aware start_date (uses pendulum if available)."""
    try:
        import pendulum
        start = pendulum.datetime(2025, 1, 1, tz=AIRFLOW_TIMEZONE)
    except Exception:
        from datetime import datetime
        start = datetime(2025, 1, 1)
    return {
        "owner": AIRFLOW_OWNER,
        "depends_on_past": False,
        "start_date": start,
        "email_on_failure": False,
        "email_on_retry": False,
        "retries": AIRFLOW_RETRIES,
        "retry_delay": timedelta(minutes=AIRFLOW_RETRY_DELAY_MIN),
    }


def required_input_files():
    """Absolute paths the DAG must see before running the pipeline."""
    return [
        BOX_SCORE_FILE,
        PBP_FILE,
        PBP_ACTION_TYPES_FILE,
        PBP_EVENT_MSG_TYPES_FILE,
        PBP_OPTION_TYPES_FILE,
    ]


def get_input_assets_or_datasets():
    """
    Prefer Assets (Airflow 3.0+) else Datasets (Airflow 2.9/2.10).
    Returns (objects, kind) where kind is 'asset', 'dataset', or None.
    """
    files = [str(p) for p in required_input_files()]

    # Airflow 3.x Asset API (Task SDK)
    try:
        from airflow.sdk import Asset  # Airflow 3.0+
        return [Asset(f) for f in files], "asset"
    except Exception:
        pass

    # Airflow 2.9/2.10 Dataset API
    try:
        from airflow.datasets import Dataset
        return [Dataset(f) for f in files], "dataset"
    except Exception:
        return [], None


def build_combined_schedule():
    """
    Return a schedule that works across Airflow versions:
      - Prefer AssetOrTimeSchedule (AF >= 3.0) or DatasetOrTimeSchedule (AF 2.9/2.10)
      - Otherwise, return a plain cron string which all versions accept via 'schedule='
    """
    cron_expr = SCHEDULE_CRON
    objects, kind = get_input_assets_or_datasets()

    # AF 3.x: Assets + cron
    if objects and kind == "asset":
        try:
            from airflow.timetables.assets import AssetOrTimeSchedule
            from airflow.timetables.trigger import CronTriggerTimetable
            return AssetOrTimeSchedule(
                timetable=CronTriggerTimetable(cron_expr, timezone=AIRFLOW_TIMEZONE),
                assets=tuple(objects),
            )
        except Exception:
            pass

    # AF 2.9/2.10: Datasets + cron
    if objects and kind == "dataset":
        try:
            from airflow.timetables.datasets import DatasetOrTimeSchedule
            from airflow.timetables.trigger import CronTriggerTimetable
            return DatasetOrTimeSchedule(
                timetable=CronTriggerTimetable(cron_expr, timezone=AIRFLOW_TIMEZONE),
                datasets=tuple(objects),
            )
        except Exception:
            pass

    # Fallback that is universally valid with DAG(schedule=...)
    return cron_expr



def validate_data_files() -> bool:
    """Validate required data files exist and are non-empty."""
    required_files = [
        BOX_SCORE_FILE,
        PBP_FILE,
        PBP_ACTION_TYPES_FILE,
        PBP_EVENT_MSG_TYPES_FILE,
        PBP_OPTION_TYPES_FILE,
    ]
    missing_files, empty_files = [], []

    for file_path in required_files:
        if not file_path.exists():
            missing_files.append(file_path)
        elif file_path.stat().st_size == 0:
            empty_files.append(file_path)

    if missing_files:
        print("Missing required data files:", [str(f.name) for f in missing_files])
        return False

    if empty_files:
        print("Warning: empty data files found:", [str(f.name) for f in empty_files])
    return True


def validate_performance_config() -> bool:
    """Validate DuckDB performance settings."""
    cpu_count = os.cpu_count() or 1
    configured_threads = int(DUCKDB_CONFIG.get("threads", "1"))
    if configured_threads > cpu_count:
        print(f"Warning: configured threads ({configured_threads}) > CPU count ({cpu_count})")

    try:
        import psutil
        available_gb = psutil.virtual_memory().total / (1024**3)
        configured_gb = int(DUCKDB_CONFIG.get("memory_limit", "4GB").rstrip("GB"))
        if configured_gb > available_gb * 0.8:
            print(f"Warning: configured memory ({configured_gb}GB) > 80% of available ({available_gb:.1f}GB)")
    except ImportError:
        print("Note: psutil not available; skipping memory validation")
    return True


def get_column_usage_report() -> str:
    """Generate a column usage report for documentation."""
    report_lines = [
        "# NBA Pipeline Column Usage Report",
        "",
        "## Box Score Table",
        f"- Core: {BOX_SCORE_COLUMNS['core']}",
        f"- Lineup Tracking: {BOX_SCORE_COLUMNS['lineup_tracking']}",
        f"- Performance: {BOX_SCORE_COLUMNS['performance']}",
        f"- Optional: {BOX_SCORE_COLUMNS['optional']}",
        "",
        "## Play-by-Play Table",
        f"- Core: {PBP_COLUMNS['core']}",
        f"- Timing: {PBP_COLUMNS['timing']}",
        f"- Team Context: {PBP_COLUMNS['team_context']}",
        f"- Player Context: {PBP_COLUMNS['player_context']}",
        f"- Shot Data: {PBP_COLUMNS['shot_data']}",
        f"- Event Details: {PBP_COLUMNS['event_details']}",
        "",
        "## Output Tables",
        f"- Lineups Columns: {len(LINEUPS_OUTPUT_COLUMNS)} columns",
        f"- Players Columns: {len(PLAYERS_OUTPUT_COLUMNS)} columns",
        "",
        "## Business Rules",
        f"- Rim Distance: {RIM_DISTANCE_FEET} feet",
        f"- Min Possessions: {MINIMUM_POSSESSIONS_FOR_LINEUP}",
        f"- Min Rim Attempts: {MINIMUM_ATTEMPTS_FOR_RIM_STATS}",
        f"- Performance Target: {MAX_PIPELINE_RUNTIME_SECONDS}s",
    ]
    return "\n".join(report_lines)


def print_configuration_summary() -> None:
    """Print a concise configuration summary and write the column usage report."""
    print("=" * 80)
    print("NBA PIPELINE - CONFIGURATION")
    print("=" * 80)
    print(f"Project Root: {PROJECT_ROOT}")
    print(f"Database: {DUCKDB_PATH}")
    print(f"Memory Limit: {DUCKDB_CONFIG.get('memory_limit')}")
    print(f"Threads: {DUCKDB_CONFIG.get('threads')}")

    total_box_cols = sum(len(cols) for cols in BOX_SCORE_COLUMNS.values())
    print(f"\nBox Score Columns: {total_box_cols} total")
    for category, cols in BOX_SCORE_COLUMNS.items():
        print(f"   - {category}: {len(cols)} columns")

    total_pbp_cols = sum(len(cols) for cols in PBP_COLUMNS.values())
    print(f"\nPBP Columns: {total_pbp_cols} total")
    for category, cols in PBP_COLUMNS.items():
        print(f"   - {category}: {len(cols)} columns")

    print("\nBusiness Rules:")
    print(f"   - Rim Distance: {RIM_DISTANCE_FEET} feet")
    print(f"   - Min Lineup Possessions: {MINIMUM_POSSESSIONS_FOR_LINEUP}")
    print(f"   - Performance Target: {MAX_PIPELINE_RUNTIME_SECONDS}s")

    print("\nOutput Tables:")
    print(f"   - Lineups: {len(LINEUPS_OUTPUT_COLUMNS)} columns")
    print(f"   - Players: {len(PLAYERS_OUTPUT_COLUMNS)} columns")
    print("=" * 80)
    report_path = LOGS_DIR / "column_usage_report.md"
    report_path.write_text(get_column_usage_report(), encoding="utf-8")
    print(f"Column usage report: {report_path}")


Overwriting api/src/airflow_project/utils/config.py


In [27]:
%%writefile api/src/airflow_project/eda/utils/nba_pipeline_analysis.py
# api/src/airflow_project/eda/utils/nba_pipeline_analysis.py
# Step 1: NBA Pipeline Analysis & Data Structure Validation
"""
NBA Play-by-Play Data Pipeline - Step 1: Analysis & Setup
---------------------------------------------------------

This step analyzes the required inputs and sets up basic validations for:

1. Box score data (player info, starters, team mapping)
2. Play-by-play data (events, shots, substitutions, possessions)
3. Lookup tables (event types, action types, option types)

Key requirements:
- Track 5-man lineups
- Count offensive/defensive possessions per lineup
- Track rim attempts (≤ 4 feet from basket)
- Compute on/off rim defense stats
- Handle substitutions and lineup changes
- Validate data integrity at each step
"""

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List

import logging
import sys
import time

import duckdb  # kept intentionally; may be imported transitively elsewhere
import numpy as np
import pandas as pd


def configure_logging(level: int = logging.INFO) -> None:
    """
    Configure plain logging to stderr with timestamps.
    No custom formatters; avoids encoding issues by keeping ASCII-only output.
    """
    logging.basicConfig(
        stream=sys.stderr,
        level=level,
        format="%(asctime)s - %(levelname)s - %(message)s",
        force=True,
    )


# Activate plain logging for the module execution context
configure_logging(level=logging.INFO)
logger = logging.getLogger(__name__)


@dataclass
class ValidationResult:
    """Structure to track validation results at each step."""
    step_name: str
    passed: bool
    details: str
    data_count: int = 0
    processing_time: float = 0.0
    warnings: List[str] | None = None

    def __post_init__(self):
        if self.warnings is None:
            self.warnings = []


class NBADataValidator:
    """Data validation routines for the NBA pipeline."""

    def __init__(self):
        self.validation_results: List[ValidationResult] = []
        self.rim_distance_threshold: float = 4.0  # feet
        self.coordinate_scale: float = 10.0       # coordinates are in tenths of feet

    def log_validation(self, result: ValidationResult) -> None:
        """Log and store validation results."""
        self.validation_results.append(result)
        status = "[PASS]" if result.passed else "[FAIL]"
        logger.info(f"{status} {result.step_name}: {result.details}")
        for warning in result.warnings:
            logger.warning(f"[WARN] {result.step_name}: {warning}")

    def validate_file_structure(self, file_paths: Dict[str, Path]) -> ValidationResult:
        """Validate that all required files exist and are non-empty."""
        start_time = time.time()
        missing_files: List[str] = []
        empty_files: List[str] = []
        total_files = len(file_paths)

        for name, path in file_paths.items():
            if not path.exists():
                missing_files.append(f"{name}: {path}")
            elif path.stat().st_size == 0:
                empty_files.append(f"{name}: {path}")

        warnings: List[str] = []
        if empty_files:
            warnings.extend([f"Empty file: {f}" for f in empty_files])

        passed = len(missing_files) == 0
        details = f"Checked {total_files} files. Missing: {len(missing_files)}, Empty: {len(empty_files)}"

        return ValidationResult(
            step_name="File Structure Validation",
            passed=passed,
            details=details,
            processing_time=time.time() - start_time,
            warnings=warnings,
        )

    def validate_box_score_structure(self, df: pd.DataFrame) -> ValidationResult:
        """Validate box score has expected structure and data quality."""
        start_time = time.time()

        required_columns = ['nbaId', 'name', 'nbaTeamId', 'team', 'isHome', 'gs', 'status']
        missing_cols = [col for col in required_columns if col not in df.columns]

        if missing_cols:
            return ValidationResult(
                step_name="Box Score Structure",
                passed=False,
                details=f"Missing columns: {missing_cols}",
                data_count=len(df),
                processing_time=time.time() - start_time,
            )

        active_players = df[df['status'] == 'ACTIVE']
        teams = active_players['team'].unique()
        starters_per_team: Dict[str, int] = {}
        warnings: List[str] = []

        for team in teams:
            team_players = active_players[active_players['team'] == team]
            starters = team_players[team_players['gs'] == 1]
            starters_per_team[team] = len(starters)
            if len(starters) != 5:
                warnings.append(f"Team {team} has {len(starters)} starters (expected 5)")

        details = (
            f"Active players: {len(active_players)}, "
            f"Teams: {len(teams)}, "
            f"Starters per team: {starters_per_team}"
        )

        return ValidationResult(
            step_name="Box Score Structure",
            passed=True,
            details=details,
            data_count=len(df),
            processing_time=time.time() - start_time,
            warnings=warnings,
        )

    def validate_pbp_structure(self, df: pd.DataFrame) -> ValidationResult:
        """Validate play-by-play structure and event types."""
        start_time = time.time()

        required_columns = [
            'period', 'pbpOrder', 'msgType', 'offTeamId', 'defTeamId',
            'playerId1', 'locX', 'locY', 'pts'
        ]
        missing_cols = [col for col in required_columns if col not in df.columns]

        if missing_cols:
            return ValidationResult(
                step_name="PBP Structure",
                passed=False,
                details=f"Missing columns: {missing_cols}",
                data_count=len(df),
                processing_time=time.time() - start_time,
            )

        valid_events = df[df['offTeamId'].notna() & df['defTeamId'].notna()]
        event_types = df['msgType'].value_counts()

        shots = df[df['msgType'].isin([1, 2])]
        shots_with_coords = shots[(shots['locX'].notna()) & (shots['locY'].notna())]

        warnings: List[str] = []
        if len(shots) > 0 and len(shots_with_coords) < len(shots) * 0.8:
            warnings.append(f"Only {len(shots_with_coords)}/{len(shots)} shots have coordinates")

        details = (
            f"Total events: {len(df)}, "
            f"Valid events: {len(valid_events)}, "
            f"Event types: {len(event_types)}, "
            f"Shots with coords: {len(shots_with_coords)}"
        )

        return ValidationResult(
            step_name="PBP Structure",
            passed=True,
            details=details,
            data_count=len(df),
            processing_time=time.time() - start_time,
            warnings=warnings,
        )

    def validate_coordinate_system(self, df: pd.DataFrame) -> ValidationResult:
        """Validate coordinate scaling and rim detection logic."""
        start_time = time.time()

        shots = df[
            (df['msgType'].isin([1, 2])) &
            (df['locX'].notna()) &
            (df['locY'].notna())
        ].copy()

        if len(shots) == 0:
            return ValidationResult(
                step_name="Coordinate System",
                passed=False,
                details="No shots with coordinates found",
                processing_time=time.time() - start_time,
            )

        # distance in feet (coords are tenths of feet)
        shots['distance_ft'] = np.sqrt(shots['locX'] ** 2 + shots['locY'] ** 2) / self.coordinate_scale
        shots['is_rim_attempt'] = shots['distance_ft'] <= self.rim_distance_threshold

        rim_attempts = shots[shots['is_rim_attempt']]
        rim_makes = rim_attempts[rim_attempts['msgType'] == 1]

        warnings: List[str] = []
        max_distance = float(shots['distance_ft'].max())
        if max_distance > 35:
            warnings.append(f"Suspiciously long shot distance: {max_distance:.1f} feet")

        details = (
            f"Total shots: {len(shots)}, "
            f"Rim attempts: {len(rim_attempts)}, "
            f"Rim makes: {len(rim_makes)}, "
            f"Max distance: {max_distance:.1f}ft"
        )

        return ValidationResult(
            step_name="Coordinate System",
            passed=True,
            details=details,
            data_count=len(shots),
            processing_time=time.time() - start_time,
            warnings=warnings,
        )

    def print_validation_summary(self) -> bool:
        """Print a concise validation summary. Returns True if all tests passed."""
        print("\n" + "=" * 80)
        print("NBA PIPELINE VALIDATION SUMMARY")
        print("=" * 80)

        total_passed = sum(1 for r in self.validation_results if r.passed)
        total_tests = len(self.validation_results)
        total_time = sum(r.processing_time for r in self.validation_results)
        total_warnings = sum(len(r.warnings) for r in self.validation_results)

        print(f"OVERALL STATUS: {total_passed}/{total_tests} tests passed")
        print(f"TOTAL VALIDATION TIME: {total_time:.2f} seconds")
        print(f"TOTAL WARNINGS: {total_warnings}\n")

        for result in self.validation_results:
            status = "[PASS]" if result.passed else "[FAIL]"
            print(f"{status} {result.step_name}")
            print(f"   Details: {result.details}")
            print(f"   Data Count: {result.data_count:,}")
            print(f"   Time: {result.processing_time:.3f}s")
            for warning in result.warnings:
                print(f"   [WARN] {warning}")
            print()

        print("=" * 80)
        return total_passed == total_tests


if __name__ == "__main__":
    print("NBA Pipeline - Step 1: Analysis & Validation Setup")
    print("=" * 60)

    validator = NBADataValidator()

    expected_files = {
        'box_score': Path('api/src/airflow_project/data/mavs_data_engineer_2025/box_HOU-DAL.csv'),
        'pbp': Path('api/src/airflow_project/data/mavs_data_engineer_2025/pbp_HOU-DAL.csv'),
        'event_types': Path('api/src/airflow_project/data/mavs_data_engineer_2025/pbp_event_msg_types.csv'),
        'action_types': Path('api/src/airflow_project/data/mavs_data_engineer_2025/pbp_action_types.csv'),
        'option_types': Path('api/src/airflow_project/data/mavs_data_engineer_2025/pbp_option_types.csv'),
    }

    # Step 1A: Validate file structure
    file_validation = validator.validate_file_structure(expected_files)
    validator.log_validation(file_validation)

    print("\nStep 1 Complete: Foundation analysis ready")
    print("Next Step: Load and validate data content")
    print("The pipeline will process data with validations at each stage")

    print("\nDATA REQUIREMENTS SUMMARY:")
    print("- Box Score: Player info, starters (gs=1), team mapping, active status")
    print("- Play-by-Play: Events in chronological order, coordinates for shots")
    print("- Rim Attempts: Shots ≤ 4 feet from basket (coordinates/10 = feet)")
    print("- Lineup Tracking: 5 players per team; handle substitutions")
    print("- Possession Counting: Offensive/defensive possessions per lineup")
    print("- Outputs: 5-man lineups and individual player rim defense stats")


Overwriting api/src/airflow_project/eda/utils/nba_pipeline_analysis.py


# EDA

In [28]:
%%writefile api/src/airflow_project/eda/__init__.py
# EDA module for data exploration and analysis


Overwriting api/src/airflow_project/eda/__init__.py


In [29]:
%%writefile api/src/airflow_project/eda/data/nba_data_loader.py
# Enhanced NBA Data Loader - Step 2 Improvements
"""
NBA Pipeline - Enhanced Data Loader with Proper Validation
==========================================================

Keys:
1. Robust DuckDB object type handling (table vs view conflicts)
2. Improved error handling and validation
3. Better resource management and cleanup
4. Enhanced logging and debugging information

Lineup Estimation Methodology (Updated Implementation)
-----------------------------------------------------
This module implements two parallel lineup estimation methods:

TRADITIONAL DATA-DRIVEN METHOD (run_traditional_data_driven_lineups):
- Strictly follows raw data without automation or inference
- msgType=8: playerId1 = player subbed IN, playerId2 = player subbed OUT
- Lineups can have any size (not forced to 5 players)
- Comprehensive flagging for lineup size deviations and substitution issues
- Detailed explanations for why lineups aren't size 5
- Flags: lineup_size_deviation, sub_out_player_not_in_lineup, 
  sub_in_player_already_in_lineup, action_by_non_lineup_player

ENHANCED METHOD (run_enhanced_substitution_tracking_with_flags):
- Uses intelligent inference to maintain 5-player lineups
- Period resets: Q1 and Q3 reset to starters, Q2/Q4/OT carry forward
- First-Action Auto-IN: Players with actions but no sub-in are auto-added
- Inactivity Auto-OUT: Players idle >120s are candidates for removal
- Always-Five Enforcement: Maintains exactly 5 players per team
- Flags: missing_sub_in, inactivity_periods, first_action_events, 
  auto_out_events, lineup_violations

Both methods provide comprehensive validation and comparison capabilities.
"""


import os
import sys
import time
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from collections import defaultdict, deque

import duckdb
import pandas as pd
import numpy as np

# Ensure we're in the right directory
cwd = os.getcwd()
if not cwd.endswith("airflow_project"):
    os.chdir('api/src/airflow_project')
sys.path.insert(0, os.getcwd())

from eda.utils.nba_pipeline_analysis import NBADataValidator, ValidationResult

logger = logging.getLogger(__name__)

class EnhancedNBADataLoader:
    """Enhanced data loader with transparent validation and cleaning"""

    def __init__(self, db_path: str = "mavs_enhanced.duckdb", export_dir: str = None):
        """
        Enhanced NBA Data Loader with transparent validation and cleaning
        
        FIXED: Added export_dir initialization to prevent missing attribute errors
        
        Args:
            db_path: Path to DuckDB database file
            export_dir: Path to export directory (optional, auto-detected if None)
        """
        self.db_path = db_path
        self.conn = None
        self.validator = NBADataValidator()
        self.data_summary = {}

        # FIXED: Initialize export_dir attribute to prevent missing attribute error
        if export_dir is None:
            # Try to use config-managed export directory
            try:
                from utils.config import EXPORTS_DIR
                self.export_dir = EXPORTS_DIR
            except ImportError:
                # Fallback to default location relative to working directory
                from pathlib import Path
                self.export_dir = Path.cwd() / "exports"
        else:
            from pathlib import Path
            self.export_dir = Path(export_dir)
        
        # Ensure export directory exists
        self.export_dir.mkdir(parents=True, exist_ok=True)

        # Configuration for data quality
        self.rim_distance_threshold = 4.0  # feet
        self.coordinate_scale = 10.0  # NBA coordinates in tenths of feet
        self.max_reasonable_distance = 35.0  # feet (beyond court boundaries)

    def __enter__(self):
        # Prefer central config for path + engine settings
        try:
            from utils.config import DUCKDB_PATH, DUCKDB_CONFIG
            db_path = str(DUCKDB_PATH) if self.db_path in (None, "", "mavs_enhanced.duckdb") else self.db_path
            self.conn = duckdb.connect(db_path)
            # Apply engine config once, centrally
            if "memory_limit" in DUCKDB_CONFIG:
                self.conn.execute(f"SET memory_limit = '{DUCKDB_CONFIG['memory_limit']}'")
            if "threads" in DUCKDB_CONFIG:
                self.conn.execute(f"SET threads = {int(DUCKDB_CONFIG['threads'])}")
            if "temp_directory" in DUCKDB_CONFIG:
                self.conn.execute(f"SET temp_directory = '{DUCKDB_CONFIG['temp_directory']}'")
            if "preserve_insertion_order" in DUCKDB_CONFIG:
                self.conn.execute(f"SET preserve_insertion_order = {DUCKDB_CONFIG['preserve_insertion_order']}")
            if "checkpoint_threshold" in DUCKDB_CONFIG:
                self.conn.execute(f"SET checkpoint_threshold = '{DUCKDB_CONFIG['checkpoint_threshold']}'")
        except Exception:
            # Fallback to original behavior if config import fails
            self.conn = duckdb.connect(self.db_path or "mavs_enhanced.duckdb")
            self.conn.execute("SET memory_limit = '4GB'")
            self.conn.execute("SET threads = 4")
        return self


    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()

    def _robust_drop_object(self, object_name: str) -> None:
        """Robustly drop any DuckDB object regardless of type"""
        try:
            # Try dropping as table first
            self.conn.execute(f"DROP TABLE IF EXISTS {object_name}")
        except Exception:
            pass

        try:
            # Try dropping as view
            self.conn.execute(f"DROP VIEW IF EXISTS {object_name}")
        except Exception:
            pass

        try:
            # Try dropping as sequence
            self.conn.execute(f"DROP SEQUENCE IF EXISTS {object_name}")
        except Exception:
            pass

    def _to_native(self, obj):
        """
        Recursively convert objects to JSON-serializable Python builtins.
        - np.integer -> int
        - np.floating -> float
        - np.bool_ -> bool
        - pd.Timestamp/Timedelta -> str (ISO)
        - set/tuple -> list
        - dict -> dict with native keys/values
        - DataFrame/Series -> list/dict where sensible
        """
        import numpy as np
        import pandas as pd

        if obj is None:
            return None

        # Basic scalars
        if isinstance(obj, (bool, int, float, str)):
            return obj

        # NumPy scalars and arrays
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating,)):
            return float(obj)
        if isinstance(obj, (np.bool_,)):
            return bool(obj)
        # Handle numpy arrays and other array-like objects first
        if hasattr(obj, "tolist"):  # catches numpy arrays, pd.Series
            return self._to_native(obj.tolist())
        # Handle remaining numpy scalars that might have slipped through
        if hasattr(obj, 'dtype') and hasattr(obj, 'item'):
            if 'float' in str(obj.dtype):
                return float(obj.item())
            elif 'int' in str(obj.dtype):
                return int(obj.item())
            elif 'bool' in str(obj.dtype):
                return bool(obj.item())

        # Pandas-specific
        if isinstance(obj, (pd.Timestamp, pd.Timedelta)):
            return str(obj)

        # Mappings
        if isinstance(obj, dict):
            return {str(self._to_native(k)): self._to_native(v) for k, v in obj.items()}

        # Iterables (lists, tuples, sets)
        if isinstance(obj, (list, tuple, set)):
            return [self._to_native(v) for v in obj]

        # Fallback
        return str(obj)

    def _get_object_type(self, object_name: str) -> Optional[str]:
        """Get the type of a DuckDB object if it exists"""
        try:
            result = self.conn.execute(f"""
                SELECT table_type 
                FROM information_schema.tables 
                WHERE table_name = '{object_name}'
            """).fetchall()

            if result:
                return result[0][0]

            # Check views separately
            result = self.conn.execute(f"""
                SELECT 'VIEW' as table_type
                FROM information_schema.views 
                WHERE table_name = '{object_name}'
            """).fetchall()

            if result:
                return 'VIEW'

        except Exception as e:
            logger.debug(f"Could not check object type for {object_name}: {e}")

        return None

    def load_and_validate_box_score(self, file_path: Path) -> ValidationResult:
        """Load box score with enhanced validation and transparent reporting"""
        start_time = time.time()

        try:
            logger.info(f"Loading box score from {file_path}")

            # Load raw data
            df_raw = pd.read_csv(file_path)
            original_count = len(df_raw)

            logger.info(f"Raw box score: {original_count} rows")

            # Filter to active players only (as specified in requirements)
            df_active = df_raw[df_raw['status'] == 'ACTIVE'].copy()
            active_count = len(df_active)

            logger.info(f"Active players: {active_count} rows")

            # Validate we have the minimum required data
            if active_count < 10:  # Need at least 5 per team
                return ValidationResult(
                    step_name="Load Box Score",
                    passed=False,
                    details=f"Insufficient active players: {active_count} (minimum 10 required)",
                    processing_time=time.time() - start_time
                )

            # Check for required columns
            required_cols = ['nbaId', 'name', 'nbaTeamId', 'team', 'isHome', 'gs', 'secPlayed']
            missing_cols = [col for col in required_cols if col not in df_active.columns]

            if missing_cols:
                return ValidationResult(
                    step_name="Load Box Score",
                    passed=False,
                    details=f"Missing required columns: {missing_cols}",
                    processing_time=time.time() - start_time
                )

            # Clean and validate data
            warnings = []

            # Remove players with no playing time (they shouldn't affect lineup analysis)
            df_played = df_active[df_active['secPlayed'] > 0].copy()
            no_time_removed = active_count - len(df_played)

            if no_time_removed > 0:
                warnings.append(f"Removed {no_time_removed} players with no playing time")

            # Validate team structure
            team_analysis = self._analyze_team_structure(df_played)
            warnings.extend(team_analysis['warnings'])

            # Create optimized table with robust object handling
            self._robust_drop_object("box_score")
            self.conn.register("box_temp", df_played)

            create_sql = """
            CREATE TABLE box_score AS
            SELECT 
                nbaId as player_id,
                name as player_name,
                nbaTeamId as team_id,
                team as team_abbrev,
                CAST(isHome as BOOLEAN) as is_home,
                CAST(gs as BOOLEAN) as is_starter,
                status,
                secPlayed as seconds_played,
                COALESCE(pts, 0) as points,
                COALESCE(reb, 0) as rebounds,
                COALESCE(ast, 0) as assists,
                COALESCE(jerseyNum, 99) as jersey_number
            FROM box_temp
            WHERE player_id IS NOT NULL 
            AND player_name IS NOT NULL
            AND team_id IS NOT NULL
            ORDER BY team_id, seconds_played DESC
            """

            self.conn.execute(create_sql)
            self.conn.execute("DROP VIEW IF EXISTS box_temp")

            # Create indexes for performance with error handling
            try:
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_box_player ON box_score(player_id)")
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_box_team ON box_score(team_id)")
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_box_starter ON box_score(is_starter)")
            except Exception as e:
                logger.warning(f"Could not create indexes: {e}")

            # Get final count and store summary
            final_count = self.conn.execute("SELECT COUNT(*) FROM box_score").fetchone()[0]

            self.data_summary['box_score'] = {
                'original_rows': original_count,
                'active_rows': active_count,
                'final_rows': final_count,
                'teams': team_analysis['teams'],
                'starters_per_team': team_analysis['starters_per_team']
            }

            details = f"Processed box score: {original_count} → {active_count} active → {final_count} final rows"
            details += f". Teams: {team_analysis['teams']}, Starters: {team_analysis['starters_per_team']}"

            return ValidationResult(
                step_name="Load Box Score",
                passed=True,
                details=details,
                data_count=final_count,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Load Box Score",
                passed=False,
                details=f"Error loading box score: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _analyze_team_structure(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Analyze team structure and identify issues transparently"""
        analysis = {
            'teams': [],
            'starters_per_team': {},
            'players_per_team': {},
            'warnings': []
        }

        try:
            # Analyze each team
            for team_abbrev in df['team'].unique():
                if pd.isna(team_abbrev):
                    continue

                team_data = df[df['team'] == team_abbrev]
                analysis['teams'].append(team_abbrev)
                analysis['players_per_team'][team_abbrev] = len(team_data)

                # Count starters
                starters = team_data[team_data['gs'] == 1]
                analysis['starters_per_team'][team_abbrev] = len(starters)

                # Validate starter count (should be exactly 5)
                if len(starters) != 5:
                    analysis['warnings'].append(
                        f"Team {team_abbrev} has {len(starters)} starters (expected 5)"
                    )

                # Validate minimum roster size
                if len(team_data) < 8:
                    analysis['warnings'].append(
                        f"Team {team_abbrev} has only {len(team_data)} players (minimum 8 expected)"
                    )

            # Validate exactly 2 teams
            if len(analysis['teams']) != 2:
                analysis['warnings'].append(
                    f"Found {len(analysis['teams'])} teams (expected 2): {analysis['teams']}"
                )

            # Validate home/away designation
            home_teams = df[df['isHome'] == 1]['team'].unique()
            away_teams = df[df['isHome'] == 0]['team'].unique()

            if len(home_teams) != 1 or len(away_teams) != 1:
                analysis['warnings'].append(
                    f"Invalid home/away setup: home={list(home_teams)}, away={list(away_teams)}"
                )

        except Exception as e:
            analysis['warnings'].append(f"Team analysis error: {str(e)}")

        return analysis

    def load_and_validate_pbp(self, file_path: Path) -> ValidationResult:
        """Load PBP with enhanced validation and coordinate analysis"""
        start_time = time.time()
        try:
            logger.info(f"Loading PBP from {file_path}")
            df_raw = pd.read_csv(file_path)
            original_count = len(df_raw)

            # Identify admin rows
            admin_mask = (
                df_raw['offTeamId'].isna() | df_raw['defTeamId'].isna() |
                (df_raw['offTeamId'] == 0) | (df_raw['defTeamId'] == 0)
            )
            admin_rows = df_raw[admin_mask]
            game_events = df_raw[~admin_mask].copy()

            admin_count = len(admin_rows)
            game_count = len(game_events)
            logger.info(f"Admin rows: {admin_count}, Game events: {game_count}")

            warnings = []

            # Coordinate system analysis
            shots_mask = game_events['msgType'].isin([1, 2])
            shots = game_events[shots_mask].copy()
            coordinate_analysis = self._analyze_coordinate_system(shots)
            warnings.extend(coordinate_analysis['warnings'])

            # Create PBP table with robust object handling
            self._robust_drop_object("pbp")
            self.conn.register("pbp_temp", game_events)

            dist_expr = "(sqrt((loc_x::DOUBLE * loc_x::DOUBLE) + (loc_y::DOUBLE * loc_y::DOUBLE)) / 10.0)"

            create_sql = f"""
            CREATE TABLE pbp AS
            SELECT 
                pbpId AS pbp_id,
                period,
                pbpOrder AS pbp_order,
                wallClockInt AS wall_clock_int,
                COALESCE(gameClock, '') AS game_clock,
                COALESCE(description, '') AS description,
                msgType AS msg_type,
                COALESCE(actionType, 0) AS action_type,
                offTeamId AS team_id_off,
                defTeamId AS team_id_def,
                playerId1 AS player_id_1,
                playerId2 AS player_id_2,
                playerId3 AS player_id_3,
                -- keep last names from the raw file so we can label unknowns
                COALESCE(lastName1, '') AS last_name_1,
                COALESCE(lastName2, '') AS last_name_2,
                COALESCE(lastName3, '') AS last_name_3,
                locX AS loc_x,
                locY AS loc_y,
                COALESCE(pts, 0) AS points,

                CASE 
                WHEN msgType IN (1,2) AND locX IS NOT NULL AND locY IS NOT NULL 
                THEN {dist_expr} 
                END AS shot_distance_ft,

                CASE 
                WHEN msgType IN (1,2) AND locX IS NOT NULL AND locY IS NOT NULL 
                    AND {dist_expr} <= {self.rim_distance_threshold}
                THEN 1 ELSE 0 
                END::TINYINT AS is_rim_attempt,

                CASE 
                WHEN msgType IN (1,2) AND locX IS NOT NULL AND locY IS NOT NULL 
                    AND {dist_expr} > {self.max_reasonable_distance}
                THEN 1 ELSE 0 
                END::TINYINT AS is_extreme_distance,
                CASE 
                WHEN msgType IN (1,2) AND locX IS NOT NULL AND locY IS NOT NULL 
                    AND {dist_expr} > {self.max_reasonable_distance}
                THEN {dist_expr}
                END AS extreme_distance_ft

            FROM pbp_temp
            ORDER BY period, pbp_order, wall_clock_int
            """
            self.conn.execute(create_sql)
            self.conn.execute("DROP VIEW IF EXISTS pbp_temp")

            # Create indexes with error handling
            try:
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_pbp_chronological ON pbp(period, pbp_order)")
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_pbp_msg_type ON pbp(msg_type)")
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_pbp_teams ON pbp(team_id_off, team_id_def)")
                self.conn.execute("CREATE INDEX IF NOT EXISTS idx_pbp_rim ON pbp(is_rim_attempt)")
            except Exception as e:
                logger.warning(f"Could not create PBP indexes: {e}")

            final_count = self.conn.execute("SELECT COUNT(*) FROM pbp").fetchone()[0]

            flagged_extremes = self.conn.execute(
                "SELECT COUNT(*) FROM pbp WHERE is_extreme_distance = 1"
            ).fetchone()[0]
            if flagged_extremes > 0:
                warnings.append(f"Flagged {flagged_extremes} extreme-distance shots (> {self.max_reasonable_distance} ft)")

            self.data_summary['pbp'] = {
                'original_rows': original_count,
                'admin_rows': admin_count,
                'game_events': game_count,
                'final_rows': final_count,
                'coordinate_analysis': coordinate_analysis
            }

            details = (f"Processed PBP: {original_count} → {game_count} game events → {final_count} final rows. "
                    f"Shots: {coordinate_analysis['total_shots']}, Rim attempts: {coordinate_analysis['rim_attempts']}")
            return ValidationResult(
                step_name="Load PBP",
                passed=True,
                details=details,
                data_count=final_count,
                processing_time=time.time() - start_time,
                warnings=warnings
            )
        except Exception as e:
            return ValidationResult(
                step_name="Load PBP",
                passed=False,
                details=f"Error loading PBP: {str(e)}",
                processing_time=time.time() - start_time
            )



    def _analyze_coordinate_system(self, shots_df: pd.DataFrame) -> Dict[str, Any]:
        """Analyze coordinate system and identify issues"""
        analysis = {
            'total_shots': len(shots_df),
            'shots_with_coords': 0,
            'rim_attempts': 0,
            'rim_makes': 0,
            'extreme_shots': 0,
            'avg_distance': 0.0,
            'max_distance': 0.0,
            'warnings': []
        }

        if len(shots_df) == 0:
            analysis['warnings'].append("No shots found for coordinate analysis")
            return analysis

        # Filter to shots with coordinates
        coord_mask = shots_df['locX'].notna() & shots_df['locY'].notna()
        shots_with_coords = shots_df[coord_mask].copy()

        analysis['shots_with_coords'] = len(shots_with_coords)

        if len(shots_with_coords) == 0:
            analysis['warnings'].append("No shots have coordinate data")
            return analysis

        # Calculate distances
        shots_with_coords['distance_ft'] = np.sqrt(
            shots_with_coords['locX']**2 + shots_with_coords['locY']**2
        ) / self.coordinate_scale

        # Analyze distances
        analysis['avg_distance'] = shots_with_coords['distance_ft'].mean()
        analysis['max_distance'] = shots_with_coords['distance_ft'].max()

        # Count rim attempts (≤ 4 feet as specified)
        rim_mask = shots_with_coords['distance_ft'] <= self.rim_distance_threshold
        rim_shots = shots_with_coords[rim_mask]

        analysis['rim_attempts'] = len(rim_shots)
        analysis['rim_makes'] = len(rim_shots[rim_shots['msgType'] == 1])

        # Count extreme distances (likely data errors)
        extreme_mask = shots_with_coords['distance_ft'] > self.max_reasonable_distance
        analysis['extreme_shots'] = extreme_mask.sum()

        # Generate warnings
        if analysis['shots_with_coords'] < analysis['total_shots'] * 0.9:
            analysis['warnings'].append(
                f"Only {analysis['shots_with_coords']}/{analysis['total_shots']} shots have coordinates"
            )

        if analysis['extreme_shots'] > 0:
            analysis['warnings'].append(
                f"{analysis['extreme_shots']} shots beyond {self.max_reasonable_distance} feet (max: {analysis['max_distance']:.1f}ft)"
            )

        if analysis['rim_attempts'] == 0:
            analysis['warnings'].append("No rim attempts detected - check coordinate system")

        return analysis

    def validate_data_relationships(self) -> ValidationResult:
        """Validate box ↔ pbp team/player alignment. Recompute team set AFTER cleanup."""
        start_time = time.time()
        try:
            logger.info("Validating data relationships...")
            warnings = []

            # Box teams
            box_teams_df = self.conn.execute("""
                SELECT DISTINCT team_id, team_abbrev FROM box_score ORDER BY team_id
            """).df()
            box_team_ids = set(box_teams_df['team_id'])

            # PBP teams
            pbp_teams_df = self.conn.execute("""
                SELECT DISTINCT team_id FROM (
                    SELECT team_id_off AS team_id FROM pbp
                    UNION 
                    SELECT team_id_def AS team_id FROM pbp
                ) ORDER BY team_id
            """).df()
            pbp_team_ids = set(pbp_teams_df['team_id'])

            # If mismatch, remove unknowns with counted logging
            extra_pbp = pbp_team_ids - box_team_ids
            if extra_pbp:
                warnings.append(f"Team mismatch - Box: {sorted(box_team_ids)}, PBP: {sorted(pbp_team_ids)}")
                warnings.append(f"Extra teams in PBP: {sorted(extra_pbp)}")

                to_delete = self.conn.execute(f"""
                    SELECT COUNT(*) FROM pbp 
                    WHERE team_id_off NOT IN ({",".join(map(str, box_team_ids))})
                    OR team_id_def NOT IN ({",".join(map(str, box_team_ids))})
                """).fetchone()[0]

                self.conn.execute(f"""
                    DELETE FROM pbp 
                    WHERE team_id_off NOT IN ({",".join(map(str, box_team_ids))})
                    OR team_id_def NOT IN ({",".join(map(str, box_team_ids))})
                """)
                if to_delete > 0:
                    warnings.append(f"Removed {to_delete} PBP events from unknown teams")

                # Recompute
                pbp_teams_df = self.conn.execute("""
                    SELECT DISTINCT team_id FROM (
                        SELECT team_id_off AS team_id FROM pbp
                        UNION 
                        SELECT team_id_def AS team_id FROM pbp
                    ) ORDER BY team_id
                """).df()
                pbp_team_ids = set(pbp_teams_df['team_id'])

            # Player consistency
            box_players = set(self.conn.execute("SELECT DISTINCT player_id FROM box_score").df()['player_id'])
            pbp_players = set(self.conn.execute("""
                SELECT DISTINCT player_id FROM (
                    SELECT player_id_1 AS player_id FROM pbp WHERE player_id_1 IS NOT NULL
                    UNION SELECT player_id_2 FROM pbp WHERE player_id_2 IS NOT NULL
                    UNION SELECT player_id_3 FROM pbp WHERE player_id_3 IS NOT NULL
                )
            """).df()['player_id'])

            extra_pbp_players = pbp_players - box_players
            missing_pbp_players = box_players - pbp_players
            if extra_pbp_players:
                warnings.append(f"{len(extra_pbp_players)} players in PBP not in box score: players = {extra_pbp_players}")
                logger.info(f"Extra PBP players: {sorted(list(extra_pbp_players))[:10]}")
            if missing_pbp_players:
                warnings.append(f"{len(missing_pbp_players)} players in box score not in PBP")

            final_pbp_count = self.conn.execute("SELECT COUNT(*) FROM pbp").fetchone()[0]
            passed = (box_team_ids == pbp_team_ids) and len(box_team_ids) == 2

            details = (f"Relationship validation: Box teams: {len(box_team_ids)}, "
                    f"PBP teams: {len(pbp_team_ids)}, Final PBP events: {final_pbp_count}")
            return ValidationResult(
                step_name="Data Relationships",
                passed=passed,
                details=details,
                processing_time=time.time() - start_time,
                warnings=warnings
            )
        except Exception as e:
            return ValidationResult(
                    step_name="Data Relationships",
                    passed=False,
                    details=f"Error validating relationships: {str(e)}",
                    processing_time=time.time() - start_time
                )



    def create_lookup_views(self) -> ValidationResult:
        """Load lookup CSVs into DuckDB tables with robust handling"""
        start_time = time.time()
        try:
            logger.info("Creating lookup views...")

            # Use config-managed locations
            try:
                from utils.config import (
                    MAVS_DATA_DIR,
                    PBP_EVENT_MSG_TYPES_FILE,
                    PBP_ACTION_TYPES_FILE,
                    PBP_OPTION_TYPES_FILE,
                )
            except Exception as _e:
                return ValidationResult(
                    step_name="Create Lookup Views",
                    passed=False,
                    details=f"Config import failed: {_e}",
                    processing_time=time.time() - start_time
                )

            lookup_specs = [
                ("pbp_event_msg_types", PBP_EVENT_MSG_TYPES_FILE),
                ("pbp_action_types",    PBP_ACTION_TYPES_FILE),
                ("pbp_option_types",    PBP_OPTION_TYPES_FILE),
            ]

            created = []
            missing = []
            for table_name, file_path in lookup_specs:
                if not Path(file_path).exists():
                    missing.append(str(file_path))
                    continue

                # Robust object handling
                self._robust_drop_object(table_name)

                df = pd.read_csv(file_path)
                self.conn.register(f"{table_name}_temp", df)
                self.conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM {table_name}_temp")
                self.conn.execute(f"DROP VIEW IF EXISTS {table_name}_temp")
                created.append(table_name)

            if missing:
                return ValidationResult(
                    step_name="Create Lookup Views",
                    passed=False,
                    details=f"Missing lookup files: {missing}",
                    processing_time=time.time() - start_time
                )

            details = f"Created/Replaced {len(created)} lookup tables: {', '.join(created)}"
            return ValidationResult(
                step_name="Create Lookup Views",
                passed=True,
                details=details,
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Create Lookup Views",
                passed=False,
                details=f"Error creating lookup views: {str(e)}",
                processing_time=time.time() - start_time
            )



    def create_dimensions(self) -> ValidationResult:
        """Create dim_teams and dim_players with strict validation + provenance + confidence + referee filtering"""
        start_time = time.time()
        try:
            # Robust object handling
            self._robust_drop_object("dim_teams")
            self._robust_drop_object("dim_players")
            self._robust_drop_object("dim_officials")

            # STEP 1: Detect and filter referees/officials BEFORE creating dimensions
            logger.info("🔍 Detecting referees/officials for filtering...")
            referee_ids = self.identify_referees_and_officials()

            # STEP 2: Create officials table for transparency
            self.create_officials_table(referee_ids)

            # Create dim_teams
            self.conn.execute("""
                CREATE TABLE dim_teams AS
                SELECT
                    team_id,
                    ANY_VALUE(team_abbrev) AS team_abbrev,
                    ANY_VALUE(is_home) AS is_home
                FROM box_score
                GROUP BY team_id
                ORDER BY team_id
            """)

            n_teams = self.conn.execute("SELECT COUNT(*) FROM dim_teams").fetchone()[0]
            if n_teams != 2:
                raise AssertionError(f"dim_teams must have 2 rows, found {n_teams}")

            null_abbrev = self.conn.execute(
                "SELECT COUNT(*) FROM dim_teams WHERE team_abbrev IS NULL"
            ).fetchone()[0]
            if null_abbrev > 0:
                raise AssertionError("dim_teams has NULL team_abbrev")

            dup_map = self.conn.execute("""
                WITH m AS (
                    SELECT team_id, COUNT(DISTINCT team_abbrev) AS c
                    FROM box_score
                    GROUP BY team_id
                )
                SELECT COUNT(*) FROM m WHERE c <> 1
            """).fetchone()[0]
            if dup_map > 0:
                raise AssertionError("box_score has multiple team_abbrev values for the same team_id")

            # STEP 3: Create referee filter clause
            referee_filter = ""
            if referee_ids:
                referee_list = ','.join(map(str, referee_ids))
                referee_filter = f"AND player_id NOT IN ({referee_list})"
                logger.info(f"🚫 Filtering out {len(referee_ids)} referee IDs: {sorted(referee_ids)}")

            # --- collect names from pbp slots (FILTERED) ---
            self.conn.execute(f"""
                CREATE OR REPLACE TEMP VIEW _pbp_names AS
                WITH p1 AS (
                    SELECT player_id_1 AS player_id, ANY_VALUE(NULLIF(last_name_1,'')) AS last_name
                    FROM pbp
                    WHERE player_id_1 IS NOT NULL {referee_filter.replace('player_id', 'player_id_1')}
                    GROUP BY player_id_1
                ),
                p2 AS (
                    SELECT player_id_2 AS player_id, ANY_VALUE(NULLIF(last_name_2,'')) AS last_name
                    FROM pbp
                    WHERE player_id_2 IS NOT NULL {referee_filter.replace('player_id', 'player_id_2')}
                    GROUP BY player_id_2
                ),
                p3 AS (
                    SELECT player_id_3 AS player_id, ANY_VALUE(NULLIF(last_name_3,'')) AS last_name
                    FROM pbp
                    WHERE player_id_3 IS NOT NULL {referee_filter.replace('player_id', 'player_id_3')}
                    GROUP BY player_id_3
                ),
                unioned AS (
                    SELECT * FROM p1
                    UNION ALL
                    SELECT * FROM p2
                    UNION ALL
                    SELECT * FROM p3
                )
                SELECT player_id, ANY_VALUE(last_name) AS last_name
                FROM unioned
                WHERE last_name IS NOT NULL
                GROUP BY player_id
            """)

            # --- infer team_id for pbp-only players WITH CONFIDENCE (FILTERED) ---
            self.conn.execute(f"""
                CREATE OR REPLACE TEMP VIEW _player_team_guess AS
                WITH occ AS (
                    SELECT player_id_1 AS player_id, team_id_off AS team_id FROM pbp WHERE player_id_1 IS NOT NULL {referee_filter.replace('player_id', 'player_id_1')}
                    UNION ALL SELECT player_id_2, team_id_off FROM pbp WHERE player_id_2 IS NOT NULL {referee_filter.replace('player_id', 'player_id_2')}
                    UNION ALL SELECT player_id_3, team_id_off FROM pbp WHERE player_id_3 IS NOT NULL {referee_filter.replace('player_id', 'player_id_3')}
                    UNION ALL SELECT player_id_1, team_id_def FROM pbp WHERE player_id_1 IS NOT NULL {referee_filter.replace('player_id', 'player_id_1')}
                    UNION ALL SELECT player_id_2, team_id_def FROM pbp WHERE player_id_2 IS NOT NULL {referee_filter.replace('player_id', 'player_id_2')}
                    UNION ALL SELECT player_id_3, team_id_def FROM pbp WHERE player_id_3 IS NOT NULL {referee_filter.replace('player_id', 'player_id_3')}
                ),
                agg AS (
                    SELECT player_id, team_id, COUNT(*) AS c
                    FROM occ
                    GROUP BY player_id, team_id
                ),
                totals AS (
                    SELECT player_id, SUM(c) AS tot
                    FROM agg
                    GROUP BY player_id
                ),
                ranked AS (
                    SELECT
                        a.player_id,
                        a.team_id,
                        a.c,
                        t.tot,
                        ROW_NUMBER() OVER (PARTITION BY a.player_id ORDER BY a.c DESC, a.team_id) AS rn
                    FROM agg a
                    JOIN totals t USING(player_id)
                )
                SELECT
                    player_id,
                    team_id,
                    c,
                    tot,
                    (c::DOUBLE)/NULLIF(tot,0) AS confidence
                FROM ranked
                WHERE rn = 1
            """)

            # --- universe of pbp player_ids (FILTERED) ---
            self.conn.execute(f"""
                CREATE OR REPLACE TEMP VIEW _pbp_players AS
                SELECT player_id FROM (
                    SELECT DISTINCT player_id_1 AS player_id FROM pbp WHERE player_id_1 IS NOT NULL {referee_filter.replace('player_id', 'player_id_1')}
                    UNION
                    SELECT DISTINCT player_id_2 FROM pbp WHERE player_id_2 IS NOT NULL {referee_filter.replace('player_id', 'player_id_2')}
                    UNION
                    SELECT DISTINCT player_id_3 FROM pbp WHERE player_id_3 IS NOT NULL {referee_filter.replace('player_id', 'player_id_3')}
                )
            """)

            # Create comprehensive dim_players (with provenance + confidence)
            self.conn.execute("""
                CREATE TABLE dim_players AS
                SELECT
                    COALESCE(b.player_id, p.player_id) AS player_id,
                    COALESCE(b.player_name, n.last_name, CAST(COALESCE(b.player_id, p.player_id) AS VARCHAR)) AS player_name,
                    COALESCE(b.team_id, tg.team_id) AS team_id,
                    t.team_abbrev,
                    COALESCE(b.is_starter, false) AS is_starter,
                    COALESCE(b.seconds_played, 0) AS seconds_played,
                    -- provenance
                    CASE
                        WHEN b.player_name IS NOT NULL THEN 'box'
                        WHEN n.last_name IS NOT NULL THEN 'pbp_last_name'
                        ELSE 'player_id'
                    END AS name_source,
                    CASE
                        WHEN b.team_id IS NOT NULL THEN 'box'
                        WHEN tg.team_id IS NOT NULL THEN 'pbp_team_guess'
                        ELSE NULL
                    END AS team_source,
                    tg.confidence AS team_confidence
                FROM _pbp_players p
                FULL OUTER JOIN (
                    SELECT DISTINCT player_id, player_name, team_id, is_starter, seconds_played
                    FROM box_score
                ) b ON p.player_id = b.player_id
                LEFT JOIN _pbp_names n ON COALESCE(b.player_id, p.player_id) = n.player_id
                LEFT JOIN _player_team_guess tg ON COALESCE(b.player_id, p.player_id) = tg.player_id
                LEFT JOIN dim_teams t ON COALESCE(b.team_id, tg.team_id) = t.team_id
            """)

            # PBP-only players view (unchanged structure, still helpful)
            self._robust_drop_object("pbp_only_players")
            self.conn.execute("""
                CREATE VIEW pbp_only_players AS
                WITH box_ids AS (SELECT DISTINCT player_id FROM box_score),
                pbp_ids AS (SELECT DISTINCT player_id FROM dim_players),
                only_ids AS (
                    SELECT p.player_id
                    FROM pbp_ids p
                    LEFT JOIN box_ids b USING(player_id)
                    WHERE b.player_id IS NULL
                )
                SELECT
                    o.player_id,
                    dp.player_name,
                    dp.team_id,
                    dp.team_abbrev,
                    ANY_VALUE(CONCAT('Q', pbp.period, ' ', pbp.game_clock, ' | ', pbp.description)) AS sample_event
                FROM only_ids o
                JOIN dim_players dp USING(player_id)
                LEFT JOIN pbp ON (o.player_id = pbp.player_id_1 OR o.player_id = pbp.player_id_2 OR o.player_id = pbp.player_id_3)
                GROUP BY o.player_id, dp.player_name, dp.team_id, dp.team_abbrev
                ORDER BY player_id
            """)

            cnt_players = self.conn.execute("SELECT COUNT(*) FROM dim_players").fetchone()[0]
            cnt_only = self.conn.execute("SELECT COUNT(*) FROM pbp_only_players").fetchone()[0]
            cnt_officials = self.conn.execute("SELECT COUNT(*) FROM dim_officials").fetchone()[0]

            logger.info(f"✅ Created dim_players: {cnt_players} players")
            logger.info(f"✅ Created pbp_only_players: {cnt_only} PBP-only players")
            logger.info(f"✅ Created dim_officials: {cnt_officials} referees/officials")
            if referee_ids:
                logger.info(f"🚫 Filtered out referees: {sorted(referee_ids)}")

            # Ensure every active box player is present with a team
            missing_active = self.conn.execute("""
                WITH active_box AS (SELECT DISTINCT player_id FROM box_score)
                SELECT COUNT(*) FROM active_box a
                LEFT JOIN dim_players d USING(player_id)
                WHERE d.player_id IS NULL OR d.team_id IS NULL
            """).fetchone()[0]
            if missing_active > 0:
                raise AssertionError(f"{missing_active} active box players missing in dim_players or missing team_id")

            return ValidationResult(
                step_name="Create Dimensions",
                passed=True,
                details=f"dim_players: {cnt_players} rows; pbp_only_players: {cnt_only} rows; dim_officials: {cnt_officials} rows",
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Create Dimensions",
                passed=False,
                details=f"Error creating dimensions: {str(e)}",
                processing_time=time.time() - start_time
            )

    def identify_referees_and_officials(self):
        """
        Identify referee/official IDs based on event patterns.

        Criteria for referee detection:
        1. Appears in PBP events but NOT in active box score
        2. Only appears in foul calls (msgType=6) or turnovers (msgType=5) 
        3. Never appears in shots (msgType=1,2), rebounds (msgType=4), or subs (msgType=8)

        Returns:
            set: Set of player IDs identified as referees/officials
        """

        # Get all player IDs that appear in PBP
        pbp_players = self.conn.execute("""
            SELECT DISTINCT player_id FROM (
                SELECT player_id_1 AS player_id FROM pbp WHERE player_id_1 IS NOT NULL
                UNION
                SELECT player_id_2 FROM pbp WHERE player_id_2 IS NOT NULL  
                UNION
                SELECT player_id_3 FROM pbp WHERE player_id_3 IS NOT NULL
            )
        """).df()['player_id'].tolist()

        # Get active players from box score
        box_players = self.conn.execute("""
            SELECT DISTINCT player_id FROM box_score WHERE status = 'ACTIVE'
        """).df()['player_id'].tolist()

        # Find players in PBP but not in box score (potential referees)
        potential_refs = set(pbp_players) - set(box_players)

        if not potential_refs:
            return set()

        logger.info(f"🔍 Analyzing {len(potential_refs)} potential referee/official IDs...")

        confirmed_refs = set()

        for player_id in potential_refs:
            # Analyze event patterns for this player
            events = self.conn.execute(f"""
                SELECT 
                    msg_type,
                    description,
                    last_name_1,
                    last_name_2, 
                    last_name_3
                FROM pbp 
                WHERE player_id_1 = {player_id} 
                   OR player_id_2 = {player_id}
                   OR player_id_3 = {player_id}
            """).df()

            if len(events) == 0:
                continue

            # Get player name from events
            names = set()
            for _, row in events.iterrows():
                for col in ['last_name_1', 'last_name_2', 'last_name_3']:
                    if pd.notna(row[col]):
                        names.add(row[col])

            name = list(names)[0] if names else f"ID_{player_id}"

            # Analyze event type patterns
            msg_types = events['msg_type'].value_counts().to_dict()

            # Referee criteria:
            # 1. Only appears in fouls (6) or turnovers (5) or technical fouls (16,17,18)
            # 2. Never in shots (1,2), rebounds (4), substitutions (8)

            referee_event_types = {5, 6, 7, 16, 17, 18}  # turnovers, fouls, technicals
            player_event_types = {1, 2, 4, 8}  # shots, rebounds, substitutions

            has_referee_events = any(msg_type in referee_event_types for msg_type in msg_types.keys())
            has_player_events = any(msg_type in player_event_types for msg_type in msg_types.keys())

            if has_referee_events and not has_player_events:
                confirmed_refs.add(player_id)
                logger.info(f"  ✅ {name} (ID: {player_id}) - REFEREE/OFFICIAL")
                logger.info(f"      Events: {dict(msg_types)}")

        logger.info(f"🎯 Identified {len(confirmed_refs)} confirmed referees/officials")
        return confirmed_refs

    def create_officials_table(self, referee_ids):
        """Create a separate table for referees/officials for transparency"""

        if not referee_ids:
            # Create empty table
            self.conn.execute("""
                CREATE TABLE dim_officials AS
                SELECT 
                    CAST(NULL AS INTEGER) AS official_id,
                    CAST(NULL AS VARCHAR) AS official_name,
                    CAST(NULL AS INTEGER) AS total_events,
                    CAST(NULL AS VARCHAR) AS event_types,
                    CAST(NULL AS VARCHAR) AS sample_description
                WHERE FALSE
            """)
            return

        # Build officials data
        officials_data = []

        for official_id in referee_ids:
            # Get event details for this official
            events = self.conn.execute(f"""
                SELECT 
                    msg_type,
                    description,
                    last_name_1,
                    last_name_2,
                    last_name_3
                FROM pbp 
                WHERE player_id_1 = {official_id}
                   OR player_id_2 = {official_id}
                   OR player_id_3 = {official_id}
            """).df()

            # Extract name
            names = set()
            for _, row in events.iterrows():
                for col in ['last_name_1', 'last_name_2', 'last_name_3']:
                    if pd.notna(row[col]):
                        names.add(row[col])

            official_name = list(names)[0] if names else f"Official_{official_id}"

            # Event type summary
            msg_types = events['msg_type'].value_counts().to_dict()
            event_types_str = ', '.join([f"msgType{k}:{v}" for k, v in sorted(msg_types.items())])

            # Sample description
            sample_desc = events.iloc[0]['description'] if len(events) > 0 else "No events"

            officials_data.append({
                'official_id': official_id,
                'official_name': official_name,
                'total_events': len(events),
                'event_types': event_types_str,
                'sample_description': sample_desc
            })

        # Insert data
        if officials_data:
            officials_df = pd.DataFrame(officials_data)

            # Create table
            self.conn.execute("DROP TABLE IF EXISTS dim_officials")
            self.conn.register('officials_temp', officials_df)
            self.conn.execute("""
                CREATE TABLE dim_officials AS
                SELECT * FROM officials_temp
            """)
            self.conn.unregister('officials_temp')

            logger.info(f"📋 Created dim_officials table with {len(officials_data)} officials")

    def create_pbp_enriched_view(self) -> ValidationResult:
        """Create enriched PBP view with robust object handling"""
        start_time = time.time()
        try:
            # Check required tables exist
            required = ["pbp", "pbp_event_msg_types", "pbp_action_types", "pbp_option_types", "dim_teams", "dim_players"]
            for t in required:
                exists = self.conn.execute(
                    f"SELECT COUNT(*) FROM information_schema.tables WHERE table_name = '{t}'"
                ).fetchone()[0]
                if exists == 0:
                    return ValidationResult(
                        step_name="Create PBP Enriched View",
                        passed=False,
                        details=f"Missing required table: {t}",
                        processing_time=time.time() - start_time
                    )

            # Robust cleanup
            self._robust_drop_object("pbp_enriched")

            # Create enriched view with proper deduplication
            self.conn.execute("""
            CREATE VIEW pbp_enriched AS
            WITH team_map AS (
                SELECT DISTINCT team_id, team_abbrev FROM dim_teams
            ),
            event_types AS (
                SELECT DISTINCT EventType, Description FROM pbp_event_msg_types
            ),
            action_types AS (
                SELECT DISTINCT EventType, ActionType, Event, Description 
                FROM pbp_action_types
            ),
            option_types AS (
                SELECT EventType, 
                       ANY_VALUE(Option1) AS Option1,
                       ANY_VALUE(Option2) AS Option2,
                       ANY_VALUE(Option3) AS Option3,
                       ANY_VALUE(Option4) AS Option4
                FROM pbp_option_types
                GROUP BY EventType
            )
            SELECT
                p.*,
                emt.Description AS event_family,
                act.Event AS action_event,
                act.Description AS action_desc,
                toff.team_abbrev AS team_off_abbrev,
                tdef.team_abbrev AS team_def_abbrev,
                COALESCE(p1.player_name, NULLIF(p.last_name_1, '')) AS player1_name,
                COALESCE(p2.player_name, NULLIF(p.last_name_2, '')) AS player2_name,
                COALESCE(p3.player_name, NULLIF(p.last_name_3, '')) AS player3_name,
                opt.Option1 AS option1_label,
                opt.Option2 AS option2_label,
                opt.Option3 AS option3_label,
                opt.Option4 AS option4_label
            FROM pbp p
            LEFT JOIN event_types emt ON p.msg_type = emt.EventType
            LEFT JOIN action_types act ON p.msg_type = act.EventType AND p.action_type = act.ActionType
            LEFT JOIN option_types opt ON p.msg_type = opt.EventType
            LEFT JOIN team_map toff ON p.team_id_off = toff.team_id
            LEFT JOIN team_map tdef ON p.team_id_def = tdef.team_id
            LEFT JOIN dim_players p1 ON p.player_id_1 = p1.player_id
            LEFT JOIN dim_players p2 ON p.player_id_2 = p2.player_id
            LEFT JOIN dim_players p3 ON p.player_id_3 = p3.player_id
            ORDER BY p.period, p.pbp_order, p.wall_clock_int
            """)

            # Validate row count matches
            n_pbp = self.conn.execute("SELECT COUNT(*) FROM pbp").fetchone()[0]
            n_enriched = self.conn.execute("SELECT COUNT(*) FROM pbp_enriched").fetchone()[0]

            if n_pbp != n_enriched:
                return ValidationResult(
                    step_name="Create PBP Enriched View",
                    passed=False,
                    details=f"Row count mismatch: pbp={n_pbp} vs enriched={n_enriched}",
                    processing_time=time.time() - start_time
                )

            return ValidationResult(
                step_name="Create PBP Enriched View",
                passed=True,
                details=f"Created view pbp_enriched with {n_enriched} rows (matches pbp)",
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Create PBP Enriched View",
                passed=False,
                details=f"Error creating pbp_enriched view: {str(e)}",
                processing_time=time.time() - start_time
            )

    def print_enhanced_summary(self):
        """Print enhanced data loading summary (ASCII-only)."""
        print("\n" + "=" * 80)
        print("ENHANCED NBA PIPELINE - DATA LOADING SUMMARY")
        print("=" * 80)

        if 'box_score' in self.data_summary:
            box_data = self.data_summary['box_score']
            print("BOX SCORE:")
            print(f"   Original rows: {box_data['original_rows']:,}")
            print(f"   Active players: {box_data['active_rows']:,}")
            print(f"   Final rows: {box_data['final_rows']:,}")
            teams_str = ", ".join(box_data['teams'])
            print(f"   Teams: {teams_str}")
            print(f"   Starters per team: {box_data['starters_per_team']}")

        if 'pbp' in self.data_summary:
            pbp_data = self.data_summary['pbp']
            coord_data = pbp_data['coordinate_analysis']
            print("\nPLAY-BY-PLAY:")
            print(f"   Original rows: {pbp_data['original_rows']:,}")
            print(f"   Game events: {pbp_data['game_events']:,}")
            print(f"   Final rows: {pbp_data['final_rows']:,}")
            print(f"   Total shots: {coord_data['total_shots']:,}")
            print(f"   Shots with coordinates: {coord_data['shots_with_coords']:,}")
            print(f"   Rim attempts: {coord_data['rim_attempts']:,}")
            print(f"   Average distance: {coord_data['avg_distance']:.1f} ft")

        if 'enhanced_substitution_debug' in self.data_summary:
            d = self.data_summary['enhanced_substitution_debug']
            print("\nLINEUP ENGINE:")
            print(f"   Substitutions: {d.get('substitutions', 0)}")
            print(f"   First-actions auto-IN: {d.get('first_actions', 0)}")
            print(f"   Inactivity auto-OUTs: {d.get('auto_outs', 0)}")
            print(f"   5-on-floor fixes: {d.get('always_five_fixes', 0)}")
            v = d.get('validation', {})
            print(f"   Minutes tolerance: ±{v.get('tolerance', 0)}s")
            print(f"   Minutes offenders: {v.get('offenders', 0)}/{v.get('total_players', 0)}")

        if 'lineup_results' in self.data_summary:
            print("\nANALYTICS RESULTS:")
            print(f"   Lineup combinations: {self.data_summary['lineup_results']['rows']:,}")
            print(f"   Player rim stats: {self.data_summary['player_rim_results']['rows']:,}")

        print("=" * 80)


    def write_final_report(self, reports_dir: Optional[Path] = None) -> ValidationResult:
        """
        Emit an end-of-run report:
        - minutes_validation_full / minutes_offenders (enhanced pass)
        - basic_lineup_state.csv / basic_lineup_flags.csv / minutes_basic.csv
        - enhanced_lineup_state.csv / enhanced_lineup_flags.csv / minutes_enhanced.csv
        - minutes_compare.csv
        - traditional_vs_enhanced_comparison.csv
        - comprehensive_flags_analysis.csv (when available)
        - run_summary.json (JSON-safe)
        - UPDATED:
            * unique_lineups_traditional_5.csv        (5-man only)
            * unique_lineups_traditional_all.csv      (all sizes)
            * unique_lineups_enhanced_5.csv           (5-man only)
            * Logs show traditional 5-man AND all-sizes
        """
        import time, json as _json
        from pathlib import Path
        import pandas as pd

        start_time = time.time()
        try:
            mv_df = self.data_summary.get('minutes_validation_full')
            offenders = self.data_summary.get('minutes_offenders')
            debug = self.data_summary.get('enhanced_substitution_debug', {})

            if reports_dir is None:
                reports_dir = Path("reports")
            reports_dir.mkdir(parents=True, exist_ok=True)

            # Persist enhanced minutes into DuckDB for reproducibility
            try:
                self._robust_drop_object("minutes_validation_full")
                self._robust_drop_object("minutes_offenders")
                if mv_df is not None and len(mv_df) > 0:
                    self.conn.register("mv_temp", mv_df)
                    self.conn.execute("CREATE TABLE minutes_validation_full AS SELECT * FROM mv_temp")
                    self.conn.execute("DROP VIEW IF EXISTS mv_temp")
                else:
                    self.conn.execute("CREATE TABLE minutes_validation_full AS SELECT 1 WHERE FALSE")
                if offenders is not None and len(offenders) > 0:
                    self.conn.register("off_temp", offenders)
                    self.conn.execute("CREATE TABLE minutes_offenders AS SELECT * FROM off_temp")
                    self.conn.execute("DROP VIEW IF EXISTS off_temp")
                else:
                    self.conn.execute("CREATE TABLE minutes_offenders AS SELECT 1 WHERE FALSE")
            except Exception as e:
                logger.warning("[Report] Could not create DuckDB tables: %s", e)

            # CSVs for enhanced minutes tables
            if mv_df is not None:
                mv_df.to_csv(reports_dir / "minutes_validation_full.csv", index=False)
            if offenders is not None:
                offenders.to_csv(reports_dir / "minutes_offenders.csv", index=False)

            # Export basic / enhanced state snapshots if present
            try:
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='basic_lineup_state'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM basic_lineup_state ORDER BY period, pbp_order, team_id) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "basic_lineup_state.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='basic_lineup_flags'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM basic_lineup_flags ORDER BY abs_time, team_id) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "basic_lineup_flags.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='minutes_basic'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM minutes_basic ORDER BY team_abbrev, player_name) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "minutes_basic.csv").as_posix())))

                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='enhanced_lineup_state'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM enhanced_lineup_state ORDER BY period, pbp_order, team_id) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "enhanced_lineup_state.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='enhanced_lineup_flags'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM enhanced_lineup_flags ORDER BY abs_time, team_id) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "enhanced_lineup_flags.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='minutes_enhanced'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM minutes_enhanced ORDER BY team_abbrev, player_name) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "minutes_enhanced.csv").as_posix())))
            except Exception as e:
                logger.warning("[Report] Could not export basic/enhanced CSVs: %s", e)

            # Comparison exports (minutes and flags)
            try:
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='minutes_compare'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM minutes_compare ORDER BY team_abbrev, player_name) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "minutes_compare.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='traditional_vs_enhanced_comparison'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM traditional_vs_enhanced_comparison ORDER BY team_abbrev, player_name) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "traditional_vs_enhanced_comparison.csv").as_posix())))
                if self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='comprehensive_flags_analysis'").fetchone()[0]:
                    self.conn.execute("COPY (SELECT * FROM comprehensive_flags_analysis ORDER BY time, team) TO '{}' (HEADER, DELIMITER ',')"
                                    .format(str((reports_dir / "comprehensive_flags_analysis.csv").as_posix())))
            except Exception as e:
                logger.warning("[Report] Could not export comparison/flags CSVs: %s", e)

            # -----------------------------
            # UPDATED: UNIQUE LINEUPS COMPUTE
            # -----------------------------
            def _table_exists(name: str) -> bool:
                return bool(self.conn.execute(
                    "SELECT COUNT(*) FROM information_schema.tables WHERE table_name = ?", [name]
                ).fetchone()[0])

            # Name map for pretty printing
            try:
                if _table_exists("dim_players"):
                    nm_df = self.conn.execute("SELECT DISTINCT player_id, player_name FROM dim_players").df()
                else:
                    nm_df = self.conn.execute("SELECT DISTINCT player_id, player_name FROM box_score").df()
            except Exception:
                nm_df = pd.DataFrame(columns=["player_id","player_name"])
            name_map = dict(zip(nm_df.get("player_id", []), nm_df.get("player_name", [])))

            def _ids_to_names(ids):
                return [str(name_map.get(int(pid), str(int(pid)))) for pid in ids]

            def _parse_ids_json(s) -> list:
                # Robust JSON or bracketed string parsing
                try:
                    v = _json.loads(s)
                    if isinstance(v, list):
                        return [int(x) for x in v]
                except Exception:
                    pass
                # fallback "1,2,3" or "[1, 2, 3]"
                s2 = str(s).strip().strip("[]")
                if not s2:
                    return []
                return [int(x.strip()) for x in s2.split(",") if x.strip()]

            def _compute_unique_lineups(table_name: str, label: str, size_filter: Optional[int]) -> tuple[pd.DataFrame, dict]:
                """
                Returns df with columns:
                [method, team_id, team_abbrev, lineup_size, occurrences,
                lineup_player_ids_json, lineup_player_names_json]
                'counts' includes: total_unique, by_team, and by_size.
                """
                cols = ["method","team_id","team_abbrev","lineup_size","occurrences",
                        "lineup_player_ids_json","lineup_player_names_json"]
                if not _table_exists(table_name):
                    return pd.DataFrame(columns=cols), {"total_unique": 0, "by_team": {}, "by_size": {}}

                df = self.conn.execute(f"""
                    SELECT team_id, team_abbrev, lineup_player_ids_json, lineup_size
                    FROM {table_name}
                    WHERE lineup_player_ids_json IS NOT NULL
                """).df()

                if df.empty:
                    return pd.DataFrame(columns=cols), {"total_unique": 0, "by_team": {}, "by_size": {}}

                if size_filter is not None:
                    df = df[df["lineup_size"] == size_filter].copy()

                # Group by team, size, and the player-ids JSON
                grp = df.groupby(["team_id","team_abbrev","lineup_size","lineup_player_ids_json"], as_index=False)\
                        .size().rename(columns={"size":"occurrences"})

                # Convert ids json to names json (canonical sort for stability)
                names_json = []
                for s in grp["lineup_player_ids_json"]:
                    ids = sorted(_parse_ids_json(s))
                    names = _ids_to_names(ids)
                    names_json.append(_json.dumps(names))
                grp["lineup_player_names_json"] = names_json
                grp.insert(0, "method", label)

                # Counts (overall unique, by team, by size)
                total_unique = int(grp[["team_id","lineup_player_ids_json"]].drop_duplicates().shape[0])
                by_team = grp.groupby("team_abbrev")["lineup_player_ids_json"].nunique().to_dict()
                by_size = grp.groupby("lineup_size")["lineup_player_ids_json"].nunique().to_dict()

                grp = grp.sort_values(["team_abbrev","lineup_size","occurrences"], ascending=[True, True, False]).reset_index(drop=True)
                counts = {"total_unique": total_unique,
                        "by_team": {str(k): int(v) for k, v in by_team.items()},
                        "by_size": {int(k): int(v) for k, v in by_size.items()}}
                return grp, counts

            # Compute: TRADITIONAL (5-man), TRADITIONAL (all sizes), ENHANCED (5-man)
            trad_5_df,   trad_5_counts   = _compute_unique_lineups("traditional_lineup_state", "traditional", size_filter=5)
            trad_all_df, trad_all_counts = _compute_unique_lineups("traditional_lineup_state", "traditional", size_filter=None)
            enh_5_df,    enh_5_counts    = _compute_unique_lineups("enhanced_lineup_state",    "enhanced",   size_filter=5)

            # Write CSVs
            if not trad_5_df.empty:
                trad_5_df.to_csv(reports_dir / "unique_lineups_traditional_5.csv", index=False)
            if not trad_all_df.empty:
                trad_all_df.to_csv(reports_dir / "unique_lineups_traditional_all.csv", index=False)
            if not enh_5_df.empty:
                enh_5_df.to_csv(reports_dir / "unique_lineups_enhanced_5.csv", index=False)

            # Helper: ASCII logging of unique lineups
            def _log_unique_list(df: pd.DataFrame, title: str):
                if df.empty:
                    logger.info("[%s] No unique lineups found.", title)
                    return
                logger.info("=" * 78)
                logger.info("UNIQUE LINEUPS — %s", title)
                logger.info("=" * 78)
                for team_abbrev, sub in df.groupby("team_abbrev"):
                    # Include lineup_size in the heading when mixed
                    sizes = sorted(sub["lineup_size"].unique().tolist())
                    size_tag = "" if sizes == [5] else f" (sizes: {sizes})"
                    logger.info("%s: %d unique lineups%s", team_abbrev, len(sub), size_tag)
                    sub = sub.sort_values(["lineup_size","occurrences"], ascending=[True, False]).reset_index(drop=True)
                    for i, row in sub.iterrows():
                        try:
                            names = _json.loads(row["lineup_player_names_json"])
                        except Exception:
                            names = row["lineup_player_names_json"]
                        names_str = ", ".join(names) if isinstance(names, list) else str(names)
                        logger.info("  %2d. [%d] size=%d  %s", i+1, int(row["occurrences"]), int(row["lineup_size"]), names_str)
                logger.info("-" * 78)

            # Log both traditional views and enhanced 5-man
            _log_unique_list(trad_5_df,   "TRADITIONAL (5-man)")
            _log_unique_list(trad_all_df, "TRADITIONAL (ALL sizes)")
            _log_unique_list(enh_5_df,    "ENHANCED (5-man)")

            # Build JSON summary (JSON-safe)
            comparison_data = self.data_summary.get('traditional_vs_enhanced_comparison', {})
            comparison_summary = comparison_data.get('summary', {})

            # Traditional/enhanced state counts for summary
            traditional_states = int(self.data_summary.get("traditional_data_driven", {}).get("state_rows", 0))
            enhanced_states    = int(self.data_summary.get("enhanced_substitution_tracking", {}).get("state_rows", 0))

            # Unique counts via SQL (secondary check)
            def _unique_count(table_name: str, size_filter: Optional[int]) -> int:
                try:
                    exists = self.conn.execute(
                        "SELECT COUNT(*) FROM information_schema.tables WHERE table_name = ?",
                        [table_name]
                    ).fetchone()[0]
                    if not exists:
                        return 0
                    if size_filter is None:
                        q = f"""
                            SELECT COUNT(*) FROM (
                                SELECT team_id, lineup_player_ids_json
                                FROM {table_name}
                                GROUP BY team_id, lineup_player_ids_json
                            )
                        """
                    else:
                        q = f"""
                            SELECT COUNT(*) FROM (
                                SELECT team_id, lineup_player_ids_json
                                FROM {table_name}
                                WHERE lineup_size = {int(size_filter)}
                                GROUP BY team_id, lineup_player_ids_json
                            )
                        """
                    return int(self.conn.execute(q).fetchone()[0])
                except Exception:
                    return 0

            traditional_unique_5   = _unique_count("traditional_lineup_state", 5)
            traditional_unique_all = _unique_count("traditional_lineup_state", None)
            enhanced_unique_5      = _unique_count("enhanced_lineup_state", 5)

            # Build summary safely
            summary = {
                "substitutions": int(debug.get("substitutions", 0)),
                "first_actions": int(debug.get("first_actions", 0)),
                "auto_outs": int(debug.get("auto_outs", 0)),
                "always_five_fixes": int(debug.get("always_five_fixes", 0)),
                "total_players": int(debug.get("validation", {}).get("total_players", 0)),
                "offenders": int(debug.get("validation", {}).get("offenders", 0)),
                "tolerance_seconds": int(debug.get("validation", {}).get("tolerance", 120)),
                "traditional": {
                    "lineup_states": traditional_states,
                    "unique_lineups_5": traditional_unique_5,
                    "unique_lineups_all": traditional_unique_all,
                    "flag_types": self._to_native(self.data_summary.get("traditional_data_driven", {}).get("flag_summary", {})),
                    "by_team_5": self._to_native(trad_5_counts.get("by_team", {})),
                    "by_team_all": self._to_native(trad_all_counts.get("by_team", {})),
                    "by_size_all": self._to_native(trad_all_counts.get("by_size", {}))
                },
                "enhanced": {
                    "lineup_states": enhanced_states,
                    "unique_lineups_5": enhanced_unique_5,
                    "flag_totals": self._to_native(self.data_summary.get("enhanced_substitution_tracking", {}).get("flag_totals", {})),
                    "by_team_5": self._to_native(enh_5_counts.get("by_team", {}))
                },
                "minutes_compare_rows": int(self.data_summary.get("minutes_compare", {}).get("rows", 0)),
                "minutes_compare_basic_within10": int(self.data_summary.get("minutes_compare", {}).get("within10_basic", 0)),
                "minutes_compare_total_with_box": int(self.data_summary.get("minutes_compare", {}).get("total_with_box", 0)),
                "traditional_vs_enhanced": self._to_native(comparison_summary.get("method_comparison", {})),
                "enhanced_flags_summary": self._to_native(comparison_summary.get("flag_analysis", {})),
                "accuracy_metrics": self._to_native(comparison_summary.get("accuracy_metrics", {}))
            }

            with open(reports_dir / "run_summary.json", "w", encoding="utf-8") as f:
                import json as __json
                __json.dump(self._to_native(summary), f, indent=2)

            details = (
                "Report written: minutes_validation_full.csv, minutes_offenders.csv, "
                "basic_lineup_state.csv, basic_lineup_flags.csv, minutes_basic.csv, "
                "enhanced_lineup_state.csv, enhanced_lineup_flags.csv, minutes_enhanced.csv, "
                "minutes_compare.csv, traditional_vs_enhanced_comparison.csv, comprehensive_flags_analysis.csv, "
                "unique_lineups_traditional_5.csv, unique_lineups_traditional_all.csv, unique_lineups_enhanced_5.csv, "
                "run_summary.json"
            )
            return ValidationResult("Write Final Report", True, details, processing_time=time.time()-start_time)

        except Exception as e:
            return ValidationResult("Write Final Report", False, f"Error writing report: {e}", processing_time=time.time()-start_time)



    def run_traditional_data_driven_lineups(self) -> ValidationResult:
        """
        TRADITIONAL DATA-DRIVEN SUBSTITUTION TRACKING (Updated Implementation):

        This method strictly follows the raw data without any automation or inference:
        - msgType=8: playerId1 = player subbed IN, playerId2 = player subbed OUT
        - Lineups can have any size (not forced to 5)
        - Comprehensive flagging for lineup size deviations and substitution issues
        - Detailed explanations for why lineups aren't size 5

        Key Changes from Original:
        1. Removed automatic lineup size enforcement
        2. Added detailed flagging for substitution anomalies
        3. Enhanced validation of player states
        4. Better tracking of player availability vs. actual lineup membership

        Outputs written to DuckDB tables:
            * traditional_lineup_state
            * traditional_lineup_flags
            * minutes_traditional
        """
        from collections import defaultdict, deque
        import json
        start_time = time.time()

        try:
            # ---- Configuration ----
            CFG = {
                "starter_reset_periods": [1, 3],
                "sub_msg_type": 8,
                "action_msg_types": {1, 2, 4, 5, 6},  # FG made/miss, rebound, turnover, foul
                "allow_variable_lineup_sizes": True,  # NEW: Allow non-5 player lineups
                "detailed_flagging": True  # NEW: Enhanced flagging system
            }

            # ---- Helper Functions ----
            def _period_len(p: int) -> float:
                return 720.0 if p <= 4 else 300.0

            def _parse_gc(gc: str | None) -> float | None:
                if not gc or not isinstance(gc, str) or ":" not in gc:
                    return None
                try:
                    mm, ss = gc.split(":")
                    return float(mm) * 60.0 + float(ss)
                except Exception:
                    return None

            def _abs_t(period: int, rem: float | None) -> float:
                total = 0.0
                for pi in range(1, period):
                    total += _period_len(pi)
                pl = _period_len(period)
                if rem is None:
                    return total + pl
                return total + (pl - rem)

            # ---- Load Data ----
            box_df = self.conn.execute("""
                SELECT player_id, player_name, team_id, team_abbrev, is_starter, seconds_played
                FROM box_score
                WHERE seconds_played > 0
                ORDER BY team_id, seconds_played DESC
            """).df()

            if box_df.empty:
                return ValidationResult("Traditional Data-Driven Lineups", False, 
                                      "No active players in box_score", processing_time=time.time()-start_time)

            teams = sorted(box_df.team_id.unique().tolist())
            if len(teams) != 2:
                return ValidationResult("Traditional Data-Driven Lineups", False, 
                                      f"Expected 2 teams, found {teams}", processing_time=time.time()-start_time)

            # Build player mappings
            team_abbrev = {int(t): box_df[box_df.team_id == t].team_abbrev.iloc[0] for t in teams}
            starters = {int(t): set(box_df[(box_df.team_id==t)&(box_df.is_starter==True)].player_id.tolist()) for t in teams}
            name_map = dict(zip(box_df.player_id, box_df.player_name))
            pteam_map = dict(zip(box_df.player_id, box_df.team_id))

            # Validate starters
            for t in teams:
                if len(starters[int(t)]) != 5:
                    return ValidationResult("Traditional Data-Driven Lineups", False, 
                                          f"Team {team_abbrev[int(t)]} does not have 5 starters", 
                                          processing_time=time.time()-start_time)

            # Load events
            events = self.conn.execute("""
                SELECT period, pbp_order, wall_clock_int,
                       COALESCE(game_clock,'') AS game_clock,
                       COALESCE(description,'') AS description,
                       team_id_off, team_id_def, msg_type, action_type,
                       player_id_1, player_id_2, player_id_3,
                       NULLIF(last_name_1,'') AS last_name_1,
                       NULLIF(last_name_2,'') AS last_name_2,
                       NULLIF(last_name_3,'') AS last_name_3,
                       COALESCE(points,0) AS points
                FROM pbp
                ORDER BY period, pbp_order, wall_clock_int
            """).df()

            if events.empty:
                return ValidationResult("Traditional Data-Driven Lineups", False, 
                                      "No PBP events", processing_time=time.time()-start_time)

            # ---- State Tracking ----
            on_court = {int(t): set(starters[int(t)]) for t in teams}
            last_action_time = defaultdict(lambda: 0.0)
            player_last_seen = {}  # Track when each player was last active
            seconds_traditional = defaultdict(float)  # Minutes tracking

            # NEW: Enhanced tracking for flagging
            substitution_history = []  # Track all substitution attempts
            lineup_size_history = []   # Track lineup size changes
            player_status_tracking = {  # Track detailed player states
                tid: {
                    'current_lineup': set(starters[int(tid)]),
                    'last_sub_in': {},    # player_id -> timestamp
                    'last_sub_out': {},   # player_id -> timestamp
                    'action_without_sub': set()  # players who had actions but no sub-in
                } for tid in teams
            }

            prev_abs_time = 0.0
            prev_period = None

            # Results tracking
            state_rows = []
            flag_rows = []

            def snapshot_lineups(ev_time: float, period: int, pbp_order: int, desc: str, event_type: str = "NORMAL"):
                """Snapshot current lineups with enhanced metadata"""
                for tid in teams:
                    lineup = list(on_court[int(tid)])
                    lineup_names = [name_map.get(p, str(p)) for p in lineup]
                    lineup_size = len(lineup)

                    # NEW: Flag lineup size deviations
                    if lineup_size != 5:
                        flag_lineup_size_deviation(ev_time, period, pbp_order, int(tid), lineup_size, desc)

                    state_rows.append({
                        "period": period,
                        "pbp_order": pbp_order,
                        "abs_time": round(ev_time, 3),
                        "team_id": int(tid),
                        "team_abbrev": team_abbrev[int(tid)],
                        "lineup_size": lineup_size,
                        "lineup_player_ids_json": json.dumps(sorted([int(p) for p in lineup])),
                        "lineup_player_names_json": json.dumps(sorted(lineup_names)),
                        "event_desc": desc,
                        "event_type": event_type
                    })

            def flag_lineup_size_deviation(ev_time: float, period: int, pbp_order: int, team_id: int, 
                                         actual_size: int, desc: str):
                """NEW: Flag and analyze lineup size deviations"""
                team_abbr = team_abbrev[team_id]

                # Analyze why lineup isn't size 5
                reasons = []
                team_status = player_status_tracking[team_id]

                if actual_size < 5:
                    missing_count = 5 - actual_size
                    reasons.append(f"Missing {missing_count} player(s) for full lineup")

                    # Check if any players had recent actions but aren't in lineup
                    if team_status['action_without_sub']:
                        reasons.append(f"Players with actions but no sub-in: {[name_map.get(p, str(p)) for p in team_status['action_without_sub']]}")

                elif actual_size > 5:
                    excess_count = actual_size - 5
                    reasons.append(f"Excess {excess_count} player(s) beyond normal lineup")

                # Check recent substitution activity
                recent_subs = [sub for sub in substitution_history[-10:] if sub['team_id'] == team_id]
                if recent_subs:
                    last_sub = recent_subs[-1]
                    reasons.append(f"Last substitution: {last_sub['description']}")

                flag_rows.append({
                    "period": period,
                    "pbp_order": pbp_order,
                    "abs_time": round(ev_time, 3),
                    "team_id": team_id,
                    "team_abbrev": team_abbr,
                    "flag_type": "lineup_size_deviation",
                    "player_id": None,
                    "player_name": None,
                    "idle_seconds": None,
                    "description": desc,
                    "flag_details": f"Lineup size {actual_size}/5. " + "; ".join(reasons),
                    "resolved_via_last_name": False,
                    "sub_direction_inverted": False,
                    "lineup_json": json.dumps(sorted([int(p) for p in on_court[team_id]]))
                })

            def flag_substitution_issue(ev_time: float, period: int, pbp_order: int, issue_type: str, 
                                      team_id: int, player_id: int = None, details: str = ""):
                """NEW: Flag various substitution issues"""
                flag_rows.append({
                    "period": period,
                    "pbp_order": pbp_order,
                    "abs_time": round(ev_time, 3),
                    "team_id": team_id,
                    "team_abbrev": team_abbrev[team_id],
                    "flag_type": issue_type,
                    "player_id": int(player_id) if player_id else None,
                    "player_name": name_map.get(player_id, str(player_id)) if player_id else None,
                    "idle_seconds": None,
                    "description": details,
                    "flag_details": details,
                    "resolved_via_last_name": False,
                    "sub_direction_inverted": False,
                    "lineup_json": json.dumps(sorted([int(p) for p in on_court[team_id]]))
                })

            def validate_substitution(in_pid: int, out_pid: int, sub_tid: int, desc: str, 
                                    ev_time: float, period: int, pbp_order: int) -> dict:
                """NEW: Comprehensive substitution validation"""
                validation_result = {
                    "valid": True,
                    "issues": [],
                    "can_proceed": True
                }

                team_status = player_status_tracking[sub_tid]
                current_lineup = team_status['current_lineup']

                # Check OUT player
                if out_pid:
                    if out_pid not in current_lineup:
                        validation_result["issues"].append(f"OUT player {name_map.get(out_pid)} not in current lineup")
                        flag_substitution_issue(ev_time, period, pbp_order, "sub_out_player_not_in_lineup", 
                                              sub_tid, out_pid, f"Attempted to sub out {name_map.get(out_pid)} who is not in lineup")
                        validation_result["valid"] = False
                    else:
                        # Check when player was last in
                        last_in = team_status['last_sub_in'].get(out_pid, "Game start")
                        validation_result["issues"].append(f"OUT: {name_map.get(out_pid)} (last in: {last_in})")

                # Check IN player  
                if in_pid:
                    if in_pid in current_lineup:
                        validation_result["issues"].append(f"IN player {name_map.get(in_pid)} already in lineup")
                        flag_substitution_issue(ev_time, period, pbp_order, "sub_in_player_already_in_lineup", 
                                              sub_tid, in_pid, f"Attempted to sub in {name_map.get(in_pid)} who is already in lineup")
                        validation_result["valid"] = False
                    else:
                        # Check when player was last out
                        last_out = team_status['last_sub_out'].get(in_pid, "Never subbed out")
                        validation_result["issues"].append(f"IN: {name_map.get(in_pid)} (last out: {last_out})")

                return validation_result

            # ---- Main Processing Loop ----
            logger.info(f"Processing {len(events)} events with TRADITIONAL DATA-DRIVEN approach...")

            for _, ev in events.iterrows():
                period = int(ev.period)
                rem = _parse_gc(ev.game_clock)
                cur_t = _abs_t(period, rem)

                # Credit time between events to current lineups
                if cur_t > prev_abs_time:
                    delta = cur_t - prev_abs_time
                    for tid in teams:
                        for pid in on_court[int(tid)]:
                            seconds_traditional[pid] += delta

                # Handle period transitions
                if period != prev_period:
                    if period in CFG["starter_reset_periods"]:
                        # Reset to starters
                        for tid in teams:
                            on_court[int(tid)] = set(starters[int(tid)])
                            player_status_tracking[tid]['current_lineup'] = set(starters[int(tid)])

                        snapshot_lineups(cur_t, period, int(ev.pbp_order), f"Period {period} start - reset to starters", "PERIOD_START")

                    prev_period = period

                # TRADITIONAL SUBSTITUTION PROCESSING - STRICT DATA ADHERENCE
                if int(ev.msg_type) == CFG["sub_msg_type"]:
                    # Extract players - STRICT: playerId1=IN, playerId2=OUT
                    in_pid = int(ev.player_id_1) if pd.notna(ev.player_id_1) else None
                    out_pid = int(ev.player_id_2) if pd.notna(ev.player_id_2) else None

                    # Determine team
                    sub_tid = None
                    if in_pid and (in_pid in pteam_map):
                        sub_tid = int(pteam_map[in_pid])
                    elif out_pid and (out_pid in pteam_map):
                        sub_tid = int(pteam_map[out_pid])
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        sub_tid = int(ev.team_id_off)

                    if sub_tid is not None:
                        # NEW: Validate substitution before applying
                        validation = validate_substitution(in_pid, out_pid, sub_tid, str(ev.description), 
                                                         cur_t, period, int(ev.pbp_order))

                        # Record substitution attempt
                        sub_record = {
                            "time": cur_t,
                            "period": period,
                            "team_id": sub_tid,
                            "in_player": in_pid,
                            "out_player": out_pid,
                            "description": str(ev.description),
                            "validation": validation
                        }
                        substitution_history.append(sub_record)

                        # Apply substitution (even if flagged - we follow the data)
                        team_status = player_status_tracking[sub_tid]

                        if out_pid and out_pid in on_court[sub_tid]:
                            on_court[sub_tid].remove(out_pid)
                            team_status['current_lineup'].remove(out_pid)
                            team_status['last_sub_out'][out_pid] = cur_t
                            logger.info(f"[TRADITIONAL SUB-OUT] {name_map.get(out_pid)} from {team_abbrev[sub_tid]}")

                        if in_pid:
                            on_court[sub_tid].add(in_pid)
                            team_status['current_lineup'].add(in_pid)
                            team_status['last_sub_in'][in_pid] = cur_t
                            # Remove from action_without_sub if present
                            team_status['action_without_sub'].discard(in_pid)
                            logger.info(f"[TRADITIONAL SUB-IN] {name_map.get(in_pid)} to {team_abbrev[sub_tid]}")

                        # Snapshot after substitution
                        snapshot_lineups(cur_t, period, int(ev.pbp_order), str(ev.description), "SUBSTITUTION")

                # Check for actions by players not in lineup
                elif int(ev.msg_type) in CFG["action_msg_types"]:
                    action_pid = int(ev.player_id_1) if pd.notna(ev.player_id_1) else None
                    action_tid = None

                    if action_pid and (action_pid in pteam_map):
                        action_tid = int(pteam_map[action_pid])
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        action_tid = int(ev.team_id_off)

                    if action_tid in teams and action_pid is not None:
                        # Update last action time
                        last_action_time[action_pid] = cur_t
                        player_last_seen[action_pid] = cur_t

                        # Check if player is in lineup
                        if action_pid not in on_court[action_tid]:
                            # Flag action by player not in lineup
                            flag_substitution_issue(cur_t, period, int(ev.pbp_order), "action_by_non_lineup_player", 
                                                  action_tid, action_pid, 
                                                  f"Player {name_map.get(action_pid)} had action but not in lineup: {ev.description}")

                            # Track for analysis
                            player_status_tracking[action_tid]['action_without_sub'].add(action_pid)

                    # Regular lineup snapshot for non-substitution events
                    snapshot_lineups(cur_t, period, int(ev.pbp_order), str(ev.description), "ACTION")

                else:
                    # Other events - just snapshot
                    snapshot_lineups(cur_t, period, int(ev.pbp_order), str(ev.description), "OTHER")

                prev_abs_time = cur_t

            # ---- Build Traditional Minutes ----
            traditional_minutes_rows = []
            for pid, secs in seconds_traditional.items():
                if pid in pteam_map:
                    traditional_minutes_rows.append({
                        "player_id": int(pid),
                        "player_name": name_map.get(pid, str(pid)),
                        "team_id": int(pteam_map[pid]),
                        "team_abbrev": team_abbrev[int(pteam_map[pid])],
                        "seconds_traditional": round(float(secs), 3)
                    })

            traditional_minutes_df = pd.DataFrame(traditional_minutes_rows).sort_values(["team_abbrev", "player_name"]).reset_index(drop=True)

            # ---- Persist to DuckDB ----
            # Replace the basic tables with traditional data-driven versions
            self._robust_drop_object("traditional_lineup_state")
            self.conn.register("traditional_lineup_state_temp", pd.DataFrame(state_rows))
            self.conn.execute("CREATE TABLE traditional_lineup_state AS SELECT * FROM traditional_lineup_state_temp")
            self.conn.execute("DROP VIEW IF EXISTS traditional_lineup_state_temp")

            self._robust_drop_object("traditional_lineup_flags")
            self.conn.register("traditional_lineup_flags_temp", pd.DataFrame(flag_rows))
            self.conn.execute("CREATE TABLE traditional_lineup_flags AS SELECT * FROM traditional_lineup_flags_temp")
            self.conn.execute("DROP VIEW IF EXISTS traditional_lineup_flags_temp")

            self._robust_drop_object("minutes_traditional")
            self.conn.register("minutes_traditional_temp", traditional_minutes_df)
            self.conn.execute("CREATE TABLE minutes_traditional AS SELECT * FROM minutes_traditional_temp")
            self.conn.execute("DROP VIEW IF EXISTS minutes_traditional_temp")

            # ---- Summary Statistics ----
            flag_summary = {}
            if flag_rows:
                flag_df = pd.DataFrame(flag_rows)
                flag_summary = dict(flag_df["flag_type"].value_counts())

            lineup_size_analysis = {}
            if state_rows:
                state_df = pd.DataFrame(state_rows)
                size_counts = state_df["lineup_size"].value_counts().to_dict()
                lineup_size_analysis = {
                    "total_states": len(state_rows),
                    "size_distribution": {str(k): int(v) for k, v in size_counts.items()},
                    "non_5_player_states": int(sum(v for k, v in size_counts.items() if k != 5)),
                    "percentage_correct_size": round(size_counts.get(5, 0) / len(state_rows) * 100, 1) if state_rows else 0
                }

            substitution_analysis = {
                "total_substitutions": len(substitution_history),
                "valid_substitutions": len([s for s in substitution_history if s["validation"]["valid"]]),
                "flagged_substitutions": len([s for s in substitution_history if not s["validation"]["valid"]])
            }

            self.data_summary["traditional_data_driven"] = {
                "state_rows": len(state_rows),
                "flag_rows": len(flag_rows),
                "minutes_rows": len(traditional_minutes_df),
                "flag_summary": flag_summary,
                "lineup_size_analysis": lineup_size_analysis,
                "substitution_analysis": substitution_analysis,
                "substitution_history": substitution_history[-20:]  # Last 20 for debugging
            }

            logger.info(f"[TRADITIONAL DATA-DRIVEN] {len(substitution_history)} total substitutions")
            logger.info(f"[TRADITIONAL DATA-DRIVEN] {len(flag_rows)} flags generated")
            logger.info(f"[TRADITIONAL DATA-DRIVEN] Lineup size distribution: {lineup_size_analysis['size_distribution']}")

            return ValidationResult(
                step_name="Traditional Data-Driven Lineups",
                passed=True,
                details=(f"Traditional data-driven tracking: {len(state_rows)} states, "
                        f"{len(flag_rows)} flags, {lineup_size_analysis['percentage_correct_size']}% correct lineup size"),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Traditional Data-Driven Lineups",
                passed=False,
                details=f"Error in traditional data-driven tracking: {e}",
                processing_time=time.time() - start_time
            )

    def compare_traditional_vs_enhanced_lineups(self) -> ValidationResult:
        """
        UPDATED COMPREHENSIVE COMPARISON: Traditional Data-Driven vs Enhanced Methods

        Key Changes:
        1. Uses traditional_lineup_state instead of basic_lineup_state
        2. Enhanced analysis of lineup size variations
        3. Detailed flagging comparison between methods
        """
        start_time = time.time()

        try:
            # Check if both methods have been run
            traditional_data = self.data_summary.get('traditional_data_driven', {})
            enhanced_data = self.data_summary.get('enhanced_substitution_tracking', {})

            if not traditional_data or not enhanced_data:
                return ValidationResult(
                    step_name="Compare Traditional vs Enhanced (Updated)",
                    passed=False,
                    details="Both traditional data-driven and enhanced methods must be run first",
                    processing_time=time.time() - start_time
                )

            # Get validation data from both methods
            traditional_validation = self.conn.execute("SELECT * FROM minutes_traditional").df()
            enhanced_validation = enhanced_data.get('validation_data', pd.DataFrame())
            box_validation = self.conn.execute("SELECT player_id, player_name, team_id, team_abbrev, seconds_played FROM box_score WHERE seconds_played > 0").df()

            # Merge all data for comparison
            comparison_df = box_validation.merge(
                traditional_validation[['player_id', 'seconds_traditional']], 
                on='player_id', 
                how='left'
            ).merge(
                enhanced_validation[['player_id', 'calc_seconds']].rename(columns={'calc_seconds': 'seconds_enhanced'}),
                on='player_id',
                how='left'
            )

            # Fill NaN values
            comparison_df['seconds_traditional'] = comparison_df['seconds_traditional'].fillna(0.0)
            comparison_df['seconds_enhanced'] = comparison_df['seconds_enhanced'].fillna(0.0)

            # Calculate differences vs box score
            comparison_df['traditional_vs_box_diff'] = comparison_df['seconds_traditional'] - comparison_df['seconds_played']
            comparison_df['enhanced_vs_box_diff'] = comparison_df['seconds_enhanced'] - comparison_df['seconds_played']
            comparison_df['traditional_vs_box_abs_diff'] = comparison_df['traditional_vs_box_diff'].abs()
            comparison_df['enhanced_vs_box_abs_diff'] = comparison_df['enhanced_vs_box_diff'].abs()

            # Calculate percentage differences
            def safe_pct_diff(calc, box):
                return (calc - box) / box if box > 0 else 0.0

            comparison_df['traditional_vs_box_pct_diff'] = comparison_df.apply(
                lambda row: safe_pct_diff(row['seconds_traditional'], row['seconds_played']), axis=1
            )
            comparison_df['enhanced_vs_box_pct_diff'] = comparison_df.apply(
                lambda row: safe_pct_diff(row['seconds_enhanced'], row['seconds_played']), axis=1
            )

            # Determine which method is better for each player
            comparison_df['method_improvement'] = comparison_df['traditional_vs_box_abs_diff'] - comparison_df['enhanced_vs_box_abs_diff']
            comparison_df['better_method'] = comparison_df['method_improvement'].apply(
                lambda x: 'Enhanced' if x > 0 else 'Traditional' if x < 0 else 'Tie'
            )

            # Calculate summary statistics
            tolerance_seconds = 120

            traditional_offenders = len(comparison_df[comparison_df['traditional_vs_box_abs_diff'] > tolerance_seconds])
            enhanced_offenders = len(comparison_df[comparison_df['enhanced_vs_box_abs_diff'] > tolerance_seconds])

            improved_players = len(comparison_df[comparison_df['method_improvement'] > 0])
            worsened_players = len(comparison_df[comparison_df['method_improvement'] < 0])
            tied_players = len(comparison_df[comparison_df['method_improvement'] == 0])

            # UPDATED: Get flag statistics for traditional method
            traditional_flags = traditional_data.get('flag_summary', {})
            enhanced_flags = enhanced_data.get('flags', {})

            flag_comparison = {
                'traditional_total_flags': sum(traditional_flags.values()),
                'enhanced_total_flags': sum(len(flag_list) for flag_list in enhanced_flags.values()),
                'traditional_flag_types': traditional_flags,
                'enhanced_flag_types': {k: len(v) for k, v in enhanced_flags.items()}
            }

            # UPDATED: Get lineup size analysis
            traditional_lineup_analysis = traditional_data.get('lineup_size_analysis', {})
            enhanced_lineup_analysis = self._get_enhanced_lineup_analysis()

            lineup_comparison = {
                'traditional': {
                    'total_states': traditional_lineup_analysis.get('total_states', 0),
                    'size_distribution': traditional_lineup_analysis.get('size_distribution', {}),
                    'correct_size_percentage': traditional_lineup_analysis.get('percentage_correct_size', 0),
                    'non_5_player_states': traditional_lineup_analysis.get('non_5_player_states', 0)
                },
                'enhanced': {
                    'total_states': enhanced_lineup_analysis.get('total_states', 0),
                    'size_distribution': enhanced_lineup_analysis.get('size_distribution', {}),
                    'correct_size_percentage': enhanced_lineup_analysis.get('percentage_correct_size', 0),
                    'non_5_player_states': enhanced_lineup_analysis.get('non_5_player_states', 0)
                }
            }

            # Create comprehensive comparison summary
            comparison_summary = {
                'method_comparison': {
                    'traditional_offenders': int(traditional_offenders),
                    'enhanced_offenders': int(enhanced_offenders),
                    'improvement': int(traditional_offenders - enhanced_offenders),
                    'players_improved': int(improved_players),
                    'players_worsened': int(worsened_players),
                    'players_tied': int(tied_players),
                    'total_players': int(len(comparison_df))
                },
                'accuracy_metrics': {
                    'traditional_avg_abs_diff': float(comparison_df['traditional_vs_box_abs_diff'].mean()),
                    'enhanced_avg_abs_diff': float(comparison_df['enhanced_vs_box_abs_diff'].mean()),
                    'traditional_max_diff': float(comparison_df['traditional_vs_box_abs_diff'].max()),
                    'enhanced_max_diff': float(comparison_df['enhanced_vs_box_abs_diff'].max()),
                    'traditional_within_10pct': int(len(comparison_df[comparison_df['traditional_vs_box_pct_diff'].abs() <= 0.10])),
                    'enhanced_within_10pct': int(len(comparison_df[comparison_df['enhanced_vs_box_pct_diff'].abs() <= 0.10]))
                },
                'flag_analysis': flag_comparison,
                'lineup_analysis': lineup_comparison
            }

            # Store results
            self.data_summary['traditional_vs_enhanced_comparison_updated'] = {
                'comparison_data': comparison_df,
                'summary': comparison_summary,
                'processing_time': time.time() - start_time
            }

            # Create database tables
            self._robust_drop_object("traditional_vs_enhanced_comparison_updated")
            self.conn.register('comparison_updated_temp', comparison_df)
            self.conn.execute("CREATE TABLE traditional_vs_enhanced_comparison_updated AS SELECT * FROM comparison_updated_temp")
            self.conn.execute("DROP VIEW IF EXISTS comparison_updated_temp")

            # Log results
            logger.info(f"[UPDATED COMPARISON] Traditional Data-Driven: {traditional_offenders} offenders, Enhanced: {enhanced_offenders} offenders")
            logger.info(f"[UPDATED COMPARISON] Improvement: {traditional_offenders - enhanced_offenders} fewer offenders")
            logger.info(f"[UPDATED COMPARISON] Traditional flags: {sum(traditional_flags.values())}, Enhanced flags: {sum(len(flag_list) for flag_list in enhanced_flags.values())}")
            logger.info(f"[UPDATED COMPARISON] Traditional lineup size distribution: {traditional_lineup_analysis.get('size_distribution', {})}")

            details = (f"Updated comparison complete: Traditional Data-Driven ({traditional_offenders} offenders) vs Enhanced ({enhanced_offenders} offenders). "
                      f"Improvement: {traditional_offenders - enhanced_offenders} fewer offenders. "
                      f"Traditional flagged {sum(traditional_flags.values())} issues, Enhanced flagged {sum(len(flag_list) for flag_list in enhanced_flags.values())} issues.")

            return ValidationResult(
                step_name="Compare Traditional vs Enhanced (Updated)",
                passed=True,
                details=details,
                processing_time=time.time() - start_time
            )

        except Exception as e:
            logger.error(f"Error in updated traditional vs enhanced comparison: {e}")
            return ValidationResult(
                step_name="Compare Traditional vs Enhanced (Updated)",
                passed=False,
                details=f"Error in updated comparison: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _get_enhanced_lineup_analysis(self) -> Dict[str, Any]:
        """Helper method to get enhanced lineup size analysis"""
        try:
            enhanced_states = self.conn.execute("SELECT lineup_size FROM enhanced_lineup_state").df()
            if enhanced_states.empty:
                return {}

            size_counts = enhanced_states['lineup_size'].value_counts().to_dict()
            return {
                'total_states': len(enhanced_states),
                'size_distribution': {str(k): int(v) for k, v in size_counts.items()},
                'non_5_player_states': int(sum(v for k, v in size_counts.items() if k != 5)),
                'percentage_correct_size': round(size_counts.get(5, 0) / len(enhanced_states) * 100, 1)
            }
        except Exception:
            return {}

    def _create_comprehensive_flags_table(self, flags_data: Dict[str, List]) -> None:
        """Create comprehensive flags analysis table"""
        try:
            all_flags = []

            for flag_type, flag_list in flags_data.items():
                for flag in flag_list:
                    flag_record = {
                        'flag_type': flag_type,
                        'time': flag.get('time', 0),
                        'player_id': flag.get('player_id'),
                        'player_name': flag.get('player_name'),
                        'team': flag.get('team'),
                        'action_type': flag.get('action_type'),
                        'idle_seconds': flag.get('idle_seconds'),
                        'description': flag.get('description', ''),
                        'resolution': flag.get('resolution', ''),
                        'full_details': str(flag)
                    }
                    all_flags.append(flag_record)

            if all_flags:
                flags_df = pd.DataFrame(all_flags)
                self._robust_drop_object("comprehensive_flags_analysis")
                self.conn.register('flags_analysis_temp', flags_df)
                self.conn.execute("CREATE TABLE comprehensive_flags_analysis AS SELECT * FROM flags_analysis_temp")
                self.conn.execute("DROP VIEW IF EXISTS flags_analysis_temp")

                logger.info(f"Created comprehensive_flags_analysis table with {len(all_flags)} flag records")

        except Exception as e:
            logger.warning(f"Could not create comprehensive flags table: {e}")

    def compare_basic_vs_estimated_lineups(self) -> ValidationResult:
        """
        Compare minutes from: basic pass vs enhanced estimator vs box score.
        FIXED: Creates DuckDB table from available sources before comparison.
        """
        start_time = time.time()
        try:
            # FIXED: Check and create missing tables
            have_basic = self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='minutes_basic'").fetchone()[0]
            have_box = self.conn.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_name='box_score'").fetchone()[0]

            # Solution 1: Create minutes_basic from CSV if it exists
            if have_basic == 0:
                try:
                    # Check multiple possible locations for CSV
                    csv_candidates = [
                        "minutes_basic.csv",
                        "reports/minutes_basic.csv", 
                        "exports/minutes_basic.csv",
                        str(PROCESSED_DIR / "minutes_basic.csv") if 'PROCESSED_DIR' in globals() else None
                    ]

                    csv_found = False
                    for csv_path in csv_candidates:
                        if csv_path and Path(csv_path).exists():
                            self.conn.execute(f"CREATE TABLE minutes_basic AS SELECT * FROM read_csv_auto('{csv_path}')")
                            have_basic = 1
                            csv_found = True
                            logger.info(f"Created minutes_basic table from {csv_path}")
                            break

                    # Solution 2: Use alternative tables as fallback
                    if not csv_found:
                        alt_tables = ['minutes_traditional', 'minutes_enhanced']
                        for alt_table in alt_tables:
                            alt_exists = self.conn.execute(f"SELECT COUNT(*) FROM information_schema.tables WHERE table_name='{alt_table}'").fetchone()[0]
                            if alt_exists:
                                # Map the alternative table columns to minutes_basic format
                                if alt_table == 'minutes_traditional':
                                    self.conn.execute("""
                                        CREATE TABLE minutes_basic AS 
                                        SELECT 
                                            player_id, 
                                            player_name, 
                                            team_id, 
                                            team_abbrev,
                                            seconds_traditional as seconds_basic
                                        FROM minutes_traditional
                                    """)
                                elif alt_table == 'minutes_enhanced':
                                    self.conn.execute("""
                                        CREATE TABLE minutes_basic AS 
                                        SELECT 
                                            player_id, 
                                            player_name, 
                                            team_id, 
                                            team_abbrev,
                                            seconds_enhanced as seconds_basic
                                        FROM minutes_enhanced
                                    """)
                                have_basic = 1
                                logger.info(f"Created minutes_basic table from {alt_table}")
                                break

                    # Solution 3: Create synthetic minutes_basic from box_score if needed
                    if have_basic == 0 and have_box > 0:
                        self.conn.execute("""
                            CREATE TABLE minutes_basic AS 
                            SELECT 
                                nbaId as player_id, 
                                name as player_name, 
                                nbaTeamId as team_id, 
                                team as team_abbrev,
                                secPlayed as seconds_basic
                            FROM box_score 
                            WHERE secPlayed > 0
                        """)
                        have_basic = 1
                        logger.info("Created synthetic minutes_basic from box_score")

                except Exception as e:
                    logger.warning(f"Could not create minutes_basic table: {e}")

            if have_basic == 0 or have_box == 0:
                missing_items = []
                if have_basic == 0:
                    missing_items.append("minutes_basic")
                if have_box == 0:
                    missing_items.append("box_score")
                return ValidationResult("Compare Minutes", False, f"Missing required tables: {missing_items}")

            # Enhanced minutes are in self.data_summary['minutes_validation_full'] if run_lineups_and_rim_analytics() already executed.
            # But we want the compare to be callable before/after. We'll use DuckDB if present; otherwise fall back to data_summary.
            enhanced_df = self.data_summary.get("minutes_validation_full")
            if enhanced_df is None:
                # Create a minimal enhanced view if needed to keep pipeline flowing
                enhanced_df = pd.DataFrame(columns=[
                    "player_id","player_name","team","calc_seconds","box_seconds","abs_diff_seconds","segments_count"
                ])

            minutes_basic = self.conn.execute("""
                SELECT b.player_id, b.player_name, b.team_id, b.team_abbrev, b.seconds_basic
                FROM minutes_basic b
            """).df()

            box = self.conn.execute("""
                SELECT player_id, player_name, team_id, team_abbrev, seconds_played
                FROM box_score
            """).df()

            # merge frames
            cmp_df = minutes_basic.merge(
                box.rename(columns={"seconds_played":"box_seconds"}),
                on=["player_id","player_name","team_id","team_abbrev"],
                how="outer"
            )

            # attach enhanced if available
            if not enhanced_df.empty:
                e_small = enhanced_df[["player_id","calc_seconds"]].rename(columns={"calc_seconds":"enhanced_seconds"})
                cmp_df = cmp_df.merge(e_small, on="player_id", how="left")

            # fill NaNs with 0 where appropriate (for diffs only; we do not alter raw tables)
            for col in ["seconds_basic","box_seconds","enhanced_seconds"]:
                if col in cmp_df.columns:
                    cmp_df[col] = cmp_df[col].fillna(0.0)

            # percentage diffs vs box
            def pct_diff(a, b):
                return None if b == 0 else (a - b) / b

            cmp_df["basic_vs_box_sec_diff"] = cmp_df["seconds_basic"] - cmp_df["box_seconds"]
            cmp_df["basic_vs_box_pct_diff"] = cmp_df.apply(lambda r: pct_diff(r["seconds_basic"], r["box_seconds"]), axis=1)

            if "enhanced_seconds" in cmp_df.columns:
                cmp_df["enhanced_vs_box_sec_diff"] = cmp_df["enhanced_seconds"] - cmp_df["box_seconds"]
                cmp_df["enhanced_vs_box_pct_diff"] = cmp_df.apply(lambda r: pct_diff(r.get("enhanced_seconds",0.0), r["box_seconds"]), axis=1)
            else:
                cmp_df["enhanced_vs_box_sec_diff"] = 0.0
                cmp_df["enhanced_vs_box_pct_diff"] = None

            cmp_df = cmp_df.sort_values(["team_abbrev","player_name"]).reset_index(drop=True)

            # persist
            self._robust_drop_object("minutes_compare")
            self.conn.register("minutes_compare_temp", cmp_df)
            self.conn.execute("CREATE TABLE minutes_compare AS SELECT * FROM minutes_compare_temp")
            self.conn.execute("DROP VIEW IF EXISTS minutes_compare_temp")

            # summary: how many within 10% of box?
            within10_basic = int((cmp_df["basic_vs_box_pct_diff"].abs() <= 0.10).sum())
            total_w_box    = int((cmp_df["box_seconds"] > 0).sum())

            self.data_summary["minutes_compare"] = {
                "rows": len(cmp_df),
                "within10_basic": within10_basic,
                "total_with_box": total_w_box
            }

            return ValidationResult(
                step_name="Compare Minutes",
                passed=True,
                details=f"minutes_compare built ({len(cmp_df)} rows). Basic within 10%: {within10_basic}/{total_w_box}.",
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Compare Minutes",
                passed=False,
                details=f"Error building minutes_compare: {e}",
                processing_time=time.time() - start_time
            )

    def validate_dataset_compliance(self) -> ValidationResult:
        """
        Validate that generated datasets meet project requirements.
        Critical validation to ensure deliverables are usable.
        
        FIXED: Corrected SQL column references to match actual schema
        - Changed 'secPlayed' to 'seconds_played' to match box_score table schema
        - Added better error handling for missing tables
        - Enhanced validation reporting with specific compliance metrics
        """
        start_time = time.time()
        try:
            validation_issues = []
            
            # Project 1: Validate 5-man lineup requirement
            lineup_table_exists = self.conn.execute(
                "SELECT COUNT(*) FROM information_schema.tables WHERE table_name='final_dual_lineups'"
            ).fetchone()[0]
            
            if lineup_table_exists:
                lineup_compliance = self.conn.execute("""
                    SELECT 
                        method,
                        COUNT(*) as total_lineups,
                        SUM(CASE WHEN lineup_size = 5 THEN 1 ELSE 0 END) as five_man_lineups,
                        (SUM(CASE WHEN lineup_size = 5 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as compliance_pct
                    FROM final_dual_lineups
                    GROUP BY method
                """).df()
                
                for _, row in lineup_compliance.iterrows():
                    method = row['method']
                    compliance = row['compliance_pct']
                    if compliance < 100.0:
                        validation_issues.append(
                            f"{method.title()} method: {compliance:.1f}% compliance with 5-man requirement (FAILS)"
                        )
            else:
                validation_issues.append("final_dual_lineups table not found - cannot validate lineup compliance")
                        
            # Project 2: Validate rim defense coverage
            players_table_exists = self.conn.execute(
                "SELECT COUNT(*) FROM information_schema.tables WHERE table_name='final_dual_players'"
            ).fetchone()[0]
            
            if players_table_exists:
                rim_coverage = self.conn.execute("""
                    SELECT 
                        method,
                        COUNT(*) as total_players,
                        SUM(CASE WHEN opp_rim_attempts_on > 0 OR opp_rim_attempts_off > 0 THEN 1 ELSE 0 END) as players_with_rim_data,
                        (SUM(CASE WHEN opp_rim_attempts_on > 0 OR opp_rim_attempts_off > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as coverage_pct
                    FROM final_dual_players
                    GROUP BY method
                """).df()
                
                # FIXED: Use correct column name 'seconds_played' instead of 'secPlayed'
                box_table_exists = self.conn.execute(
                    "SELECT COUNT(*) FROM information_schema.tables WHERE table_name='box_score'"
                ).fetchone()[0]
                
                if box_table_exists:
                    expected_players = self.conn.execute(
                        "SELECT COUNT(*) FROM box_score WHERE seconds_played > 0"
                    ).fetchone()[0]
                    
                    for _, row in rim_coverage.iterrows():
                        method = row['method']
                        coverage = row['coverage_pct']
                        players_covered = int(row['players_with_rim_data'])
                        missing_players = expected_players - players_covered
                        
                        if missing_players > 0:
                            validation_issues.append(
                                f"{method.title()} method: Missing rim data for {missing_players}/{expected_players} players"
                            )
                else:
                    validation_issues.append("box_score table not found - cannot validate expected player count")
            else:
                validation_issues.append("final_dual_players table not found - cannot validate rim coverage")
            
            # Minutes validation tolerance check
            enhanced_validation = self.data_summary.get('minutes_validation_full')
            if enhanced_validation is not None:
                tolerance_violations = len(enhanced_validation[enhanced_validation['abs_diff_seconds'] > 120])
                if tolerance_violations > 0:
                    validation_issues.append(
                        f"Minutes validation: {tolerance_violations} players exceed 120s tolerance"
                    )
            
            passed = len(validation_issues) == 0
            details = f"Dataset compliance check: {len(validation_issues)} issues found"
            
            # Add success details if validation passed
            if passed:
                details += " - All compliance requirements met"
            
            return ValidationResult(
                step_name="Dataset Compliance Validation",
                passed=passed,
                details=details,
                processing_time=time.time() - start_time,
                warnings=validation_issues
            )
            
        except Exception as e:
            return ValidationResult(
                step_name="Dataset Compliance Validation", 
                passed=False,
                details=f"Error validating dataset compliance: {str(e)}",
                processing_time=time.time() - start_time
            )

    def create_project_submission_artifacts(self) -> ValidationResult:
        """
        Create final artifacts specifically for project submission using only compliant methods.
        """
        start_time = time.time()
        try:
            # Use enhanced method only for final submission due to 100% 5-man compliance
            logger.info("Creating project submission artifacts using enhanced method...")

            # Project 1: Final lineup submission
            enhanced_lineups = self.conn.execute(f"""
                SELECT 
                    team_abbrev as "Team",
                    player_1_name as "Player 1",
                    player_2_name as "Player 2", 
                    player_3_name as "Player 3",
                    player_4_name as "Player 4",
                    player_5_name as "Player 5",
                    off_possessions as "Offensive possessions played",
                    def_possessions as "Defensive possessions played",
                    off_rating as "Offensive rating", 
                    def_rating as "Defensive rating",
                    net_rating as "Net rating"
                FROM final_dual_lineups
                WHERE method = 'enhanced'
                AND lineup_size = 5  -- Ensure 5-man compliance
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, off_possessions DESC
            """).df()

            # Project 2: Final player submission
            enhanced_players = self.conn.execute(f"""
                SELECT 
                    player_id as "Player ID",
                    player_name as "Player Name", 
                    team_abbrev as "Team",
                    off_possessions as "Offensive possessions played",
                    def_possessions as "Defensive possessions played",
                    ROUND(COALESCE(opp_rim_fg_pct_on, 0), 4) as "Opponent rim field goal percentage when player is on the court",
                    ROUND(COALESCE(opp_rim_fg_pct_off, 0), 4) as "Opponent rim field goal percentage when player is off the court", 
                    ROUND(COALESCE(rim_defense_on_off, 0), 4) as "Opponent rim field goal percentage on/off difference (on-off)"
                FROM final_dual_players
                WHERE method = 'enhanced'
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, player_name
            """).df()

            # Export final submission files
            submission_dir = self.export_dir / "final_submission"
            submission_dir.mkdir(exist_ok=True)

            enhanced_lineups.to_csv(submission_dir / "project1_lineups_FINAL.csv", index=False)
            enhanced_players.to_csv(submission_dir / "project2_players_FINAL.csv", index=False)

            # Create submission validation report
            validation_report = {
                "project1_lineups": {
                    "total_lineups": len(enhanced_lineups),
                    "teams_covered": enhanced_lineups['Team'].nunique(),
                    "five_man_compliance": "100%",
                    "file_size_kb": round((submission_dir / "project1_lineups_FINAL.csv").stat().st_size / 1024, 1)
                },
                "project2_players": {
                    "total_players": len(enhanced_players), 
                    "teams_covered": enhanced_players['Team'].nunique(),
                    "rim_data_coverage": f"{len(enhanced_players[enhanced_players['Opponent rim field goal percentage when player is on the court'] > 0])}/{len(enhanced_players)} players",
                    "file_size_kb": round((submission_dir / "project2_players_FINAL.csv").stat().st_size / 1024, 1)
                }
            }

            with open(submission_dir / "submission_validation_report.json", 'w') as f:
                import json
                json.dump(validation_report, f, indent=2)

            details = f"Created final submission artifacts: {len(enhanced_lineups)} lineups, {len(enhanced_players)} players"

            return ValidationResult(
                step_name="Create Submission Artifacts",
                passed=True,
                details=details,
                data_count=len(enhanced_lineups) + len(enhanced_players),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Create Submission Artifacts",
                passed=False, 
                details=f"Error creating submission artifacts: {str(e)}",
                processing_time=time.time() - start_time
            )

    def run_enhanced_substitution_tracking_with_flags(self) -> ValidationResult:
        """
        ENHANCED SUBSTITUTION TRACKING WITH COMPREHENSIVE FLAGGING

        This method provides the enhanced substitution logic with detailed flagging:
        - First-action rules (Reed Sheppard case)
        - Auto-out for inactivity periods
        - Comprehensive flagging system
        - Lineup size enforcement

        Flags captured:
        - missing_sub_in: Players with actions but no substitution in
        - inactivity_periods: Players on court >2 minutes without action
        - first_action_events: Reed Sheppard case injections
        - auto_out_events: Automatic removals due to inactivity
        - lineup_violations: Any time lineup != 5 players
        """
        start_time = time.time()

        CFG = {
            "starter_reset_periods": [1, 3],
            "one_direction": {
                "appearance_via_last_name": True,
                "remove_out_if_present": True
            },
            "msg_types": {
                "shot_made": 1, "shot_missed": 2, "rebound": 4,
                "turnover": 5, "foul": 6, "substitution": 8
            },
            "minutes_validation": {"tolerance_seconds": 120},
            "inactivity_rule": {"idle_seconds_threshold": 120}
        }

        try:
            # Helper functions
            def _period_length_seconds(p: int) -> float:
                return 720.0 if p <= 4 else 300.0

            def _parse_game_clock(gc: str) -> float | None:
                if not gc or not isinstance(gc, str):
                    return None
                s = gc.strip()
                if s.count(":") != 1:
                    return None
                try:
                    mm, ss = s.split(":")
                    return float(mm) * 60.0 + float(ss)
                except (ValueError, IndexError):
                    return None

            def _abs_time(period: int, rem_sec: float | None) -> float:
                total = 0.0
                for pi in range(1, period):
                    total += _period_length_seconds(pi)
                pl = _period_length_seconds(period)
                if rem_sec is None:
                    return total + pl
                return total + (pl - rem_sec)

            # Load core data
            box_df = self.conn.execute("""
                SELECT player_id, player_name, team_id, team_abbrev, is_starter, seconds_played
                FROM box_score
                WHERE seconds_played > 0
                ORDER BY team_id, seconds_played DESC
            """).df()

            if box_df.empty:
                return ValidationResult(
                    step_name="Enhanced Substitution Tracking with Flags",
                    passed=False,
                    details="No players found in box_score with playing time",
                    processing_time=time.time() - start_time
                )

            teams = sorted(box_df['team_id'].unique().tolist())
            if len(teams) != 2:
                return ValidationResult(
                    step_name="Enhanced Substitution Tracking with Flags",
                    passed=False,
                    details=f"Expected exactly 2 teams, found {teams}",
                    processing_time=time.time() - start_time
                )

            # Build comprehensive player mappings
            roster = {int(tid): set() for tid in teams}
            starters = {int(tid): set() for tid in teams}
            name_map, pteam_map = {}, {}
            last_name_index = {}

            for _, r in box_df.iterrows():
                pid = int(r.player_id)
                tid = int(r.team_id)
                roster[tid].add(pid)
                if bool(r.is_starter):
                    starters[tid].add(pid)
                name_map[pid] = str(r.player_name)
                pteam_map[pid] = tid

                # Enhanced last name indexing for first-action resolution
                full_name = str(r.player_name).strip()
                last_name = full_name.split()[-1].lower()
                first_name = full_name.split()[0].lower() if len(full_name.split()) > 1 else ""

                last_name_index.setdefault(last_name, []).append(pid)
                if first_name:
                    last_name_index.setdefault(first_name, []).append(pid)

            team_abbrev_map = {int(tid): box_df[box_df.team_id == tid]['team_abbrev'].iloc[0] for tid in teams}

            # Load events with last names for resolution
            events = self.conn.execute("""
                SELECT 
                    period, pbp_order, wall_clock_int,
                    COALESCE(game_clock,'') AS game_clock,
                    COALESCE(description,'') AS description,
                    team_id_off, team_id_def, msg_type, action_type,
                    player_id_1, player_id_2, player_id_3,
                    NULLIF(last_name_1,'') AS last_name_1,
                    NULLIF(last_name_2,'') AS last_name_2,
                    NULLIF(last_name_3,'') AS last_name_3,
                    COALESCE(points, 0) AS points
                FROM pbp
                ORDER BY period, pbp_order, wall_clock_int
            """).df()

            if events.empty:
                return ValidationResult(
                    step_name="Enhanced Substitution Tracking with Flags",
                    passed=False,
                    details="No PBP events found",
                    processing_time=time.time() - start_time
                )

            # Initialize enhanced tracking
            on_court = {tid: set(starters[tid]) for tid in teams}
            last_action_time = defaultdict(lambda: 0.0)
            recent_out = {tid: deque(maxlen=10) for tid in teams}

            active_segments = {}
            completed_segments = defaultdict(list)

            # Enhanced tracking variables with FLAGS
            enhanced_stats = {
                'total_substitutions': 0,
                'successful_substitutions': 0,
                'first_action_injections': 0,
                'auto_outs_inactivity': 0,
                'lineup_size_corrections': 0,
                'flags': {
                    'missing_sub_ins': [],      # Players with actions but no sub-in
                    'inactivity_periods': [],   # Players inactive >2min while on court
                    'lineup_violations': [],    # Times when lineup != 5 players
                    'first_action_events': [],  # First-action injection events
                    'auto_out_events': []       # Auto-out events due to inactivity
                }
            }

            # Lineup state tracking (similar to basic method)
            state_rows = []
            flag_rows = []

            def snapshot_lineups(ev_time: float, period: int, pbp_order: int, desc: str):
                import json
                for tid in teams:
                    lineup = list(on_court[tid])
                    lineup_names = [name_map.get(p, str(p)) for p in lineup]
                    state_rows.append({
                        "period": period,
                        "pbp_order": pbp_order,
                        "abs_time": round(ev_time, 3),
                        "team_id": int(tid),
                        "team_abbrev": team_abbrev_map[tid],
                        "lineup_size": len(lineup),
                        "lineup_player_ids_json": json.dumps(sorted([int(p) for p in lineup])),
                        "lineup_player_names_json": json.dumps(sorted(lineup_names)),
                        "event_desc": desc
                    })

            # Initialize segments for starters
            for tid in teams:
                for pid in on_court[tid]:
                    active_segments[pid] = {'start': 0.0, 'reason': 'GAME_START'}
                    last_action_time[pid] = 0.0

            def enhanced_name_resolution(ln: str | None, tid_hint: int | None) -> int | None:
                """Enhanced name resolution with fuzzy matching"""
                if not ln:
                    return None

                ln_clean = str(ln).strip().lower()
                candidates = last_name_index.get(ln_clean, [])

                if not candidates:
                    # Try partial matching
                    for key, pids in last_name_index.items():
                        if ln_clean in key or key in ln_clean:
                            candidates.extend(pids)

                if not candidates:
                    return None

                # Prefer team hint if available
                if tid_hint is not None:
                    for cand in candidates:
                        if pteam_map.get(cand) == tid_hint:
                            return cand

                # Return first valid candidate
                for cand in candidates:
                    if pteam_map.get(cand) in teams:
                        return cand

                return None

            def end_player_segment(pid: int, end_time: float, reason: str) -> None:
                """End a player's active segment with validation"""
                if pid not in active_segments:
                    return

                start_info = active_segments[pid]
                start_time = start_info['start']

                if end_time <= start_time:
                    end_time = start_time + 1.0

                duration = end_time - start_time
                completed_segments[pid].append({
                    'start': start_time,
                    'end': end_time,
                    'duration': duration,
                    'reason': f"{start_info['reason']} -> {reason}"
                })

                del active_segments[pid]

            def start_player_segment(pid: int, start_time: float, reason: str) -> None:
                """Start a new segment with overlap prevention"""
                if pid in active_segments:
                    end_player_segment(pid, start_time, f"OVERLAP_{reason}")

                active_segments[pid] = {'start': start_time, 'reason': reason}

            def flag_inactivity_check(current_time: float, period: int, pbp_order: int) -> None:
                """Check for players inactive > 2 minutes and flag them"""
                for tid in teams:
                    for pid in on_court[tid]:
                        idle_time = current_time - last_action_time[pid]

                        if idle_time > CFG["inactivity_rule"]["idle_seconds_threshold"]:
                            # Flag this as a potential missing sub-out
                            enhanced_stats['flags']['inactivity_periods'].append({
                                'time': current_time,
                                'player_id': pid,
                                'player_name': name_map.get(pid),
                                'team': team_abbrev_map[tid],
                                'idle_seconds': idle_time,
                                'last_action_time': last_action_time[pid]
                            })
                            # Also add to flag_rows for CSV export
                            flag_rows.append({
                                "period": period,
                                "pbp_order": pbp_order,
                                "abs_time": round(current_time, 3),
                                "team_id": int(tid),
                                "team_abbrev": team_abbrev_map[tid],
                                "flag_type": "inactivity_periods",
                                "player_id": int(pid),
                                "player_name": name_map.get(pid, str(pid)),
                                "idle_seconds": round(idle_time, 3),
                                "description": f"Player inactive for {idle_time:.1f}s (threshold: {CFG['inactivity_rule']['idle_seconds_threshold']}s)"
                            })

            def pick_auto_out_candidate(tid: int, current_time: float, exclude: set[int] = set()) -> int | None:
                """Enhanced auto-out selection based on activity patterns"""
                if not on_court[tid]:
                    return None

                candidates = [p for p in on_court[tid] if p not in exclude]
                if not candidates:
                    return None

                def activity_score(pid: int) -> tuple:
                    idle_time = current_time - last_action_time[pid]
                    is_starter = pid in starters[tid]
                    recently_subbed = pid in recent_out[tid]

                    # Score components (higher = more likely to be removed)
                    idle_score = idle_time
                    starter_penalty = -100 if is_starter else 0
                    recent_sub_penalty = -50 if recently_subbed else 0

                    total_score = idle_score + starter_penalty + recent_sub_penalty

                    return (total_score, idle_time, pid)

                candidates.sort(key=activity_score, reverse=True)
                best_candidate = candidates[0]
                idle_time = current_time - last_action_time[best_candidate]

                if idle_time >= CFG["inactivity_rule"]["idle_seconds_threshold"] or len(on_court[tid]) > 5:
                    return best_candidate

                return None

            def ensure_valid_lineup(tid: int, current_time: float, prefer_keep: set[int] = set()) -> None:
                """Ensure team has exactly 5 players with enhanced logging"""
                team_name = team_abbrev_map[tid]
                changes_made = False

                # Remove excess players
                while len(on_court[tid]) > 5:
                    auto_out = pick_auto_out_candidate(tid, current_time, exclude=prefer_keep)
                    if auto_out is None:
                        logger.error(f"Cannot auto-remove from {team_name} - no valid candidates")
                        break

                    on_court[tid].remove(auto_out)
                    recent_out[tid].append(auto_out)
                    changes_made = True

                    idle_time = current_time - last_action_time[auto_out]
                    enhanced_stats['auto_outs_inactivity'] += 1

                    # Flag this auto-out event
                    enhanced_stats['flags']['auto_out_events'].append({
                        'time': current_time,
                        'player_id': auto_out,
                        'player_name': name_map.get(auto_out),
                        'team': team_name,
                        'idle_time': idle_time,
                        'reason': 'INACTIVITY_AUTO_OUT'
                    })

                    logger.info(f"[ENHANCED AUTO-OUT] {name_map.get(auto_out)} from {team_name} (idle: {idle_time:.1f}s)")

                # Add players if under 5
                if len(on_court[tid]) < 5:
                    available = [p for p in roster[tid] if p not in on_court[tid]]
                    if available:
                        def fill_priority(pid: int) -> tuple:
                            recently_out_priority = 0 if pid in recent_out[tid] else 1
                            starter_priority = 0 if pid in starters[tid] else 1
                            activity_priority = -(current_time - last_action_time[pid])

                            return (recently_out_priority, starter_priority, activity_priority)

                        available.sort(key=fill_priority)

                        needed = 5 - len(on_court[tid])
                        for i in range(min(needed, len(available))):
                            fill_player = available[i]
                            on_court[tid].add(fill_player)
                            changes_made = True
                            logger.info(f"[ENHANCED AUTO-IN] {name_map.get(fill_player)} to {team_name} (fill to 5)")

                # Update segments if changes were made
                if changes_made:
                    enhanced_stats['lineup_size_corrections'] += 1
                    for pid in on_court[tid]:
                        if pid not in active_segments:
                            start_player_segment(pid, current_time, "LINEUP_CORRECTION")

                    # Flag lineup correction
                    enhanced_stats['flags']['lineup_violations'].append({
                        'time': current_time,
                        'team': team_name,
                        'correction_type': 'AUTO_CORRECTION',
                        'final_size': len(on_court[tid])
                    })

            # MAIN PROCESSING LOOP - ENHANCED APPROACH WITH FLAGS
            prev_period = None

            logger.info(f"Processing {len(events)} events with ENHANCED substitution rules and flagging...")

            for idx, ev in events.iterrows():
                period = int(ev.period)
                clock_str = ev.game_clock
                parsed_clock = _parse_game_clock(clock_str)
                current_time = _abs_time(period, parsed_clock)
                msg_type = int(ev.msg_type)

                # Handle period transitions
                if period != prev_period and prev_period is not None:
                    period_end_time = _abs_time(prev_period, 0.0)

                    for pid in list(active_segments.keys()):
                        end_player_segment(pid, period_end_time, f"PERIOD_{prev_period}_END")

                if period != prev_period:
                    if period in CFG["starter_reset_periods"]:
                        on_court = {tid: set(starters[tid]) for tid in teams}
                        logger.info(f"[ENHANCED PERIOD {period}] Reset to starters")
                    else:
                        logger.info(f"[ENHANCED PERIOD {period}] Continue lineups")

                    for tid in teams:
                        for pid in on_court[tid]:
                            if pid not in active_segments:
                                start_player_segment(pid, current_time, f"PERIOD_{period}_START")

                    prev_period = period

                # SUBSTITUTION PROCESSING
                if msg_type == CFG["msg_types"]["substitution"]:
                    enhanced_stats['total_substitutions'] += 1

                    out_pid = int(ev.player_id_1) if not pd.isna(ev.player_id_1) else None
                    in_pid = int(ev.player_id_2) if not pd.isna(ev.player_id_2) else None
                    out_ln, in_ln = ev.last_name_1, ev.last_name_2

                    # Determine team
                    sub_tid = None
                    if in_pid and in_pid in pteam_map:
                        sub_tid = pteam_map[in_pid]
                    elif out_pid and out_pid in pteam_map:
                        sub_tid = pteam_map[out_pid]
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        sub_tid = int(ev.team_id_off)

                    # Enhanced name resolution
                    if in_pid is None and CFG["one_direction"]["appearance_via_last_name"] and in_ln:
                        in_pid = enhanced_name_resolution(in_ln, sub_tid)
                    if out_pid is None and out_ln:
                        out_pid = enhanced_name_resolution(out_ln, sub_tid)

                    if sub_tid is not None:
                        team_name = team_abbrev_map[sub_tid]

                        # Process OUT first
                        if CFG["one_direction"]["remove_out_if_present"] and out_pid and out_pid in on_court[sub_tid]:
                            on_court[sub_tid].remove(out_pid)
                            recent_out[sub_tid].append(out_pid)
                            end_player_segment(out_pid, current_time, "SUB_OUT")
                            logger.info(f"[ENHANCED SUB-OUT] {name_map.get(out_pid)} from {team_name}")

                        # Process IN
                        if in_pid and in_pid not in on_court[sub_tid]:
                            # Make room if needed
                            if len(on_court[sub_tid]) >= 5:
                                auto_out = pick_auto_out_candidate(sub_tid, current_time, exclude={in_pid})
                                if auto_out:
                                    on_court[sub_tid].remove(auto_out)
                                    recent_out[sub_tid].append(auto_out)
                                    end_player_segment(auto_out, current_time, "MAKE_ROOM")
                                    logger.info(f"[ENHANCED MAKE-ROOM] {name_map.get(auto_out)} out for {name_map.get(in_pid)}")

                            on_court[sub_tid].add(in_pid)
                            start_player_segment(in_pid, current_time, "SUB_IN")
                            enhanced_stats['successful_substitutions'] += 1
                            logger.info(f"[ENHANCED SUB-IN] {name_map.get(in_pid)} to {team_name}")

                    # Update activity times
                    if out_pid:
                        last_action_time[out_pid] = current_time
                    if in_pid:
                        last_action_time[in_pid] = current_time

                    # Snapshot lineup after substitution
                    snapshot_lineups(current_time, period, int(ev.pbp_order), str(ev.description))

                # FIRST ACTION PROCESSING (Reed Sheppard rule) WITH FLAGS
                elif msg_type in [1, 2, 4, 5, 6] and CFG["one_direction"]["appearance_via_last_name"]:
                    action_pid = int(ev.player_id_1) if not pd.isna(ev.player_id_1) else None
                    action_ln = ev.last_name_1
                    action_tid = None

                    if action_pid and action_pid in pteam_map:
                        action_tid = pteam_map[action_pid]
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        action_tid = int(ev.team_id_off)

                    # Resolve via last name if needed
                    if action_pid is None and action_ln:
                        action_pid = enhanced_name_resolution(action_ln, action_tid)
                        if action_pid:
                            action_tid = pteam_map[action_pid]

                    # Apply Reed Sheppard rule with FLAG
                    if action_tid in teams and action_pid and action_pid not in on_court[action_tid]:
                        # FLAG: This is a missing sub-in scenario
                        enhanced_stats['flags']['missing_sub_ins'].append({
                            'time': current_time,
                            'player_id': action_pid,
                            'player_name': name_map.get(action_pid),
                            'team': team_abbrev_map[action_tid],
                            'action_type': msg_type,
                            'description': ev.description,
                            'resolution': 'FIRST_ACTION_INJECTION'
                        })

                        # Also add to flag_rows for CSV export
                        flag_rows.append({
                            "period": period,
                            "pbp_order": int(ev.pbp_order),
                            "abs_time": round(current_time, 3),
                            "team_id": int(action_tid),
                            "team_abbrev": team_abbrev_map[action_tid],
                            "flag_type": "missing_sub_in",
                            "player_id": int(action_pid),
                            "player_name": name_map.get(action_pid, str(action_pid)),
                            "idle_seconds": None,
                            "description": str(ev.description or "")
                        })

                        # Inject player via first-action rule
                        on_court[action_tid].add(action_pid)
                        enhanced_stats['first_action_injections'] += 1

                        logger.info(f"[ENHANCED FIRST-ACTION] {name_map.get(action_pid)} -> {team_abbrev_map[action_tid]} (msg: {msg_type})")

                        # Flag the first-action event
                        enhanced_stats['flags']['first_action_events'].append({
                            'time': current_time,
                            'player_id': action_pid,
                            'player_name': name_map.get(action_pid),
                            'team': team_abbrev_map[action_tid],
                            'action': ev.description
                        })

                        start_player_segment(action_pid, current_time, "FIRST_ACTION")
                        ensure_valid_lineup(action_tid, current_time, prefer_keep={action_pid})

                        # Snapshot lineup after first-action injection
                        snapshot_lineups(current_time, period, int(ev.pbp_order), f"FIRST_ACTION: {ev.description}")

                    # Update activity time
                    if action_pid:
                        last_action_time[action_pid] = current_time

                # Periodic inactivity checking
                if idx % 20 == 0:  # Check every 20 events
                    flag_inactivity_check(current_time, period, int(ev.pbp_order))

                # Ensure lineup validity
                for tid in teams:
                    if len(on_court[tid]) != 5:
                        ensure_valid_lineup(tid, current_time)

            # Final processing
            final_time = _abs_time(4, 0.0) if prev_period and prev_period <= 4 else 2880.0
            for pid in list(active_segments.keys()):
                end_player_segment(pid, final_time, "GAME_END")

            # Calculate minutes
            calculated_minutes = {}
            for pid in set(list(completed_segments.keys()) + list(box_df['player_id'])):
                segments = completed_segments[pid]
                total_seconds = sum(seg['duration'] for seg in segments)
                calculated_minutes[pid] = total_seconds

            # Build validation results
            enhanced_validation = []
            for pid in set(list(calculated_minutes.keys()) + list(box_df['player_id'])):
                calc_secs = calculated_minutes.get(pid, 0.0)
                box_row = box_df[box_df['player_id'] == pid]
                box_secs = float(box_row['seconds_played'].iloc[0]) if not box_row.empty else 0.0
                diff = calc_secs - box_secs

                enhanced_validation.append({
                    "player_id": pid,
                    "player_name": name_map.get(pid, f"ID_{pid}"),
                    "team": team_abbrev_map.get(pteam_map.get(pid), "UNK"),
                    "calc_seconds": round(calc_secs, 1),
                    "box_seconds": round(box_secs, 1),
                    "abs_diff_seconds": round(abs(diff), 1),
                    "segments_count": len(completed_segments.get(pid, []))
                })

            enhanced_validation_df = pd.DataFrame(enhanced_validation).sort_values(["team", "player_name"]).reset_index(drop=True)
            enhanced_offenders = enhanced_validation_df[enhanced_validation_df["abs_diff_seconds"] > CFG["minutes_validation"]["tolerance_seconds"]]

            # Create lineup state and flag tables
            state_df = pd.DataFrame(state_rows).sort_values(["period", "pbp_order", "team_id"]).reset_index(drop=True)
            flag_df = pd.DataFrame(flag_rows).sort_values(["abs_time", "team_id"]).reset_index(drop=True)

            # Enhanced minutes table (similar to basic_minutes)
            enhanced_minutes_rows = []
            for pid, secs in calculated_minutes.items():
                if pid in pteam_map:
                    enhanced_minutes_rows.append({
                        "player_id": int(pid),
                        "player_name": name_map.get(pid, str(pid)),
                        "team_id": int(pteam_map[pid]),
                        "team_abbrev": team_abbrev_map[int(pteam_map[pid])],
                        "seconds_enhanced": round(float(secs), 3)
                    })
            enhanced_minutes_df = pd.DataFrame(enhanced_minutes_rows).sort_values(["team_abbrev", "player_name"]).reset_index(drop=True)

            # Persist to DuckDB
            self._robust_drop_object("enhanced_lineup_state")
            self.conn.register("enhanced_lineup_state_temp", state_df)
            self.conn.execute("CREATE TABLE enhanced_lineup_state AS SELECT * FROM enhanced_lineup_state_temp")
            self.conn.execute("DROP VIEW IF EXISTS enhanced_lineup_state_temp")

            self._robust_drop_object("enhanced_lineup_flags")
            self.conn.register("enhanced_lineup_flags_temp", flag_df)
            self.conn.execute("CREATE TABLE enhanced_lineup_flags AS SELECT * FROM enhanced_lineup_flags_temp")
            self.conn.execute("DROP VIEW IF EXISTS enhanced_lineup_flags_temp")

            self._robust_drop_object("minutes_enhanced")
            self.conn.register("minutes_enhanced_temp", enhanced_minutes_df)
            self.conn.execute("CREATE TABLE minutes_enhanced AS SELECT * FROM minutes_enhanced_temp")
            self.conn.execute("DROP VIEW IF EXISTS minutes_enhanced_temp")

            # Store results with enhanced data summary
            flag_totals = {
                "missing_sub_in": len([f for f in flag_rows if f.get("flag_type") == "missing_sub_in"]),
                "inactivity_periods": len([f for f in flag_rows if f.get("flag_type") == "inactivity_periods"]),
                "first_action_events": enhanced_stats['first_action_injections'],
                "auto_out_events": enhanced_stats['auto_outs_inactivity'],
                "lineup_violations": len([f for f in enhanced_stats['flags']['lineup_violations']])
            }

            self.data_summary['enhanced_substitution_tracking'] = {
                'state_rows': len(state_rows),
                'flag_rows': len(flag_rows),
                'flag_totals': flag_totals,
                'validation_data': enhanced_validation_df,
                'offenders_data': enhanced_offenders,
                'flags': enhanced_stats['flags'],
                'statistics': enhanced_stats
            }

            # Store validation data for minutes report
            self.data_summary['minutes_validation_full'] = enhanced_validation_df.copy()
            self.data_summary['minutes_offenders'] = enhanced_offenders.copy()

            # Store debug summary for final report
            self.data_summary['enhanced_substitution_debug'] = {
                "substitutions": enhanced_stats['total_substitutions'],
                "first_actions": enhanced_stats['first_action_injections'],
                "auto_outs": enhanced_stats['auto_outs_inactivity'],
                "always_five_fixes": enhanced_stats['lineup_size_corrections'],
                "validation": {
                    "tolerance": CFG["minutes_validation"]["tolerance_seconds"],
                    "offenders": len(enhanced_offenders),
                    "total_players": len(enhanced_validation_df)
                }
            }

            # Create database tables for flags
            self._create_enhanced_flags_tables(enhanced_stats['flags'])

            # Summary statistics with FLAGS
            summary = {
                'method': 'ENHANCED_WITH_FLAGS',
                'processing_time': time.time() - start_time,
                'substitutions': enhanced_stats,
                'validation': {
                    'total_players': len(enhanced_validation_df),
                    'offenders': len(enhanced_offenders),
                    'tolerance_seconds': CFG["minutes_validation"]["tolerance_seconds"],
                    'max_difference': enhanced_validation_df['abs_diff_seconds'].max() if not enhanced_validation_df.empty else 0
                },
                'flags_summary': {
                    'missing_sub_ins': len(enhanced_stats['flags']['missing_sub_ins']),
                    'inactivity_periods': len(enhanced_stats['flags']['inactivity_periods']),
                    'lineup_violations': len(enhanced_stats['flags']['lineup_violations']),
                    'first_action_events': len(enhanced_stats['flags']['first_action_events']),
                    'auto_out_events': len(enhanced_stats['flags']['auto_out_events'])
                }
            }

            logger.info(f"[ENHANCED COMPLETE] {enhanced_stats['successful_substitutions']} subs, {enhanced_stats['first_action_injections']} first-actions, {enhanced_stats['auto_outs_inactivity']} auto-outs")
            logger.info(f"[ENHANCED FLAGS] Missing sub-ins: {len(enhanced_stats['flags']['missing_sub_ins'])}, Inactivity periods: {len(enhanced_stats['flags']['inactivity_periods'])}")

            total_flags = sum(len(flag_list) for flag_list in enhanced_stats['flags'].values())

            return ValidationResult(
                step_name="Enhanced Substitution Tracking with Flags",
                passed=True,
                details=f"Enhanced tracking complete: {enhanced_stats['successful_substitutions']} subs, {enhanced_stats['first_action_injections']} first-actions, {total_flags} total flags",
                processing_time=time.time() - start_time
            )

        except Exception as e:
            logger.error(f"Error in enhanced substitution tracking: {e}")
            import traceback
            logger.error(f"Traceback: {traceback.format_exc()}")
            return ValidationResult(
                step_name="Enhanced Substitution Tracking with Flags",
                passed=False,
                details=f"Error in enhanced tracking: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _create_enhanced_flags_tables(self, flags_data: Dict[str, List]) -> None:
        """Create database tables for enhanced flags"""
        try:
            # Create comprehensive flags table
            all_flags = []

            for flag_type, flag_list in flags_data.items():
                for flag in flag_list:
                    flag_record = {
                        'flag_type': flag_type,
                        'time': flag.get('time', 0),
                        'player_id': flag.get('player_id'),
                        'player_name': flag.get('player_name'),
                        'team': flag.get('team'),
                        'description': str(flag)
                    }
                    all_flags.append(flag_record)

            if all_flags:
                flags_df = pd.DataFrame(all_flags)
                self._robust_drop_object("enhanced_flags")
                self.conn.register('enhanced_flags_temp', flags_df)
                self.conn.execute("CREATE TABLE enhanced_flags AS SELECT * FROM enhanced_flags_temp")
                self.conn.execute("DROP VIEW IF EXISTS enhanced_flags_temp")

                logger.info(f"Created enhanced_flags table with {len(all_flags)} flag records")

        except Exception as e:
            logger.warning(f"Could not create enhanced flags tables: {e}")

    def run_lineups_and_rim_analytics(self) -> ValidationResult:
        """
        CORRECTED substitution engine that systematically fixes all identified issues:

        1. Prevents double-crediting of period remainder time
        2. Properly implements Reed Sheppard first-action rule  
        3. Fixes time tracking between events
        4. Implements intelligent 2-minute inactivity auto-out
        5. Maintains strict 5-man lineups with gap-filling logic

        Key Fixes:
        - Single responsibility for period remainder crediting
        - Enhanced last name resolution for first actions
        - Proper time segment tracking without overlaps
        - Comprehensive debugging and validation
        - Activity-based auto-out selection to prevent inappropriate removals
        """
        start_time = time.time()

        CFG = {
            "starter_reset_periods": [1, 3],
            "one_direction": {
                "appearance_via_last_name": True,
                "remove_out_if_present": True
            },
            "msg_types": {
                "shot_made": 1, "shot_missed": 2, "rebound": 4,
                "turnover": 5, "foul": 6, "substitution": 8
            },
            "minutes_validation": {"tolerance_seconds": 120},  # Reasonable tolerance
            "inactivity_rule": {"idle_seconds_threshold": 120}
        }

        try:
            # Helper functions with better error handling
            def _period_length_seconds(p: int) -> float:
                return 720.0 if p <= 4 else 300.0

            def _parse_game_clock(gc: str) -> float | None:
                if not gc or not isinstance(gc, str):
                    return None
                s = gc.strip()
                if s.count(":") != 1:
                    return None
                try:
                    mm, ss = s.split(":")
                    return float(mm) * 60.0 + float(ss)
                except (ValueError, IndexError):
                    return None

            def _abs_time(period: int, rem_sec: float | None) -> float:
                """Calculate absolute game time elapsed"""
                total = 0.0
                for pi in range(1, period):
                    total += _period_length_seconds(pi)
                pl = _period_length_seconds(period)
                if rem_sec is None:
                    return total + pl  # If no clock, assume period end
                return total + (pl - rem_sec)

            # Load and validate core data
            box_df = self.conn.execute("""
                SELECT player_id, player_name, team_id, team_abbrev, is_starter, seconds_played
                FROM box_score
                WHERE seconds_played > 0
                ORDER BY team_id, seconds_played DESC
            """).df()

            if box_df.empty:
                return ValidationResult(
                    step_name="Enhanced Lineups & Rim Analytics",
                    passed=False,
                    details="No players found in box_score with playing time",
                    processing_time=time.time() - start_time
                )

            teams = sorted(box_df['team_id'].unique().tolist())
            if len(teams) != 2:
                return ValidationResult(
                    step_name="Enhanced Lineups & Rim Analytics",
                    passed=False,
                    details=f"Expected exactly 2 teams, found {teams}",
                    processing_time=time.time() - start_time
                )

            # Build comprehensive player mappings
            roster = {int(tid): set() for tid in teams}
            starters = {int(tid): set() for tid in teams}
            name_map, pteam_map = {}, {}
            last_name_index = {}

            for _, r in box_df.iterrows():
                pid = int(r.player_id)
                tid = int(r.team_id)
                roster[tid].add(pid)
                if bool(r.is_starter):
                    starters[tid].add(pid)
                name_map[pid] = str(r.player_name)
                pteam_map[pid] = tid

                # Enhanced last name indexing
                full_name = str(r.player_name).strip()
                last_name = full_name.split()[-1].lower()
                first_name = full_name.split()[0].lower() if len(full_name.split()) > 1 else ""

                last_name_index.setdefault(last_name, []).append(pid)
                if first_name:
                    last_name_index.setdefault(first_name, []).append(pid)

            # Validate team structure
            if any(len(starters[tid]) != 5 for tid in teams):
                detail = {tid: [name_map[p] for p in sorted(starters[tid])] for tid in teams}
                return ValidationResult(
                    step_name="Enhanced Lineups & Rim Analytics",
                    passed=False,
                    details=f"Invalid starters: {detail}",
                    processing_time=time.time() - start_time
                )

            team_abbrev_map = {int(tid): box_df[box_df.team_id == tid]['team_abbrev'].iloc[0] for tid in teams}

            # Load events with validation
            events = self.conn.execute("""
                SELECT 
                    period, pbp_order, wall_clock_int,
                    COALESCE(game_clock,'') AS game_clock,
                    COALESCE(description,'') AS description,
                    team_id_off, team_id_def, msg_type, action_type,
                    player_id_1, player_id_2, player_id_3,
                    NULLIF(last_name_1,'') AS last_name_1,
                    NULLIF(last_name_2,'') AS last_name_2,
                    NULLIF(last_name_3,'') AS last_name_3,
                    COALESCE(points, 0) AS points
                FROM pbp
                ORDER BY period, pbp_order, wall_clock_int
            """).df()

            if events.empty:
                return ValidationResult(
                    step_name="Enhanced Lineups & Rim Analytics",
                    passed=False,
                    details="No PBP events found",
                    processing_time=time.time() - start_time
                )

            # Initialize state tracking
            from collections import defaultdict, deque

            # CORRECTED: Enhanced state tracking with comprehensive segment management
            on_court = {tid: set(starters[tid]) for tid in teams}
            last_action_time = defaultdict(lambda: 0.0)
            recent_out = {tid: deque(maxlen=10) for tid in teams}

            # CORRECTED: Enhanced segment tracking with validation
            active_segments = {}  # player_id -> {'start': time, 'reason': str}
            completed_segments = defaultdict(list)  # player_id -> [{'start': time, 'end': time, 'duration': dur, 'reason': str}]
            period_end_times = {}  # Track when we last ended a period to prevent double-crediting

            # Initialize segments for starters
            for tid in teams:
                for pid in on_court[tid]:
                    active_segments[pid] = {'start': 0.0, 'reason': 'GAME_START'}
                    last_action_time[pid] = 0.0

            # Enhanced debugging
            debug_events = []
            sub_count = 0
            first_action_count = 0
            auto_out_count = 0
            lineup_violation_fixes = 0
            validation_errors = []

            idle_thresh = CFG.get("inactivity_rule", {}).get("idle_seconds_threshold", 120)
            tol = CFG["minutes_validation"]["tolerance_seconds"]

            def enhanced_name_resolution(ln: str | None, tid_hint: int | None) -> int | None:
                """Enhanced name resolution with fuzzy matching"""
                if not ln:
                    return None

                ln_clean = str(ln).strip().lower()
                candidates = last_name_index.get(ln_clean, [])

                if not candidates:
                    # Try partial matching
                    for key, pids in last_name_index.items():
                        if ln_clean in key or key in ln_clean:
                            candidates.extend(pids)

                if not candidates:
                    return None

                # Prefer team hint if available
                if tid_hint is not None:
                    for cand in candidates:
                        if pteam_map.get(cand) == tid_hint:
                            return cand

                # Return first valid candidate
                for cand in candidates:
                    if pteam_map.get(cand) in teams:
                        return cand

                return None

            def end_player_segment(pid: int, end_time: float, reason: str) -> None:
                """CORRECTED: End a player's active segment with validation"""
                if pid not in active_segments:
                    return  # No active segment to end

                start_info = active_segments[pid]
                start_time = start_info['start']

                if end_time <= start_time:
                    # Safety fix: use a minimum duration of 1 second
                    logger.warning(f"Invalid segment timing fixed: end_time {end_time} <= start_time {start_time} for {name_map.get(pid)}")
                    end_time = start_time + 1.0
                    validation_errors.append(f"Fixed invalid segment timing for {name_map.get(pid)}")

                duration = end_time - start_time
                completed_segments[pid].append({
                    'start': start_time,
                    'end': end_time,
                    'duration': duration,
                    'reason': f"{start_info['reason']} -> {reason}"
                })

                del active_segments[pid]
                logger.debug(f"Ended segment for {name_map.get(pid)}: {duration:.1f}s ({reason})")

            def start_player_segment(pid: int, start_time: float, reason: str) -> None:
                """CORRECTED: Start a new segment with overlap prevention"""
                if pid in active_segments:
                    # End existing segment first to prevent overlaps
                    end_player_segment(pid, start_time, f"OVERLAP_{reason}")

                active_segments[pid] = {'start': start_time, 'reason': reason}
                logger.debug(f"Started segment for {name_map.get(pid)}: {start_time:.1f}s ({reason})")

            def handle_lineup_change(tid: int, current_time: float, reason: str) -> None:
                """CORRECTED: Handle lineup changes with proper segment management"""
                team_name = team_abbrev_map[tid]

                # Get all players who should be tracked for this team
                team_players = [pid for pid in pteam_map if pteam_map.get(pid) == tid]

                # End segments for players no longer on court
                for pid in team_players:
                    if pid in active_segments and pid not in on_court[tid]:
                        end_player_segment(pid, current_time, f"{reason}_OUT")

                # Start segments for new players on court
                for pid in on_court[tid]:
                    if pid not in active_segments:
                        start_player_segment(pid, current_time, f"{reason}_IN")

            def pick_auto_out_candidate(tid: int, current_time: float, exclude: set[int] = set()) -> int | None:
                """CORRECTED: Enhanced auto-out selection based on activity patterns"""
                if not on_court[tid]:
                    return None

                candidates = [p for p in on_court[tid] if p not in exclude]
                if not candidates:
                    return None

                # CORRECTED: Enhanced scoring system for auto-out selection
                def activity_score(pid: int) -> tuple:
                    idle_time = current_time - last_action_time[pid]

                    # Primary factors (lower is better for removal):
                    # 1. Idle time (higher idle = more likely to remove)
                    # 2. Starter status (prefer to keep starters)
                    # 3. Recent sub activity (avoid ping-ponging)

                    is_starter = pid in starters[tid]
                    recently_subbed = pid in recent_out[tid]

                    # Score components (higher = more likely to be removed)
                    idle_score = idle_time
                    starter_penalty = -100 if is_starter else 0  # Keep starters longer
                    recent_sub_penalty = -50 if recently_subbed else 0  # Avoid ping-pong

                    total_score = idle_score + starter_penalty + recent_sub_penalty

                    return (total_score, idle_time, pid)  # Use pid for deterministic tiebreaking

                # Sort by activity score (highest score = best candidate for removal)
                candidates.sort(key=activity_score, reverse=True)

                best_candidate = candidates[0]
                idle_time = current_time - last_action_time[best_candidate]

                # Only auto-remove if idle >= threshold OR we have >5 players
                if idle_time >= idle_thresh or len(on_court[tid]) > 5:
                    return best_candidate

                return None

            def ensure_valid_lineup(tid: int, current_time: float, prefer_keep: set[int] = set()) -> None:
                """CORRECTED: Ensure team has exactly 5 players with better logic"""
                team_name = team_abbrev_map[tid]
                changes_made = False

                # Remove excess players
                while len(on_court[tid]) > 5:
                    auto_out = pick_auto_out_candidate(tid, current_time, exclude=prefer_keep)
                    if auto_out is None:
                        logger.error(f"Cannot auto-remove from {team_name} - no valid candidates")
                        validation_errors.append(f"Cannot auto-remove from {team_name}")
                        break

                    on_court[tid].remove(auto_out)
                    recent_out[tid].append(auto_out)
                    changes_made = True

                    idle_time = current_time - last_action_time[auto_out]
                    logger.info(f"[AUTO-OUT] {name_map.get(auto_out)} from {team_name} (idle: {idle_time:.1f}s)")

                    debug_events.append({
                        'time': current_time,
                        'type': 'AUTO_OUT',
                        'player': name_map.get(auto_out),
                        'team': team_name,
                        'idle_time': idle_time
                    })

                    nonlocal auto_out_count
                    auto_out_count += 1

                # Add players if under 5
                if len(on_court[tid]) < 5:
                    available = [p for p in roster[tid] if p not in on_court[tid]]
                    if available:
                        # CORRECTED: Prioritize based on game context
                        def fill_priority(pid: int) -> tuple:
                            # Priority factors (lower is better):
                            # 1. Recently out (prefer recent subs)
                            # 2. Starter status (prefer starters)  
                            # 3. Recent activity (prefer active players)

                            recently_out_priority = 0 if pid in recent_out[tid] else 1
                            starter_priority = 0 if pid in starters[tid] else 1
                            activity_priority = -(current_time - last_action_time[pid])  # More recent = lower number

                            return (recently_out_priority, starter_priority, activity_priority)

                        available.sort(key=fill_priority)

                        needed = 5 - len(on_court[tid])
                        for i in range(min(needed, len(available))):
                            fill_player = available[i]
                            on_court[tid].add(fill_player)
                            changes_made = True
                            logger.info(f"[AUTO-IN] {name_map.get(fill_player)} to {team_name} (fill to 5)")

                # Update segments if changes were made
                if changes_made:
                    handle_lineup_change(tid, current_time, "LINEUP_CORRECTION")

            def guard_always_five(current_time: float):
                """Fix any deviation from 5 and count it."""
                nonlocal lineup_violation_fixes
                for tid in teams:
                    if len(on_court[tid]) != 5:
                        lineup_violation_fixes += 1
                        ensure_valid_lineup(tid, current_time)

            # MAIN PROCESSING LOOP
            prev_period = None
            prev_time = None

            logger.info(f"Starting lineup processing: {team_abbrev_map[teams[0]]} vs {team_abbrev_map[teams[1]]}")

            for idx, ev in events.iterrows():
                period = int(ev.period)
                clock_str = ev.game_clock
                parsed_clock = _parse_game_clock(clock_str)
                current_time = _abs_time(period, parsed_clock)
                msg_type = int(ev.msg_type)

                # CORRECTED: Handle period transitions without double-crediting
                if period != prev_period and prev_period is not None:
                    # Calculate period end time
                    period_end_time = _abs_time(prev_period, 0.0)

                    # Only credit period end time if we haven't already done so
                    if prev_period not in period_end_times:
                        # End all active segments at period end
                        for pid in list(active_segments.keys()):
                            end_player_segment(pid, period_end_time, f"PERIOD_{prev_period}_END")

                        period_end_times[prev_period] = period_end_time
                        logger.debug(f"Ended period {prev_period} at {period_end_time:.1f}s")

                # Initialize new period
                if period != prev_period:
                    if period in CFG["starter_reset_periods"]:
                        # Reset to starters
                        on_court = {tid: set(starters[tid]) for tid in teams}
                        logger.info(f"[PERIOD {period}] Reset to starters")
                    else:
                        # Continue previous lineups
                        logger.info(f"[PERIOD {period}] Continue lineups")
                        for tid in teams:
                            ensure_valid_lineup(tid, current_time)

                    # Start new segments for all on-court players
                    for tid in teams:
                        for pid in on_court[tid]:
                            if pid not in active_segments:
                                start_player_segment(pid, current_time, f"PERIOD_{period}_START")

                    prev_period = period

                # SUBSTITUTION PROCESSING
                if msg_type == CFG["msg_types"]["substitution"]:
                    sub_count += 1

                    out_pid = int(ev.player_id_1) if not pd.isna(ev.player_id_1) else None
                    in_pid = int(ev.player_id_2) if not pd.isna(ev.player_id_2) else None
                    out_ln, in_ln = ev.last_name_1, ev.last_name_2

                    # Determine team
                    sub_tid = None
                    if in_pid and in_pid in pteam_map:
                        sub_tid = pteam_map[in_pid]
                    elif out_pid and out_pid in pteam_map:
                        sub_tid = pteam_map[out_pid]
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        sub_tid = int(ev.team_id_off)

                    # Enhanced name resolution
                    if in_pid is None and CFG["one_direction"]["appearance_via_last_name"] and in_ln:
                        in_pid = enhanced_name_resolution(in_ln, sub_tid)
                    if out_pid is None and out_ln:
                        out_pid = enhanced_name_resolution(out_ln, sub_tid)

                    if sub_tid is not None:
                        team_name = team_abbrev_map[sub_tid]

                        # Process OUT first
                        if CFG["one_direction"]["remove_out_if_present"] and out_pid and out_pid in on_court[sub_tid]:
                            on_court[sub_tid].remove(out_pid)
                            recent_out[sub_tid].append(out_pid)
                            logger.info(f"[SUB-OUT] {name_map.get(out_pid)} from {team_name}")

                        # Process IN
                        if in_pid and in_pid not in on_court[sub_tid]:
                            # Make room if needed
                            if len(on_court[sub_tid]) >= 5:
                                auto_out = pick_auto_out_candidate(sub_tid, current_time, exclude={in_pid})
                                if auto_out:
                                    on_court[sub_tid].remove(auto_out)
                                    recent_out[sub_tid].append(auto_out)
                                    logger.info(f"[MAKE-ROOM] {name_map.get(auto_out)} out for {name_map.get(in_pid)}")

                            on_court[sub_tid].add(in_pid)
                            logger.info(f"[SUB-IN] {name_map.get(in_pid)} to {team_name}")

                        # Update segments for substitution
                        handle_lineup_change(sub_tid, current_time, "SUBSTITUTION")

                    # Update activity times
                    if out_pid:
                        last_action_time[out_pid] = current_time
                    if in_pid:
                        last_action_time[in_pid] = current_time

                # FIRST ACTION PROCESSING (Reed Sheppard rule)
                elif msg_type in [1, 2, 4, 5, 6] and CFG["one_direction"]["appearance_via_last_name"]:
                    action_pid = int(ev.player_id_1) if not pd.isna(ev.player_id_1) else None
                    action_ln = ev.last_name_1
                    action_tid = None

                    if action_pid and action_pid in pteam_map:
                        action_tid = pteam_map[action_pid]
                    elif pd.notna(ev.team_id_off) and int(ev.team_id_off) in teams:
                        action_tid = int(ev.team_id_off)

                    # Resolve via last name if needed
                    if action_pid is None and action_ln:
                        action_pid = enhanced_name_resolution(action_ln, action_tid)
                        if action_pid:
                            action_tid = pteam_map[action_pid]

                    # CORRECTED: Apply Reed Sheppard rule with proper time tracking
                    if action_tid in teams and action_pid and action_pid not in on_court[action_tid]:
                        # This is a first action - inject player
                        on_court[action_tid].add(action_pid)
                        first_action_count += 1

                        logger.info(f"[FIRST-ACTION] {name_map.get(action_pid)} -> {team_abbrev_map[action_tid]} (msg: {msg_type})")

                        debug_events.append({
                            'time': current_time,
                            'type': 'FIRST_ACTION',
                            'player': name_map.get(action_pid),
                            'team': team_abbrev_map[action_tid],
                            'action': ev.description
                        })

                        # CORRECTED: Start segment for first-action player
                        start_player_segment(action_pid, current_time, "FIRST_ACTION")

                        # Ensure valid lineup after injection
                        ensure_valid_lineup(action_tid, current_time, prefer_keep={action_pid})

                    # Update activity time
                    if action_pid:
                        last_action_time[action_pid] = current_time

                # after each event: enforce always-5
                guard_always_five(current_time)

                prev_time = current_time

            # CORRECTED: Final processing - end all remaining segments
            # Use the last actual event time or calculate proper game end time
            if prev_period and current_time:
                final_time = current_time
            else:
                final_time = 2880.0  # 48 minutes total

            for pid in list(active_segments.keys()):
                end_player_segment(pid, final_time, "GAME_END")

            # CORRECTED: Calculate final minutes with comprehensive validation
            calculated_minutes = {}
            for pid in set(list(completed_segments.keys()) + list(box_df['player_id'])):
                segments = completed_segments[pid]
                total_seconds = sum(seg['duration'] for seg in segments)
                calculated_minutes[pid] = total_seconds

            # Build validation results
            mv = []
            for pid in set(list(calculated_minutes.keys()) + list(box_df['player_id'])):
                calc_secs = calculated_minutes.get(pid, 0.0)
                box_row = box_df[box_df['player_id'] == pid]
                box_secs = float(box_row['seconds_played'].iloc[0]) if not box_row.empty else 0.0
                diff = calc_secs - box_secs

                mv.append({
                    "player_id": pid,
                    "player_name": name_map.get(pid, f"ID_{pid}"),
                    "team": team_abbrev_map.get(pteam_map.get(pid), "UNK"),
                    "calc_seconds": round(calc_secs, 1),
                    "box_seconds": round(box_secs, 1),
                    "abs_diff_seconds": round(abs(diff), 1),
                    "segments_count": len(completed_segments.get(pid, []))
                })

            mv_df = pd.DataFrame(mv).sort_values(["team", "player_name"]).reset_index(drop=True)
            offenders = mv_df[mv_df["abs_diff_seconds"] > tol]

            # Enhanced logging for validation
            if len(offenders) > 0:
                logger.warning(f"CORRECTED: Minutes validation: {len(offenders)} players exceed {tol}s tolerance")
                for _, row in offenders.iterrows():
                    logger.warning(f"  {row.player_name} ({row.team}): calc={row.calc_seconds}s vs box={row.box_seconds}s (diff={row.abs_diff_seconds}s)")

                    # Debug segment details for offenders
                    segments = completed_segments.get(row.player_id, [])
                    logger.debug(f"    Segments for {row.player_name}: {len(segments)} total")
                    for i, seg in enumerate(segments[:5]):  # Show first 5 segments
                        logger.debug(f"      {i+1}: {seg['duration']:.1f}s ({seg['reason']})")

            if validation_errors:
                logger.warning(f"Validation errors encountered: {len(validation_errors)}")
                for error in validation_errors[:5]:
                    logger.warning(f"  {error}")

            # Store enhanced debug data
            self.data_summary['enhanced_substitution_debug'] = {
                'total_events': len(events),
                'substitutions': sub_count,
                'first_actions': first_action_count,
                'auto_outs': auto_out_count,
                'always_five_fixes': lineup_violation_fixes,
                'validation_errors': len(validation_errors),
                'validation': {
                    'total_players': len(mv_df),
                    'offenders': len(offenders),
                    'tolerance': tol,
                    'max_difference': mv_df['abs_diff_seconds'].max() if not mv_df.empty else 0
                }
            }
            # also stash full minutes table for the report writer
            self.data_summary['minutes_validation_full'] = mv_df
            self.data_summary['minutes_offenders'] = offenders

            logger.info(f"SUBSTITUTION SUMMARY: {sub_count} subs, {first_action_count} first-actions, {auto_out_count} auto-outs")

            # IMPORTANT: non-fatal -> passed = True (warnings carry the issues)
            details = (f"Enhanced engine: {len(events)} events, {sub_count} subs, {first_action_count} first-actions. "
                       f"Validation: {len(offenders)}/{len(mv_df)} offenders; 5-on-floor fixes: {lineup_violation_fixes}")
            return ValidationResult(
                step_name="Enhanced Lineups & Rim Analytics",
                passed=True,
                details=details,
                processing_time=time.time() - start_time,
                warnings=[] if len(offenders)==0 else [f"{len(offenders)} players exceed {tol}s tolerance"]
            )

        except Exception as e:
            import traceback
            logger.error(f"Exception in corrected substitution engine: {e}")
            logger.error(f"Traceback: {traceback.format_exc()}")
            return ValidationResult(
                step_name="Enhanced Lineups & Rim Analytics",
                passed=False,
                details=f"Error in corrected engine: {str(e)}",
                processing_time=time.time() - start_time
            )

    def debug_segment_analysis(self, player_id: int = None) -> Dict[str, Any]:
        """
        Enhanced debugging function to analyze segment calculation for specific players.
        Useful for diagnosing Reed Sheppard cases and other timing issues.
        """
        if not hasattr(self, 'data_summary') or 'enhanced_substitution_debug' not in self.data_summary:
            return {"error": "No enhanced substitution data available"}

        debug_data = self.data_summary['enhanced_substitution_debug']

        # If specific player requested, focus on them
        if player_id:
            return self._analyze_player_segments(player_id)

        # Otherwise provide overall analysis
        return {
            'summary': debug_data,
            'recommendations': self._generate_debug_recommendations(debug_data)
        }

    def _analyze_player_segments(self, player_id: int) -> Dict[str, Any]:
        """Detailed analysis for a specific player's segments"""
        return {
            'player_id': player_id,
            'analysis': 'Detailed segment analysis would go here',
            'recommendations': []
        }

    def _generate_debug_recommendations(self, debug_data: Dict) -> List[str]:
        """Generate recommendations based on debug data"""
        recommendations = []

        validation = debug_data.get('validation', {})
        offenders = validation.get('offenders', 0)
        max_diff = validation.get('max_difference', 0)

        if offenders > 0:
            recommendations.append(f"Still have {offenders} players with timing issues")

        if max_diff > 300:  # 5 minutes
            recommendations.append("Large timing discrepancies detected - check period transitions")
        elif max_diff > 120:  # 2 minutes  
            recommendations.append("Moderate timing issues - check first-action logic")

        if debug_data.get('validation_errors', 0) > 0:
            recommendations.append("Segment validation errors detected - check overlap prevention")

        auto_outs = debug_data.get('auto_outs', 0)
        if auto_outs > 20:
            recommendations.append("High auto-out count - may indicate lineup instability")

        return recommendations

    def create_minutes_validation_report(self) -> str:
        """
        Create a detailed validation report for minutes calculation.
        Useful for verifying the corrected engine performance.
        """
        if not hasattr(self, 'data_summary'):
            return "No validation data available"

        # Get validation data from enhanced runs if available
        enhanced_data = self.data_summary.get('enhanced_substitution_debug', {})

        report_lines = [
            "MINUTES VALIDATION REPORT",
            "=" * 50,
            ""
        ]

        if enhanced_data:
            validation = enhanced_data.get('validation', {})
            report_lines.extend([
                "ENHANCED ENGINE RESULTS:",
                f"Total players: {validation.get('total_players', 0)}",
                f"Players exceeding tolerance: {validation.get('offenders', 0)}",
                f"Maximum difference: {validation.get('max_difference', 0):.1f}s",
                f"Tolerance threshold: {validation.get('tolerance', 120)}s",
                f"5-on-floor fixes: {enhanced_data.get('always_five_fixes', 0)}",
                ""
            ])

            # Add recommendations
            recommendations = self._generate_debug_recommendations(enhanced_data)
            if recommendations:
                report_lines.extend([
                    "RECOMMENDATIONS:",
                    *[f"- {rec}" for rec in recommendations],
                    ""
                ])

        return "\n".join(report_lines)

    def debug_substitution_analysis(self) -> Dict[str, Any]:
        """
        Comprehensive debugger to identify substitution and minutes calculation issues.

        This function analyzes:
        1. Reed Sheppard's specific case and similar players
        2. Minutes calculation discrepancies
        3. Substitution event patterns
        4. Missing first-action events
        """

        logger.info("🔍 STARTING COMPREHENSIVE SUBSTITUTION ANALYSIS")
        logger.info("=" * 80)

        analysis_results = {
            'reed_sheppard_analysis': {},
            'minutes_discrepancies': {},
            'substitution_patterns': {},
            'first_action_missing': {},
            'timeline_analysis': {}
        }

        # Get all data we need
        box_df = self.conn.execute("""
            SELECT player_id, player_name, team_id, team_abbrev, is_starter, seconds_played
            FROM box_score WHERE seconds_played > 0 ORDER BY team_id, seconds_played DESC
        """).df()

        events_df = self.conn.execute("""
            SELECT period, pbp_order, wall_clock_int, game_clock, description,
                   team_id_off, team_id_def, msg_type, action_type,
                   player_id_1, player_id_2, player_id_3,
                   last_name_1, last_name_2, last_name_3, points
            FROM pbp ORDER BY period, pbp_order, wall_clock_int
        """).df()

        # Create name mappings
        name_map = dict(zip(box_df['player_id'], box_df['player_name']))
        team_map = dict(zip(box_df['player_id'], box_df['team_abbrev']))

        logger.info(f"📊 Analyzing {len(events_df)} events for {len(box_df)} players")

        # 1. REED SHEPPARD SPECIFIC ANALYSIS
        logger.info("\n🎯 REED SHEPPARD CASE ANALYSIS")
        logger.info("-" * 50)

        reed_sheppard_id = 1642263  # From the provided data
        if reed_sheppard_id in name_map:
            reed_events = []

            # Find all events involving Reed Sheppard
            for _, ev in events_df.iterrows():
                if (ev['player_id_1'] == reed_sheppard_id or 
                    ev['player_id_2'] == reed_sheppard_id or 
                    ev['player_id_3'] == reed_sheppard_id or
                    'sheppard' in str(ev['description']).lower()):

                    reed_events.append({
                        'period': ev['period'],
                        'game_clock': ev['game_clock'],
                        'msg_type': ev['msg_type'],
                        'description': ev['description'],
                        'player_1': ev['player_id_1'],
                        'player_2': ev['player_id_2'],
                        'player_3': ev['player_id_3'],
                        'last_name_1': ev['last_name_1']
                    })

            logger.info(f"Reed Sheppard events found: {len(reed_events)}")
            for i, event in enumerate(reed_events):
                logger.info(f"  Event {i+1}: Q{event['period']} {event['game_clock']} | {event['description']}")
                logger.info(f"    MsgType: {event['msg_type']}, Players: {event['player_1']}, {event['player_2']}, {event['player_3']}")

            analysis_results['reed_sheppard_analysis'] = {
                'total_events': len(reed_events),
                'events': reed_events,
                'box_minutes': box_df[box_df['player_id'] == reed_sheppard_id]['seconds_played'].iloc[0] if reed_sheppard_id in box_df['player_id'].values else 0
            }

        # 2. FIND PLAYERS WITH FIRST ACTIONS BUT NO SUB-IN
        logger.info("\n🚨 PLAYERS WITH ACTIONS BUT NO SUB-IN")
        logger.info("-" * 50)

        # Get all substitution events
        sub_events = events_df[events_df['msg_type'] == 8].copy()

        # Get all players who sub IN
        subbed_in_players = set()
        for _, sub in sub_events.iterrows():
            if pd.notna(sub['player_id_2']):
                subbed_in_players.add(int(sub['player_id_2']))

        # Get starters
        starters = set(box_df[box_df['is_starter'] == True]['player_id'].tolist())

        # Find players with actions but never subbed in (and not starters)
        action_events = events_df[events_df['msg_type'].isin([1, 2, 4, 5, 6])].copy()

        players_with_actions = set()
        for _, ev in action_events.iterrows():
            for col in ['player_id_1', 'player_id_2', 'player_id_3']:
                if pd.notna(ev[col]):
                    players_with_actions.add(int(ev[col]))

        # Players who have actions but no sub-in and aren't starters
        missing_sub_in = players_with_actions - subbed_in_players - starters

        logger.info(f"Players with actions but no sub-in: {len(missing_sub_in)}")
        for pid in missing_sub_in:
            if pid in name_map:
                logger.info(f"  {name_map[pid]} (ID: {pid}) - {team_map.get(pid, 'Unknown team')}")

                # Find their first action
                first_action = None
                for _, ev in action_events.iterrows():
                    if (ev['player_id_1'] == pid or ev['player_id_2'] == pid or ev['player_id_3'] == pid):
                        first_action = ev
                        break

                if first_action is not None:
                    logger.info(f"    First action: Q{first_action['period']} {first_action['game_clock']} | {first_action['description']}")

        analysis_results['first_action_missing'] = {
            'count': len(missing_sub_in),
            'players': [{'id': pid, 'name': name_map.get(pid), 'team': team_map.get(pid)} for pid in missing_sub_in if pid in name_map]
        }

        # 3. SUBSTITUTION PATTERN ANALYSIS
        logger.info("\n🔄 SUBSTITUTION PATTERN ANALYSIS")
        logger.info("-" * 50)

        sub_analysis = {
            'total_subs': len(sub_events),
            'subs_with_both_players': 0,
            'subs_with_only_in': 0,
            'subs_with_only_out': 0,
            'subs_with_neither': 0
        }

        for _, sub in sub_events.iterrows():
            has_out = pd.notna(sub['player_id_1'])
            has_in = pd.notna(sub['player_id_2'])

            if has_out and has_in:
                sub_analysis['subs_with_both_players'] += 1
            elif has_in and not has_out:
                sub_analysis['subs_with_only_in'] += 1
            elif has_out and not has_in:
                sub_analysis['subs_with_only_out'] += 1
            else:
                sub_analysis['subs_with_neither'] += 1

        logger.info(f"Total substitutions: {sub_analysis['total_subs']}")
        logger.info(f"  Both players: {sub_analysis['subs_with_both_players']}")
        logger.info(f"  Only IN player: {sub_analysis['subs_with_only_in']}")
        logger.info(f"  Only OUT player: {sub_analysis['subs_with_only_out']}")
        logger.info(f"  Neither player: {sub_analysis['subs_with_neither']}")

        analysis_results['substitution_patterns'] = sub_analysis

        # 4. MINUTES CALCULATION SIMULATION
        logger.info("\n⏱️ MINUTES CALCULATION SIMULATION")
        logger.info("-" * 50)

        def parse_game_clock(gc):
            if not gc or not isinstance(gc, str):
                return None
            try:
                if ':' in gc:
                    mm, ss = gc.split(':')
                    return float(mm) * 60.0 + float(ss)
            except:
                pass
            return None

        def abs_time(period, rem_sec):
            total = 0.0
            for p in range(1, period):
                total += 720.0 if p <= 4 else 300.0
            period_length = 720.0 if period <= 4 else 300.0
            if rem_sec is None:
                return total
            return total + (period_length - rem_sec)

        # Simulate simple starter minutes (baseline)
        baseline_minutes = {}
        starters_per_team = {}

        for team in box_df['team_id'].unique():
            team_starters = box_df[(box_df['team_id'] == team) & (box_df['is_starter'] == True)]['player_id'].tolist()
            starters_per_team[team] = team_starters

            # Assume starters play full quarters 1 and 3, then continue in 2 and 4
            for pid in team_starters:
                baseline_minutes[pid] = 2 * 720.0  # Q1 + Q3 = 24 minutes baseline

        logger.info(f"Baseline starter minutes (Q1+Q3 only): {sum(baseline_minutes.values())/60:.1f} total minutes")

        # Find actual box score total
        actual_total = box_df['seconds_played'].sum()
        logger.info(f"Actual box score total: {actual_total/60:.1f} minutes")
        logger.info(f"Expected game total: {48*10:.1f} minutes (48 min × 10 players)")

        analysis_results['minutes_discrepancies'] = {
            'baseline_total': sum(baseline_minutes.values()),
            'actual_total': actual_total,
            'expected_total': 48 * 60 * 10,
            'baseline_vs_actual_diff': actual_total - sum(baseline_minutes.values())
        }

        # 5. TIMELINE ANALYSIS
        logger.info("\n📈 TIMELINE ANALYSIS")
        logger.info("-" * 50)

        timeline = []
        for _, ev in events_df.iterrows():
            if ev['msg_type'] == 8:  # Substitutions
                timeline.append({
                    'time': abs_time(ev['period'], parse_game_clock(ev['game_clock'])),
                    'period': ev['period'],
                    'clock': ev['game_clock'],
                    'event_type': 'SUB',
                    'description': ev['description']
                })
            elif ev['msg_type'] in [1, 2, 4, 5, 6] and ev['player_id_1'] in missing_sub_in:
                timeline.append({
                    'time': abs_time(ev['period'], parse_game_clock(ev['game_clock'])),
                    'period': ev['period'],
                    'clock': ev['game_clock'],
                    'event_type': 'MISSING_PLAYER_ACTION',
                    'player': name_map.get(ev['player_id_1'], f"ID_{ev['player_id_1']}"),
                    'description': ev['description']
                })

        timeline.sort(key=lambda x: x['time'])

        logger.info("Key timeline events:")
        for event in timeline[:20]:  # First 20 events
            if event['event_type'] == 'SUB':
                logger.info(f"  {event['time']:>6.1f}s Q{event['period']} {event['clock']} | SUB: {event['description']}")
            else:
                logger.info(f"  {event['time']:>6.1f}s Q{event['period']} {event['clock']} | MISSING: {event['player']} | {event['description']}")

        analysis_results['timeline_analysis'] = timeline[:50]  # Store first 50 for reference

        logger.info("\n✅ ANALYSIS COMPLETE")
        logger.info("=" * 80)

        return analysis_results

    def debug_minutes_tracker(self) -> Dict[str, Any]:
        """
        Create a detailed minute-by-minute tracker to identify exactly where minutes are being miscalculated.
        """
        from collections import defaultdict

        logger.info("🔍 CREATING DETAILED MINUTES TRACKER")

        # This will track every single second of every player's time
        minute_tracker = {
            'player_segments': defaultdict(list),
            'period_summaries': {},
            'discrepancies': {},
            'debug_log': []
        }

        def log_debug(message):
            minute_tracker['debug_log'].append(message)
            logger.debug(message)

        # Get data
        box_df = self.conn.execute("""
            SELECT player_id, player_name, team_id, team_abbrev, is_starter, seconds_played
            FROM box_score WHERE seconds_played > 0 ORDER BY team_id, seconds_played DESC
        """).df()

        events_df = self.conn.execute("""
            SELECT period, pbp_order, wall_clock_int, game_clock, description,
                   team_id_off, team_id_def, msg_type, action_type,
                   player_id_1, player_id_2, player_id_3,
                   last_name_1, last_name_2, last_name_3
            FROM pbp ORDER BY period, pbp_order, wall_clock_int
        """).df()

        name_map = dict(zip(box_df['player_id'], box_df['player_name']))
        team_map = dict(zip(box_df['player_id'], box_df['team_id']))

        # Initialize with starters
        teams = sorted(box_df['team_id'].unique())
        starters = {team: set(box_df[(box_df['team_id'] == team) & (box_df['is_starter'] == True)]['player_id'].tolist()) for team in teams}

        current_lineups = {team: set(starters[team]) for team in teams}

        log_debug(f"Initial lineups: {current_lineups}")

        def parse_clock(gc):
            if not gc or not isinstance(gc, str) or ':' not in gc:
                return None
            try:
                mm, ss = gc.split(':')
                return float(mm) * 60.0 + float(ss)
            except:
                pass
            return None

        def abs_time(period, rem_sec):
            total = sum(720.0 if p <= 4 else 300.0 for p in range(1, period))
            if rem_sec is None:
                return total
            period_length = 720.0 if period <= 4 else 300.0
            return total + (period_length - rem_sec)

        # Track time
        prev_abs_time = 0.0
        prev_period = 0

        for idx, ev in events_df.iterrows():
            period = int(ev['period'])
            curr_clock = parse_clock(ev['game_clock'])
            curr_abs_time = abs_time(period, curr_clock)

            # Handle period transitions
            if period != prev_period:
                if prev_period > 0:
                    # Credit end of previous period
                    period_end_time = abs_time(prev_period, 0.0)
                    if period_end_time > prev_abs_time:
                        duration = period_end_time - prev_abs_time
                        for team in teams:
                            for pid in current_lineups[team]:
                                minute_tracker['player_segments'][pid].append({
                                    'start': prev_abs_time,
                                    'end': period_end_time,
                                    'duration': duration,
                                    'reason': f'PERIOD_{prev_period}_END'
                                })
                        log_debug(f"Period {prev_period} end: credited {duration:.1f}s to {sum(len(current_lineups[t]) for t in teams)} players")

                # Reset or continue lineups for new period
                if period in [1, 3]:  # Starter reset periods
                    current_lineups = {team: set(starters[team]) for team in teams}
                    log_debug(f"Period {period}: Reset to starters")
                else:
                    log_debug(f"Period {period}: Continue lineups")

                prev_period = period

            # Credit time between events
            if curr_abs_time > prev_abs_time and prev_abs_time > 0:
                duration = curr_abs_time - prev_abs_time
                players_credited = 0
                for team in teams:
                    for pid in current_lineups[team]:
                        minute_tracker['player_segments'][pid].append({
                            'start': prev_abs_time,
                            'end': curr_abs_time,
                            'duration': duration,
                            'reason': f'PERIOD_{period}_PLAY'
                        })
                        players_credited += 1

                if duration > 60:  # Log significant time gaps
                    log_debug(f"Large time gap: {duration:.1f}s credited to {players_credited} players")

            # Handle substitutions
            if ev['msg_type'] == 8:
                out_pid = int(ev['player_id_1']) if pd.notna(ev['player_id_1']) else None
                in_pid = int(ev['player_id_2']) if pd.notna(ev['player_id_2']) else None

                # Find which team this substitution is for
                sub_team = None
                if in_pid and in_pid in team_map:
                    sub_team = team_map[in_pid]
                elif out_pid and out_pid in team_map:
                    sub_team = team_map[out_pid]

                if sub_team:
                    if out_pid and out_pid in current_lineups[sub_team]:
                        current_lineups[sub_team].remove(out_pid)
                        log_debug(f"SUB OUT: {name_map.get(out_pid, out_pid)} from team {sub_team}")

                    if in_pid and in_pid not in current_lineups[sub_team]:
                        current_lineups[sub_team].add(in_pid)
                        log_debug(f"SUB IN: {name_map.get(in_pid, in_pid)} to team {sub_team}")

            prev_abs_time = curr_abs_time

        # Final period end
        if prev_period > 0:
            final_end = abs_time(prev_period, 0.0)
            if final_end > prev_abs_time:
                duration = final_end - prev_abs_time
                for team in teams:
                    for pid in current_lineups[team]:
                        minute_tracker['player_segments'][pid].append({
                            'start': prev_abs_time,
                            'end': final_end,
                            'duration': duration,
                            'reason': f'PERIOD_{prev_period}_FINAL_END'
                        })
                log_debug(f"Final period end: credited {duration:.1f}s")

        # Calculate totals and compare
        for pid in minute_tracker['player_segments']:
            calculated_total = sum(seg['duration'] for seg in minute_tracker['player_segments'][pid])
            box_total = box_df[box_df['player_id'] == pid]['seconds_played'].iloc[0] if pid in box_df['player_id'].values else 0
            diff = calculated_total - box_total

            minute_tracker['discrepancies'][pid] = {
                'calculated': calculated_total,
                'box_score': box_total,
                'difference': diff,
                'segments_count': len(minute_tracker['player_segments'][pid]),
                'player_name': name_map.get(pid, f"ID_{pid}")
            }

        return minute_tracker

    def test_reed_sheppard_case(self):
        """Test function to verify Reed Sheppard case is handled correctly"""
        # Check if Reed Sheppard (ID: 1642263) appears in events
        reed_events = self.conn.execute("""
            SELECT period, game_clock, description, msg_type,
                   player_id_1, player_id_2, player_id_3,
                   last_name_1, last_name_2, last_name_3
            FROM pbp 
            WHERE player_id_1 = 1642263 
               OR player_id_2 = 1642263 
               OR player_id_3 = 1642263
               OR LOWER(description) LIKE '%sheppard%'
            ORDER BY period, pbp_order
        """).df()

        print(f"Reed Sheppard events: {len(reed_events)}")
        for _, ev in reed_events.iterrows():
            print(f"  Q{ev['period']} {ev['game_clock']} | {ev['description']}")

        return reed_events

    def create_missing_player_report(self) -> ValidationResult:
        """
        Summarize PBP-only players with names, inferred team, confidence, first/last seen, and event breakdown.

        Debug-first policy:
        - Do NOT hide missing data via COALESCE in the final outputs. Expose raw + resolved columns.
        - Add preflight checks and log actual row counts of intermediates (pbp_only_players, occ, sums).
        - Rebuild _pbp_names in this scope if it is not present to avoid hidden coupling.
        - Dump the FULL report (all columns, all rows) and schema to the logs when done.
        """
        start_time = time.time()
        try:
            # --- Preconditions: required base tables/views must exist ---
            need_tables = ["pbp", "dim_players", "dim_teams"]
            for t in need_tables:
                exists = self.conn.execute(
                    f"SELECT COUNT(*) FROM information_schema.tables WHERE table_name = '{t}'"
                ).fetchone()[0]
                if exists == 0:
                    return ValidationResult(
                        step_name="Missing Player Report",
                        passed=False,
                        details=f"Missing required table: {t}",
                        processing_time=time.time() - start_time
                    )

            # pbp_only_players is created in create_dimensions()
            has_pbp_only = self.conn.execute(
                "SELECT COUNT(*) FROM information_schema.views WHERE table_name = 'pbp_only_players'"
            ).fetchone()[0]
            if has_pbp_only == 0:
                return ValidationResult(
                    step_name="Missing Player Report",
                    passed=False,
                    details="pbp_only_players view not found. Run create_dimensions() first.",
                    processing_time=time.time() - start_time
                )

            # --- Ensure _pbp_names exists (recreate locally if absent) ---
            has_pbp_names = self.conn.execute(
                "SELECT COUNT(*) FROM information_schema.views WHERE table_name = '_pbp_names'"
            ).fetchone()[0]
            if has_pbp_names == 0:
                logger.info("[Missing Player Report] _pbp_names not found; rebuilding TEMP view locally")
                self.conn.execute("""
                    CREATE OR REPLACE TEMP VIEW _pbp_names AS
                    WITH p1 AS (
                        SELECT player_id_1 AS player_id, ANY_VALUE(NULLIF(last_name_1,'')) AS last_name
                        FROM pbp
                        WHERE player_id_1 IS NOT NULL
                        GROUP BY player_id_1
                    ),
                    p2 AS (
                        SELECT player_id_2 AS player_id, ANY_VALUE(NULLIF(last_name_2,'')) AS last_name
                        FROM pbp
                        WHERE player_id_2 IS NOT NULL
                        GROUP BY player_id_2
                    ),
                    p3 AS (
                        SELECT player_id_3 AS player_id, ANY_VALUE(NULLIF(last_name_3,'')) AS last_name
                        FROM pbp
                        WHERE player_id_3 IS NOT NULL
                        GROUP BY player_id_3
                    ),
                    unioned AS (
                        SELECT * FROM p1
                        UNION ALL
                        SELECT * FROM p2
                        UNION ALL
                        SELECT * FROM p3
                    )
                    SELECT player_id, ANY_VALUE(last_name) AS last_name
                    FROM unioned
                    WHERE last_name IS NOT NULL
                    GROUP BY player_id
                """)
            else:
                logger.info("[Missing Player Report] Reusing existing _pbp_names TEMP view")

            # Rebuild team guess confidence (same logic as earlier)
            self.conn.execute("""
                CREATE OR REPLACE TEMP VIEW _team_guess_conf AS
                WITH occ AS (
                    SELECT player_id_1 AS player_id, team_id_off AS team_id FROM pbp WHERE player_id_1 IS NOT NULL
                    UNION ALL SELECT player_id_2, team_id_off FROM pbp WHERE player_id_2 IS NOT NULL
                    UNION ALL SELECT player_id_3, team_id_off FROM pbp WHERE player_id_3 IS NOT NULL
                    UNION ALL SELECT player_id_1, team_id_def FROM pbp WHERE player_id_1 IS NOT NULL
                    UNION ALL SELECT player_id_2, team_id_def FROM pbp WHERE player_id_2 IS NOT NULL
                    UNION ALL SELECT player_id_3, team_id_def FROM pbp WHERE player_id_3 IS NOT NULL
                ),
                agg AS (
                    SELECT player_id, team_id, COUNT(*) AS c
                    FROM occ
                    GROUP BY player_id, team_id
                ),
                totals AS (
                    SELECT player_id, SUM(c) AS tot
                    FROM agg
                    GROUP BY player_id
                ),
                ranked AS (
                    SELECT
                        a.player_id, a.team_id, a.c, t.tot,
                        ROW_NUMBER() OVER (PARTITION BY a.player_id ORDER BY a.c DESC, a.team_id) AS rn
                    FROM agg a
                    JOIN totals t USING(player_id)
                )
                SELECT
                    player_id,
                    team_id AS guessed_team_id,
                    c AS guessed_count,
                    tot,
                    (c::DOUBLE)/NULLIF(tot,0) AS team_confidence
                FROM ranked
                WHERE rn = 1
            """)

            # --- Preflight debug: how many pbp-only players? ---
            num_only = self.conn.execute("SELECT COUNT(*) FROM pbp_only_players").fetchone()[0]
            logger.info(f"[Missing Player Report] pbp_only_players count = {num_only}")

            # --- Build the report table (JOIN sums as `s`) ---
            self._robust_drop_object("missing_player_report")
            self.conn.execute("""
                CREATE TABLE missing_player_report AS
                WITH occ AS (
                    SELECT
                        o.player_id,
                        p.period,
                        p.pbp_order,
                        p.wall_clock_int,
                        p.game_clock,
                        p.description,
                        p.msg_type,
                        p.points
                    FROM pbp_only_players o
                    LEFT JOIN pbp p
                    ON o.player_id = p.player_id_1
                    OR o.player_id = p.player_id_2
                    OR o.player_id = p.player_id_3
                ),
                sums AS (
                    SELECT
                        player_id,
                        COUNT(*) AS total_events,
                        SUM(points) AS points,
                        SUM(CASE WHEN msg_type IN (1,2) THEN 1 ELSE 0 END) AS shot_events,
                        SUM(CASE WHEN msg_type = 1 THEN 1 ELSE 0 END) AS made_fg,
                        SUM(CASE WHEN msg_type = 2 THEN 1 ELSE 0 END) AS missed_fg,
                        SUM(CASE WHEN msg_type = 3 THEN 1 ELSE 0 END) AS free_throws,
                        SUM(CASE WHEN msg_type = 4 THEN 1 ELSE 0 END) AS rebounds,
                        SUM(CASE WHEN msg_type = 5 THEN 1 ELSE 0 END) AS turnovers,
                        SUM(CASE WHEN msg_type = 6 THEN 1 ELSE 0 END) AS fouls,
                        SUM(CASE WHEN msg_type = 8 THEN 1 ELSE 0 END) AS substitutions,
                        arg_min(CONCAT('Q', period, ' ', game_clock, ' | ', description), wall_clock_int) AS first_event,
                        arg_max(CONCAT('Q', period, ' ', game_clock, ' | ', description), wall_clock_int) AS last_event
                    FROM occ
                    GROUP BY player_id
                )
                SELECT
                    -- identity
                    o.player_id,

                    -- names: show raw sources + resolved (do NOT hide missing in raw columns)
                    dp.player_name                           AS box_player_name,
                    n.last_name                              AS pbp_last_name,
                    CASE
                        WHEN dp.player_name IS NOT NULL THEN dp.player_name
                        WHEN n.last_name   IS NOT NULL THEN n.last_name
                        ELSE CAST(o.player_id AS VARCHAR)
                    END                                      AS resolved_name,

                    -- team IDs & labels: expose raw + guessed + resolved
                    dp.team_id                               AS box_team_id,
                    tgc.guessed_team_id                      AS guessed_team_id,
                    tgc.team_confidence                      AS team_confidence,
                    CASE
                        WHEN dp.team_id IS NOT NULL THEN dp.team_id
                        WHEN tgc.guessed_team_id IS NOT NULL THEN tgc.guessed_team_id
                        ELSE NULL
                    END                                      AS resolved_team_id,

                    dt_res.team_abbrev                       AS resolved_team_abbrev,
                    dt_box.team_abbrev                       AS box_team_abbrev,
                    dt_guess.team_abbrev                     AS guessed_team_abbrev,

                    -- sample text from pbp_only_players
                    o.sample_event,

                    -- event rollups from sums
                    s.total_events,
                    s.points,
                    s.shot_events,
                    s.made_fg,
                    s.missed_fg,
                    s.free_throws,
                    s.rebounds,
                    s.turnovers,
                    s.fouls,
                    s.substitutions,
                    s.first_event,
                    s.last_event

                FROM pbp_only_players o
                LEFT JOIN dim_players dp           ON o.player_id = dp.player_id
                LEFT JOIN _pbp_names n             ON o.player_id = n.player_id
                LEFT JOIN _team_guess_conf tgc     ON o.player_id = tgc.player_id
                LEFT JOIN sums s                   ON o.player_id = s.player_id
                LEFT JOIN dim_teams dt_res         ON CASE
                                                        WHEN dp.team_id IS NOT NULL THEN dp.team_id
                                                        ELSE tgc.guessed_team_id
                                                    END = dt_res.team_id
                LEFT JOIN dim_teams dt_box         ON dp.team_id = dt_box.team_id
                LEFT JOIN dim_teams dt_guess       ON tgc.guessed_team_id = dt_guess.team_id
                ORDER BY o.player_id
            """)

            # --- Postflight debug metrics (surface issues instead of masking) ---
            n_rows = self.conn.execute("SELECT COUNT(*) FROM missing_player_report").fetchone()[0]
            logger.info(f"[Missing Player Report] built with {n_rows} rows")

            n_no_sums = self.conn.execute("SELECT COUNT(*) FROM missing_player_report WHERE total_events IS NULL").fetchone()[0]
            if n_no_sums > 0:
                logger.warning(f"[Missing Player Report] {n_no_sums} row(s) have NULL total_events (no matching occ/sums)")

            n_no_team = self.conn.execute("SELECT COUNT(*) FROM missing_player_report WHERE resolved_team_id IS NULL").fetchone()[0]
            if n_no_team > 0:
                logger.warning(f"[Missing Player Report] {n_no_team} row(s) missing resolved_team_id")

            n_no_name = self.conn.execute("SELECT COUNT(*) FROM missing_player_report WHERE resolved_name IS NULL").fetchone()[0]
            if n_no_name > 0:
                logger.warning(f"[Missing Player Report] {n_no_name} row(s) missing resolved_name")

            # --- NEW: Dump the FULL report & schema to the output (no truncation) ---
            try:
                df = self.conn.execute("""
                    SELECT *
                    FROM missing_player_report
                    ORDER BY player_id
                """).df()

                # Print schema (column name + dtype)
                schema_lines = [f"  - {col}: {str(df[col].dtype)}" for col in df.columns]
                logger.info("\n" + "="*80 + "\nMISSING PLAYER REPORT — SCHEMA\n" + "="*80 + "\n" + "\n".join(schema_lines))

                # Print full table without truncation
                with pd.option_context('display.max_columns', None, 'display.max_colwidth', None, 'display.width', 10000):
                    table_str = df.to_string(index=False)
                logger.info("\n" + "="*80 + "\nMISSING PLAYER REPORT — FULL DATA\n" + "="*80 + f"\nrows: {len(df)}\n" + table_str + "\n" + "="*80)

            except Exception as dump_e:
                logger.warning(f"[Missing Player Report] Could not print full report to logs: {dump_e}")

            return ValidationResult(
                step_name="Missing Player Report",
                passed=True,
                details=f"Built missing_player_report with {n_rows} rows",
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Missing Player Report",
                passed=False,
                details=f"Error building report: {e}",
                processing_time=time.time() - start_time
            )


def load_all_data_enhanced(data_dir: Path | None = None, db_path: str = "mavs_enhanced.duckdb") -> Tuple[bool, Dict[str, Any]]:
    """Enhanced data loading with comprehensive validation + analytics outputs"""
    print("NBA Pipeline - Enhanced Data Loading & Validation")
    print("="*60)

    results = {
        'success': True,
        'validation_results': [],
        'data_summary': {}
    }

    # Prefer config-managed data directory if not provided
    if data_dir is None:
        try:
            from utils.config import MAVS_DATA_DIR
            data_dir = MAVS_DATA_DIR
        except Exception:
            data_dir = Path('data/mavs_data_engineer_2025')

    with EnhancedNBADataLoader(db_path) as loader:

        # 1) Box
        box_result = loader.load_and_validate_box_score(data_dir / 'box_HOU-DAL.csv')
        loader.validator.log_validation(box_result)
        results['validation_results'].append(box_result)
        if not box_result.passed:
            results['success'] = False
            return results['success'], results

        # 2) PBP
        pbp_result = loader.load_and_validate_pbp(data_dir / 'pbp_HOU-DAL.csv')
        loader.validator.log_validation(pbp_result)
        results['validation_results'].append(pbp_result)
        if not pbp_result.passed:
            results['success'] = False
            return results['success'], results

        # 3) Relationships
        rel_result = loader.validate_data_relationships()
        loader.validator.log_validation(rel_result)
        results['validation_results'].append(rel_result)
        if not rel_result.passed:
            results['success'] = False
            return results['success'], results

        # 4) Lookups
        lookup_result = loader.create_lookup_views()
        loader.validator.log_validation(lookup_result)
        results['validation_results'].append(lookup_result)
        if not lookup_result.passed:
            results['success'] = False
            return results['success'], results

        # 5) Dimensions
        dims_result = loader.create_dimensions()
        loader.validator.log_validation(dims_result)
        results['validation_results'].append(dims_result)
        if not dims_result.passed:
            results['success'] = False
            return results['success'], results

        # 6) Enriched view
        enrich_result = loader.create_pbp_enriched_view()
        loader.validator.log_validation(enrich_result)
        results['validation_results'].append(enrich_result)
        if not enrich_result.passed:
            results['success'] = False
            return results['success'], results

        # 6.3) UPDATED: Traditional Data-Driven Substitution Tracking (replaces basic)
        traditional_result = loader.run_traditional_data_driven_lineups()
        loader.validator.log_validation(traditional_result)
        results['validation_results'].append(traditional_result)
        if not traditional_result.passed:
            results['success'] = False
            return results['success'], results

        # 6.4) Enhanced substitution tracking with comprehensive flagging
        enhanced_result = loader.run_enhanced_substitution_tracking_with_flags()
        loader.validator.log_validation(enhanced_result)
        results['validation_results'].append(enhanced_result)
        if not enhanced_result.passed:
            results['success'] = False
            return results['success'], results

        # 6.5) Missing player report (optional but useful)
        missing_result = loader.create_missing_player_report()
        loader.validator.log_validation(missing_result)
        results['validation_results'].append(missing_result)
        if not missing_result.passed:
            results['success'] = False
            return results['success'], results

        # 7) Enhanced estimation engine (projects 1 & 2) - original method
        analytics_result = loader.run_lineups_and_rim_analytics()
        loader.validator.log_validation(analytics_result)
        results['validation_results'].append(analytics_result)

        # 7.5) UPDATED: Comprehensive comparison: Traditional Data-Driven vs Enhanced vs Box
        compare_result = loader.compare_traditional_vs_enhanced_lineups()
        loader.validator.log_validation(compare_result)
        results['validation_results'].append(compare_result)

        # 7.6) Legacy comparison (for backward compatibility)
        legacy_compare_result = loader.compare_basic_vs_estimated_lineups()
        loader.validator.log_validation(legacy_compare_result)
        results['validation_results'].append(legacy_compare_result)

        # 7.7) NEW: Dataset compliance validation
        compliance_result = loader.validate_dataset_compliance()
        loader.validator.log_validation(compliance_result)
        results['validation_results'].append(compliance_result)

        # 7.8) NEW: Create final submission artifacts
        submission_result = loader.create_project_submission_artifacts()
        loader.validator.log_validation(submission_result)
        results['validation_results'].append(submission_result)

        # 8) Final report
        report_result = loader.write_final_report()
        loader.validator.log_validation(report_result)
        results['validation_results'].append(report_result)

        # Summary
        results['data_summary'] = loader.data_summary
        loader.print_enhanced_summary()
        success = loader.validator.print_validation_summary()
        results['success'] = success
        return success, results





# Example usage
if __name__ == "__main__":
    data_directory = Path('data/mavs_data_engineer_2025')
    database_path = "mavs_enhanced.duckdb"

    success, results = load_all_data_enhanced(data_directory, database_path)

    if success:
        print("\n✅ Enhanced data loading completed successfully")
        print("🎯 Ready for entity extraction and lineup analysis")
    else:
        print("\n❌ Enhanced data loading failed")
        print("🔧 Review validation messages above")


Overwriting api/src/airflow_project/eda/data/nba_data_loader.py


In [30]:
%%writefile api/src/airflow_project/eda/data/nba_entities_extractor.py
# Robust NBA Entity Extractor - Step 3 Improvements  
"""
NBA Pipeline - Robust Entity Extraction with Proper Validation
==============================================================

This robust extractor handles the identified issues:
1. Uses actual starter data from box score (gs=1)
2. Handles missing players transparently 
3. Creates proper team mappings
4. Validates entity completeness without hiding issues

Key Features:
- Extract exactly 5 starters per team from box score
- Handle team mapping correctly (HOU: 1610612745, DAL: 1610612742)
- Create canonical player and team entities
- Transparent validation and error reporting
"""

import os
import sys
import time
import logging
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Set
from dataclasses import dataclass, field

import duckdb
import pandas as pd
import numpy as np

# Ensure we're in the right directory
cwd = os.getcwd()
if not cwd.endswith("airflow_project"):
    os.chdir('api/src/airflow_project')
sys.path.insert(0, os.getcwd())

from eda.utils.nba_pipeline_analysis import NBADataValidator, ValidationResult

logger = logging.getLogger(__name__)

@dataclass
class GameEntities:
    """Container for all canonical game entities"""
    unique_players: pd.DataFrame = None
    starters: Dict[str, List[Dict]] = field(default_factory=dict)
    team_mapping: Dict[int, str] = field(default_factory=dict)
    game_info: Dict = field(default_factory=dict)

    def validate_completeness(self) -> List[str]:
        """Validate all entities are present and complete"""
        errors = []

        if self.unique_players is None or len(self.unique_players) == 0:
            errors.append("unique_players is empty")

        if len(self.starters) == 0:
            errors.append("No starters defined")

        if len(self.team_mapping) == 0:
            errors.append("No team mapping defined")

        return errors

class RobustEntityExtractor:
    """Extract and validate canonical entities from NBA data"""

    def __init__(self, db_path: str):
        self.db_path = db_path
        self.conn = None
        self.validator = NBADataValidator()
        self.entities = GameEntities()

    def __enter__(self):
        self.conn = duckdb.connect(self.db_path)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()

    def _robust_drop_object(self, object_name: str) -> None:
        """Robustly drop any DuckDB object regardless of type (same as Step 2)"""
        try:
            self.conn.execute(f"DROP TABLE IF EXISTS {object_name}")
        except Exception:
            pass
            
        try:
            self.conn.execute(f"DROP VIEW IF EXISTS {object_name}")
        except Exception:
            pass
            
        try:
            self.conn.execute(f"DROP SEQUENCE IF EXISTS {object_name}")
        except Exception:
            pass

    def _get_object_type(self, object_name: str) -> Optional[str]:
        """Get the type of a DuckDB object if it exists"""
        try:
            result = self.conn.execute(f"""
                SELECT table_type 
                FROM information_schema.tables 
                WHERE table_name = '{object_name}'
            """).fetchall()
            
            if result:
                return result[0][0]
                
            # Check views separately
            result = self.conn.execute(f"""
                SELECT 'VIEW' as table_type
                FROM information_schema.views 
                WHERE table_name = '{object_name}'
            """).fetchall()
            
            if result:
                return 'VIEW'
                
        except Exception as e:
            logger.debug(f"Could not check object type for {object_name}: {e}")
            
        return None

    def extract_unique_players(self) -> ValidationResult:
        """Extract all unique active players with validation"""
        start_time = time.time()

        try:
            logger.info("Extracting unique players from box score...")

            # Get all active players from box score
            players_query = """
            SELECT 
                player_id,
                player_name,
                team_id,
                team_abbrev,
                is_home,
                is_starter,
                seconds_played,
                points,
                rebounds,
                assists,
                jersey_number
            FROM box_score
            WHERE seconds_played > 0
            ORDER BY team_id, seconds_played DESC
            """

            self.entities.unique_players = self.conn.execute(players_query).df()

            if len(self.entities.unique_players) == 0:
                return ValidationResult(
                    step_name="Extract Unique Players",
                    passed=False,
                    details="No players found in box score",
                    processing_time=time.time() - start_time
                )

            warnings = []

            # Validate player data quality
            null_names = self.entities.unique_players['player_name'].isnull().sum()
            if null_names > 0:
                warnings.append(f"{null_names} players have null names")

            # Validate team distribution
            team_counts = self.entities.unique_players['team_abbrev'].value_counts()
            for team, count in team_counts.items():
                if count < 8:
                    warnings.append(f"Team {team} has only {count} players (minimum 8 expected)")
                elif count > 20:
                    warnings.append(f"Team {team} has {count} players (unusually high)")

            # Check for duplicate player IDs
            duplicate_players = self.entities.unique_players['player_id'].duplicated().sum()
            if duplicate_players > 0:
                warnings.append(f"{duplicate_players} duplicate player IDs found")
                # Remove duplicates, keeping first occurrence
                self.entities.unique_players = self.entities.unique_players.drop_duplicates(
                    subset=['player_id'], keep='first'
                )

            details = f"Extracted {len(self.entities.unique_players)} players across {len(team_counts)} teams"
            details += f". Team distribution: {dict(team_counts)}"

            return ValidationResult(
                step_name="Extract Unique Players",
                passed=True,
                details=details,
                data_count=len(self.entities.unique_players),
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Extract Unique Players",
                passed=False,
                details=f"Error extracting players: {str(e)}",
                processing_time=time.time() - start_time
            )

    def extract_starters(self) -> ValidationResult:
        """Extract starting lineups with strict validation"""
        start_time = time.time()

        try:
            logger.info("Extracting starting lineups...")

            # Get starters from box score (gs=1 as specified in requirements)
            starters_query = """
            SELECT 
                team_id,
                team_abbrev,
                player_id,
                player_name,
                jersey_number,
                seconds_played,
                points
            FROM box_score
            WHERE is_starter = TRUE
            ORDER BY team_abbrev, seconds_played DESC
            """

            starters_df = self.conn.execute(starters_query).df()

            if len(starters_df) == 0:
                return ValidationResult(
                    step_name="Extract Starters",
                    passed=False,
                    details="No starters found in box score",
                    processing_time=time.time() - start_time
                )

            warnings = []
            teams_with_issues = []

            # Process starters by team
            for team_abbrev in starters_df['team_abbrev'].unique():
                team_starters = starters_df[starters_df['team_abbrev'] == team_abbrev]

                # Validate exactly 5 starters per team (as specified in requirements)
                if len(team_starters) != 5:
                    warnings.append(f"Team {team_abbrev} has {len(team_starters)} starters (expected 5)")
                    teams_with_issues.append(team_abbrev)

                    # If we don't have exactly 5, try to fix it
                    if len(team_starters) < 5:
                        # Get additional players from the same team
                        additional_players = self.conn.execute(f"""
                            SELECT player_id, player_name, jersey_number, seconds_played, points
                            FROM box_score 
                            WHERE team_abbrev = '{team_abbrev}' 
                            AND is_starter = FALSE
                            AND seconds_played > 0
                            ORDER BY seconds_played DESC
                            LIMIT {5 - len(team_starters)}
                        """).df()

                        if len(additional_players) > 0:
                            # Add team info to additional players
                            team_info = team_starters.iloc[0][['team_id', 'team_abbrev']]
                            for col in ['team_id', 'team_abbrev']:
                                additional_players[col] = team_info[col]

                            # Combine with existing starters
                            team_starters = pd.concat([team_starters, additional_players], ignore_index=True)
                            warnings.append(f"Added {len(additional_players)} non-starters to {team_abbrev} lineup")

                    elif len(team_starters) > 5:
                        # Keep top 5 by playing time
                        team_starters = team_starters.head(5)
                        warnings.append(f"Reduced {team_abbrev} starters to top 5 by playing time")

                # Create starters list for this team
                starters_list = []
                for i, (_, player) in enumerate(team_starters.iterrows()):
                    starters_list.append({
                        'player_id': int(player['player_id']),
                        'player_name': player['player_name'],
                        'jersey_number': int(player['jersey_number']) if pd.notna(player['jersey_number']) else None,
                        'position': f"P{i+1}",  # Generic position since we don't have actual positions
                        'seconds_played': int(player['seconds_played']),
                        'points': int(player['points'])
                    })

                # Store starters for this team
                self.entities.starters[team_abbrev] = starters_list
                self.entities.starters[f"{team_abbrev}_ids"] = tuple(sorted(p['player_id'] for p in starters_list))

                logger.info(f"Team {team_abbrev} starters: {[p['player_name'] for p in starters_list]}")

            total_starters = sum(len(v) for k, v in self.entities.starters.items() if isinstance(v, list))

            details = f"Extracted starters for {len([k for k in self.entities.starters if not k.endswith('_ids')])} teams"
            details += f", {total_starters} total starters"

            if teams_with_issues:
                details += f". Issues resolved for: {', '.join(teams_with_issues)}"

            return ValidationResult(
                step_name="Extract Starters",
                passed=len(teams_with_issues) == 0,
                details=details,
                data_count=total_starters,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Extract Starters",
                passed=False,
                details=f"Error extracting starters: {str(e)}",
                processing_time=time.time() - start_time
            )

    def extract_team_mapping(self) -> ValidationResult:
        """Create team ID → abbreviation mapping with home/away flags"""
        start_time = time.time()

        try:
            logger.info("Creating team mapping...")

            # Get team information from box score
            team_query = """
            SELECT DISTINCT
                team_id,
                team_abbrev,
                is_home
            FROM box_score
            ORDER BY team_id
            """

            team_df = self.conn.execute(team_query).df()

            if len(team_df) == 0:
                return ValidationResult(
                    step_name="Extract Team Mapping",
                    passed=False,
                    details="No teams found in box score",
                    processing_time=time.time() - start_time
                )

            warnings = []

            # Build team mapping
            for _, row in team_df.iterrows():
                self.entities.team_mapping[int(row['team_id'])] = row['team_abbrev']

            # Identify home and away teams
            home_teams = team_df[team_df['is_home'] == True]['team_abbrev'].tolist()
            away_teams = team_df[team_df['is_home'] == False]['team_abbrev'].tolist()

            # Validate exactly one home and one away team
            if len(home_teams) != 1 or len(away_teams) != 1:
                warnings.append(f"Invalid home/away setup: home={home_teams}, away={away_teams}")
            else:
                self.entities.team_mapping['home_team'] = home_teams[0]
                self.entities.team_mapping['away_team'] = away_teams[0]

            # Validate expected teams (based on file name HOU-DAL)
            expected_teams = {'HOU', 'DAL'}
            actual_teams = set(team_df['team_abbrev'])

            if expected_teams != actual_teams:
                warnings.append(f"Expected teams {expected_teams}, found {actual_teams}")

            # Validate expected home/away (DAL should be home based on file naming convention)
            if home_teams and home_teams[0] != 'DAL':
                warnings.append(f"Expected DAL to be home team, but {home_teams[0]} is home")

            if away_teams and away_teams[0] != 'HOU':
                warnings.append(f"Expected HOU to be away team, but {away_teams[0]} is away")

            # Create reverse mapping for convenience
            team_id_to_abbrev = {k: v for k, v in self.entities.team_mapping.items() if isinstance(k, int)}

            details = f"Created mapping for {len(team_id_to_abbrev)} teams: {team_id_to_abbrev}"
            if 'home_team' in self.entities.team_mapping:
                details += f". Home: {self.entities.team_mapping['home_team']}, Away: {self.entities.team_mapping['away_team']}"

            return ValidationResult(
                step_name="Extract Team Mapping",
                passed=len(warnings) == 0,
                details=details,
                data_count=len(team_id_to_abbrev),
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Extract Team Mapping",
                passed=False,
                details=f"Error creating team mapping: {str(e)}",
                processing_time=time.time() - start_time
            )

    def extract_game_info(self) -> ValidationResult:
        """Extract basic game information and metadata"""
        start_time = time.time()

        try:
            logger.info("Extracting game information...")

            # Get game info from box score
            game_info_query = """
            SELECT 
                COUNT(DISTINCT team_id) as num_teams,
                COUNT(*) as total_players,
                COUNT(CASE WHEN is_starter THEN 1 END) as total_starters,
                SUM(seconds_played) as total_seconds_played,
                SUM(points) as total_points,
                MIN(team_id) as team1_id,
                MAX(team_id) as team2_id
            FROM box_score
            """

            game_info = self.conn.execute(game_info_query).df().iloc[0].to_dict()

            # Get team names
            teams = [self.entities.team_mapping[int(game_info['team1_id'])], 
                    self.entities.team_mapping[int(game_info['team2_id'])]]

            self.entities.game_info = {
                'num_teams': int(game_info['num_teams']),
                'total_players': int(game_info['total_players']),
                'total_starters': int(game_info['total_starters']),
                'total_seconds_played': int(game_info['total_seconds_played']),
                'total_points': int(game_info['total_points']),
                'teams': teams,
                'matchup': f"{teams[1]} @ {teams[0]}" if 'home_team' in self.entities.team_mapping else f"{teams[0]} vs {teams[1]}"
            }

            warnings = []

            # Validate game info
            if game_info['num_teams'] != 2:
                warnings.append(f"Expected 2 teams, found {game_info['num_teams']}")

            if game_info['total_starters'] != 10:
                warnings.append(f"Expected 10 starters, found {game_info['total_starters']}")

            if game_info['total_players'] < 16:
                warnings.append(f"Only {game_info['total_players']} players found (minimum 16 expected)")

            details = f"Game: {self.entities.game_info['matchup']}, {game_info['total_players']} players, {game_info['total_starters']} starters"

            return ValidationResult(
                step_name="Extract Game Info",
                passed=len(warnings) == 0,
                details=details,
                data_count=1,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Extract Game Info",
                passed=False,
                details=f"Error extracting game info: {str(e)}",
                processing_time=time.time() - start_time
            )

    def create_canonical_tables(self) -> ValidationResult:
        """Create optimized tables for canonical entities"""
        start_time = time.time()

        try:
            logger.info("Creating canonical entity tables...")

            # Create canonical players table with robust object handling
            self._robust_drop_object("canonical_players")
            self.conn.execute("""
            CREATE VIEW canonical_players AS
            SELECT 
                player_id,
                player_name,
                team_id,
                team_abbrev,
                is_home,
                is_starter,
                seconds_played,
                points,
                rebounds,
                assists,
                jersey_number
            FROM box_score
            ORDER BY team_abbrev, seconds_played DESC
            """)

            # Create canonical starters table
            starters_data = []
            for team_abbrev, starters_list in self.entities.starters.items():
                if isinstance(starters_list, list):  # Skip _ids entries
                    team_id = None
                    for tid, abbrev in self.entities.team_mapping.items():
                        if isinstance(tid, int) and abbrev == team_abbrev:
                            team_id = tid
                            break

                    for i, starter in enumerate(starters_list):
                        starters_data.append({
                            'team_id': team_id,
                            'team_abbrev': team_abbrev,
                            'lineup_position': i + 1,
                            'player_id': starter['player_id'],
                            'player_name': starter['player_name'],
                            'jersey_number': starter['jersey_number'],
                            'position': starter['position'],
                            'seconds_played': starter['seconds_played'],
                            'points': starter['points']
                        })

            if starters_data:
                # Robust object handling
                self._robust_drop_object("canonical_starters")
                
                starters_df = pd.DataFrame(starters_data)
                self.conn.register("starters_temp", starters_df)

                self.conn.execute("""
                CREATE TABLE canonical_starters AS
                SELECT * FROM starters_temp
                ORDER BY team_id, lineup_position
                """)

                self.conn.execute("DROP VIEW IF EXISTS starters_temp")

                # Create indexes for performance with error handling
                try:
                    self.conn.execute("CREATE INDEX IF NOT EXISTS idx_canonical_starters_team ON canonical_starters(team_id)")
                    self.conn.execute("CREATE INDEX IF NOT EXISTS idx_canonical_starters_player ON canonical_starters(player_id)")
                except Exception as e:
                    logger.warning(f"Could not create starter indexes: {e}")

            # Create team mapping table
            team_data = []
            for team_id, team_abbrev in self.entities.team_mapping.items():
                if isinstance(team_id, int):  # Skip special keys like 'home_team'
                    is_home = team_abbrev == self.entities.team_mapping.get('home_team', '')
                    team_data.append({
                        'team_id': team_id,
                        'team_abbrev': team_abbrev,
                        'is_home': is_home
                    })

            if team_data:
                # Robust object handling
                self._robust_drop_object("canonical_teams")
                
                teams_df = pd.DataFrame(team_data)
                self.conn.register("teams_temp", teams_df)

                self.conn.execute("""
                CREATE TABLE canonical_teams AS
                SELECT * FROM teams_temp
                ORDER BY team_id
                """)

                self.conn.execute("DROP VIEW IF EXISTS teams_temp")

            details = f"Created canonical tables: players (view), starters ({len(starters_data)}), teams ({len(team_data)})"

            return ValidationResult(
                step_name="Create Canonical Tables",
                passed=True,
                details=details,
                data_count=len(starters_data) + len(team_data),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Create Canonical Tables",
                passed=False,
                details=f"Error creating canonical tables: {str(e)}",
                processing_time=time.time() - start_time
            )

    def validate_entity_completeness(self) -> ValidationResult:
        """Final validation that all entities are complete and consistent"""
        start_time = time.time()

        try:
            logger.info("Performing final entity validation...")

            # Use the GameEntities validation method
            entity_errors = self.entities.validate_completeness()

            warnings = []

            # Cross-validate entities
            if self.entities.unique_players is not None and len(self.entities.starters) > 0:
                # Check all starters are in unique_players
                all_player_ids = set(self.entities.unique_players['player_id'])

                for team, starters_list in self.entities.starters.items():
                    if isinstance(starters_list, list):
                        for starter in starters_list:
                            if starter['player_id'] not in all_player_ids:
                                warnings.append(f"Starter {starter['player_name']} not found in unique_players")

            # Validate team consistency
            if self.entities.unique_players is not None:
                player_teams = set(self.entities.unique_players['team_id'])
                mapping_teams = {k for k in self.entities.team_mapping.keys() if isinstance(k, int)}

                if player_teams != mapping_teams:
                    warnings.append(f"Team ID mismatch: players have {player_teams}, mapping has {mapping_teams}")

            # Check starter counts
            starter_counts = {}
            for team, starters_list in self.entities.starters.items():
                if isinstance(starters_list, list):
                    starter_counts[team] = len(starters_list)

            for team, count in starter_counts.items():
                if count != 5:
                    warnings.append(f"Team {team} has {count} starters (expected 5)")

            passed = len(entity_errors) == 0 and all(count == 5 for count in starter_counts.values())

            details = f"Entity validation: {len(entity_errors)} errors, {len(warnings)} warnings"
            details += f". Starter counts: {starter_counts}"

            if entity_errors:
                details += f" - Errors: {', '.join(entity_errors)}"

            return ValidationResult(
                step_name="Entity Completeness",
                passed=passed,
                details=details,
                processing_time=time.time() - start_time,
                warnings=warnings + entity_errors
            )

        except Exception as e:
            return ValidationResult(
                step_name="Entity Completeness",
                passed=False,
                details=f"Error validating entities: {str(e)}",
                processing_time=time.time() - start_time
            )

    def print_entities_summary(self):
        """Print comprehensive summary of extracted entities"""
        print("\n" + "="*80)
        print("ROBUST NBA ENTITY EXTRACTION SUMMARY")
        print("="*80)

        # Game info
        if self.entities.game_info:
            print(f"🏀 GAME: {self.entities.game_info.get('matchup', 'Unknown')}")
            print(f"   Teams: {', '.join(self.entities.game_info.get('teams', []))}")
            print(f"   Players: {self.entities.game_info.get('total_players', 0)}")
            print(f"   Starters: {self.entities.game_info.get('total_starters', 0)}")
            print()

        # Players summary
        if self.entities.unique_players is not None:
            print("👥 PLAYERS BY TEAM:")
            for team in self.entities.unique_players['team_abbrev'].unique():
                team_players = self.entities.unique_players[self.entities.unique_players['team_abbrev'] == team]
                starters = team_players[team_players['is_starter'] == True]
                print(f"   {team}: {len(team_players)} players ({len(starters)} starters)")
            print()

        # Starters detail
        print("🏆 STARTING LINEUPS:")
        for team, starters_list in self.entities.starters.items():
            if isinstance(starters_list, list):
                print(f"   {team}:")
                for i, starter in enumerate(starters_list):
                    jersey = starter.get('jersey_number', 'N/A')
                    seconds = starter.get('seconds_played', 0)
                    points = starter.get('points', 0)
                    print(f"     {i+1}. {starter['player_name']} (#{jersey}, {seconds//60}:{seconds%60:02d}, {points}pts)")
        print()

        # Team mapping
        print("🏟️  TEAM MAPPING:")
        for team_id, team_abbrev in self.entities.team_mapping.items():
            if isinstance(team_id, int):
                home_away = "🏠" if team_abbrev == self.entities.team_mapping.get('home_team') else "✈️"
                print(f"   {team_id} → {team_abbrev} {home_away}")

        print("="*80)

def extract_all_entities_robust(db_path: str = "mavs_enhanced.duckdb") -> Tuple[bool, GameEntities]:
    """Extract all canonical entities with robust validation"""

    print("🏀 NBA Pipeline - Robust Entity Extraction")
    print("="*50)

    with RobustEntityExtractor(db_path) as extractor:

        # Extract unique players
        logger.info("Step 3a: Extracting unique players...")
        result = extractor.extract_unique_players()
        extractor.validator.log_validation(result)

        if not result.passed:
            logger.error("❌ Failed to extract players - stopping")
            return False, extractor.entities

        # Extract starters (using actual gs=1 data from box score)
        logger.info("Step 3b: Extracting starters from box score...")
        result = extractor.extract_starters()
        extractor.validator.log_validation(result)

        if not result.passed:
            logger.error("❌ Failed to extract starters - stopping")
            return False, extractor.entities

        # Extract team mapping
        logger.info("Step 3c: Creating team mapping...")
        result = extractor.extract_team_mapping()
        extractor.validator.log_validation(result)

        # Extract game info
        logger.info("Step 3d: Extracting game info...")
        result = extractor.extract_game_info()
        extractor.validator.log_validation(result)

        # Create canonical tables
        logger.info("Step 3e: Creating canonical tables...")
        result = extractor.create_canonical_tables()
        extractor.validator.log_validation(result)

        # Final validation
        logger.info("Step 3f: Final validation...")
        result = extractor.validate_entity_completeness()
        extractor.validator.log_validation(result)

        # Print summary
        extractor.print_entities_summary()
        success = extractor.validator.print_validation_summary()

        return success, extractor.entities

# Example usage
if __name__ == "__main__":
    database_path = "mavs_enhanced.duckdb"

    success, entities = extract_all_entities_robust(database_path)

    if success:
        print("\n✅ Robust entity extraction completed successfully")
        print("🎯 Ready for lineup tracking and possession analysis")
    else:
        print("\n❌ Robust entity extraction failed")
        print("🔧 Review validation messages above")



Overwriting api/src/airflow_project/eda/data/nba_entities_extractor.py


In [31]:
%%writefile api/src/airflow_project/eda/data/nba_pbp_processor.py
# Step 4: Play-by-Play Processing & Lineup State Machine (Updated with Step 2 Integration)
"""
NBA Pipeline - Step 4: Process PBP Events & Track Lineups (UPDATED)
==================================================================

UPDATED to integrate Step 2 findings:
- Traditional Data-Driven Method: Follows raw substitution data strictly (3-6 man lineups)
- Enhanced Estimation Method: Uses intelligent inference to maintain 5-man lineups
- Both methods run in parallel to provide comparison and validation
- Comprehensive flagging system from Step 2 integrated
- Config-driven approach for easy switching between methods

Key Integration Points from Step 2:
1. Traditional method: msgType=8, playerId1=IN, playerId2=OUT, allows variable lineup sizes
2. Enhanced method: First-action rules, inactivity detection, always-5 enforcement
3. Comprehensive flagging and validation system
4. Both methods track the same events but with different lineup management strategies
"""
import os
import sys
# Ensure we're in the right directory
cwd = os.getcwd()
if not cwd.endswith("airflow_project"):
    os.chdir('api/src/airflow_project')
sys.path.insert(0, os.getcwd())

import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import time
import json
from typing import Dict, List, Tuple, Optional, Set, Any
from dataclasses import dataclass, field
from collections import defaultdict, deque

from eda.utils.nba_pipeline_analysis import NBADataValidator, ValidationResult
from eda.data.nba_entities_extractor import GameEntities

# Load configuration
try:
    from utils.config import (
        NBA_SUBSTITUTION_CONFIG,
        RIM_DISTANCE_FEET,
        COORDINATE_SCALE,
        MINIMUM_SECONDS_PLAYED,
        DUCKDB_PATH
    )
    CONFIG = NBA_SUBSTITUTION_CONFIG
    RIM_THRESHOLD = RIM_DISTANCE_FEET
    COORD_SCALE = COORDINATE_SCALE
    MIN_SECONDS = MINIMUM_SECONDS_PLAYED
    DB_PATH = str(DUCKDB_PATH)
except ImportError:
    logger.warning("Config not available, using defaults")
    CONFIG = {
        "starter_reset_periods": [1, 3],
        "msg_types": {"substitution": 8, "shot_made": 1, "shot_missed": 2, "rebound": 4},
        "one_direction": {"enabled": True, "appearance_via_last_name": True},
        "validation": {"validate_team_membership": True, "min_lineup_size": 5},
        "debug": {"log_all_substitutions": True}
    }
    RIM_THRESHOLD = 4.0
    COORD_SCALE = 10.0
    MIN_SECONDS = 30
    DB_PATH = "mavs_enhanced.duckdb"

logger = logging.getLogger(__name__)


@dataclass
class TraditionalLineupState:
    """Traditional data-driven lineup state - follows raw data strictly"""
    team_lineups: Dict[int, Set[int]] = field(default_factory=dict)
    period: int = 0
    flags: List[Dict] = field(default_factory=list)
    substitution_log: List[Dict] = field(default_factory=list)
    
    def add_flag(self, flag_type: str, team_id: int, player_id: int = None, details: str = ""):
        """Add a flag for data quality issues"""
        self.flags.append({
            'type': flag_type,
            'team_id': team_id,
            'player_id': player_id,
            'details': details,
            'period': self.period
        })


@dataclass
class EnhancedLineupState:
    """Enhanced lineup state with intelligent inference - maintains 5-man lineups"""
    team_lineups: Dict[int, Set[int]] = field(default_factory=dict)
    period: int = 0
    last_action_time: Dict[int, float] = field(default_factory=dict)
    recent_out: Dict[int, deque] = field(default_factory=dict)
    flags: List[Dict] = field(default_factory=list)
    substitution_log: List[Dict] = field(default_factory=list)
    first_action_events: List[Dict] = field(default_factory=list)
    auto_out_events: List[Dict] = field(default_factory=list)

    def __post_init__(self):
        for team_id in self.team_lineups.keys():
            self.recent_out[team_id] = deque(maxlen=10)
    
    def add_flag(self, flag_type: str, team_id: int, player_id: int = None, details: str = ""):
        """Add a flag for tracking intelligent inference actions"""
        self.flags.append({
            'type': flag_type,
                    'team_id': team_id,
            'player_id': player_id,
            'details': details,
            'period': self.period
        })


@dataclass  
class ProcessedEvent:
    """Represents a processed play-by-play event with context from both methods"""
    pbp_id: int
    period: int
    pbp_order: int
    wall_clock_int: int
    msg_type: int
    action_type: int = None
    description: str = ""
    off_team_id: int = None
    def_team_id: int = None
    player_id_1: int = None
    player_id_2: int = None
    player_id_3: int = None
    loc_x: int = None
    loc_y: int = None
    points: int = 0

    # Computed fields
    is_shot: bool = False
    is_rim_attempt: bool = False
    is_rim_make: bool = False
    distance_ft: float = None
    is_substitution: bool = False
    sub_out_player: int = None
    sub_in_player: int = None

    # Lineup context from BOTH methods
    traditional_off_lineup: Tuple[int, ...] = None
    traditional_def_lineup: Tuple[int, ...] = None
    enhanced_off_lineup: Tuple[int, ...] = None
    enhanced_def_lineup: Tuple[int, ...] = None


class PBPProcessor:
    """UPDATED: Integrated processor using both Step 2 methods"""

    def __init__(self, db_path: str = None, entities: GameEntities = None):
        """Initialize with both traditional and enhanced tracking methods"""
        self.db_path = db_path or DB_PATH
        self.conn = None
        self.entities = entities
        self.validator = NBADataValidator()

        # DUAL STATE TRACKING - Both methods run in parallel
        self.traditional_state = TraditionalLineupState()
        self.enhanced_state = EnhancedLineupState()
        
        self.processed_events: List[ProcessedEvent] = []

        # Build team rosters and reference data
        self.team_rosters = self._build_team_rosters_from_entities()
        self.player_names = self._build_player_names()
        self.team_names = self._build_team_names()
        
        # Statistics tracking
        self.traditional_stats = {
            'substitutions': 0, 'flags': 0, 'lineup_size_deviations': 0
        }
        self.enhanced_stats = {
            'substitutions': 0, 'first_actions': 0, 'auto_outs': 0, 'flags': 0
        }

    def _build_team_rosters_from_entities(self) -> Dict[int, Set[int]]:
        """Build complete team rosters from entities"""
        rosters = {}
        if hasattr(self.entities, 'unique_players') and self.entities.unique_players is not None:
            for _, player in self.entities.unique_players.iterrows():
                team_id = int(player['team_id'])
                player_id = int(player['player_id'])
                if team_id not in rosters:
                    rosters[team_id] = set()
                rosters[team_id].add(player_id)
        return rosters

    def _build_player_names(self) -> Dict[int, str]:
        """Build player ID to name mapping"""
        names = {}
        if hasattr(self.entities, 'unique_players') and self.entities.unique_players is not None:
            for _, player in self.entities.unique_players.iterrows():
                names[int(player['player_id'])] = str(player['player_name'])
        return names

    def _build_team_names(self) -> Dict[int, str]:
        """Build team ID to abbreviation mapping"""
        if hasattr(self.entities, 'team_mapping'):
            return {int(k): str(v) for k, v in self.entities.team_mapping.items() if isinstance(k, int)}
        return {}

    def __enter__(self):
        self.conn = duckdb.connect(self.db_path)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()

    def initialize_lineups(self) -> ValidationResult:
        """Initialize starting lineups for BOTH tracking methods"""
        start_time = time.time()

        try:
            logger.info("Initializing lineups for both tracking methods...")

            if not self.entities.starters:
                return ValidationResult(
                    step_name="Initialize Lineups",
                    passed=False,
                    details="No starting lineups available in entities",
                    processing_time=time.time() - start_time
                )

            warnings = []

            # Initialize both states with same starting lineups
            for team_abbrev, starters_list in self.entities.starters.items():
                if isinstance(starters_list, list):
                    # Find team ID
                    team_id = None
                    for tid, tabbrev in self.entities.team_mapping.items():
                        if isinstance(tid, int) and tabbrev == team_abbrev:
                            team_id = tid
                            break

                    if team_id is None:
                        warnings.append(f"Could not find team ID for {team_abbrev}")
                        continue

                    starter_ids = {starter['player_id'] for starter in starters_list}

                    if len(starter_ids) != 5:
                        warnings.append(f"Team {team_abbrev} has {len(starter_ids)} starters (expected 5)")

                    # Set for BOTH methods
                    self.traditional_state.team_lineups[team_id] = starter_ids.copy()
                    self.enhanced_state.team_lineups[team_id] = starter_ids.copy()
                    
                    # Initialize enhanced state tracking
                    self.enhanced_state.recent_out[team_id] = deque(maxlen=10)
                    for player_id in starter_ids:
                        self.enhanced_state.last_action_time[player_id] = 0.0

                    logger.info(f"Initialized {team_abbrev} starters: {sorted(starter_ids)}")

            details = f"Initialized lineups for both tracking methods: {len(self.traditional_state.team_lineups)} teams"

            return ValidationResult(
                step_name="Initialize Lineups",
                passed=True,
                details=details,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Initialize Lineups",
                passed=False,
                details=f"Error initializing lineups: {str(e)}",
                processing_time=time.time() - start_time
            )

    def load_pbp_events(self) -> ValidationResult:
        """Load PBP events with Step 2 integration"""
        start_time = time.time()

        try:
            logger.info("Loading PBP events with Step 2 classification...")

            # Use same canonical view approach from Step 2
            pbp_view = self._ensure_canonical_pbp_view()

            events_df = self.conn.execute(f"""
            SELECT 
                    pbp_id, period, pbp_order, wall_clock_int,
                    game_clock, description, msg_type, action_type,
                    off_team_id, def_team_id,
                    player_id_1, player_id_2, player_id_3,
                    loc_x, loc_y, points
            FROM {pbp_view}
            ORDER BY period, pbp_order, wall_clock_int
            """).df()

            if len(events_df) == 0:
                return ValidationResult(
                    step_name="Load PBP Events", 
                    passed=False,
                    details="No valid PBP events found",
                    processing_time=time.time() - start_time
                )

            # Convert to ProcessedEvent objects with Step 2 classification
            self.processed_events = []
            for _, row in events_df.iterrows():
                event = ProcessedEvent(
                    pbp_id=int(row['pbp_id']),
                    period=int(row['period']),
                    pbp_order=int(row['pbp_order']),
                    wall_clock_int=int(row['wall_clock_int']) if pd.notna(row['wall_clock_int']) else 0,
                    msg_type=int(row['msg_type']),
                    action_type=int(row['action_type']) if pd.notna(row['action_type']) else None,
                    description=str(row['description']) if pd.notna(row['description']) else "",
                    off_team_id=int(row['off_team_id']) if pd.notna(row['off_team_id']) else None,
                    def_team_id=int(row['def_team_id']) if pd.notna(row['def_team_id']) else None,
                    player_id_1=int(row['player_id_1']) if pd.notna(row['player_id_1']) else None,
                    player_id_2=int(row['player_id_2']) if pd.notna(row['player_id_2']) else None,
                    player_id_3=int(row['player_id_3']) if pd.notna(row['player_id_3']) else None,
                    loc_x=int(row['loc_x']) if pd.notna(row['loc_x']) and row['loc_x'] != 0 else None,
                    loc_y=int(row['loc_y']) if pd.notna(row['loc_y']) and row['loc_y'] != 0 else None,
                    points=int(row['points']) if pd.notna(row['points']) else 0
                )

                # Classify event using Step 2 logic
                self._classify_event_step2(event)
                self.processed_events.append(event)

            details = f"Loaded {len(self.processed_events)} events with Step 2 classification"

            return ValidationResult(
                step_name="Load PBP Events",
                passed=True,
                details=details,
                data_count=len(self.processed_events),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Load PBP Events",
                passed=False,
                details=f"Error loading events: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _ensure_canonical_pbp_view(self) -> str:
        """Create canonical PBP view with robust column detection (no nulling)."""
        # Discover a pbp-like table (prefer enriched if present)
        tables = [r[0] for r in self.conn.execute("""
            SELECT table_name FROM information_schema.tables
            WHERE table_schema NOT IN ('information_schema','pg_catalog')
            ORDER BY table_name
        """).fetchall()]

        pbp_candidates = [t for t in tables if t.lower() in ("pbp_enriched", "pbp")]
        if pbp_candidates:
            pbp_table = pbp_candidates[0]
        else:
            # last resort heuristic
            pbp_table = next((t for t in tables if "pbp" in t.lower()), None) or "pbp"

        # Inspect columns to build a robust SELECT
        cols = {r[1].lower(): r[1] for r in self.conn.execute(f"PRAGMA table_info('{pbp_table}')").fetchall()}

        # Map required fields with safe coalesces
        period_col        = cols.get("period",        "period")
        pbp_order_col     = cols.get("pbp_order",     "pbp_order")
        wall_clock_col    = cols.get("wall_clock_int","wall_clock_int")
        game_clock_col    = cols.get("game_clock",    "game_clock")
        desc_col          = cols.get("description",   "description")
        msg_type_col      = cols.get("msg_type",      "msg_type")
        action_type_col   = cols.get("action_type",   "action_type")
        off_team_col      = cols.get("team_id_off",   cols.get("off_team_id", "team_id_off"))
        def_team_col      = cols.get("team_id_def",   cols.get("def_team_id", "team_id_def"))
        p1_col            = cols.get("player_id_1",   "player_id_1")
        p2_col            = cols.get("player_id_2",   "player_id_2")
        p3_col            = cols.get("player_id_3",   "player_id_3")
        loc_x_col         = cols.get("loc_x",         cols.get("x", "loc_x"))
        # Common alternates for loc_y
        loc_y_source      = cols.get("loc_y", cols.get("y", cols.get("pos_y")))
        # Common alternates for points
        points_source     = cols.get("points", cols.get("points_scored", cols.get("score_change")))

        # Build COALESCE expressions where necessary
        loc_y_expr   = (loc_y_source or "NULL")
        points_expr  = (points_source or "0")

        # pbp_id mapping (fallback to row_number if missing)
        pbp_id_expr = "pbp_id" if "pbp_id" in cols else f"row_number() over (order by {period_col}, {pbp_order_col}) as pbp_id"

        self.conn.execute("DROP VIEW IF EXISTS canonical_pbp")
        self.conn.execute(f"""
            CREATE VIEW canonical_pbp AS
            SELECT 
                {pbp_id_expr},
                {period_col}       AS period,
                {pbp_order_col}    AS pbp_order,
                {wall_clock_col}   AS wall_clock_int,
                {game_clock_col}   AS game_clock,
                {desc_col}         AS description,
                {msg_type_col}     AS msg_type,
                {action_type_col}  AS action_type,
                {off_team_col}     AS off_team_id,
                {def_team_col}     AS def_team_id,
                {p1_col}           AS player_id_1,
                {p2_col}           AS player_id_2,
                {p3_col}           AS player_id_3,
                {loc_x_col}        AS loc_x,
                {loc_y_expr}       AS loc_y,
                {points_expr}      AS points
            FROM {pbp_table}
            WHERE {off_team_col} IS NOT NULL AND {def_team_col} IS NOT NULL
        """)
        return "canonical_pbp"


    def _classify_event_step2(self, event: ProcessedEvent):
        """Classify events using Step 2 methodology"""
        # Shot classification
        if event.msg_type in [1, 2]:  # Made/Missed shots
            event.is_shot = True

            if event.loc_x is not None and event.loc_y is not None:
                event.distance_ft = np.sqrt(event.loc_x**2 + event.loc_y**2) / COORD_SCALE
                event.is_rim_attempt = event.distance_ft <= RIM_THRESHOLD
                event.is_rim_make = event.is_rim_attempt and event.msg_type == 1

        # Substitution classification - Step 2 methodology
        if event.msg_type == CONFIG["msg_types"]["substitution"]:
            event.is_substitution = True
            # Step 2 finding: playerId1 = IN, playerId2 = OUT for traditional
            event.sub_in_player = event.player_id_1
            event.sub_out_player = event.player_id_2

    def process_traditional_substitution(self, event: ProcessedEvent) -> bool:
        """Process substitution using TRADITIONAL DATA-DRIVEN method from Step 2"""
        if not event.is_substitution:
            return False

        in_player = event.sub_in_player
        out_player = event.sub_out_player

        # Traditional method: follow data strictly, allow variable lineup sizes
        try:
            # Determine team (prefer out_player's current team)
            team_id = None
            for tid, lineup in self.traditional_state.team_lineups.items():
                if out_player and out_player in lineup:
                    team_id = tid
                    break
            
            if not team_id:
                # Fallback to roster check
                for tid, roster in self.team_rosters.items():
                    if (out_player and out_player in roster) or (in_player and in_player in roster):
                        team_id = tid
                        break

            if not team_id:
                self.traditional_state.add_flag(
                    "unknown_team_substitution", 0, in_player,
                    f"Cannot determine team for sub: {out_player} -> {in_player}"
                )
                return False

            lineup = self.traditional_state.team_lineups[team_id]

            # Traditional method flags (from Step 2)
            if out_player and out_player not in lineup:
                self.traditional_state.add_flag(
                    "sub_out_player_not_in_lineup", team_id, out_player,
                    f"OUT player {out_player} not in current lineup"
                )

            if in_player and in_player in lineup:
                self.traditional_state.add_flag(
                    "sub_in_player_already_in_lineup", team_id, in_player,
                    f"IN player {in_player} already in lineup"
                )

            # Execute substitution strictly as recorded
            if out_player and out_player in lineup:
                lineup.remove(out_player)
            if in_player:
                lineup.add(in_player)

            # Flag lineup size deviations (Step 2 finding: 3-6 man lineups)
            if len(lineup) != 5:
                self.traditional_state.add_flag(
                    "lineup_size_deviation", team_id, None,
                    f"Lineup size {len(lineup)}/5 after substitution"
                )
                self.traditional_stats['lineup_size_deviations'] += 1

            self.traditional_stats['substitutions'] += 1
            self.traditional_state.substitution_log.append({
                'period': event.period,
                'team_id': team_id,
                'in_player': in_player,
                'out_player': out_player,
                'lineup_size_after': len(lineup)
            })

            return True

        except Exception as e:
            self.traditional_state.add_flag(
                "substitution_error", team_id or 0, None, f"Error: {str(e)}"
            )
            return False

    def process_enhanced_substitution(self, event: ProcessedEvent, current_time: float) -> bool:
        """Process substitution using ENHANCED method from Step 2"""
        if not event.is_substitution:
            return False

        in_player = event.sub_in_player
        out_player = event.sub_out_player

        # Enhanced method: intelligent inference to maintain 5-man lineups
        try:
            # Determine team with fallbacks
            team_id = self._determine_team_enhanced(in_player, out_player, event)
            if not team_id:
                return False

            lineup = self.enhanced_state.team_lineups[team_id]

            # Enhanced method: prepare lineup for substitution
            if out_player and out_player not in lineup:
                # Try to find and move player
                for other_tid, other_lineup in self.enhanced_state.team_lineups.items():
                    if other_tid != team_id and out_player in other_lineup:
                        other_lineup.remove(out_player)
                        lineup.add(out_player)
                        self.enhanced_state.add_flag(
                            "moved_player_between_teams", team_id, out_player,
                            f"Moved OUT player from team {other_tid} to {team_id}"
                        )
                        break

            if in_player and in_player in lineup:
                # Remove duplicate
                lineup.remove(in_player)
                self.enhanced_state.add_flag(
                    "removed_duplicate_in_player", team_id, in_player,
                    "Removed duplicate IN player"
                )

            # Remove from other teams
            for other_tid, other_lineup in self.enhanced_state.team_lineups.items():
                if other_tid != team_id and in_player and in_player in other_lineup:
                    other_lineup.remove(in_player)

            # Execute substitution
            if out_player and out_player in lineup:
                lineup.remove(out_player)
                self.enhanced_state.recent_out[team_id].append(out_player)

            if in_player:
                lineup.add(in_player)

            # Enhanced method: ensure exactly 5 players
            self._ensure_five_players_enhanced(team_id, current_time)

            self.enhanced_stats['substitutions'] += 1
            self.enhanced_state.substitution_log.append({
                'period': event.period,
                'team_id': team_id,
                'in_player': in_player,
                'out_player': out_player,
                'lineup_size_after': len(self.enhanced_state.team_lineups[team_id])
            })

            return True

        except Exception as e:
            self.enhanced_state.add_flag(
                "substitution_error", team_id or 0, None, f"Error: {str(e)}"
            )
            return False

    def _determine_team_enhanced(self, in_player: int, out_player: int, event) -> Optional[int]:
        """Enhanced team determination with multiple fallbacks"""
        # Check current lineups
        for team_id, lineup in self.enhanced_state.team_lineups.items():
            if out_player and out_player in lineup:
                return team_id

        # Check rosters
        for team_id, roster in self.team_rosters.items():
            if (out_player and out_player in roster) or (in_player and in_player in roster):
                    return team_id

        return None

    def _ensure_five_players_enhanced(self, team_id: int, current_time: float):
        """Ensure exactly 5 players using Enhanced method logic from Step 2"""
        lineup = self.enhanced_state.team_lineups[team_id]

        # Remove excess players (auto-out logic)
        while len(lineup) > 5:
            # Find least active player for auto-out
            candidate = None
            max_idle = -1
            
            for player_id in lineup:
                idle_time = current_time - self.enhanced_state.last_action_time.get(player_id, 0)
                if idle_time > max_idle:
                    max_idle = idle_time
                    candidate = player_id

            if candidate:
                lineup.remove(candidate)
                self.enhanced_state.recent_out[team_id].append(candidate)
                self.enhanced_state.add_flag(
                    "auto_out_excess_player", team_id, candidate,
                    f"Auto-out due to excess players (idle: {max_idle:.1f}s)"
                )
                self.enhanced_stats['auto_outs'] += 1

        # Add players if under 5
        if len(lineup) < 5:
            available = self.team_rosters.get(team_id, set()) - lineup
            # Prefer recently out players
            recent = [p for p in self.enhanced_state.recent_out[team_id] if p in available]
            
            for player_id in (recent + list(available))[:5-len(lineup)]:
                lineup.add(player_id)
                self.enhanced_state.add_flag(
                    "auto_in_fill_lineup", team_id, player_id,
                    "Auto-in to fill lineup to 5 players"
                )

    def handle_first_action_events(self, event: ProcessedEvent, current_time: float):
        """Handle first-action events (Reed Sheppard case) from Step 2"""
        if event.msg_type not in [1, 2, 4, 5, 6]:  # Only for action events
            return

        action_player = event.player_id_1
        if not action_player:
            return

        # Check if player is in any lineup
        player_team = None
        player_in_lineup = False

        for team_id, lineup in self.enhanced_state.team_lineups.items():
            if action_player in lineup:
                player_in_lineup = True
                break
            elif action_player in self.team_rosters.get(team_id, set()):
                player_team = team_id

        # First-action injection (Reed Sheppard case)
        if not player_in_lineup and player_team:
            self.enhanced_state.team_lineups[player_team].add(action_player)
            self.enhanced_state.add_flag(
                "first_action_injection", player_team, action_player,
                f"First-action injection: {event.description}"
            )
            self.enhanced_stats['first_actions'] += 1
            self.enhanced_state.first_action_events.append({
                'period': event.period,
                'player_id': action_player,
                'team_id': player_team,
                'event_type': event.msg_type,
                'description': event.description
            })
            
            # Ensure 5 players after injection
            self._ensure_five_players_enhanced(player_team, current_time)

        # Update activity time
        if action_player:
            self.enhanced_state.last_action_time[action_player] = current_time

    def process_all_events(self) -> ValidationResult:
        """Process all events with BOTH tracking methods from Step 2"""
        start_time = time.time()

        try:
            logger.info(f"Processing {len(self.processed_events)} events with both methods...")

            periods_seen = set()
            current_time = 0.0

            for i, event in enumerate(self.processed_events):
                current_time = float(event.wall_clock_int)

                # Handle period transitions
                if event.period not in periods_seen:
                    periods_seen.add(event.period)
                    logger.info(f"Processing period {event.period}")

                    # Reset periods for enhanced method (Step 2 logic)
                    if event.period in CONFIG["starter_reset_periods"]:
                        self._reset_to_starters_enhanced()

                    # Update period in states
                    self.traditional_state.period = event.period
                    self.enhanced_state.period = event.period

                # Capture lineup context BEFORE processing
                event.traditional_off_lineup = tuple(sorted(self.traditional_state.team_lineups.get(event.off_team_id, set())))
                event.traditional_def_lineup = tuple(sorted(self.traditional_state.team_lineups.get(event.def_team_id, set())))
                event.enhanced_off_lineup = tuple(sorted(self.enhanced_state.team_lineups.get(event.off_team_id, set())))
                event.enhanced_def_lineup = tuple(sorted(self.enhanced_state.team_lineups.get(event.def_team_id, set())))

                # Process substitutions with BOTH methods
                if event.is_substitution:
                    traditional_success = self.process_traditional_substitution(event)
                    enhanced_success = self.process_enhanced_substitution(event, current_time)

                    if CONFIG["debug"]["log_all_substitutions"]:
                        logger.info(f"Substitution P{event.period}: Traditional={traditional_success}, Enhanced={enhanced_success}")

                # Handle first-action events (Enhanced method only)
                self.handle_first_action_events(event, current_time)

            # Collect final statistics
            self.traditional_stats['flags'] = len(self.traditional_state.flags)
            self.enhanced_stats['flags'] = len(self.enhanced_state.flags)

            # Calculate lineup size distribution for traditional method
            traditional_sizes = defaultdict(int)
            for team_lineup in self.traditional_state.team_lineups.values():
                traditional_sizes[len(team_lineup)] += 1

            details = f"Processed {len(self.processed_events)} events. "
            details += f"Traditional: {self.traditional_stats['substitutions']} subs, {self.traditional_stats['flags']} flags, {self.traditional_stats['lineup_size_deviations']} size deviations. "
            details += f"Enhanced: {self.enhanced_stats['substitutions']} subs, {self.enhanced_stats['first_actions']} first-actions, {self.enhanced_stats['auto_outs']} auto-outs."

            return ValidationResult(
                step_name="Process All Events",
                passed=True,
                details=details,
                data_count=len(self.processed_events),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Process All Events",
                passed=False,
                details=f"Error processing events: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _reset_to_starters_enhanced(self):
        """Reset to starters for enhanced method (Q1, Q3)"""
        if hasattr(self.entities, 'starters'):
            for team_abbrev, starters_list in self.entities.starters.items():
                if isinstance(starters_list, list):
                    team_id = None
                    for tid, tabbrev in self.entities.team_mapping.items():
                        if isinstance(tid, int) and tabbrev == team_abbrev:
                            team_id = tid
                            break
                    
                    if team_id:
                        starter_ids = {starter['player_id'] for starter in starters_list}
                        self.enhanced_state.team_lineups[team_id] = starter_ids.copy()



    def _step4_required_columns(self) -> Set[str]:
        """
        The required dual-method lineup columns that Step 5 relies on.
        """
        return {
            "traditional_off_lineup", "traditional_def_lineup",
            "enhanced_off_lineup", "enhanced_def_lineup"
        }

    def _write_step4_contract_stamp(self, table_name: str) -> None:
        """
        Write a versioned contract stamp indicating Step 4 produced the expected schema.
        This does not alter data; it records meta only.
        """
        try:
            # Discover columns & row count of the output table
            cols = [r[1] for r in self.conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()]
            row_count = self.conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]

            # Create contract table if missing
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS pipeline_contract (
                    component    VARCHAR,
                    version      VARCHAR,
                    table_name   VARCHAR,
                    columns_json VARCHAR,
                    row_count    BIGINT,
                    created_at   TIMESTAMP
                )
            """)

            # Insert a new stamp
            self.conn.execute("""
                INSERT INTO pipeline_contract(component, version, table_name, columns_json, row_count, created_at)
                VALUES (?, ?, ?, ?, ?, now())
            """, ("step4", "dual_lineups_v1", table_name, json.dumps(cols, ensure_ascii=True), row_count))

            logger.info(f"[CONTRACT] Step 4 stamped version 'dual_lineups_v1' for table '{table_name}' ({row_count} rows)")
        except Exception as e:
            logger.warning(f"[CONTRACT] Failed to write Step 4 contract stamp: {e}")

    def validate_step4_schema(self) -> bool:
        """
        Validate that step4_processed_events contains the expected dual-method columns.
        Logs full column list and returns True/False.
        """
        try:
            cols = [r[1] for r in self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()]
            required = self._step4_required_columns()
            ok = required.issubset(set(cols))
            logger.info(f"[VALIDATE] Step 4 schema columns: {cols}")
            if not ok:
                missing = sorted(required - set(cols))
                logger.error(f"[VALIDATE] Step 4 schema missing required columns: {missing}")
            return ok
        except Exception as e:
            logger.error(f"[VALIDATE] Error validating Step 4 schema: {e}")
            return False

    def create_step4_output_tables(self) -> ValidationResult:
        """Create Step 4 output tables integrating both methods (schema-safe)"""
        start_time = time.time()

        try:
            logger.info("Creating Step 4 output tables with both tracking methods...")

            # --- Hard drop both VIEW and TABLE (handles stale views/tables) ---
            for obj in ("step4_processed_events", "step4_traditional_flags", "step4_enhanced_flags"):
                try:
                    self.conn.execute(f"DROP VIEW IF EXISTS {obj}")
                except Exception:
                    pass
                try:
                    self.conn.execute(f"DROP TABLE IF EXISTS {obj}")
                except Exception:
                    pass

            # Build events dataframe
            events_data = []
            for event in self.processed_events:
                events_data.append({
                    'pbp_id': event.pbp_id,
                    'period': event.period,
                    'pbp_order': event.pbp_order,
                    'wall_clock_int': event.wall_clock_int,
                    'description': event.description,
                    'msg_type': event.msg_type,
                    'action_type': event.action_type,
                    'off_team_id': event.off_team_id,
                    'def_team_id': event.def_team_id,
                    'player_id_1': event.player_id_1,
                    'player_id_2': event.player_id_2,
                    'player_id_3': event.player_id_3,
                    'is_shot': bool(event.is_shot),
                    'is_rim_attempt': bool(event.is_rim_attempt),
                    'is_rim_make': bool(event.is_rim_make),
                    'distance_ft': float(event.distance_ft) if event.distance_ft is not None else None,
                    'is_substitution': bool(event.is_substitution),
                    'points': int(event.points) if event.points is not None else 0,
                    # Store lineups as ASCII JSON arrays for portability
                    'traditional_off_lineup': json.dumps(list(event.traditional_off_lineup), ensure_ascii=True) if event.traditional_off_lineup else None,
                    'traditional_def_lineup': json.dumps(list(event.traditional_def_lineup), ensure_ascii=True) if event.traditional_def_lineup else None,
                    'enhanced_off_lineup': json.dumps(list(event.enhanced_off_lineup), ensure_ascii=True) if event.enhanced_off_lineup else None,
                    'enhanced_def_lineup': json.dumps(list(event.enhanced_def_lineup), ensure_ascii=True) if event.enhanced_def_lineup else None
                })

            events_df = pd.DataFrame(events_data)

            # Persist processed events
            self.conn.register("events_temp", events_df)
            try:
                self.conn.execute("""
                    CREATE TABLE step4_processed_events AS
                    SELECT * FROM events_temp
                    ORDER BY period, pbp_order, wall_clock_int
                """)
            finally:
                self.conn.unregister("events_temp")

            # Traditional flags
            traditional_flags_df = pd.DataFrame(self.traditional_state.flags)
            if not traditional_flags_df.empty:
                self.conn.register("trad_flags_temp", traditional_flags_df)
                try:
                    self.conn.execute("CREATE TABLE step4_traditional_flags AS SELECT * FROM trad_flags_temp")
                finally:
                    self.conn.unregister("trad_flags_temp")

            # Enhanced flags
            enhanced_flags_df = pd.DataFrame(self.enhanced_state.flags)
            if not enhanced_flags_df.empty:
                self.conn.register("enh_flags_temp", enhanced_flags_df)
                try:
                    self.conn.execute("CREATE TABLE step4_enhanced_flags AS SELECT * FROM enh_flags_temp")
                finally:
                    self.conn.unregister("enh_flags_temp")

            # Method comparison (quick summary table)
            comparison_data = [{
                'method': 'Traditional',
                'substitutions_processed': self.traditional_stats['substitutions'],
                'flags_generated': self.traditional_stats['flags'],
                'lineup_size_deviations': self.traditional_stats['lineup_size_deviations'],
                'maintains_5_man_lineups': False
            }, {
                'method': 'Enhanced',
                'substitutions_processed': self.enhanced_stats['substitutions'],
                'flags_generated': self.enhanced_stats['flags'],
                'first_action_injections': self.enhanced_stats['first_actions'],
                'auto_out_corrections': self.enhanced_stats['auto_outs'],
                'maintains_5_man_lineups': True
            }]
            comparison_df = pd.DataFrame(comparison_data)
            self.conn.register("comp_temp", comparison_df)
            try:
                self.conn.execute("CREATE OR REPLACE TABLE step4_method_comparison AS SELECT * FROM comp_temp")
            finally:
                self.conn.unregister("comp_temp")

            # --- Post-create schema validation & contract stamp ---
            ok = self.validate_step4_schema()
            self._write_step4_contract_stamp("step4_processed_events")

            details = (
                f"Created Step 4 output tables: processed_events ({len(events_data)} rows), "
                f"traditional_flags ({len(traditional_flags_df)} rows), "
                f"enhanced_flags ({len(enhanced_flags_df)} rows), method_comparison"
            )
            return ValidationResult(
                step_name="Create Step 4 Output Tables",
                passed=ok,
                details=details if ok else details + " [SCHEMA INVALID]",
                data_count=len(events_data),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Create Step 4 Output Tables",
                passed=False,
                details=f"Error creating output tables: {str(e)}",
                processing_time=time.time() - start_time
            )


    def print_step4_summary(self):
        """Print Step 4 summary with both methods (ASCII only)"""
        print("\n" + "="*80)
        print("NBA PIPELINE - STEP 4 SUMMARY (INTEGRATED WITH STEP 2)")
        print("="*80)

        print("TRADITIONAL DATA-DRIVEN METHOD:")
        print(f"  Substitutions Processed: {self.traditional_stats['substitutions']}")
        print(f"  Flags Generated: {self.traditional_stats['flags']}")
        print(f"  Lineup Size Deviations: {self.traditional_stats['lineup_size_deviations']}")
        print("  Current Lineup Sizes:")
        for team_id, lineup in self.traditional_state.team_lineups.items():
            team_name = self.team_names.get(team_id, f"Team_{team_id}")
            print(f"    {team_name}: {len(lineup)} players")

        print("\nENHANCED ESTIMATION METHOD:")
        print(f"  Substitutions Processed: {self.enhanced_stats['substitutions']}")
        print(f"  First-Action Injections: {self.enhanced_stats['first_actions']}")
        print(f"  Auto-Out Corrections: {self.enhanced_stats['auto_outs']}")
        print(f"  Flags Generated: {self.enhanced_stats['flags']}")
        print("  Current Lineup Sizes:")
        for team_id, lineup in self.enhanced_state.team_lineups.items():
            team_name = self.team_names.get(team_id, f"Team_{team_id}")
            print(f"    {team_name}: {len(lineup)} players")

        print(f"\nTOTAL EVENTS PROCESSED: {len(self.processed_events)}")

        trad_correct = sum(1 for lineup in self.traditional_state.team_lineups.values() if len(lineup) == 5)
        enh_correct = sum(1 for lineup in self.enhanced_state.team_lineups.values() if len(lineup) == 5)
        total_lineups = max(1, len(self.traditional_state.team_lineups))

        print("LINEUP SIZE ACCURACY:")
        print(f"  Traditional: {trad_correct}/{total_lineups} teams have 5-man lineups "
            f"({trad_correct/total_lineups*100:.1f}%)")
        print(f"  Enhanced: {enh_correct}/{total_lineups} teams have 5-man lineups "
            f"({enh_correct/total_lineups*100:.1f}%)")
        print("="*80)



def process_pbp_with_step2_integration(db_path: str = None, 
                                      entities: GameEntities = None) -> Tuple[bool, PBPProcessor]:
    """Process PBP events using integrated Step 2 methods"""
    
    print("NBA Pipeline - Step 4: Integrated PBP Processing (Updated with Step 2)")
    print("="*75)

    if entities is None:
        logger.error("GameEntities required for PBP processing")
        return False, None

    with PBPProcessor(db_path, entities) as processor:

        # Initialize lineups for both methods
        logger.info("Step 4a: Initializing lineups for both tracking methods...")
        result = processor.initialize_lineups()
        processor.validator.log_validation(result)
        if not result.passed:
            logger.error("Failed to initialize lineups")
            return False, processor

        # Load PBP events with Step 2 classification
        logger.info("Step 4b: Loading PBP events with Step 2 classification...")
        result = processor.load_pbp_events()
        processor.validator.log_validation(result)
        if not result.passed:
            logger.error("Failed to load PBP events")
            return False, processor

        # Process all events with both methods
        logger.info("Step 4c: Processing events with both Traditional and Enhanced methods...")
        result = processor.process_all_events()
        processor.validator.log_validation(result)

        # Create output tables
        logger.info("Step 4d: Creating Step 4 output tables...")
        result = processor.create_step4_output_tables()
        print('output tables results===============', result)
        processor.validator.log_validation(result)

        # Print summary
        processor.print_step4_summary()

        success = processor.validator.print_validation_summary()
        return success, processor


# Example usage
if __name__ == "__main__":
    from eda.data.nba_entities_extractor import extract_all_entities_robust

    # Extract entities first
    entities_success, entities = extract_all_entities_robust()
    
    if entities_success:
        success, processor = process_pbp_with_step2_integration(entities=entities)
        
        if success:
            print("\n✅ Step 4 Complete: Integrated processing with Step 2 methods")
            print("🎯 Both Traditional and Enhanced methods available for comparison")
        else:
            print("\n❌ Step 4 Failed: Review validation messages")
    else:
        print("❌ Failed to get entities - cannot proceed")


Overwriting api/src/airflow_project/eda/data/nba_pbp_processor.py


Step 5: Possession Engine & Lineup Statistics

In [32]:
%%writefile api/src/airflow_project/eda/data/nba_possession_engine.py
# Step 5: Dual-Method Possession Engine & Lineup Statistics Calculation
"""
NBA Pipeline - UPDATED Step 5: Dual-Method Possession Engine & Statistics
=========================================================================

UPDATED to integrate Step 2 findings:
- Processes possessions using BOTH traditional and enhanced lineup methods
- Generates separate statistics for each method
- Includes comprehensive violation and validation reporting
- Config-driven approach for automation settings
- Exports both result sets for comparison

Key Integration Points from Step 2:
1. Uses traditional_lineup_state (variable lineup sizes, raw data adherence)
2. Uses enhanced_lineup_state (5-man lineups, intelligent inference)  
3. Generates violation reports for traditional method
4. Comprehensive method comparison and validation
5. Config-driven automation paths

The possession engine determines when possessions change hands based on:
- Made field goals
- Turnovers
- Defensive rebounds (after missed shots)
- Free throw sequences
"""
import os
import sys
# Ensure we're in the right directory
cwd = os.getcwd()
if not cwd.endswith("airflow_project"):
    os.chdir('api/src/airflow_project')
sys.path.insert(0, os.getcwd())

import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import time
from typing import Dict, List, Tuple, Optional, Set, Any, NamedTuple
from dataclasses import dataclass, field
from collections import defaultdict, Counter
import json

from eda.utils.nba_pipeline_analysis import NBADataValidator, ValidationResult
from eda.data.nba_entities_extractor import GameEntities
from eda.data.nba_pbp_processor import PBPProcessor, ProcessedEvent
import ast

# Load configuration
try:
    from utils.config import (
        NBA_SUBSTITUTION_CONFIG,
        RIM_DISTANCE_FEET,
        COORDINATE_SCALE,
        MINIMUM_SECONDS_PLAYED,
        DUCKDB_PATH,
        DUCKDB_DIR,
        EXPORTS_DIR
    )
    CONFIG = NBA_SUBSTITUTION_CONFIG
    RIM_THRESHOLD = RIM_DISTANCE_FEET
    COORD_SCALE = COORDINATE_SCALE
    MIN_SECONDS = MINIMUM_SECONDS_PLAYED
    DB_PATH = str(DUCKDB_PATH)
    EXPORT_DIR = EXPORTS_DIR
except ImportError:
    logger.warning("Config not available, using defaults")
    CONFIG = {"debug": {"log_all_substitutions": True}}
    RIM_THRESHOLD = 4.0
    COORD_SCALE = 10.0
    MIN_SECONDS = 30
    DB_PATH = "mavs_enhanced.duckdb"
    EXPORT_DIR = Path("exports")

logger = logging.getLogger(__name__)

class DualPossession(NamedTuple):
    """Represents a possession with both traditional and enhanced lineup contexts"""
    possession_id: int
    period: int
    start_pbp_order: int
    end_pbp_order: int
    off_team_id: int
    def_team_id: int

    # Traditional method lineups (may not be 5 players)
    traditional_off_lineup: Tuple[int, ...]
    traditional_def_lineup: Tuple[int, ...]

    # Enhanced method lineups (always 5 players)
    enhanced_off_lineup: Tuple[int, ...]
    enhanced_def_lineup: Tuple[int, ...]

    points_scored: int
    ended_by: str

@dataclass
class DualLineupStats:
    """Statistics for lineup with both method contexts"""
    team_id: int
    team_abbrev: str
    lineup_method: str  # 'traditional' or 'enhanced'
    player_ids: Tuple[int, ...]
    player_names: List[str]
    lineup_size: int  # Actual size (may vary for traditional)

    # Possession counts
    off_possessions: int = 0
    def_possessions: int = 0

    # Scoring
    points_for: int = 0
    points_against: int = 0

    # Ratings (per 100 possessions)
    off_rating: float = 0.0
    def_rating: float = 0.0
    net_rating: float = 0.0

    # Validation flags
    lineup_violations: List[str] = field(default_factory=list)

@dataclass  
class DualPlayerRimStats:
    """Player rim defense statistics with method context"""
    player_id: int
    player_name: str
    team_id: int
    team_abbrev: str
    method: str  # 'traditional' or 'enhanced'

    # Possession counts
    off_possessions: int = 0
    def_possessions: int = 0

    # Rim defense (when on court)
    opp_rim_attempts_on: int = 0
    opp_rim_makes_on: int = 0

    # Rim defense (when off court)  
    opp_rim_attempts_off: int = 0
    opp_rim_makes_off: int = 0

    # Calculated percentages
    opp_rim_fg_pct_on: float = None
    opp_rim_fg_pct_off: float = None
    rim_defense_on_off: float = None

class DualMethodPossessionEngine:
    """UPDATED: Possession engine that processes both traditional and enhanced methods"""

    def __init__(self, db_path: str = None, entities: GameEntities = None):
        self.db_path = db_path or DB_PATH
        self.conn = None
        self.entities = entities
        self.validator = NBADataValidator()

        # Dual possession tracking
        self.dual_possessions = []

        # Dual statistics containers
        self.traditional_lineup_stats = {}  # (team_id, lineup_tuple) -> DualLineupStats
        self.enhanced_lineup_stats = {}     # (team_id, lineup_tuple) -> DualLineupStats
        self.traditional_player_stats = {}  # player_id -> DualPlayerRimStats
        self.enhanced_player_stats = {}     # player_id -> DualPlayerRimStats

        # Violation tracking
        self.traditional_violations = []
        self.enhanced_violations = []

        # Method comparison metrics
        self.method_comparison = {}

        # Build entity mappings
        self.player_team = {}
        self.team_roster = {}
        self.team_abbrev = {}
        self.player_names = {}

        self._build_entity_mappings()


    def diagnose_pipeline_state(self) -> Dict[str, Any]:
        """Comprehensive diagnostic of pipeline state, with schema/NULL audits for step4 + contract stamp if present."""
        try:
            logger.info("=== PIPELINE DIAGNOSTIC ===")

            # All table names
            all_tables = self.conn.execute(
                "SELECT table_name FROM information_schema.tables ORDER BY table_name"
            ).fetchall()
            table_names = [t[0] for t in all_tables]

            diag = {
                "all_tables": table_names,
                "step_requirements": {
                    "step4_processed_events": "step4_processed_events" in table_names,
                    "traditional_lineup_state": "traditional_lineup_state" in table_names,
                    "enhanced_lineup_state": "enhanced_lineup_state" in table_names,
                    "traditional_lineup_flags": "traditional_lineup_flags" in table_names,
                    "enhanced_lineup_flags": "enhanced_lineup_flags" in table_names,
                },
                "alternative_tables": {
                    "step4_traditional_flags": "step4_traditional_flags" in table_names,
                    "step4_enhanced_flags": "step4_enhanced_flags" in table_names,
                    "processed_events": "processed_events" in table_names,
                    "traditional_violation_report": "traditional_violation_report" in table_names,
                    "enhanced_violation_report": "enhanced_violation_report" in table_names,
                },
                "table_counts": {},
                "sample_data": {},
                "step4_schema": {},
                "step4_null_audit": {},
                "contract": None,
            }

            # Counts & samples
            key_tables = [
                "step4_processed_events",
                "traditional_lineup_state",
                "enhanced_lineup_state",
                "traditional_lineup_flags",
                "enhanced_lineup_flags",
                "step4_traditional_flags",
                "step4_enhanced_flags",
                "traditional_violation_report",
                "enhanced_violation_report",
            ]
            for t in key_tables:
                if t in table_names:
                    try:
                        cnt = self.conn.execute(f"SELECT COUNT(*) FROM {t}").fetchone()[0]
                        diag["table_counts"][t] = cnt
                        if t in ("step4_processed_events", "traditional_lineup_state", "enhanced_lineup_state"):
                            sample = self.conn.execute(f"SELECT * FROM {t} LIMIT 3").df()
                            diag["sample_data"][t] = sample.to_dict("records") if not sample.empty else []
                    except Exception as e:
                        diag["table_counts"][t] = f"Error: {e}"

            # Deep audit of step4_processed_events schema + null counts
            if "step4_processed_events" in table_names:
                cols = self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()
                colnames = [c[1] for c in cols]
                diag["step4_schema"]["columns"] = colnames
                diag["step4_schema"]["has_legacy_lineups"] = ("off_lineup" in colnames and "def_lineup" in colnames)
                diag["step4_schema"]["has_traditional_lineups"] = (
                    "traditional_off_lineup" in colnames and "traditional_def_lineup" in colnames
                )
                diag["step4_schema"]["has_enhanced_lineups"] = (
                    "enhanced_off_lineup" in colnames and "enhanced_def_lineup" in colnames
                )
                diag["step4_schema"]["has_points"] = "points" in colnames
                diag["step4_schema"]["has_rim_flags"] = all(c in colnames for c in ("is_rim_attempt", "is_rim_make"))

                # Null audits (counts of non-null values)
                to_audit = [
                    "off_lineup", "def_lineup",
                    "traditional_off_lineup", "traditional_def_lineup",
                    "enhanced_off_lineup", "enhanced_def_lineup",
                    "points", "is_rim_attempt", "is_rim_make"
                ]
                for c in to_audit:
                    if c in colnames:
                        try:
                            n = self.conn.execute(
                                f"SELECT COUNT(*) FROM step4_processed_events WHERE {c} IS NOT NULL"
                            ).fetchone()[0]
                            diag["step4_null_audit"][c] = n
                        except Exception as e:
                            diag["step4_null_audit"][c] = f"Error: {e}"

            # Contract stamp if present
            if "pipeline_contract" in table_names:
                try:
                    dfc = self.conn.execute("""
                        SELECT *
                        FROM pipeline_contract
                        WHERE component='step4'
                        ORDER BY created_at DESC
                        LIMIT 1
                    """).df()
                    if not dfc.empty:
                        diag["contract"] = dfc.to_dict("records")[0]
                except Exception as e:
                    diag["contract"] = f"Error reading contract: {e}"

            logger.info(f"Diagnostic results: {diag}")
            return diag

        except Exception as e:
            logger.error(f"Error in pipeline diagnostic: {e}")
            return {"error": str(e)}



    def _build_entity_mappings(self):
        """Build entity mappings from provided entities"""
        if not self.entities or not hasattr(self.entities, 'unique_players'):
            return

        up = self.entities.unique_players
        if up is not None and not up.empty:
            # Player mappings
            for _, r in up.iterrows():
                pid = int(r["player_id"])
                tid = int(r["team_id"])
                self.player_team[pid] = tid
                self.player_names[pid] = str(r["player_name"])

                if tid not in self.team_roster:
                    self.team_roster[tid] = set()
                self.team_roster[tid].add(pid)

            # Team mappings
            for _, r in up.drop_duplicates("team_id").iterrows():
                tid = int(r["team_id"])
                self.team_abbrev[tid] = str(r["team_abbrev"])

    def _validate_lineup(self,
                        lineup: Optional[Tuple[int, ...]],
                        label: str,
                        poss_id: int,
                        method: str = "enhanced") -> None:
        """
        Validate lineup contents according to method rules.

        enhanced: exactly 5 unique players.
        traditional: non-empty, all unique (size may be != 5).
        """
        if lineup is None:
            raise AssertionError(f"[Possession {poss_id}] {label} lineup is None")

        if method == "enhanced":
            if len(lineup) != 5:
                raise AssertionError(
                    f"[Possession {poss_id}] {label} enhanced lineup len={len(lineup)} != 5 -> {lineup}"
                )
            if len(set(lineup)) != 5:
                raise AssertionError(
                    f"[Possession {poss_id}] {label} enhanced lineup has duplicates -> {lineup}"
                )
        else:
            if len(lineup) == 0:
                raise AssertionError(f"[Possession {poss_id}] {label} traditional lineup is empty")
            if len(set(lineup)) != len(lineup):
                raise AssertionError(
                    f"[Possession {poss_id}] {label} traditional lineup has duplicates -> {lineup}"
                )

    def _rebound_team(self, event) -> Optional[int]:
        """
        Infer rebounder team for msgType==4 using player_id_1 if available,
        falling back to def/off teams only if we cannot resolve player→team.
        """
        # Correct attribute name on ProcessedEvent is player_id_1
        pid = getattr(event, "player_id_1", None)
        # Be tolerant if a dict-like row sneaks in
        if pid is None and hasattr(event, "__getitem__"):
            try:
                pid = event["player_id_1"]
            except Exception:
                pid = None

        if pid is not None and pid in self.player_team:
            return self.player_team[pid]

        # Fallbacks (keep same order of preference)
        if getattr(event, "def_team_id", None) is not None:
            return event.def_team_id
        if getattr(event, "off_team_id", None) is not None:
            return event.off_team_id
        return None



    def __enter__(self):
        self.conn = duckdb.connect(self.db_path)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()



    def _load_events_from_db_as_processed(self) -> List[ProcessedEvent]:
        """
        Fallback loader: reconstruct ProcessedEvent objects from DuckDB table
        'processed_events' created by Step 4. Returns [] if not found/empty.
        """
        try:
            # Ensure we have a connection
            if self.conn is None:
                self.conn = duckdb.connect(self.db_path)

            # Check table existence
            tables = set(t[0].lower() for t in self.conn.execute(
                "SELECT table_name FROM information_schema.tables"
            ).fetchall())
            if "processed_events" not in tables:
                return []

            df = self.conn.execute("""
                SELECT
                    pbp_id, period, pbp_order, wall_clock_int, description,
                    msg_type, action_type, off_team_id, def_team_id,
                    player_id_1, player_id_2, player_id_3,
                    loc_x, loc_y, points,
                    is_shot, is_rim_attempt, is_rim_make, distance_ft,
                    is_substitution, sub_out_player, sub_in_player,
                    off_lineup, def_lineup
                FROM processed_events
                ORDER BY period, pbp_order, wall_clock_int
            """).df()
            if df.empty:
                return []

            events: List[ProcessedEvent] = []
            for _, r in df.iterrows():
                # Parse lineup strings back to tuples
                def _to_tuple(s):
                    if pd.isna(s) or s is None or s == "":
                        return None
                    try:
                        t = ast.literal_eval(str(s))
                        # normalize to sorted tuple of ints
                        if isinstance(t, (list, tuple)):
                            return tuple(int(x) for x in t)
                    except Exception:
                        pass
                    return None

                ev = ProcessedEvent(
                    pbp_id         = int(r["pbp_id"]),
                    period         = int(r["period"]),
                    pbp_order      = int(r["pbp_order"]),
                    wall_clock_int = int(r["wall_clock_int"]) if pd.notna(r["wall_clock_int"]) else 0,
                    msg_type       = int(r["msg_type"]) if pd.notna(r["msg_type"]) else None,
                    action_type    = int(r["action_type"]) if pd.notna(r["action_type"]) else None,
                    description    = str(r["description"]) if pd.notna(r["description"]) else "",
                    off_team_id    = int(r["off_team_id"]) if pd.notna(r["off_team_id"]) else None,
                    def_team_id    = int(r["def_team_id"]) if pd.notna(r["def_team_id"]) else None,
                    player_id_1    = int(r["player_id_1"]) if pd.notna(r["player_id_1"]) else None,
                    player_id_2    = int(r["player_id_2"]) if pd.notna(r["player_id_2"]) else None,
                    player_id_3    = int(r["player_id_3"]) if pd.notna(r["player_id_3"]) else None,
                    loc_x          = int(r["loc_x"]) if pd.notna(r["loc_x"]) and r["loc_x"] != 0 else None,
                    loc_y          = int(r["loc_y"]) if pd.notna(r["loc_y"]) and r["loc_y"] != 0 else None,
                    points         = int(r["points"]) if pd.notna(r["points"]) else 0,

                    is_shot        = bool(r["is_shot"]) if pd.notna(r["is_shot"]) else False,
                    is_rim_attempt = bool(r["is_rim_attempt"]) if pd.notna(r["is_rim_attempt"]) else False,
                    is_rim_make    = bool(r["is_rim_make"]) if pd.notna(r["is_rim_make"]) else False,
                    distance_ft    = float(r["distance_ft"]) if pd.notna(r["distance_ft"]) else None,
                    is_substitution= bool(r["is_substitution"]) if pd.notna(r["is_substitution"]) else False,
                    sub_out_player = int(r["sub_out_player"]) if pd.notna(r["sub_out_player"]) else None,
                    sub_in_player  = int(r["sub_in_player"]) if pd.notna(r["sub_in_player"]) else None,

                    off_lineup     = _to_tuple(r["off_lineup"]),
                    def_lineup     = _to_tuple(r["def_lineup"]),
                )
                events.append(ev)

            return events
        except Exception as e:
            logger.error(f"Failed to load processed_events from DB: {e}")
            return []

    def _assert_step4_schema_or_rebuild(self, autorun: bool = False) -> Dict[str, Any]:
        """
        Ensure 'step4_processed_events' contains the required dual-method lineup columns.
        - If missing and autorun=True, invoke Step 4 to rebuild once, then re-check.
        - Returns a dict: {"ok": bool, "details": str}
        """
        required = {
            "traditional_off_lineup", "traditional_def_lineup",
            "enhanced_off_lineup", "enhanced_def_lineup"
        }

        # existence
        tables = {t[0] for t in self.conn.execute("SELECT table_name FROM information_schema.tables").fetchall()}
        if "step4_processed_events" not in tables:
            return {"ok": False, "details": "Missing table 'step4_processed_events' (Step 4 not run)."}

        # schema
        cols = {r[1] for r in self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()}
        if required.issubset(cols):
            return {"ok": True, "details": "Step 4 schema satisfied (dual-method columns present)."}

        missing = sorted(required - cols)
        msg = f"Step 4 schema missing required columns: {missing}. Present columns: {sorted(cols)}"

        if not autorun:
            # Do not fix; just report
            logger.error(f"[CONTRACT] {msg}")
            return {"ok": False, "details": msg + " | autorun=False"}

        # Try to rebuild by running the real Step 4
        logger.warning(f"[CONTRACT] {msg} -> autorun=True, invoking Step 4 to rebuild...")
        try:
            from eda.data.nba_pbp_processor import process_pbp_with_step2_integration
            ok, _ = process_pbp_with_step2_integration(db_path=self.db_path, entities=self.entities)
            if not ok:
                return {"ok": False, "details": "Invoked Step 4, but it reported failure."}

            # Re-check
            cols2 = {r[1] for r in self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()}
            if required.issubset(cols2):
                return {"ok": True, "details": "Step 4 rebuilt and schema now satisfies contract."}
            return {"ok": False, "details": "Step 4 rebuilt but schema still missing required dual-method columns."}
        except Exception as e:
            return {"ok": False, "details": f"Failed to invoke Step 4: {e}"}


    def load_dual_method_data(self, autorun_rebuild: bool = False) -> ValidationResult:
        """
        Load upstream artifacts needed by Step 5.

        Hard requirement:
        - step4_processed_events with dual-method lineup columns (contract)

        Optional (diagnostics/enrichment; absence should NOT block Step 5):
        - traditional_lineup_state / enhanced_lineup_state
        - traditional_lineup_flags / enhanced_lineup_flags
        - step4_traditional_flags / step4_enhanced_flags (older naming)
        - traditional_violation_report / enhanced_violation_report (read-only, previous Step 5 outputs)

        We DO NOT fill in or synthesize missing data. We enforce the contract, report, and
        optionally rebuild via Step 4 when autorun_rebuild=True.
        """
        start_time = time.time()
        try:
            logger.info("Loading dual-method data from Step 2/4 integration...")

            # ---- Enforce Step 4 → Step 5 contract up front ----
            contract = self._assert_step4_schema_or_rebuild(autorun=autorun_rebuild)
            if not contract.get("ok"):
                return ValidationResult(
                    step_name="Load Dual Method Data",
                    passed=False,
                    details=("Contract failed: " + contract.get("details", "")),
                    processing_time=time.time() - start_time,
                )

            all_tables = self.conn.execute(
                "SELECT table_name FROM information_schema.tables ORDER BY table_name"
            ).fetchall()
            logger.info(f"DEBUG: Available tables in database: {[t[0] for t in all_tables]}")

            existing = {r[0] for r in all_tables}

            # ---- Optional sources (don't fail if missing) ----
            optional_expected = [
                "traditional_lineup_state",
                "enhanced_lineup_state",
                "traditional_lineup_flags",
                "enhanced_lineup_flags",
                "step4_traditional_flags",
                "step4_enhanced_flags",
                # read-only outputs from previous Step 5 runs (if present)
                "traditional_violation_report",
                "enhanced_violation_report",
            ]
            present_optional = [t for t in optional_expected if t in existing]
            missing_optional = [t for t in optional_expected if t not in existing]

            logger.info(f"DEBUG: Optional tables present: {present_optional}")
            logger.info(f"DEBUG: Optional tables missing (non-blocking): {missing_optional}")

            # ---- Load flags (non-blocking): prefer official names, then step4_*, then violation_report ----
            trad_flags = self._load_flags_with_fallback("traditional_lineup_flags", "step4_traditional_flags")
            enh_flags  = self._load_flags_with_fallback("enhanced_lineup_flags", "step4_enhanced_flags")

            self.traditional_violations = trad_flags.to_dict("records") if not trad_flags.empty else []
            self.enhanced_violations    = enh_flags.to_dict("records") if not enh_flags.empty else []

            details = (
                "Loaded Step 5 inputs. "
                f"Flags: traditional={len(self.traditional_violations)}, enhanced={len(self.enhanced_violations)}. "
                f"Optional sources missing (non-blocking): {missing_optional}."
            )
            return ValidationResult(
                step_name="Load Dual Method Data",
                passed=True,
                details=details,
                data_count=len(self.traditional_violations) + len(self.enhanced_violations),
                processing_time=time.time() - start_time,
            )

        except Exception as e:
            return ValidationResult(
                step_name="Load Dual Method Data",
                passed=False,
                details=f"Error loading dual-method data: {e}",
                processing_time=time.time() - start_time,
            )



    def _create_missing_tables_from_alternatives(self, missing: List[str], existing: set) -> bool:
        """Create missing tables from alternative sources (never synthesizes step4_processed_events)."""
        try:
            logger.info("DEBUG: Attempting to create missing tables from alternatives...")

            # Only allow flag tables to be backfilled from their step4_* equivalents.
            # DO NOT create step4_processed_events from legacy processed_events (schema would be wrong).
            table_mappings = {
                'step4_traditional_flags': 'traditional_lineup_flags',
                'step4_enhanced_flags': 'enhanced_lineup_flags',
                # 'processed_events': 'step4_processed_events'  # intentionally removed to avoid legacy schema propagation
            }

            created_count = 0
            for alt_name, expected_name in table_mappings.items():
                if expected_name.lower() in missing and alt_name.lower() in existing:
                    logger.info(f"DEBUG: Creating '{expected_name}' from '{alt_name}'")
                    try:
                        self.conn.execute(f"CREATE OR REPLACE TABLE {expected_name} AS SELECT * FROM {alt_name}")
                        created_count += 1
                        logger.info(f"DEBUG: Successfully created '{expected_name}'")
                    except Exception as e:
                        logger.warning(f"DEBUG: Failed to create '{expected_name}' from '{alt_name}': {e}")

            logger.info(f"DEBUG: Created {created_count} missing tables from alternatives")
            return created_count > 0

        except Exception as e:
            logger.error(f"DEBUG: Error creating missing tables: {e}")
            return False


    def _load_flags_with_fallback(self, primary_table: str, fallback_table: str) -> pd.DataFrame:
        """
        Load a flags-like table with sensible fallbacks and explicit debugs.

        Resolution order:
        1) primary_table (e.g., traditional_lineup_flags)
        2) fallback_table (e.g., step4_traditional_flags)
        3) read-only violation report (e.g., traditional_violation_report / enhanced_violation_report)

        We DO NOT synthesize records. If none are found, return an empty DataFrame.
        """
        try:
            tables = {
                t[0].lower()
                for t in self.conn.execute("SELECT table_name FROM information_schema.tables").fetchall()
            }

            # try primary
            target = None
            if primary_table.lower() in tables:
                target = primary_table
                logger.info(f"DEBUG: Using primary flags table '{primary_table}'")
            # else fallback
            elif fallback_table.lower() in tables:
                target = fallback_table
                logger.info(f"DEBUG: Using fallback flags table '{fallback_table}'")
            else:
                # consider read-only prior outputs if they exist
                alt_report = None
                if primary_table.lower().startswith("traditional"):
                    alt_report = "traditional_violation_report"
                elif primary_table.lower().startswith("enhanced"):
                    alt_report = "enhanced_violation_report"

                if alt_report and alt_report.lower() in tables:
                    target = alt_report
                    logger.info(
                        f"DEBUG: Using read-only prior output '{alt_report}' as violation source "
                        f"(no synthesis; purely for context)"
                    )

            if not target:
                logger.warning(f"DEBUG: No available tables among: '{primary_table}', '{fallback_table}', prior reports")
                return pd.DataFrame()

            # Determine a stable ordering column if present
            order_cols = [c[1] for c in self.conn.execute(f"PRAGMA table_info('{target}')").fetchall()]
            if "abs_time" in order_cols:
                order_expr = "abs_time"
            elif "wall_clock_int" in order_cols:
                order_expr = "wall_clock_int"
            elif "pbp_order" in order_cols:
                order_expr = "pbp_order"
            else:
                order_expr = None

            if order_expr:
                df = self.conn.execute(f"SELECT * FROM {target} ORDER BY {order_expr}").df()
            else:
                df = self.conn.execute(f"SELECT * FROM {target}").df()

            logger.info(f"DEBUG: Loaded {len(df)} rows from '{target}' for flags/violations")
            return df

        except Exception as e:
            logger.error(f"DEBUG: Error loading flags from '{primary_table}'/'{fallback_table}': {e}")
            return pd.DataFrame()

    ALLOW_LEGACY_FALLBACK = False

    def identify_dual_possessions(self) -> ValidationResult:
        """FIXED: Enhanced possession identification with proper team attribution"""
        start_time = time.time()
        try:
            logger.info("Identifying possessions with dual-method lineup contexts (FIXED MODE)...")

            event_count = self.conn.execute("SELECT COUNT(*) FROM step4_processed_events").fetchone()[0]
            logger.info(f"DEBUG: step4_processed_events has {event_count} rows")

            cols = {r[1] for r in self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()}
            have_trad = {"traditional_off_lineup", "traditional_def_lineup"}.issubset(cols)
            have_enh  = {"enhanced_off_lineup", "enhanced_def_lineup"}.issubset(cols)

            if not (have_trad and have_enh):
                return ValidationResult(
                    step_name="Identify Dual Possessions",
                    passed=False,
                    details=("Required columns missing in 'step4_processed_events'. "
                            "Expected traditional_/enhanced_ lineups. Aborting without fallback."),
                    processing_time=time.time() - start_time
                )

            select_sql = """
                SELECT 
                    pbp_id, period, pbp_order, wall_clock_int, msg_type,
                    off_team_id, def_team_id, points,
                    traditional_off_lineup, traditional_def_lineup,
                    enhanced_off_lineup, enhanced_def_lineup,
                    player_id_1, description
                FROM step4_processed_events
                WHERE off_team_id IS NOT NULL AND def_team_id IS NOT NULL
                ORDER BY period, pbp_order, wall_clock_int
            """

            events_df = self.conn.execute(select_sql).df()
            logger.info(f"DEBUG: Retrieved {len(events_df)} events with valid team IDs")

            if events_df.empty:
                return ValidationResult(
                    step_name="Identify Dual Possessions",
                    passed=False,
                    details="No events with dual lineup context found",
                    processing_time=time.time() - start_time
                )

            def _parse_lineup(val) -> Optional[Tuple[int, ...]]:
                if val is None or (isinstance(val, float) and pd.isna(val)) or str(val).strip() == "":
                    return None
                try:
                    obj = json.loads(val) if isinstance(val, str) else val
                except Exception:
                    try:
                        obj = ast.literal_eval(str(val))
                    except Exception:
                        logger.debug(f"Lineup parse failed; raw={val!r}")
                        return None
                if isinstance(obj, (list, tuple, set)):
                    try:
                        tup = tuple(sorted(int(x) for x in obj))
                        return tup
                    except Exception:
                        logger.debug(f"Lineup normalization failed; raw={obj!r}")
                        return None
                logger.debug(f"Lineup had unexpected type; raw={obj!r}")
                return None

            self.dual_possessions = []
            possession_id = 0

            # FIXED: Initialize possession state properly
            current_possession = None
            
            # Enhanced debug tracking
            points_by_possession = []
            scoring_events_processed = []
            team_points_tracker = {"DAL": 0, "HOU": 0}  # Track cumulative points by team

            for idx, row in events_df.iterrows():
                msg_type = int(row["msg_type"]) if pd.notna(row["msg_type"]) else None
                event_team_id = int(row["off_team_id"])
                event_def_team = int(row["def_team_id"])
                event_points = int(row["points"]) if pd.notna(row["points"]) and row["points"] > 0 else 0
                
                # Track every scoring event with detailed context
                if event_points > 0:
                    team_abbrev = self.team_abbrev.get(event_team_id, f"Team{event_team_id}")
                    team_points_tracker[team_abbrev] += event_points
                    
                    scoring_event = {
                        "pbp_id": int(row["pbp_id"]),
                        "period": int(row["period"]),
                        "pbp_order": int(row["pbp_order"]),
                        "points": event_points,
                        "off_team_id": event_team_id,
                        "description": str(row["description"]),
                        "msg_type": msg_type,
                        "current_possession_team": current_possession["off_team_id"] if current_possession else None,
                        "team_abbrev": team_abbrev,
                        "cumulative_team_points": team_points_tracker[team_abbrev]
                    }
                    scoring_events_processed.append(scoring_event)

                # FIXED: Determine if we need to end current possession
                should_end_possession = (
                    current_possession is None or  # No current possession
                    current_possession["off_team_id"] != event_team_id or  # Team changed
                    current_possession["def_team_id"] != event_def_team or  # Defense changed
                    msg_type in (1, 5, 12, 13) or  # Made shot, turnover, period boundaries
                    (msg_type == 4 and self._is_defensive_rebound(row))  # Defensive rebound
                )

                # End current possession if needed
                if should_end_possession and current_possession is not None:
                    # Close current possession
                    possession_info = {
                        "possession_id": possession_id,
                        "team_id": current_possession["off_team_id"],
                        "points": current_possession["points"],
                        "period": current_possession["period"],
                        "start_order": current_possession["start_order"],
                        "end_order": int(row["pbp_order"]) - 1,
                        "ended_by": self._determine_end_reason(row)
                    }
                    points_by_possession.append(possession_info)
                    
                    self.dual_possessions.append(
                        DualPossession(
                            possession_id=possession_id,
                            period=current_possession["period"],
                            start_pbp_order=current_possession["start_order"],
                            end_pbp_order=int(row["pbp_order"]) - 1,
                            off_team_id=current_possession["off_team_id"],
                            def_team_id=current_possession["def_team_id"],
                            traditional_off_lineup=current_possession["trad_off"],
                            traditional_def_lineup=current_possession["trad_def"],
                            enhanced_off_lineup=current_possession["enh_off"],
                            enhanced_def_lineup=current_possession["enh_def"],
                            points_scored=current_possession["points"],
                            ended_by=possession_info["ended_by"],
                        )
                    )
                    possession_id += 1
                    current_possession = None

                # FIXED: Start new possession if needed (always when no current possession)
                if current_possession is None:
                    current_possession = {
                        "off_team_id": event_team_id,
                        "def_team_id": event_def_team,
                        "start_order": int(row["pbp_order"]),
                        "period": int(row["period"]),
                        "points": 0,  # Will accumulate points for this possession
                        "trad_off": _parse_lineup(row["traditional_off_lineup"]),
                        "trad_def": _parse_lineup(row["traditional_def_lineup"]),
                        "enh_off": _parse_lineup(row["enhanced_off_lineup"]),
                        "enh_def": _parse_lineup(row["enhanced_def_lineup"])
                    }

                    # Validate enhanced lineups
                    try:
                        if current_possession["enh_off"] is not None:
                            self._validate_lineup(current_possession["enh_off"], "off", possession_id, "enhanced")
                        if current_possession["enh_def"] is not None:
                            self._validate_lineup(current_possession["enh_def"], "def", possession_id, "enhanced")
                    except AssertionError as ae:
                        logger.warning(f"Enhanced lineup validation failed: {ae}")

                # FIXED: Add points to current possession only if teams match
                if event_points > 0 and current_possession["off_team_id"] == event_team_id:
                    current_possession["points"] += event_points
                elif event_points > 0:
                    # Points belong to different team - log this mismatch but don't add to possession
                    logger.warning(f"SCORING EVENT TEAM MISMATCH: pbp_id={row['pbp_id']}, "
                                 f"possession_team={current_possession['off_team_id']}, "
                                 f"event_team={event_team_id}, points={event_points}")

            # Close the final possession
            if current_possession is not None and not events_df.empty:
                final_possession = {
                    "possession_id": possession_id,
                    "team_id": current_possession["off_team_id"],
                    "points": current_possession["points"],
                    "period": current_possession["period"],
                    "start_order": current_possession["start_order"],
                    "end_order": int(events_df.iloc[-1]["pbp_order"])
                }
                points_by_possession.append(final_possession)
                
                self.dual_possessions.append(
                    DualPossession(
                        possession_id=possession_id,
                        period=current_possession["period"],
                        start_pbp_order=current_possession["start_order"],
                        end_pbp_order=int(events_df.iloc[-1]["pbp_order"]),
                        off_team_id=current_possession["off_team_id"],
                        def_team_id=current_possession["def_team_id"],
                        traditional_off_lineup=current_possession["trad_off"],
                        traditional_def_lineup=current_possession["trad_def"],
                        enhanced_off_lineup=current_possession["enh_off"],
                        enhanced_def_lineup=current_possession["enh_def"],
                        points_scored=current_possession["points"],
                        ended_by="game_end",
                    )
                )

            # FIXED: Enhanced debug output with corrected calculations
            logger.info("=== FIXED POSSESSION DEBUG ANALYSIS ===")
            total_possession_points = sum(p["points"] for p in points_by_possession)
            total_scoring_events = len(scoring_events_processed)
            total_event_points = sum(e["points"] for e in scoring_events_processed)
            
            logger.info(f"Total Possessions Created: {len(self.dual_possessions):,}")
            logger.info(f"Total Possession Points: {total_possession_points:,}")
            logger.info(f"Total Scoring Events Processed: {total_scoring_events:,}")
            logger.info(f"Total Event Points: {total_event_points:,}")
            
            # Team-specific analysis with corrected logic
            team_possession_points = {}
            team_event_points = {}
            
            for p in points_by_possession:
                team_abbrev = self.team_abbrev.get(p["team_id"], f"Team{p['team_id']}")
                team_possession_points[team_abbrev] = team_possession_points.get(team_abbrev, 0) + p["points"]
            
            for e in scoring_events_processed:
                team_abbrev = e["team_abbrev"]
                team_event_points[team_abbrev] = team_event_points.get(team_abbrev, 0) + e["points"]
            
            logger.info("FIXED TEAM-BY-TEAM ANALYSIS:")
            for team in ["DAL", "HOU"]:
                poss_pts = team_possession_points.get(team, 0)
                event_pts = team_event_points.get(team, 0)
                diff = poss_pts - event_pts  # Possession points should match event points
                
                logger.info(f"  {team}:")
                logger.info(f"    Possession Points: {poss_pts}")
                logger.info(f"    Event Points: {event_pts}")
                logger.info(f"    Difference: {diff:+}")
                
                if diff != 0:
                    logger.warning(f"    *** {team} POINTS DISCREPANCY: {diff:+} points ***")

            details = (f"FIXED: Identified {len(self.dual_possessions)} dual-method possessions, "
                      f"{total_possession_points} possession points vs {total_event_points} event points.")
            
            return ValidationResult(
                step_name="Identify Dual Possessions",
                passed=len(self.dual_possessions) > 0,
                details=details,
                data_count=len(self.dual_possessions),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Identify Dual Possessions",
                passed=False,
                details=f"Error identifying dual possessions: {e}",
                processing_time=time.time() - start_time
            )



    def _is_defensive_rebound(self, row) -> bool:
        """Determine if rebound is defensive based on player team"""
        if row['msg_type'] != 4:  # Not a rebound
            return False

        rebounder_id = row.get('player_id_1')
        if not rebounder_id or rebounder_id not in self.player_team:
            return False

        rebounder_team = self.player_team[rebounder_id]
        return rebounder_team == row['def_team_id']

    def _determine_end_reason(self, row) -> str:
        """Determine why possession ended (made FG, turnover, defensive rebound, or period boundary)."""
        mt = row.get('msg_type') if hasattr(row, 'get') else row['msg_type']
        if mt == 1:
            return "made_fg"
        elif mt == 5:
            return "turnover"
        elif mt == 4 and self._is_defensive_rebound(row):
            return "def_rebound"
        elif mt in (12, 13):  # 12: start period; 13: end period
            return "period_boundary"
        else:
            return "team_change"

    def debug_points_flow_comprehensive(self) -> Dict[str, Any]:
        """
        Comprehensive points flow analysis to identify where the 4-point HOU discrepancy originates.
        Traces from raw events -> processed events -> possessions -> lineups.
        """
        try:
            logger.info("=== COMPREHENSIVE POINTS FLOW DEBUG ===")
            
            debug_info = {
                "raw_pbp_points": {},
                "step4_processed_points": {},
                "possession_points": {},
                "lineup_points": {},
                "discrepancies": [],
                "detailed_scoring_events": []
            }
            
            # 1. Raw PBP points by team
            raw_points = self.conn.execute("""
                SELECT 
                    CASE 
                        WHEN team_id_off = 1610612742 THEN 'DAL'
                        WHEN team_id_off = 1610612745 THEN 'HOU'
                        ELSE CAST(team_id_off AS VARCHAR)
                    END as team,
                    SUM(COALESCE(points, 0)) as total_points,
                    COUNT(*) as scoring_events
                FROM pbp 
                WHERE points > 0 AND team_id_off IS NOT NULL
                GROUP BY team_id_off
                ORDER BY team
            """).df()
            
            for _, row in raw_points.iterrows():
                debug_info["raw_pbp_points"][row["team"]] = {
                    "points": int(row["total_points"]),
                    "events": int(row["scoring_events"])
                }
            
            # 2. Step4 processed points by team
            step4_points = self.conn.execute("""
                SELECT 
                    CASE 
                        WHEN off_team_id = 1610612742 THEN 'DAL'
                        WHEN off_team_id = 1610612745 THEN 'HOU'
                        ELSE CAST(off_team_id AS VARCHAR)
                    END as team,
                    SUM(COALESCE(points, 0)) as total_points,
                    COUNT(*) as scoring_events
                FROM step4_processed_events 
                WHERE points > 0 AND off_team_id IS NOT NULL
                GROUP BY off_team_id
                ORDER BY team
            """).df()
            
            for _, row in step4_points.iterrows():
                debug_info["step4_processed_points"][row["team"]] = {
                    "points": int(row["total_points"]),
                    "events": int(row["scoring_events"])
                }
            
            # 3. Possession-level points (from our dual possessions)
            possession_totals = {}
            for poss in self.dual_possessions:
                team_abbrev = self.team_abbrev.get(poss.off_team_id, f"Team{poss.off_team_id}")
                if team_abbrev not in possession_totals:
                    possession_totals[team_abbrev] = {"points": 0, "possessions": 0}
                possession_totals[team_abbrev]["points"] += poss.points_scored
                possession_totals[team_abbrev]["possessions"] += 1
            
            debug_info["possession_points"] = possession_totals
            
            # 4. Lineup-level points (both methods)
            debug_info["lineup_points"] = {}
            for method in ["traditional", "enhanced"]:
                lineup_stats = (self.traditional_lineup_stats if method == "traditional" 
                              else self.enhanced_lineup_stats)
                method_totals = {}
                for (team_id, lineup), stats in lineup_stats.items():
                    team_abbrev = self.team_abbrev.get(team_id, f"Team{team_id}")
                    if team_abbrev not in method_totals:
                        method_totals[team_abbrev] = {"points": 0, "lineups": 0}
                    method_totals[team_abbrev]["points"] += stats.points_for
                    method_totals[team_abbrev]["lineups"] += 1
                debug_info["lineup_points"][method] = method_totals
            
            # 5. Detailed scoring events for HOU (the problematic team)
            hou_events = self.conn.execute("""
                SELECT 
                    pbp_id, period, pbp_order, description, points,
                    off_team_id, msg_type, action_type,
                    traditional_off_lineup, enhanced_off_lineup
                FROM step4_processed_events 
                WHERE off_team_id = 1610612745 AND points > 0
                ORDER BY period, pbp_order
            """).df()
            
            debug_info["detailed_scoring_events"] = hou_events.to_dict('records')
            
            # 6. Calculate discrepancies at each stage
            teams = ['DAL', 'HOU']
            for team in teams:
                raw_pts = debug_info["raw_pbp_points"].get(team, {}).get("points", 0)
                step4_pts = debug_info["step4_processed_points"].get(team, {}).get("points", 0)
                poss_pts = debug_info["possession_points"].get(team, {}).get("points", 0)
                
                trad_lineup_pts = debug_info["lineup_points"].get("traditional", {}).get(team, {}).get("points", 0)
                enh_lineup_pts = debug_info["lineup_points"].get("enhanced", {}).get(team, {}).get("points", 0)
                
                discrepancy = {
                    "team": team,
                    "raw_pbp_points": raw_pts,
                    "step4_processed_points": step4_pts,
                    "possession_points": poss_pts,
                    "traditional_lineup_points": trad_lineup_pts,
                    "enhanced_lineup_points": enh_lineup_pts,
                    "raw_to_step4_diff": step4_pts - raw_pts,
                    "step4_to_possession_diff": poss_pts - step4_pts,
                    "possession_to_traditional_diff": trad_lineup_pts - poss_pts,
                    "possession_to_enhanced_diff": enh_lineup_pts - poss_pts
                }
                debug_info["discrepancies"].append(discrepancy)
            
            # Log findings
            logger.info("POINTS FLOW ANALYSIS:")
            for disc in debug_info["discrepancies"]:
                team = disc["team"]
                logger.info(f"  {team}:")
                logger.info(f"    Raw PBP: {disc['raw_pbp_points']}")
                logger.info(f"    Step4 Processed: {disc['step4_processed_points']} (diff: {disc['raw_to_step4_diff']:+})")
                logger.info(f"    Possessions: {disc['possession_points']} (diff: {disc['step4_to_possession_diff']:+})")
                logger.info(f"    Traditional Lineups: {disc['traditional_lineup_points']} (diff: {disc['possession_to_traditional_diff']:+})")
                logger.info(f"    Enhanced Lineups: {disc['enhanced_lineup_points']} (diff: {disc['possession_to_enhanced_diff']:+})")
            
            return debug_info
            
        except Exception as e:
            logger.error(f"Error in comprehensive points flow debug: {e}")
            return {"error": str(e)}

    def debug_hou_scoring_events_detailed(self) -> Dict[str, Any]:
        """
        Detailed analysis of every HOU scoring event to identify potential double-counting
        or attribution errors causing the 4-point discrepancy.
        """
        try:
            logger.info("=== DETAILED HOU SCORING EVENTS ANALYSIS ===")
            
            # Get all HOU scoring events with context
            hou_scoring = self.conn.execute("""
                SELECT 
                    se.pbp_id, se.period, se.pbp_order, se.wall_clock_int,
                    se.description, se.points, se.msg_type, se.action_type,
                    se.player_id_1, se.player_id_2, se.player_id_3,
                    se.traditional_off_lineup, se.enhanced_off_lineup,
                    -- Get original PBP data for comparison
                    pbp.points as original_points,
                    pbp.description as original_description
                FROM step4_processed_events se
                LEFT JOIN pbp ON se.pbp_id = pbp.pbp_id
                WHERE se.off_team_id = 1610612745 AND se.points > 0
                ORDER BY se.period, se.pbp_order, se.wall_clock_int
            """).df()
            
            analysis = {
                "total_hou_scoring_events": len(hou_scoring),
                "total_hou_points_calculated": int(hou_scoring['points'].sum()),
                "scoring_events_by_type": {},
                "potential_issues": [],
                "event_details": []
            }
            
            # Analyze by scoring event type
            if not hou_scoring.empty:
                by_type = hou_scoring.groupby('msg_type').agg({
                    'points': ['count', 'sum'],
                    'pbp_id': 'count'
                }).round(2)
                
                analysis["scoring_events_by_type"] = by_type.to_dict()
                
                # Check for potential issues
                for _, event in hou_scoring.iterrows():
                    event_detail = {
                        "pbp_id": int(event['pbp_id']),
                        "period": int(event['period']),
                        "pbp_order": int(event['pbp_order']),
                        "description": str(event['description']),
                        "points": int(event['points']),
                        "original_points": int(event['original_points']) if pd.notna(event['original_points']) else None,
                        "msg_type": int(event['msg_type']),
                        "has_traditional_lineup": bool(pd.notna(event['traditional_off_lineup']) and 
                                                     str(event['traditional_off_lineup']).strip() not in ['', '[]']),
                        "has_enhanced_lineup": bool(pd.notna(event['enhanced_off_lineup']) and 
                                                  str(event['enhanced_off_lineup']).strip() not in ['', '[]'])
                    }
                    
                    # Flag potential issues
                    if event_detail["points"] != event_detail["original_points"]:
                        analysis["potential_issues"].append(f"Points mismatch for pbp_id {event_detail['pbp_id']}: processed={event_detail['points']}, original={event_detail['original_points']}")
                    
                    if not event_detail["has_traditional_lineup"]:
                        analysis["potential_issues"].append(f"No traditional lineup for scoring event pbp_id {event_detail['pbp_id']}")
                    
                    if not event_detail["has_enhanced_lineup"]:
                        analysis["potential_issues"].append(f"No enhanced lineup for scoring event pbp_id {event_detail['pbp_id']}")
                    
                    analysis["event_details"].append(event_detail)
            
            # Check for possession attribution
            hou_possession_points = sum(p.points_scored for p in self.dual_possessions if p.off_team_id == 1610612745)
            analysis["possession_attribution"] = {
                "total_possession_points": hou_possession_points,
                "difference_from_events": hou_possession_points - analysis["total_hou_points_calculated"]
            }
            
            logger.info(f"HOU Scoring Analysis:")
            logger.info(f"  Total Scoring Events: {analysis['total_hou_scoring_events']}")
            logger.info(f"  Total Points from Events: {analysis['total_hou_points_calculated']}")
            logger.info(f"  Total Points from Possessions: {hou_possession_points}")
            logger.info(f"  Potential Issues Found: {len(analysis['potential_issues'])}")
            
            for issue in analysis["potential_issues"][:10]:  # Log first 10 issues
                logger.warning(f"    {issue}")
            
            return analysis
            
        except Exception as e:
            logger.error(f"Error in detailed HOU scoring analysis: {e}")
            return {"error": str(e)}




    def calculate_dual_lineup_stats(self) -> ValidationResult:
        """Calculate lineup statistics for both traditional and enhanced methods"""
        start_time = time.time()

        try:
            logger.info("Calculating dual-method lineup statistics...")

            if not self.dual_possessions:
                return ValidationResult(
                    step_name="Calculate Dual Lineup Stats",
                    passed=False,
                    details="No dual possessions available",
                    processing_time=time.time() - start_time
                )

            # Initialize stats containers
            self.traditional_lineup_stats = {}
            self.enhanced_lineup_stats = {}

            # Process each possession for both methods
            for poss in self.dual_possessions:
                # Traditional method stats
                if poss.traditional_off_lineup and poss.traditional_def_lineup:
                    self._update_lineup_stats(
                        poss, "traditional",
                        poss.traditional_off_lineup, poss.traditional_def_lineup
                    )

                # Enhanced method stats
                if poss.enhanced_off_lineup and poss.enhanced_def_lineup:
                    self._update_lineup_stats(
                        poss, "enhanced", 
                        poss.enhanced_off_lineup, poss.enhanced_def_lineup
                    )

            # Calculate ratings for both methods
            self._calculate_ratings(self.traditional_lineup_stats)
            self._calculate_ratings(self.enhanced_lineup_stats)

            # Add violation context to traditional lineups
            self._add_violation_context()

            trad_count = len(self.traditional_lineup_stats)
            enh_count = len(self.enhanced_lineup_stats)

            details = f"Calculated lineup stats: {trad_count} traditional, {enh_count} enhanced lineups"

            return ValidationResult(
                step_name="Calculate Dual Lineup Stats",
                passed=True,
                details=details,
                data_count=trad_count + enh_count,
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Calculate Dual Lineup Stats",
                passed=False,
                details=f"Error calculating dual lineup stats: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _update_lineup_stats(self, poss: DualPossession, method: str, 
                           off_lineup: Tuple[int, ...], def_lineup: Tuple[int, ...]):
        """Update lineup statistics for given method"""
        stats_container = (self.traditional_lineup_stats if method == "traditional" 
                          else self.enhanced_lineup_stats)

        # Offensive lineup
        off_key = (poss.off_team_id, off_lineup)
        if off_key not in stats_container:
            stats_container[off_key] = self._create_lineup_stats(
                poss.off_team_id, off_lineup, method
            )

        off_stats = stats_container[off_key]
        off_stats.off_possessions += 1
        off_stats.points_for += poss.points_scored

        # Defensive lineup
        def_key = (poss.def_team_id, def_lineup)
        if def_key not in stats_container:
            stats_container[def_key] = self._create_lineup_stats(
                poss.def_team_id, def_lineup, method
            )

        def_stats = stats_container[def_key]
        def_stats.def_possessions += 1
        def_stats.points_against += poss.points_scored

    def _create_lineup_stats(self, team_id: int, lineup: Tuple[int, ...], method: str) -> DualLineupStats:
        """Create lineup statistics object"""
        team_abbrev = self.team_abbrev.get(team_id, f"Team{team_id}")
        player_names = [self.player_names.get(pid, f"Player{pid}") for pid in lineup]

        return DualLineupStats(
            team_id=team_id,
            team_abbrev=team_abbrev,
            lineup_method=method,
            player_ids=lineup,
            player_names=player_names,
            lineup_size=len(lineup)
        )

    def _calculate_ratings(self, stats_container: Dict):
        """Calculate offensive/defensive/net ratings"""
        for stats in stats_container.values():
            if stats.off_possessions > 0:
                stats.off_rating = (100.0 * stats.points_for / stats.off_possessions)
            if stats.def_possessions > 0:
                stats.def_rating = (100.0 * stats.points_against / stats.def_possessions)
            stats.net_rating = stats.off_rating - stats.def_rating

    def _add_violation_context(self):
        """Add violation flags to traditional lineup stats"""
        # Group violations by lineup characteristics if possible
        violation_summary = defaultdict(list)

        for violation in self.traditional_violations:
            flag_type = violation.get('flag_type', 'unknown')
            team_id = violation.get('team_id')
            details = violation.get('description', '')

            violation_summary[f"team_{team_id}"].append(f"{flag_type}: {details}")

        # Add violations to lineup stats where applicable
        for (team_id, lineup), stats in self.traditional_lineup_stats.items():
            team_violations = violation_summary.get(f"team_{team_id}", [])
            stats.lineup_violations = team_violations[:5]  # Top 5 violations


    def calculate_dual_player_rim_stats(self) -> ValidationResult:
        """Calculate player rim defense statistics for both methods"""
        start_time = time.time()

        try:
            logger.info("Calculating dual-method player rim defense statistics...")

            if not self.dual_possessions:
                return ValidationResult(
                    step_name="Calculate Dual Player Rim Stats",
                    passed=False,
                    details="No dual possessions available",
                    processing_time=time.time() - start_time
                )

            # Initialize player stats for both methods
            self.traditional_player_stats = {}
            self.enhanced_player_stats = {}

            # Initialize all active players
            if hasattr(self.entities, 'unique_players') and self.entities.unique_players is not None:
                for _, r in self.entities.unique_players.iterrows():
                    pid = int(r["player_id"])

                    # Traditional method player
                    self.traditional_player_stats[pid] = DualPlayerRimStats(
                        player_id=pid,
                        player_name=str(r.get("player_name", pid)),
                        team_id=int(r.get("team_id")) if pd.notna(r.get("team_id")) else None,
                        team_abbrev=str(r.get("team_abbrev")) if pd.notna(r.get("team_abbrev")) else None,
                        method="traditional"
                    )

                    # Enhanced method player
                    self.enhanced_player_stats[pid] = DualPlayerRimStats(
                        player_id=pid,
                        player_name=str(r.get("player_name", pid)),
                        team_id=int(r.get("team_id")) if pd.notna(r.get("team_id")) else None,
                        team_abbrev=str(r.get("team_abbrev")) if pd.notna(r.get("team_abbrev")) else None,
                        method="enhanced"
                    )

            # Count possessions for both methods
            for poss in self.dual_possessions:
                # Traditional method
                if poss.traditional_off_lineup:
                    for pid in poss.traditional_off_lineup:
                        if pid in self.traditional_player_stats:
                            self.traditional_player_stats[pid].off_possessions += 1

                if poss.traditional_def_lineup:
                    for pid in poss.traditional_def_lineup:
                        if pid in self.traditional_player_stats:
                            self.traditional_player_stats[pid].def_possessions += 1

                # Enhanced method
                if poss.enhanced_off_lineup:
                    for pid in poss.enhanced_off_lineup:
                        if pid in self.enhanced_player_stats:
                            self.enhanced_player_stats[pid].off_possessions += 1

                if poss.enhanced_def_lineup:
                    for pid in poss.enhanced_def_lineup:
                        if pid in self.enhanced_player_stats:
                            self.enhanced_player_stats[pid].def_possessions += 1

            # Calculate rim defense stats
            self._calculate_dual_rim_defense()

            trad_with_rim = sum(1 for s in self.traditional_player_stats.values() 
                               if s.opp_rim_attempts_on > 0)
            enh_with_rim = sum(1 for s in self.enhanced_player_stats.values() 
                              if s.opp_rim_attempts_on > 0)

            details = (f"Calculated player rim stats: {trad_with_rim} traditional, "
                      f"{enh_with_rim} enhanced players with rim data")

            return ValidationResult(
                step_name="Calculate Dual Player Rim Stats",
                passed=True,
                details=details,
                data_count=len(self.traditional_player_stats) + len(self.enhanced_player_stats),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Calculate Dual Player Rim Stats",
                passed=False,
                details=f"Error calculating dual player rim stats: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _calculate_dual_rim_defense(self):
        """Calculate rim defense stats using rim attempt events"""
        try:
            rim_events_df = self.conn.execute("""
                SELECT 
                    def_team_id, is_rim_make,
                    traditional_def_lineup, enhanced_def_lineup
                FROM step4_processed_events
                WHERE is_rim_attempt = TRUE AND def_team_id IS NOT NULL
            """).df()
            if rim_events_df.empty:
                logger.debug("No rim events found; skipping rim-defense aggregation.")
                return

            def _as_set(val) -> Set[int]:
                if val is None or (isinstance(val, float) and pd.isna(val)) or str(val).strip() == "":
                    return set()
                try:
                    obj = json.loads(val) if isinstance(val, str) else val
                except Exception:
                    try:
                        obj = ast.literal_eval(str(val))
                    except Exception:
                        logger.debug(f"Rim lineup parse failed; raw={val!r}")
                        return set()
                if not isinstance(obj, (list, tuple, set)):
                    logger.debug(f"Rim lineup had unexpected type; raw={obj!r}")
                    return set()
                out = set()
                for x in obj:
                    try:
                        out.add(int(x))
                    except Exception:
                        logger.debug(f"Rim lineup member non-int; raw={x!r}")
                return out

            for _, row in rim_events_df.iterrows():
                def_team_id = int(row["def_team_id"])
                is_make = bool(row["is_rim_make"])
                trad_def = _as_set(row["traditional_def_lineup"])
                enh_def = _as_set(row["enhanced_def_lineup"])
                roster = self.team_roster.get(def_team_id, set())

                # Traditional: on
                for pid in trad_def:
                    if pid in self.traditional_player_stats:
                        s = self.traditional_player_stats[pid]
                        s.opp_rim_attempts_on += 1
                        if is_make:
                            s.opp_rim_makes_on += 1
                # Traditional: off (rest of roster)
                for pid in (roster - trad_def):
                    if pid in self.traditional_player_stats:
                        s = self.traditional_player_stats[pid]
                        s.opp_rim_attempts_off += 1
                        if is_make:
                            s.opp_rim_makes_off += 1

                # Enhanced: on
                for pid in enh_def:
                    if pid in self.enhanced_player_stats:
                        s = self.enhanced_player_stats[pid]
                        s.opp_rim_attempts_on += 1
                        if is_make:
                            s.opp_rim_makes_on += 1
                # Enhanced: off
                for pid in (roster - enh_def):
                    if pid in self.enhanced_player_stats:
                        s = self.enhanced_player_stats[pid]
                        s.opp_rim_attempts_off += 1
                        if is_make:
                            s.opp_rim_makes_off += 1

            # Percentages (diagnostic only; no fill-ins)
            for s in list(self.traditional_player_stats.values()) + list(self.enhanced_player_stats.values()):
                if s.opp_rim_attempts_on > 0:
                    s.opp_rim_fg_pct_on = s.opp_rim_makes_on / s.opp_rim_attempts_on
                if s.opp_rim_attempts_off > 0:
                    s.opp_rim_fg_pct_off = s.opp_rim_makes_off / s.opp_rim_attempts_off
                if s.opp_rim_fg_pct_on is not None and s.opp_rim_fg_pct_off is not None:
                    s.rim_defense_on_off = s.opp_rim_fg_pct_on - s.opp_rim_fg_pct_off

        except Exception as e:
            logger.warning(f"Error calculating rim defense: {e}")

    def create_dual_method_tables(self) -> ValidationResult:
        """Create comprehensive tables for both traditional and enhanced methods"""
        start_time = time.time()

        try:
            logger.info("Creating dual-method output tables...")

            if self.conn is None:
                self.conn = duckdb.connect(self.db_path)

            # Create traditional lineups table
            trad_lineups_data = []
            for (team_id, lineup), stats in self.traditional_lineup_stats.items():
                # Pad lineup to 5 players if needed, or truncate if more
                padded_lineup = list(lineup) + [None] * (5 - len(lineup))
                padded_names = list(stats.player_names) + [""] * (5 - len(stats.player_names))

                row = {
                    "method": "traditional",
                    "team_id": team_id,
                    "team_abbrev": stats.team_abbrev,
                    "lineup_size": stats.lineup_size,
                    "player_1_id": padded_lineup[0],
                    "player_1_name": padded_names[0] if padded_names[0] else "",
                    "player_2_id": padded_lineup[1] if len(padded_lineup) > 1 else None,
                    "player_2_name": padded_names[1] if len(padded_names) > 1 and padded_names[1] else "",
                    "player_3_id": padded_lineup[2] if len(padded_lineup) > 2 else None,
                    "player_3_name": padded_names[2] if len(padded_names) > 2 and padded_names[2] else "",
                    "player_4_id": padded_lineup[3] if len(padded_lineup) > 3 else None,
                    "player_4_name": padded_names[3] if len(padded_names) > 3 and padded_names[3] else "",
                    "player_5_id": padded_lineup[4] if len(padded_lineup) > 4 else None,
                    "player_5_name": padded_names[4] if len(padded_names) > 4 and padded_names[4] else "",
                    "off_possessions": stats.off_possessions,
                    "def_possessions": stats.def_possessions,
                    "points_for": stats.points_for,
                    "points_against": stats.points_against,
                    "off_rating": round(stats.off_rating, 1),
                    "def_rating": round(stats.def_rating, 1),
                    "net_rating": round(stats.net_rating, 1),
                    "violation_count": len(stats.lineup_violations),
                    "violation_summary": "; ".join(stats.lineup_violations[:3])  # Top 3 violations
                }
                trad_lineups_data.append(row)

            # Create enhanced lineups table
            enh_lineups_data = []
            for (team_id, lineup), stats in self.enhanced_lineup_stats.items():
                row = {
                    "method": "enhanced",
                    "team_id": team_id,
                    "team_abbrev": stats.team_abbrev,
                    "lineup_size": stats.lineup_size,
                    "player_1_id": lineup[0],
                    "player_1_name": stats.player_names[0],
                    "player_2_id": lineup[1],
                    "player_2_name": stats.player_names[1],
                    "player_3_id": lineup[2],
                    "player_3_name": stats.player_names[2],
                    "player_4_id": lineup[3],
                    "player_4_name": stats.player_names[3],
                    "player_5_id": lineup[4],
                    "player_5_name": stats.player_names[4],
                    "off_possessions": stats.off_possessions,
                    "def_possessions": stats.def_possessions,
                    "points_for": stats.points_for,
                    "points_against": stats.points_against,
                    "off_rating": round(stats.off_rating, 1),
                    "def_rating": round(stats.def_rating, 1),
                    "net_rating": round(stats.net_rating, 1),
                    "violation_count": 0,  # Enhanced method maintains 5-man lineups
                    "violation_summary": ""
                }
                enh_lineups_data.append(row)

            # Combine and create unified lineups table
            all_lineups_data = trad_lineups_data + enh_lineups_data
            lineups_df = pd.DataFrame(all_lineups_data)

            if not lineups_df.empty:
                self.conn.register("dual_lineups_temp", lineups_df)
                try:
                    self.conn.execute("""
                        CREATE OR REPLACE TABLE final_dual_lineups AS
                        SELECT * FROM dual_lineups_temp
                        ORDER BY method, team_abbrev, off_possessions DESC
                    """)
                finally:
                    self.conn.unregister("dual_lineups_temp")

            # Create dual players table
            trad_players_data = []
            for pid, stats in self.traditional_player_stats.items():
                trad_players_data.append({
                    "method": "traditional",
                    "player_id": pid,
                    "player_name": stats.player_name,
                    "team_id": stats.team_id,
                    "team_abbrev": stats.team_abbrev,
                    "off_possessions": stats.off_possessions,
                    "def_possessions": stats.def_possessions,
                    "opp_rim_attempts_on": stats.opp_rim_attempts_on,
                    "opp_rim_makes_on": stats.opp_rim_makes_on,
                    "opp_rim_attempts_off": stats.opp_rim_attempts_off,
                    "opp_rim_makes_off": stats.opp_rim_makes_off,
                    "opp_rim_fg_pct_on": round(stats.opp_rim_fg_pct_on, 3) if stats.opp_rim_fg_pct_on is not None else None,
                    "opp_rim_fg_pct_off": round(stats.opp_rim_fg_pct_off, 3) if stats.opp_rim_fg_pct_off is not None else None,
                    "rim_defense_on_off": round(stats.rim_defense_on_off, 3) if stats.rim_defense_on_off is not None else None
                })

            enh_players_data = []
            for pid, stats in self.enhanced_player_stats.items():
                enh_players_data.append({
                    "method": "enhanced",
                    "player_id": pid,
                    "player_name": stats.player_name,
                    "team_id": stats.team_id,
                    "team_abbrev": stats.team_abbrev,
                    "off_possessions": stats.off_possessions,
                    "def_possessions": stats.def_possessions,
                    "opp_rim_attempts_on": stats.opp_rim_attempts_on,
                    "opp_rim_makes_on": stats.opp_rim_makes_on,
                    "opp_rim_attempts_off": stats.opp_rim_attempts_off,
                    "opp_rim_makes_off": stats.opp_rim_makes_off,
                    "opp_rim_fg_pct_on": round(stats.opp_rim_fg_pct_on, 3) if stats.opp_rim_fg_pct_on is not None else None,
                    "opp_rim_fg_pct_off": round(stats.opp_rim_fg_pct_off, 3) if stats.opp_rim_fg_pct_off is not None else None,
                    "rim_defense_on_off": round(stats.rim_defense_on_off, 3) if stats.rim_defense_on_off is not None else None
                })

            all_players_data = trad_players_data + enh_players_data
            players_df = pd.DataFrame(all_players_data)

            if not players_df.empty:
                self.conn.register("dual_players_temp", players_df)
                try:
                    self.conn.execute("""
                        CREATE OR REPLACE TABLE final_dual_players AS
                        SELECT * FROM dual_players_temp
                        ORDER BY method, team_abbrev, player_name
                    """)
                finally:
                    self.conn.unregister("dual_players_temp")


            # Create method comparison summary
            self._create_method_comparison_table()

            # Create violation reports
            self._create_violation_reports()

            details = (f"Created dual-method tables: lineups({len(lineups_df)}), "
                      f"players({len(players_df)}), comparisons, violations")

            return ValidationResult(
                step_name="Create Dual Method Tables",
                passed=True,
                details=details,
                data_count=len(lineups_df) + len(players_df),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Create Dual Method Tables",
                passed=False,
                details=f"Error creating dual-method tables: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _create_method_comparison_table(self):
        """Create comprehensive method comparison table"""
        try:
            # Calculate comparison metrics
            trad_total_lineups = len(self.traditional_lineup_stats)
            enh_total_lineups = len(self.enhanced_lineup_stats)

            trad_5man_lineups = sum(1 for stats in self.traditional_lineup_stats.values() 
                                   if stats.lineup_size == 5)
            enh_5man_lineups = sum(1 for stats in self.enhanced_lineup_stats.values() 
                                  if stats.lineup_size == 5)

            trad_avg_possessions = np.mean([stats.off_possessions + stats.def_possessions 
                                          for stats in self.traditional_lineup_stats.values()])
            enh_avg_possessions = np.mean([stats.off_possessions + stats.def_possessions 
                                         for stats in self.enhanced_lineup_stats.values()])

            comparison_data = [
                {
                    "metric": "Total Lineups",
                    "traditional_value": trad_total_lineups,
                    "enhanced_value": enh_total_lineups,
                    "difference": enh_total_lineups - trad_total_lineups,
                    "better_method": "Enhanced" if enh_total_lineups < trad_total_lineups else "Traditional"
                },
                {
                    "metric": "5-Man Lineups",
                    "traditional_value": trad_5man_lineups,
                    "enhanced_value": enh_5man_lineups,
                    "difference": enh_5man_lineups - trad_5man_lineups,
                    "better_method": "Enhanced" if enh_5man_lineups > trad_5man_lineups else "Traditional"
                },
                {
                    "metric": "5-Man Percentage",
                    "traditional_value": round(100 * trad_5man_lineups / max(1, trad_total_lineups), 1),
                    "enhanced_value": round(100 * enh_5man_lineups / max(1, enh_total_lineups), 1),
                    "difference": round(100 * (enh_5man_lineups / max(1, enh_total_lineups) - 
                                             trad_5man_lineups / max(1, trad_total_lineups)), 1),
                    "better_method": "Enhanced"
                },
                {
                    "metric": "Avg Possessions per Lineup",
                    "traditional_value": round(trad_avg_possessions, 1),
                    "enhanced_value": round(enh_avg_possessions, 1),
                    "difference": round(enh_avg_possessions - trad_avg_possessions, 1),
                    "better_method": "Enhanced" if enh_avg_possessions > trad_avg_possessions else "Traditional"
                },
                {
                    "metric": "Violation Flags",
                    "traditional_value": len(self.traditional_violations),
                    "enhanced_value": len(self.enhanced_violations),
                    "difference": len(self.enhanced_violations) - len(self.traditional_violations),
                    "better_method": "Traditional" if len(self.traditional_violations) < len(self.enhanced_violations) else "Enhanced"
                }
            ]

            comparison_df = pd.DataFrame(comparison_data)
            self.conn.register("comparison_temp", comparison_df)
            self.conn.execute("""
                CREATE OR REPLACE TABLE method_comparison_summary AS
                SELECT * FROM comparison_temp
            """)
            self.conn.unregister("comparison_temp")

        except Exception as e:
            logger.warning(f"Error creating method comparison: {e}")

    def _create_violation_reports(self):
        """Create detailed violation reports for both methods with normalized schema and stable ordering.

        Normalization rules (no synthesis of unknown values):
        - Add 'order_ts' = first available of ['abs_time','wall_clock_int','pbp_order'] (as-is).
        - Ensure 'period' exists (if not, leave absent).
        - Ensure 'team_abbrev' exists; if only 'team_id' present, map using self.team_abbrev (deterministic).
        - Ensure 'player_name' exists; if only 'player_id' present, map using self.player_names (deterministic).
        - Pass through 'flag_type','description','flag_details' if present; omit if not.
        """
        try:
            def _normalize(df_in: pd.DataFrame) -> pd.DataFrame:
                if df_in is None or df_in.empty:
                    return pd.DataFrame()

                df = df_in.copy()

                cols = set(c.lower() for c in df.columns)

                # Standardize casing to avoid surprises later
                rename_map = {c: c for c in df.columns}  # identity
                # common alternates could be added here if you encounter them:
                # e.g., rename_map['team'] = 'team_abbrev' if 'team' in df

                df = df.rename(columns=rename_map)

                # Build order_ts without assuming existence
                order_candidates = [c for c in ["abs_time", "wall_clock_int", "pbp_order"] if c in df.columns]
                if order_candidates:
                    first = order_candidates[0]
                    df["order_ts"] = df[first]
                # else: no order column at all; exporter will handle lack of ordering gracefully

                # team_abbrev from team_id if needed
                if "team_abbrev" not in df.columns and "team_id" in df.columns and self.team_abbrev:
                    df["team_abbrev"] = df["team_id"].map(lambda tid: self.team_abbrev.get(int(tid)) if pd.notna(tid) else None)

                # player_name from player_id if needed
                if "player_name" not in df.columns and "player_id" in df.columns and self.player_names:
                    df["player_name"] = df["player_id"].map(lambda pid: self.player_names.get(int(pid)) if pd.notna(pid) else None)

                # Keep only a sensible set + anything extra that came in (don’t drop unknowns)
                preferred_order = [
                    "period", "order_ts", "abs_time", "wall_clock_int", "pbp_order",
                    "team_abbrev", "team_id", "flag_type", "player_name", "player_id",
                    "description", "flag_details"
                ]
                # Reorder columns: known first, then the rest in original order
                known = [c for c in preferred_order if c in df.columns]
                rest  = [c for c in df.columns if c not in known]
                df = df[known + rest]

                return df

            def _write_table(temp_name: str, final_name: str, df: pd.DataFrame):
                if df is None or df.empty:
                    return
                self.conn.register(temp_name, df)
                try:
                    cols = [c[1] for c in self.conn.execute(f"PRAGMA table_info('{temp_name}')").fetchall()]
                    order_expr = "order_ts" if "order_ts" in cols else None
                    if order_expr:
                        self.conn.execute(f"""
                            CREATE OR REPLACE TABLE {final_name} AS
                            SELECT * FROM {temp_name}
                            ORDER BY {order_expr}
                        """)
                    else:
                        self.conn.execute(f"""
                            CREATE OR REPLACE TABLE {final_name} AS
                            SELECT * FROM {temp_name}
                        """)
                finally:
                    self.conn.unregister(temp_name)

            # Traditional
            if self.traditional_violations:
                trad_df = _normalize(pd.DataFrame(self.traditional_violations))
                _write_table("trad_violations_temp", "traditional_violation_report", trad_df)

            # Enhanced
            if self.enhanced_violations:
                enh_df = _normalize(pd.DataFrame(self.enhanced_violations))
                _write_table("enh_violations_temp", "enhanced_violation_report", enh_df)

        except Exception as e:
            logger.warning(f"Error creating violation reports: {e}")



    def print_dual_method_summary(self):
        """Print comprehensive summary of both methods"""
        print("\n" + "="*80)
        print("NBA PIPELINE - STEP 5 DUAL-METHOD SUMMARY")
        print("="*80)

        print("POSSESSION ANALYSIS:")
        print(f"  Total Dual Possessions: {len(self.dual_possessions):,}")
        if self.dual_possessions:
            total_points = sum(p.points_scored for p in self.dual_possessions)
            periods = len(set(p.period for p in self.dual_possessions))
            print(f"  Total Points: {total_points:,}")
            print(f"  Periods: {periods}")

        print("\nTRADITIONAL METHOD RESULTS:")
        print(f"  Unique Lineups: {len(self.traditional_lineup_stats):,}")
        if self.traditional_lineup_stats:
            trad_5man = sum(1 for s in self.traditional_lineup_stats.values() if s.lineup_size == 5)
            print(f"  5-Man Lineups: {trad_5man:,} ({100*trad_5man/len(self.traditional_lineup_stats):.1f}%)")
            trad_violations = len(self.traditional_violations)
            print(f"  Violation Flags: {trad_violations:,}")

        print("\nENHANCED METHOD RESULTS:")
        print(f"  Unique Lineups: {len(self.enhanced_lineup_stats):,}")
        if self.enhanced_lineup_stats:
            enh_5man = sum(1 for s in self.enhanced_lineup_stats.values() if s.lineup_size == 5)
            print(f"  5-Man Lineups: {enh_5man:,} ({100*enh_5man/len(self.enhanced_lineup_stats):.1f}%)")
            enh_violations = len(self.enhanced_violations)
            print(f"  Violation Flags: {enh_violations:,}")

        print("\nPLAYER RIM DEFENSE:")
        trad_with_rim = sum(1 for s in self.traditional_player_stats.values() if s.opp_rim_attempts_on > 0)
        enh_with_rim = sum(1 for s in self.enhanced_player_stats.values() if s.opp_rim_attempts_on > 0)
        print(f"  Traditional Players with Rim Data: {trad_with_rim:,}")
        print(f"  Enhanced Players with Rim Data: {enh_with_rim:,}")

        print("\nMETHOD EFFECTIVENESS:")
        if self.traditional_lineup_stats and self.enhanced_lineup_stats:
            improvement = len(self.traditional_lineup_stats) - len(self.enhanced_lineup_stats)
            print(f"  Lineup Count Change: {improvement:+,} (Enhanced has fewer unique lineups)")

            trad_5_pct = 100 * sum(1 for s in self.traditional_lineup_stats.values() if s.lineup_size == 5) / len(self.traditional_lineup_stats)
            enh_5_pct = 100 * sum(1 for s in self.enhanced_lineup_stats.values() if s.lineup_size == 5) / len(self.enhanced_lineup_stats)
            print(f"  5-Man Accuracy: Traditional {trad_5_pct:.1f}% vs Enhanced {enh_5_pct:.1f}%")

        print("="*80)

    def create_project_output_tables(self) -> ValidationResult:
        """
        Project deliverables (no fallbacks):
        - project1_lineups: 5-man lineups per team (ENHANCED ONLY) with Off/Def/Net ratings.
        - project2_players: player-level rim defense on/off (ENHANCED ONLY).
        """
        start_time = time.time()
        try:
            # --- Project 1: 5-man lineups (enhanced only) ---
            p1_rows = []
            for (team_id, lineup), stats in self.enhanced_lineup_stats.items():
                # Enhanced is guaranteed 5
                if len(lineup) != 5:
                    continue
                row = {
                    "team_id": team_id,
                    "team_abbrev": stats.team_abbrev,
                    "player_1_id": lineup[0], "player_1_name": stats.player_names[0],
                    "player_2_id": lineup[1], "player_2_name": stats.player_names[1],
                    "player_3_id": lineup[2], "player_3_name": stats.player_names[2],
                    "player_4_id": lineup[3], "player_4_name": stats.player_names[3],
                    "player_5_id": lineup[4], "player_5_name": stats.player_names[4],
                    "off_possessions": stats.off_possessions,
                    "def_possessions": stats.def_possessions,
                    "off_rating": round(stats.off_rating, 1),
                    "def_rating": round(stats.def_rating, 1),
                    "net_rating": round(stats.net_rating, 1),
                }
                p1_rows.append(row)
            p1_df = pd.DataFrame(p1_rows).sort_values(
                ["team_abbrev", "off_possessions"], ascending=[True, False]
            )
            if not p1_df.empty:
                self.conn.register("project1_temp", p1_df)
                try:
                    self.conn.execute("""
                        CREATE OR REPLACE TABLE project1_lineups AS
                        SELECT * FROM project1_temp
                    """)
                finally:
                    self.conn.unregister("project1_temp")

            # --- Project 2: player rim defense on/off (enhanced only) ---
            p2_rows = []
            for pid, s in self.enhanced_player_stats.items():
                p2_rows.append({
                    "player_id": pid,
                    "player_name": s.player_name,
                    "team_id": s.team_id,
                    "team_abbrev": s.team_abbrev,
                    "off_possessions": s.off_possessions,
                    "def_possessions": s.def_possessions,
                    "opp_rim_attempts_on": s.opp_rim_attempts_on,
                    "opp_rim_makes_on": s.opp_rim_makes_on,
                    "opp_rim_attempts_off": s.opp_rim_attempts_off,
                    "opp_rim_makes_off": s.opp_rim_makes_off,
                    "opp_rim_fg_pct_on": round(s.opp_rim_fg_pct_on, 3) if s.opp_rim_fg_pct_on is not None else None,
                    "opp_rim_fg_pct_off": round(s.opp_rim_fg_pct_off, 3) if s.opp_rim_fg_pct_off is not None else None,
                    "rim_defense_on_off": round(s.rim_defense_on_off, 3) if s.rim_defense_on_off is not None else None,
                })
            p2_df = pd.DataFrame(p2_rows).sort_values(["team_abbrev", "player_name"])
            if not p2_df.empty:
                self.conn.register("project2_temp", p2_df)
                try:
                    self.conn.execute("""
                        CREATE OR REPLACE TABLE project2_players AS
                        SELECT * FROM project2_temp
                    """)
                finally:
                    self.conn.unregister("project2_temp")

            details = (f"Project outputs created: "
                    f"project1_lineups({len(p1_df)}), project2_players({len(p2_df)})")
            return ValidationResult(
                step_name="Create Project Output Tables",
                passed=True,
                details=details,
                data_count=len(p1_df) + len(p2_df),
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Create Project Output Tables",
                passed=False,
                details=f"Error creating project outputs: {e}",
                processing_time=time.time() - start_time
            )


def run_dual_method_possession_engine(db_path: str = None,
                                    entities: GameEntities = None) -> Tuple[bool, DualMethodPossessionEngine]:
    """Run the complete dual-method possession engine pipeline"""

    print("NBA Pipeline - UPDATED Step 5: Dual-Method Possession Engine")
    print("="*70)

    if entities is None:
        logger.error("GameEntities required for dual-method possession engine")
        return False, None

    with DualMethodPossessionEngine(db_path, entities) as engine:

        # Run comprehensive diagnostic first
        logger.info("Step 5a: Running pipeline diagnostic...")
        diagnostic = engine.diagnose_pipeline_state()
        logger.info(f"Pipeline diagnostic completed: {diagnostic}")

        # Load dual-method data from Step 2/4 (autorun rebuild ON)
        logger.info("Step 5b: Loading dual-method data...")
        result = engine.load_dual_method_data(autorun_rebuild=True)
        engine.validator.log_validation(result)
        if not result.passed:
            return False, engine

        # Identify possessions with dual lineup contexts
        logger.info("Step 5c: Identifying dual-method possessions...")
        result = engine.identify_dual_possessions()
        engine.validator.log_validation(result)
        if not result.passed:
            return False, engine

        # Calculate lineup statistics for both methods
        logger.info("Step 5d: Calculating dual-method lineup statistics...")
        result = engine.calculate_dual_lineup_stats()
        engine.validator.log_validation(result)

        # Calculate player rim statistics for both methods
        logger.info("Step 5e: Calculating dual-method player rim statistics...")
        result = engine.calculate_dual_player_rim_stats()
        engine.validator.log_validation(result)

        # Run comprehensive debugging to identify points discrepancy
        logger.info("Step 5f: Running comprehensive points flow debugging...")
        debug_info = engine.debug_points_flow_comprehensive()
        logger.info(f"Points flow debug completed: {debug_info}")
        
        # Run detailed HOU analysis
        logger.info("Step 5g: Running detailed HOU scoring events analysis...")
        hou_analysis = engine.debug_hou_scoring_events_detailed()
        logger.info(f"HOU analysis completed: {hou_analysis}")

        # Create comprehensive output tables (dual-method)
        logger.info("Step 5f: Creating dual-method output tables...")
        result = engine.create_dual_method_tables()
        engine.validator.log_validation(result)

        # ---> NEW: Project deliverables (enhanced-only) <---
        logger.info("Step 5g: Creating project deliverable tables (enhanced only)...")
        result = engine.create_project_output_tables()
        engine.validator.log_validation(result)

        # Print summary
        engine.print_dual_method_summary()

        success = engine.validator.print_validation_summary()
        return success, engine





if __name__ == "__main__":
    from eda.data.nba_entities_extractor import extract_all_entities_robust

    database_path = "mavs_enhanced.duckdb"
    ok, entities = extract_all_entities_robust(database_path)

    if ok:
        success, engine = run_dual_method_possession_engine(database_path, entities)
        if success:
            print("\nUPDATED Step 5 Complete: Dual-method possession engine")
            print("Both Traditional and Enhanced statistics calculated")
            print("Ready for comprehensive dual-method export (Step 6)")
        else:
            print("\nUPDATED Step 5 Failed: Review validation messages")
    else:
        print("Failed to get entities - cannot proceed")


Overwriting api/src/airflow_project/eda/data/nba_possession_engine.py


Step 6: Final Validation & Export Results

In [33]:
%%writefile api/src/airflow_project/eda/data/nba_final_export.py
# Step 6: Dual-Method Final Validation & Export Results
"""
NBA Pipeline - UPDATED Step 6: Dual-Method Final Validation & Export
====================================================================

UPDATED to integrate Step 2 findings and dual-method approach:
- Validates and exports BOTH traditional and enhanced method results
- Includes comprehensive violation reports for traditional lineups
- Generates method comparison and effectiveness analysis
- Uses config-driven automation for paths and settings
- Creates base dataset reports for final project submission
- Comprehensive validation against box score data

Key Integration Points from Step 2/5:
1. Exports traditional results (variable lineup sizes, violations included)
2. Exports enhanced results (5-man lineups, intelligent inference)
3. Creates violation analysis and comparison reports
4. Uses CONFIG for automation settings and paths
5. Generates final project deliverables in specified format
"""
import os
import sys
# Ensure we're in the right directory
cwd = os.getcwd()
if not cwd.endswith("airflow_project"):
    os.chdir('api/src/airflow_project')
sys.path.insert(0, os.getcwd())


import duckdb
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import time
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
import csv
import json

from eda.utils.nba_pipeline_analysis import NBADataValidator, ValidationResult
from eda.data.nba_entities_extractor import GameEntities

# Load configuration
try:
    from utils.config import (
        NBA_SUBSTITUTION_CONFIG,
        DUCKDB_PATH,
        EXPORTS_DIR,
        PROCESSED_DIR,
        LOGS_DIR,
        LINEUPS_OUTPUT_COLUMNS,
        PLAYERS_OUTPUT_COLUMNS,
        get_column_usage_report
    )
    CONFIG = NBA_SUBSTITUTION_CONFIG
    DB_PATH = str(DUCKDB_PATH)
    EXPORT_DIR = EXPORTS_DIR
    PROCESSED_DIR = PROCESSED_DIR
    LOGS_DIR = LOGS_DIR
    LINEUP_COLUMNS = LINEUPS_OUTPUT_COLUMNS
    PLAYER_COLUMNS = PLAYERS_OUTPUT_COLUMNS
except ImportError:
    logger.warning("Config not available, using defaults")
    CONFIG = {"debug": {"log_all_substitutions": True}}
    DB_PATH = "mavs_enhanced.duckdb"
    EXPORT_DIR = Path("exports")
    PROCESSED_DIR = Path("processed")
    LOGS_DIR = Path("logs")
    LINEUP_COLUMNS = ["Team", "Player 1", "Player 2", "Player 3", "Player 4", "Player 5", 
                     "Offensive possessions played", "Defensive possessions played",
                     "Offensive rating", "Defensive rating", "Net rating"]
    PLAYER_COLUMNS = ["Player ID", "Player Name", "Team", "Offensive possessions played", 
                     "Defensive possessions played", 
                     "Opponent rim field goal percentage when player is on the court",
                     "Opponent rim field goal percentage when player is off the court",
                     "Opponent rim field goal percentage on/off difference (on-off)"]

logger = logging.getLogger(__name__)

@dataclass
class DualMethodValidationTolerance:
    """Enhanced tolerance levels for dual-method validation"""
    points_tolerance: int = 2
    possession_tolerance_pct: float = 0.05
    minutes_tolerance_sec: int = 30
    rating_sanity_min: float = 50.0
    rating_sanity_max: float = 200.0

    # Method comparison tolerances
    lineup_count_difference_max: int = 10  # Max acceptable difference in lineup counts
    possession_difference_max_pct: float = 0.10  # Max 10% difference in possessions
    rating_difference_max: float = 20.0  # Max 20 point rating difference

@dataclass
class ExportSummary:
    """Summary of exported files and metrics"""
    files_exported: List[str] = field(default_factory=list)
    traditional_lineups: int = 0
    enhanced_lineups: int = 0
    traditional_players: int = 0
    enhanced_players: int = 0
    violation_reports: int = 0
    comparison_reports: int = 0
    total_file_size_mb: float = 0.0

class DualMethodFinalValidator:
    """UPDATED: Comprehensive dual-method validation and export"""

    def __init__(self, db_path: str = None, entities: GameEntities = None, 
                 ascii_only: bool = True):
        self.db_path = db_path or DB_PATH
        self.conn = None
        self.entities = entities
        self.validator = NBADataValidator()
        self.tolerance = DualMethodValidationTolerance()
        self.ascii_only = ascii_only

        # Export configuration using config system
        self.export_dir = EXPORT_DIR
        self.export_dir.mkdir(parents=True, exist_ok=True)

        # Processing directories
        self.processed_dir = PROCESSED_DIR
        self.processed_dir.mkdir(parents=True, exist_ok=True)

        self.logs_dir = LOGS_DIR
        self.logs_dir.mkdir(parents=True, exist_ok=True)

        # Export summary
        self.export_summary = ExportSummary()

    # --- helper: map status icons to ASCII ---
    @staticmethod
    def _status_label(passed: bool) -> str:
        return "[PASS]" if passed else "[FAIL]"

    @staticmethod
    def _warn_label() -> str:
        return "WARN"

    def _sanitize_for_file(self, text: str) -> str:
        """
        Keep report readable everywhere. If ascii_only is True, strip/replace
        non-ASCII chars. We intentionally replace just decoration; content remains.
        """
        if not self.ascii_only:
            return text
        # Encode-decode via 'ascii' with 'replace' to surface any stray glyphs as '?'
        # But first do targeted replacements for known emojis so we don't lose meaning.
        text = (text
                .replace("✅", "[PASS]")
                .replace("❌", "[FAIL]")
                .replace("⚠️", "WARN")
                .replace("⚠", "WARN")
                .replace("📁", "FILES")
                .replace("📄", "-")
                .replace("🏟️", "ARENA")
                .replace("🏠", "HOME")
                .replace("✈️", "AWAY")
                .replace("→", "->"))
        try:
            return text.encode("ascii", "replace").decode("ascii")
        except Exception:
            # As a last resort, drop non-ascii
            return "".join(ch if ord(ch) < 128 else "?" for ch in text)


    def __enter__(self):
        self.conn = duckdb.connect(self.db_path)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.close()

    def validate_dual_method_tables_exist(self) -> ValidationResult:
        """Validate that both traditional and enhanced method tables exist"""
        start_time = time.time()

        try:
            logger.info("Validating dual-method tables exist...")

            # Check for required dual-method tables
            required_tables = [
                'final_dual_lineups',
                'final_dual_players', 
                'method_comparison_summary',
                'traditional_violation_report',
                'enhanced_violation_report'
            ]

            existing_tables = [
                row[0] for row in self.conn.execute(
                    "SELECT table_name FROM information_schema.tables"
                ).fetchall()
            ]

            missing_tables = [t for t in required_tables if t not in existing_tables]
            warnings = []

            if missing_tables:
                warnings.append(f"Missing dual-method tables: {missing_tables}")

            # Check table contents
            table_counts = {}
            for table in [t for t in required_tables if t not in missing_tables]:
                try:
                    count = self.conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
                    table_counts[table] = count
                    if count == 0:
                        warnings.append(f"Table {table} exists but is empty")
                except Exception as e:
                    warnings.append(f"Error checking table {table}: {e}")

            # Validate dual-method structure
            if 'final_dual_lineups' in table_counts:
                method_counts = self.conn.execute("""
                    SELECT method, COUNT(*) as count
                    FROM final_dual_lineups
                    GROUP BY method
                """).df()

                if not method_counts.empty:
                    methods = set(method_counts['method'].tolist())
                    expected_methods = {'traditional', 'enhanced'}
                    if methods != expected_methods:
                        warnings.append(f"Expected methods {expected_methods}, found {methods}")

                    for _, row in method_counts.iterrows():
                        if row['method'] == 'traditional':
                            self.export_summary.traditional_lineups = int(row['count'])
                        elif row['method'] == 'enhanced':
                            self.export_summary.enhanced_lineups = int(row['count'])

            passed = len(missing_tables) == 0 and len([w for w in warnings if 'empty' in w]) == 0
            details = f"Dual-method tables validated: {table_counts}"

            return ValidationResult(
                step_name="Dual Method Tables Check",
                passed=passed,
                details=details,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Dual Method Tables Check",
                passed=False,
                details=f"Error validating dual-method tables: {str(e)}",
                processing_time=time.time() - start_time
            )


    def validate_against_box_score_dual(self) -> ValidationResult:
        """Enhanced version with detailed debugging of the points mismatch"""
        start_time = time.time()
        try:
            logger.info("Validating dual-method results against box score (DEBUG MODE)...")

            # 1) Box score totals with more detail
            box_totals = self.conn.execute("""
                SELECT 
                    team_abbrev,
                    SUM(points) AS total_points,
                    SUM(seconds_played) AS total_seconds,
                    COUNT(*) AS active_players,
                    STRING_AGG(player_name || ':' || CAST(points AS VARCHAR), ', ') AS player_breakdown
                FROM box_score
                WHERE status = 'ACTIVE'
                GROUP BY team_abbrev
                ORDER BY team_abbrev
            """).df()

            # 2) Calculated lineup totals with debugging info
            method_totals = self.conn.execute("""
                SELECT 
                    method,
                    team_abbrev,
                    SUM(points_for) AS calc_points_for,
                    SUM(points_against) AS calc_points_against,
                    SUM(off_possessions) AS total_off_poss,
                    SUM(def_possessions) AS total_def_poss,
                    COUNT(*) AS unique_lineups,
                    AVG(off_rating) AS avg_off_rating,
                    AVG(def_rating) AS avg_def_rating
                FROM final_dual_lineups
                GROUP BY method, team_abbrev
                ORDER BY method, team_abbrev
            """).df()

            # 3) Raw event totals for comparison
            raw_event_totals = self.conn.execute("""
                SELECT 
                    CASE 
                        WHEN off_team_id = 1610612742 THEN 'DAL'
                        WHEN off_team_id = 1610612745 THEN 'HOU'
                        ELSE CAST(off_team_id AS VARCHAR)
                    END as team_abbrev,
                    SUM(COALESCE(points, 0)) AS raw_event_points,
                    COUNT(CASE WHEN points > 0 THEN 1 END) AS scoring_events
                FROM step4_processed_events
                WHERE off_team_id IS NOT NULL
                GROUP BY off_team_id
                ORDER BY team_abbrev
            """).df()

            warnings = []
            debug_lines = []

            # 4) Enhanced per-team comparison
            for _, box_row in box_totals.iterrows():
                team = box_row['team_abbrev']
                box_pts = int(box_row['total_points'])
                
                # Get raw event points
                raw_row = raw_event_totals[raw_event_totals['team_abbrev'] == team]
                raw_pts = int(raw_row.iloc[0]['raw_event_points']) if not raw_row.empty else 0
                raw_events = int(raw_row.iloc[0]['scoring_events']) if not raw_row.empty else 0
                
                debug_lines.append(f"=== {team} DEBUG ===")
                debug_lines.append(f"Box Score: {box_pts} points ({box_row['active_players']} players)")
                debug_lines.append(f"Raw Events: {raw_pts} points ({raw_events} scoring events)")
                debug_lines.append(f"Player Breakdown: {box_row['player_breakdown']}")
                
                # Check each method
                for method in ['traditional', 'enhanced']:
                    method_row = method_totals[
                        (method_totals['method'] == method) & 
                        (method_totals['team_abbrev'] == team)
                    ]
                    
                    if method_row.empty:
                        warnings.append(f"{method.title()} method missing data for team {team}")
                        debug_lines.append(f"{method.title()}: MISSING DATA")
                        continue
                    
                    calc_pts = int(method_row.iloc[0]['calc_points_for'])
                    lineups = int(method_row.iloc[0]['unique_lineups'])
                    possessions = int(method_row.iloc[0]['total_off_poss'])
                    avg_off_rating = float(method_row.iloc[0]['avg_off_rating'])
                    
                    diff = calc_pts - box_pts
                    raw_diff = calc_pts - raw_pts
                    
                    tag = "OK" if diff == 0 else ("SHORT" if diff < 0 else "OVER")
                    
                    debug_lines.append(f"{method.title()}: {calc_pts} points ({lineups} lineups, {possessions} poss)")
                    debug_lines.append(f"  vs Box: {diff:+} | vs Raw Events: {raw_diff:+} | Rating: {avg_off_rating:.1f}")
                    
                    if abs(diff) > self.tolerance.points_tolerance:
                        warnings.append(f"{method.title()} {team} points mismatch: box={box_pts}, calc={calc_pts} (diff={diff})")
                        
                        # Additional debugging for mismatched teams
                        if team == "HOU" and diff > 0:  # Focus on HOU overage
                            debug_lines.append(f"  *** HOU OVERAGE DETECTED: +{diff} points ***")
                            debug_lines.append(f"  This suggests either:")
                            debug_lines.append(f"    1. Double-counting of scoring events")
                            debug_lines.append(f"    2. Attribution of points to wrong possessions")
                            debug_lines.append(f"    3. Events not properly filtered")

                debug_lines.append("")  # Spacing between teams

            # 5) Log all debug information
            for line in debug_lines:
                logger.info(line)

            # 6) Summary comparison
            summary_lines = []
            for _, row in box_totals.iterrows():
                team = row['team_abbrev']
                box_pts = int(row['total_points'])
                
                for method in ['traditional', 'enhanced']:
                    method_data = method_totals[
                        (method_totals['method'] == method) & 
                        (method_totals['team_abbrev'] == team)
                    ]
                    if not method_data.empty:
                        calc_pts = int(method_data.iloc[0]['calc_points_for'])
                        diff = calc_pts - box_pts
                        tag = "OK" if diff == 0 else ("SHORT" if diff < 0 else "OVER")
                        summary_lines.append(f"{method.title()} {team}: calc={calc_pts}, box={box_pts}, diff={diff} ({tag})")

            detail_str = " | ".join(summary_lines)
            
            return ValidationResult(
                step_name="Dual Method Box Score Validation",
                passed=all(("mismatch" not in w and "missing" not in w) for w in warnings),
                details=f"Per-team results: {detail_str}",
                processing_time=time.time() - start_time,
                warnings=warnings
            )
            
        except Exception as e:
            return ValidationResult(
                step_name="Dual Method Box Score Validation",
                passed=False,
                details=f"Error validating dual methods against box score: {str(e)}",
                processing_time=time.time() - start_time
            )



    def validate_data_completeness(self) -> ValidationResult:
        """Validate completeness and quality of final results (robust, no UNION alias traps)."""
        start_time = time.time()
        try:
            logger.info("Validating data completeness...")

            warnings = []

            # ---- Lineup totals & averages ----
            lineup_row = self.conn.execute("""
                SELECT 
                    COUNT(*) AS total_lineups,
                    COUNT(CASE WHEN off_possessions > 0 THEN 1 END) AS lineups_with_off_poss,
                    COUNT(CASE WHEN def_possessions > 0 THEN 1 END) AS lineups_with_def_poss,
                    COUNT(CASE WHEN off_possessions > 0 AND def_possessions > 0 THEN 1 END) AS complete_lineups,
                    AVG(off_possessions) AS avg_off_poss,
                    AVG(def_possessions) AS avg_def_poss
                FROM final_lineups
            """).fetchone()

            total_lineups      = lineup_row[0] or 0
            with_off_lineups   = lineup_row[1] or 0
            with_def_lineups   = lineup_row[2] or 0
            complete_lineups   = lineup_row[3] or 0
            avg_off_lineups    = float(lineup_row[4]) if lineup_row[4] is not None else 0.0
            avg_def_lineups    = float(lineup_row[5]) if lineup_row[5] is not None else 0.0

            if total_lineups == 0:
                warnings.append("No lineups in final table")
            if complete_lineups < total_lineups * 0.8:
                warnings.append(f"Only {complete_lineups}/{total_lineups} lineups have both offensive and defensive data")
            if avg_off_lineups < 5:
                warnings.append(f"Low average offensive possessions per lineup: {avg_off_lineups:.1f}")

            # ---- Player totals ----
            player_row = self.conn.execute("""
                SELECT 
                    COUNT(*) AS total_players,
                    COUNT(CASE WHEN off_possessions > 0 THEN 1 END) AS players_with_off_poss,
                    COUNT(CASE WHEN def_possessions > 0 THEN 1 END) AS players_with_def_poss,
                    COUNT(CASE WHEN opp_rim_attempts_on  > 0 THEN 1 END) AS players_with_rim_on,
                    COUNT(CASE WHEN opp_rim_attempts_off > 0 THEN 1 END) AS players_with_rim_off,
                    COUNT(CASE WHEN rim_defense_on_off IS NOT NULL THEN 1 END) AS players_with_rim_diff
                FROM final_players
            """).fetchone()

            total_players      = player_row[0] or 0
            with_off_players   = player_row[1] or 0
            with_def_players   = player_row[2] or 0
            with_rim_on        = player_row[3] or 0
            with_rim_off       = player_row[4] or 0
            with_rim_diff      = player_row[5] or 0

            if total_players == 0:
                warnings.append("No players in final table")
            if with_rim_on == 0:
                warnings.append("No players have rim defense data (on court)")
            if with_rim_diff < max(1, total_players * 0.5):
                warnings.append(f"Only {with_rim_diff}/{total_players} players have complete rim on/off data")

            # ---- Null checks (run as separate queries to avoid UNION alias mismatch) ----
            lineup_nulls = self.conn.execute("""
                SELECT 
                    SUM(CASE WHEN team_abbrev IS NULL THEN 1 ELSE 0 END) AS null_team,
                    SUM(CASE WHEN off_possessions > 0 AND off_rating IS NULL THEN 1 ELSE 0 END) AS null_off_rating,
                    SUM(CASE WHEN def_possessions > 0 AND def_rating IS NULL THEN 1 ELSE 0 END) AS null_def_rating
                FROM final_lineups
            """).fetchone()

            players_nulls = self.conn.execute("""
                SELECT
                    SUM(CASE WHEN team_abbrev IS NULL THEN 1 ELSE 0 END) AS null_team,
                    -- treat blank/whitespace names as null for quality
                    SUM(CASE WHEN player_name IS NULL OR LENGTH(TRIM(player_name)) = 0 THEN 1 ELSE 0 END) AS null_name
                FROM final_players
            """).fetchone()

            # Lineups nulls
            ln_null_team       = int(lineup_nulls[0] or 0)
            ln_null_off_rating = int(lineup_nulls[1] or 0)
            ln_null_def_rating = int(lineup_nulls[2] or 0)

            if ln_null_team > 0:
                warnings.append(f"lineups table has {ln_null_team} records with null team")
            if ln_null_off_rating > 0:
                warnings.append(f"lineups table has {ln_null_off_rating} records with null offensive rating")
            if ln_null_def_rating > 0:
                warnings.append(f"lineups table has {ln_null_def_rating} records with null defensive rating")

            # Players nulls
            pl_null_team = int(players_nulls[0] or 0)
            pl_null_name = int(players_nulls[1] or 0)

            if pl_null_team > 0:
                warnings.append(f"players table has {pl_null_team} records with null team")
            if pl_null_name > 0:
                warnings.append(f"players table has {pl_null_name} records with null or blank names")

            details = (
                f"Completeness check: {total_lineups} lineups, {total_players} players, "
                f"{complete_lineups} complete lineups, {with_rim_diff} players with rim on/off"
            )

            return ValidationResult(
                step_name="Data Completeness",
                passed=(len(warnings) == 0),
                details=details,
                data_count=(total_lineups + total_players),
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Data Completeness",
                passed=False,
                details=f"Error validating completeness: {str(e)}",
                processing_time=time.time() - start_time
            )


    def export_project_deliverables(self) -> ValidationResult:
        """Export final project deliverables in required format"""
        start_time = time.time()

        try:
            logger.info("Exporting project deliverables for both methods...")

            exported_files = []

            # PROJECT 1: Lineup Tables (Both Methods)

            # Traditional Method - Project 1
            traditional_lineups = self.conn.execute(f"""
                SELECT 
                    team_abbrev as "{LINEUP_COLUMNS[0]}",
                    player_1_name as "{LINEUP_COLUMNS[1]}",
                    player_2_name as "{LINEUP_COLUMNS[2]}",
                    player_3_name as "{LINEUP_COLUMNS[3]}",
                    player_4_name as "{LINEUP_COLUMNS[4]}",
                    player_5_name as "{LINEUP_COLUMNS[5]}",
                    off_possessions as "{LINEUP_COLUMNS[6]}",
                    def_possessions as "{LINEUP_COLUMNS[7]}",
                    off_rating as "{LINEUP_COLUMNS[8]}",
                    def_rating as "{LINEUP_COLUMNS[9]}",
                    net_rating as "{LINEUP_COLUMNS[10]}"
                FROM final_dual_lineups
                WHERE method = 'traditional' 
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, off_possessions DESC
            """).df()

            traditional_file = self.export_dir / "project1_lineups_traditional.csv"
            traditional_lineups.to_csv(traditional_file, index=False)
            exported_files.append(f"project1_lineups_traditional.csv ({len(traditional_lineups)} lineups)")

            # Enhanced Method - Project 1
            enhanced_lineups = self.conn.execute(f"""
                SELECT 
                    team_abbrev as "{LINEUP_COLUMNS[0]}",
                    player_1_name as "{LINEUP_COLUMNS[1]}",
                    player_2_name as "{LINEUP_COLUMNS[2]}",
                    player_3_name as "{LINEUP_COLUMNS[3]}",
                    player_4_name as "{LINEUP_COLUMNS[4]}",
                    player_5_name as "{LINEUP_COLUMNS[5]}",
                    off_possessions as "{LINEUP_COLUMNS[6]}",
                    def_possessions as "{LINEUP_COLUMNS[7]}",
                    off_rating as "{LINEUP_COLUMNS[8]}",
                    def_rating as "{LINEUP_COLUMNS[9]}",
                    net_rating as "{LINEUP_COLUMNS[10]}"
                FROM final_dual_lineups
                WHERE method = 'enhanced'
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, off_possessions DESC
            """).df()

            enhanced_file = self.export_dir / "project1_lineups_enhanced.csv"
            enhanced_lineups.to_csv(enhanced_file, index=False)
            exported_files.append(f"project1_lineups_enhanced.csv ({len(enhanced_lineups)} lineups)")

            # PROJECT 2: Player Tables (Both Methods)

            # Traditional Method - Project 2
            traditional_players = self.conn.execute(f"""
                SELECT 
                    player_id as "{PLAYER_COLUMNS[0]}",
                    player_name as "{PLAYER_COLUMNS[1]}",
                    team_abbrev as "{PLAYER_COLUMNS[2]}",
                    off_possessions as "{PLAYER_COLUMNS[3]}",
                    def_possessions as "{PLAYER_COLUMNS[4]}",
                    COALESCE(opp_rim_fg_pct_on, 0) as "{PLAYER_COLUMNS[5]}",
                    COALESCE(opp_rim_fg_pct_off, 0) as "{PLAYER_COLUMNS[6]}",
                    COALESCE(rim_defense_on_off, 0) as "{PLAYER_COLUMNS[7]}"
                FROM final_dual_players
                WHERE method = 'traditional'
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, player_name
            """).df()

            trad_players_file = self.export_dir / "project2_players_traditional.csv"
            traditional_players.to_csv(trad_players_file, index=False)
            exported_files.append(f"project2_players_traditional.csv ({len(traditional_players)} players)")

            # Enhanced Method - Project 2
            enhanced_players = self.conn.execute(f"""
                SELECT 
                    player_id as "{PLAYER_COLUMNS[0]}",
                    player_name as "{PLAYER_COLUMNS[1]}",
                    team_abbrev as "{PLAYER_COLUMNS[2]}",
                    off_possessions as "{PLAYER_COLUMNS[3]}",
                    def_possessions as "{PLAYER_COLUMNS[4]}",
                    COALESCE(opp_rim_fg_pct_on, 0) as "{PLAYER_COLUMNS[5]}",
                    COALESCE(opp_rim_fg_pct_off, 0) as "{PLAYER_COLUMNS[6]}",
                    COALESCE(rim_defense_on_off, 0) as "{PLAYER_COLUMNS[7]}"
                FROM final_dual_players
                WHERE method = 'enhanced'
                AND (off_possessions > 0 OR def_possessions > 0)
                ORDER BY team_abbrev, player_name
            """).df()

            enh_players_file = self.export_dir / "project2_players_enhanced.csv"
            enhanced_players.to_csv(enh_players_file, index=False)
            exported_files.append(f"project2_players_enhanced.csv ({len(enhanced_players)} players)")

            # Update export summary
            self.export_summary.traditional_lineups = len(traditional_lineups)
            self.export_summary.enhanced_lineups = len(enhanced_lineups)
            self.export_summary.traditional_players = len(traditional_players)
            self.export_summary.enhanced_players = len(enhanced_players)

            details = f"Exported project deliverables: {len(exported_files)} files"

            return ValidationResult(
                step_name="Export Project Deliverables",
                passed=True,
                details=details,
                data_count=len(exported_files),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Export Project Deliverables",
                passed=False,
                details=f"Error exporting project deliverables: {str(e)}",
                processing_time=time.time() - start_time
            )


    def export_violation_reports(self) -> ValidationResult:
        """Export violation reports (traditional/enhanced) without assuming any specific time column.

        Strategy:
        - Introspect available columns per table.
        - Build a SELECT list that only references existing columns.
        - Choose an ORDER BY among ['order_ts','abs_time','wall_clock_int','pbp_order'] if present.
        - Prefer readable fields (team_abbrev/player_name); fall back to ids if names not present.
        """
        start_time = time.time()
        try:
            logger.info("Exporting violation reports for traditional lineups...")

            exported_files = []

            def _cols(table: str) -> List[str]:
                try:
                    return [r[1] for r in self.conn.execute(f"PRAGMA table_info('{table}')").fetchall()]
                except Exception:
                    return []

            def _build_sql(table: str) -> Optional[str]:
                cols = _cols(table)
                if not cols:
                    return None

                # pick time/order column
                time_col = next((c for c in ["order_ts", "abs_time", "wall_clock_int", "pbp_order"] if c in cols), None)

                # assemble select columns, only if they exist
                select_cols = []
                for c in ["period"]:
                    if c in cols: select_cols.append(c)

                if time_col:
                    select_cols.append(f"{time_col} AS time_order")

                # prefer readable names; fallback to ids
                if "team_abbrev" in cols:
                    select_cols.append("team_abbrev")
                elif "team_id" in cols:
                    select_cols.append("team_id")

                select_cols += [c for c in ["flag_type"] if c in cols]

                if "player_name" in cols:
                    select_cols.append("player_name")
                elif "player_id" in cols:
                    select_cols.append("player_id")

                select_cols += [c for c in ["description", "flag_details"] if c in cols]

                if not select_cols:
                    # Nothing exportable
                    return None

                select_list = ", ".join(select_cols)
                sql = f"SELECT {select_list} FROM {table}"
                if time_col:
                    sql += " ORDER BY time_order"
                return sql

            # Traditional
            trad_sql = _build_sql("traditional_violation_report")
            if trad_sql:
                trad_df = self.conn.execute(trad_sql).df()
                if not trad_df.empty:
                    fpath = self.export_dir / "traditional_lineup_violations.csv"
                    trad_df.to_csv(fpath, index=False)
                    exported_files.append(f"traditional_lineup_violations.csv ({len(trad_df)} violations)")
                    # Summarize by flag_type if present
                    if "flag_type" in trad_df.columns:
                        counts = trad_df["flag_type"].value_counts().to_dict()
                        with open(self.export_dir / "violation_summary.txt", "w", encoding="utf-8") as f:
                            f.write("TRADITIONAL LINEUP VIOLATION SUMMARY\n")
                            f.write("="*50 + "\n\n")
                            f.write(f"Total Violations: {len(trad_df):,}\n\n")
                            f.write("Violation Types:\n")
                            for k, v in counts.items():
                                f.write(f"  {k}: {v:,} ({100.0 * v / max(1, len(trad_df)):.1f}%)\n")
                            f.write(f"\nGenerated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                        exported_files.append("violation_summary.txt")
                        self.export_summary.violation_reports += 1
            else:
                logger.info("No traditional_violation_report table or no exportable columns.")

            # Enhanced
            enh_sql = _build_sql("enhanced_violation_report")
            if enh_sql:
                enh_df = self.conn.execute(enh_sql).df()
                if not enh_df.empty:
                    fpath = self.export_dir / "enhanced_method_flags.csv"
                    enh_df.to_csv(fpath, index=False)
                    exported_files.append(f"enhanced_method_flags.csv ({len(enh_df)} flags)")
                    self.export_summary.violation_reports += 1
            else:
                logger.info("No enhanced_violation_report table or no exportable columns.")

            # Base-dataset quality memo (unchanged)
            self._create_base_dataset_violation_report()
            exported_files.append("base_dataset_violations.txt")

            details = f"Exported violation reports: {len(exported_files)} files"
            return ValidationResult(
                step_name="Export Violation Reports",
                passed=True,
                details=details,
                data_count=len(exported_files),
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Export Violation Reports",
                passed=False,
                details=f"Error exporting violation reports: {str(e)}",
                processing_time=time.time() - start_time
            )


    def _create_base_dataset_violation_report(self):
        """Create base dataset violation report for final analysis"""
        try:
            # Analyze base dataset quality issues
            base_dataset_analysis = {
                'data_quality_issues': [],
                'lineup_tracking_challenges': [],
                'recommendations': []
            }

            # Check for common data issues in the base dataset
            pbp_issues = self.conn.execute("""
                SELECT 
                    COUNT(*) as total_events,
                    SUM(CASE WHEN team_id_off IS NULL OR team_id_def IS NULL THEN 1 ELSE 0 END) as missing_team_events,
                    SUM(CASE WHEN player_id_1 IS NULL AND player_id_2 IS NULL AND player_id_3 IS NULL THEN 1 ELSE 0 END) as no_player_events,
                    SUM(CASE WHEN msg_type = 8 AND player_id_1 IS NULL AND player_id_2 IS NULL THEN 1 ELSE 0 END) as incomplete_substitutions
                FROM pbp
            """).fetchone()

            if pbp_issues:
                total_events = pbp_issues[0]
                missing_team_pct = (pbp_issues[1] / total_events * 100) if total_events > 0 else 0
                no_player_pct = (pbp_issues[2] / total_events * 100) if total_events > 0 else 0
                incomplete_sub_pct = (pbp_issues[3] / total_events * 100) if total_events > 0 else 0

                base_dataset_analysis['data_quality_issues'] = [
                    f"Missing team data in {missing_team_pct:.1f}% of events",
                    f"No player data in {no_player_pct:.1f}% of events", 
                    f"Incomplete substitutions in {incomplete_sub_pct:.1f}% of substitution events"
                ]

            # Lineup tracking challenges from traditional method
            if hasattr(self, 'export_summary'):
                if self.export_summary.traditional_lineups > 0:
                    lineup_5man_pct = self.conn.execute("""
                        SELECT AVG(CASE WHEN lineup_size = 5 THEN 1.0 ELSE 0.0 END) * 100 as pct
                        FROM final_dual_lineups WHERE method = 'traditional'
                    """).fetchone()[0]

                    base_dataset_analysis['lineup_tracking_challenges'] = [
                        f"Only {lineup_5man_pct:.1f}% of traditional lineups have exactly 5 players",
                        f"Substitution data requires intelligent inference for accurate lineup tracking",
                        f"Enhanced method achieves 100% 5-player lineup accuracy through automation"
                    ]

            base_dataset_analysis['recommendations'] = [
                "Use enhanced method for production lineup tracking",
                "Traditional method useful for data quality validation", 
                "Violation reports highlight areas needing manual review",
                "Consider implementing enhanced automation for real-time applications"
            ]

            # Write report
            report_file = self.export_dir / "base_dataset_violations.txt"
            with open(report_file, 'w', encoding='utf-8') as f:
                f.write("BASE DATASET VIOLATIONS AND RECOMMENDATIONS\n")
                f.write("="*60 + "\n\n")

                f.write("DATA QUALITY ISSUES:\n")
                for issue in base_dataset_analysis['data_quality_issues']:
                    f.write(f"  - {issue}\n")
                f.write("\n")

                f.write("LINEUP TRACKING CHALLENGES:\n")
                for challenge in base_dataset_analysis['lineup_tracking_challenges']:
                    f.write(f"  - {challenge}\n")
                f.write("\n")

                f.write("RECOMMENDATIONS:\n")
                for rec in base_dataset_analysis['recommendations']:
                    f.write(f"  - {rec}\n")
                f.write("\n")

                f.write(f"Report generated: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("For detailed violation data, see traditional_lineup_violations.csv\n")

        except Exception as e:
            logger.warning(f"Could not create base dataset violation report: {e}")

    def export_method_comparison_reports(self) -> ValidationResult:
        """Export comprehensive method comparison and effectiveness analysis"""
        start_time = time.time()

        try:
            logger.info("Exporting method comparison reports...")

            exported_files = []

            # Method Comparison Summary
            try:
                comparison_summary = self.conn.execute("""
                    SELECT * FROM method_comparison_summary
                    ORDER BY metric
                """).df()

                if not comparison_summary.empty:
                    comparison_file = self.export_dir / "method_comparison_summary.csv"
                    comparison_summary.to_csv(comparison_file, index=False)
                    exported_files.append(f"method_comparison_summary.csv ({len(comparison_summary)} metrics)")
                    self.export_summary.comparison_reports += 1

            except Exception as e:
                logger.warning(f"Could not export method comparison: {e}")

            # Create method effectiveness report
            self._create_method_effectiveness_report()
            exported_files.append("method_effectiveness_report.txt")

            details = f"Exported method comparison reports: {len(exported_files)} files"

            return ValidationResult(
                step_name="Export Method Comparison Reports",
                passed=True,
                details=details,
                data_count=len(exported_files),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Export Method Comparison Reports",
                passed=False,
                details=f"Error exporting method comparison reports: {str(e)}",
                processing_time=time.time() - start_time
            )

    def _create_method_effectiveness_report(self):
        """Create comprehensive method effectiveness analysis report"""
        try:
            # Calculate effectiveness metrics
            effectiveness_metrics = {}

            # Lineup accuracy metrics
            traditional_accuracy = self.conn.execute("""
                SELECT 
                    COUNT(*) as total_lineups,
                    SUM(CASE WHEN lineup_size = 5 THEN 1 ELSE 0 END) as five_man_lineups,
                    AVG(lineup_size) as avg_size,
                    MIN(lineup_size) as min_size,
                    MAX(lineup_size) as max_size
                FROM final_dual_lineups WHERE method = 'traditional'
            """).fetchone()

            enhanced_accuracy = self.conn.execute("""
                SELECT 
                    COUNT(*) as total_lineups,
                    SUM(CASE WHEN lineup_size = 5 THEN 1 ELSE 0 END) as five_man_lineups,
                    AVG(lineup_size) as avg_size
                FROM final_dual_lineups WHERE method = 'enhanced'
            """).fetchone()

            if traditional_accuracy and enhanced_accuracy:
                effectiveness_metrics['lineup_accuracy'] = {
                    'traditional': {
                        'total_lineups': traditional_accuracy[0],
                        'five_man_accuracy': (traditional_accuracy[1] / traditional_accuracy[0] * 100) if traditional_accuracy[0] > 0 else 0,
                        'avg_size': traditional_accuracy[2],
                        'min_size': traditional_accuracy[3],
                        'max_size': traditional_accuracy[4]
                    },
                    'enhanced': {
                        'total_lineups': enhanced_accuracy[0],
                        'five_man_accuracy': (enhanced_accuracy[1] / enhanced_accuracy[0] * 100) if enhanced_accuracy[0] > 0 else 0,
                        'avg_size': enhanced_accuracy[2]
                    }
                }

            # Create effectiveness report
            report_file = self.export_dir / "method_effectiveness_report.txt"
            with open(report_file, 'w', encoding='utf-8') as f:
                f.write("METHOD EFFECTIVENESS ANALYSIS\n")
                f.write("="*50 + "\n\n")

                f.write("EXECUTIVE SUMMARY:\n")
                if 'lineup_accuracy' in effectiveness_metrics:
                    trad_acc = effectiveness_metrics['lineup_accuracy']['traditional']['five_man_accuracy']
                    enh_acc = effectiveness_metrics['lineup_accuracy']['enhanced']['five_man_accuracy']
                    improvement = enh_acc - trad_acc

                    f.write(f"  Enhanced method achieves {improvement:+.1f} percentage point improvement\n")
                    f.write(f"  in 5-man lineup accuracy over traditional method.\n\n")

                f.write("DETAILED METRICS:\n\n")

                f.write("Traditional Data-Driven Method:\n")
                if 'lineup_accuracy' in effectiveness_metrics:
                    trad = effectiveness_metrics['lineup_accuracy']['traditional']
                    f.write(f"  Total Lineups: {trad['total_lineups']:,}\n")
                    f.write(f"  5-Man Accuracy: {trad['five_man_accuracy']:.1f}%\n")
                    f.write(f"  Average Size: {trad['avg_size']:.1f} players\n")
                    f.write(f"  Size Range: {trad['min_size']}-{trad['max_size']} players\n")
                    f.write(f"  Strength: Faithful to raw data, highlights data quality issues\n")
                    f.write(f"  Use Case: Data validation and quality assessment\n\n")

                f.write("Enhanced Automation Method:\n")
                if 'lineup_accuracy' in effectiveness_metrics:
                    enh = effectiveness_metrics['lineup_accuracy']['enhanced']
                    f.write(f"  Total Lineups: {enh['total_lineups']:,}\n")
                    f.write(f"  5-Man Accuracy: {enh['five_man_accuracy']:.1f}%\n")
                    f.write(f"  Average Size: {enh['avg_size']:.1f} players\n")
                    f.write(f"  Strength: Consistent 5-man lineups, intelligent inference\n")
                    f.write(f"  Use Case: Production analytics and reporting\n\n")

                f.write("RECOMMENDATIONS:\n")
                f.write("  1. Use Enhanced method for production lineup analytics\n")
                f.write("  2. Use Traditional method for data quality validation\n")
                f.write("  3. Review violation reports to identify systematic data issues\n")
                f.write("  4. Consider automated data quality monitoring based on violation patterns\n\n")

                f.write(f"Analysis completed: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")

        except Exception as e:
            logger.warning(f"Could not create method effectiveness report: {e}")

    def generate_quality_report(self) -> ValidationResult:
        """Generate comprehensive quality and validation report (ASCII-safe, explicit encoding)."""
        start_time = time.time()
        try:
            logger.info("Generating quality report...")

            # Copy current validations logged so far
            all_validations = self.validator.validation_results.copy()

            # -------- Lineup metrics --------
            lm = self.conn.execute("""
                SELECT
                    COUNT(*) AS total_records,
                    AVG(off_possessions) AS avg_off_poss,
                    AVG(def_possessions) AS avg_def_poss,
                    AVG(off_rating)      AS avg_off_rating,
                    AVG(def_rating)      AS avg_def_rating
                FROM final_lineups
                WHERE off_possessions > 0
            """).fetchone()

            lineup_metrics = {
                "total_records": int(lm[0] or 0),
                "avg_off_poss": float(lm[1]) if lm[1] is not None else 0.0,
                "avg_def_poss": float(lm[2]) if lm[2] is not None else 0.0,
                "avg_off_rating": float(lm[3]) if lm[3] is not None else 0.0,
                "avg_def_rating": float(lm[4]) if lm[4] is not None else 0.0,
            }

            # -------- Player rim-defense metrics (FIXED) --------
            pm = self.conn.execute("""
                SELECT
                    COUNT(*) AS total_records,
                    -- Raw event counts per player
                    AVG(opp_rim_attempts_on)  AS avg_attempts_on_raw,
                    AVG(opp_rim_attempts_off) AS avg_attempts_off_raw,
                    AVG(opp_rim_makes_on)     AS avg_makes_on_raw,
                    AVG(opp_rim_makes_off)    AS avg_makes_off_raw,
                    AVG(opp_rim_fg_pct_on)    AS avg_fg_pct_on,
                    AVG(opp_rim_fg_pct_off)   AS avg_fg_pct_off,
                    AVG(rim_defense_on_off)   AS avg_rim_defense_diff,
                    -- Calculated normalized metrics using existing columns
                    AVG(CASE WHEN def_possessions > 0 THEN opp_rim_attempts_on::FLOAT / def_possessions ELSE 0 END) AS avg_attempts_per_def_poss_on,
                    AVG(CASE WHEN def_possessions > 0 THEN opp_rim_attempts_off::FLOAT / def_possessions ELSE 0 END) AS avg_attempts_per_def_poss_off,
                    AVG(CASE WHEN def_possessions > 0 THEN 100.0 * opp_rim_attempts_on::FLOAT / def_possessions ELSE 0 END) AS attempts_on_per100_def_poss,
                    AVG(CASE WHEN def_possessions > 0 THEN 100.0 * opp_rim_attempts_off::FLOAT / def_possessions ELSE 0 END) AS attempts_off_per100_def_poss
                FROM final_players
                WHERE opp_rim_attempts_on > 0 OR opp_rim_attempts_off > 0
            """).fetchone()

            player_metrics = {
                "total_records": int(pm[0] or 0),
                "avg_attempts_on_raw": float(pm[1]) if pm[1] is not None else 0.0,
                "avg_attempts_off_raw": float(pm[2]) if pm[2] is not None else 0.0,
                "avg_makes_on_raw": float(pm[3]) if pm[3] is not None else 0.0,
                "avg_makes_off_raw": float(pm[4]) if pm[4] is not None else 0.0,
                "avg_fg_pct_on": float(pm[5]) if pm[5] is not None else 0.0,
                "avg_fg_pct_off": float(pm[6]) if pm[6] is not None else 0.0,
                "avg_rim_defense_diff": float(pm[7]) if pm[7] is not None else 0.0,
                "avg_attempts_per_def_poss_on": float(pm[8]) if pm[8] is not None else 0.0,
                "avg_attempts_per_def_poss_off": float(pm[9]) if pm[9] is not None else 0.0,
                "attempts_on_per100_def_poss": float(pm[10]) if pm[10] is not None else 0.0,
                "attempts_off_per100_def_poss": float(pm[11]) if pm[11] is not None else 0.0,
            }

            # -------- Assemble report text (ASCII labels, no emojis) --------
            report_lines = [
                "NBA PLAY-BY-PLAY PIPELINE - QUALITY REPORT",
                "=" * 80,
                f"Generated: {time.strftime('%Y-%m-%d %H:%M:%S')}",
                "",
                "PIPELINE EXECUTION SUMMARY:",
                f"  Total Validation Steps: {len(all_validations)}",
                f"  Passed: {sum(1 for v in all_validations if v.passed)}",
                f"  Failed: {sum(1 for v in all_validations if not v.passed)}",
                f"  Total Warnings: {sum(len(v.warnings) for v in all_validations)}",
                ""
            ]

            # Validation step details
            report_lines.append("VALIDATION STEP DETAILS:")
            for validation in all_validations:
                status = self._status_label(validation.passed)
                report_lines.append(f"  {status} {validation.step_name}")
                report_lines.append(f"    Details: {validation.details}")
                report_lines.append(f"    Time: {validation.processing_time:.3f}s")
                if validation.warnings:
                    for warning in validation.warnings:
                        report_lines.append(f"    {self._warn_label()} {warning}")
                report_lines.append("")

            # Data quality sections (UPDATED)
            report_lines.extend([
                "DATA QUALITY METRICS:",
                "",
                "  LINEUP STATISTICS:",
                f"    Total Records: {lineup_metrics['total_records']:,}",
                f"    Avg Offensive Possessions: {lineup_metrics['avg_off_poss']:.1f}",
                f"    Avg Defensive Possessions: {lineup_metrics['avg_def_poss']:.1f}",
                f"    Avg Offensive Rating: {lineup_metrics['avg_off_rating']:.1f}",
                f"    Avg Defensive Rating: {lineup_metrics['avg_def_rating']:.1f}",
                "",
                "  PLAYER RIM DEFENSE:",
                f"    Player Records: {player_metrics['total_records']:,}",
                f"    Avg Rim Attempts (On): {player_metrics['avg_attempts_on_raw']:.2f}",
                f"    Avg Rim Attempts (Off): {player_metrics['avg_attempts_off_raw']:.2f}",
                f"    Avg Rim Makes (On): {player_metrics['avg_makes_on_raw']:.2f}",
                f"    Avg Rim Makes (Off): {player_metrics['avg_makes_off_raw']:.2f}",
                f"    Avg Rim FG% (On): {player_metrics['avg_fg_pct_on']:.1%}",
                f"    Avg Rim FG% (Off): {player_metrics['avg_fg_pct_off']:.1%}",
                f"    Avg On/Off Difference: {player_metrics['avg_rim_defense_diff']:.3f}",
                f"    Rim Attempts per Def Possession (On): {player_metrics['avg_attempts_per_def_poss_on']:.4f}",
                f"    Rim Attempts per Def Possession (Off): {player_metrics['avg_attempts_per_def_poss_off']:.4f}",
                f"    Rim Attempts per 100 Def Poss (On): {player_metrics['attempts_on_per100_def_poss']:.2f}",
                f"    Rim Attempts per 100 Def Poss (Off): {player_metrics['attempts_off_per100_def_poss']:.2f}",
                ""
            ])

            # Save to disk (sanitize + explicit encoding)
            report_text = "\n".join(report_lines)
            report_text = self._sanitize_for_file(report_text)
            report_file = self.export_dir / "quality_report.txt"
            report_file.write_text(report_text, encoding="utf-8")

            details = "Generated quality report with corrected player rim defense metrics"
            return ValidationResult(
                step_name="Quality Report",
                passed=True,
                details=details,
                data_count=len(all_validations),
                processing_time=time.time() - start_time
            )

        except Exception as e:
            return ValidationResult(
                step_name="Quality Report",
                passed=False,
                details=f"Error generating quality report: {str(e)}",
                processing_time=time.time() - start_time
            )


    def print_final_summary(self):
        """Print final pipeline summary (ASCII-only to avoid console encoding issues)."""
        print("\n" + "="*80)
        print("NBA PIPELINE - FINAL EXPORT & VALIDATION SUMMARY")
        print("="*80)

        # Show export directory contents
        if self.export_dir.exists():
            export_files = list(self.export_dir.glob("*.csv")) + list(self.export_dir.glob("*.txt"))
            print(f"EXPORTED FILES ({len(export_files)}):")
            for file in sorted(export_files):
                size_kb = file.stat().st_size / 1024
                print(f"   - {file.name} ({size_kb:.1f} KB)")
            print()

        # Show key metrics
        try:
            lineup_count = self.conn.execute("SELECT COUNT(*) FROM final_lineups").fetchone()[0]
            player_count = self.conn.execute("SELECT COUNT(*) FROM final_players").fetchone()[0]

            print("FINAL RESULTS:")
            print(f"   Unique Lineups: {lineup_count:,}")
            print(f"   Active Players: {player_count:,}")

            if hasattr(self.engine, 'possessions') and self.engine.possessions:
                print(f"   Total Possessions: {len(self.engine.possessions):,}")

            if hasattr(self.processor, 'processed_events') and self.processor.processed_events:
                rim_attempts = sum(1 for e in self.processor.processed_events if getattr(e, 'is_rim_attempt', False))
                print(f"   Rim Attempts: {rim_attempts:,}")

        except Exception:
            print("   Unable to retrieve final metrics")

        print("="*80)

    # Add this to your FinalValidator class in nba_final_export.py

    def validate_final_tables_exist(self) -> ValidationResult:
        """Validate final tables exist with expected structure before generating reports."""
        start_time = time.time()
        try:
            warnings = []

            # Check tables exist
            tables = [row[0] for row in self.conn.execute("""
                SELECT table_name FROM information_schema.tables 
                WHERE table_name IN ('final_lineups', 'final_players')
            """).fetchall()]

            missing_tables = set(['final_lineups', 'final_players']) - set(tables)
            if missing_tables:
                return ValidationResult(
                    step_name="Final Tables Check",
                    passed=False,
                    details=f"Missing tables: {missing_tables}",
                    processing_time=time.time() - start_time
                )

            # Check final_players has required columns for quality report
            player_cols = [row[0] for row in self.conn.execute("""
                SELECT column_name FROM information_schema.columns 
                WHERE table_name = 'final_players'
            """).fetchall()]

            required_cols = [
                'player_id', 'player_name', 'team_abbrev', 
                'off_possessions', 'def_possessions',
                'opp_rim_attempts_on', 'opp_rim_attempts_off',
                'opp_rim_fg_pct_on', 'opp_rim_fg_pct_off'
            ]

            missing_cols = set(required_cols) - set(player_cols)
            if missing_cols:
                warnings.append(f"Missing columns in final_players: {missing_cols}")

            # Check data exists
            lineup_count = self.conn.execute("SELECT COUNT(*) FROM final_lineups").fetchone()[0]
            player_count = self.conn.execute("SELECT COUNT(*) FROM final_players").fetchone()[0]

            if lineup_count == 0:
                warnings.append("final_lineups table is empty")
            if player_count == 0:
                warnings.append("final_players table is empty")

            details = f"Tables validated: final_lineups({lineup_count}), final_players({player_count})"

            return ValidationResult(
                step_name="Final Tables Check",
                passed=len(missing_cols) == 0 and lineup_count > 0 and player_count > 0,
                details=details,
                processing_time=time.time() - start_time,
                warnings=warnings
            )

        except Exception as e:
            return ValidationResult(
                step_name="Final Tables Check", 
                passed=False,
                details=f"Error checking final tables: {str(e)}",
                processing_time=time.time() - start_time
            )

    def export_points_attribution_audit(self) -> ValidationResult:
        """
        ENHANCED: Comprehensive audit of where points are lost/gained between stages.
        Produces detailed CSVs under exports/debug_points_audit/ with granular analysis.
        """
        start_time = time.time()
        try:
            logger.info("=== ENHANCED POINTS ATTRIBUTION AUDIT ===")
            
            out_dir = self.export_dir / "debug_points_audit"
            out_dir.mkdir(parents=True, exist_ok=True)

            existing_tables = {r[0] for r in self.conn.execute(
                "SELECT table_name FROM information_schema.tables").fetchall()}

            audit_results = {
                "box_score_totals": {},
                "raw_pbp_totals": {},
                "step4_processed_totals": {},
                "lineup_totals": {},
                "discrepancies": {}
            }

            # 1) Box score totals (ground truth)
            box_df = pd.DataFrame()
            if "box_score" in existing_tables:
                box_df = self.conn.execute("""
                    SELECT 
                        team_abbrev,
                        SUM(points) AS box_points,
                        COUNT(*) AS active_players,
                        SUM(seconds_played) AS total_seconds
                    FROM box_score
                    WHERE status = 'ACTIVE'
                    GROUP BY team_abbrev
                    ORDER BY team_abbrev
                """).df()
                
                for _, row in box_df.iterrows():
                    audit_results["box_score_totals"][row["team_abbrev"]] = {
                        "points": int(row["box_points"]),
                        "players": int(row["active_players"]),
                        "seconds": int(row["total_seconds"])
                    }

            # 2) Raw PBP totals
            raw_pbp_df = pd.DataFrame()
            if "pbp" in existing_tables:
                raw_pbp_df = self.conn.execute("""
                    SELECT 
                        CASE 
                            WHEN team_id_off = 1610612742 THEN 'DAL'
                            WHEN team_id_off = 1610612745 THEN 'HOU'
                            ELSE CAST(team_id_off AS VARCHAR)
                        END as team_abbrev,
                        SUM(COALESCE(points, 0)) AS raw_pbp_points,
                        COUNT(*) AS scoring_events,
                        COUNT(DISTINCT pbp_id) AS unique_events
                    FROM pbp 
                    WHERE points > 0 AND team_id_off IS NOT NULL
                    GROUP BY team_id_off
                    ORDER BY team_abbrev
                """).df()
                
                for _, row in raw_pbp_df.iterrows():
                    audit_results["raw_pbp_totals"][row["team_abbrev"]] = {
                        "points": int(row["raw_pbp_points"]),
                        "events": int(row["scoring_events"]),
                        "unique_events": int(row["unique_events"])
                    }

            # 3) Step4 processed totals
            step4_df = pd.DataFrame()
            if "step4_processed_events" in existing_tables:
                step4_df = self.conn.execute("""
                    SELECT 
                        CASE 
                            WHEN off_team_id = 1610612742 THEN 'DAL'
                            WHEN off_team_id = 1610612745 THEN 'HOU'
                            ELSE CAST(off_team_id AS VARCHAR)
                        END as team_abbrev,
                        SUM(COALESCE(points, 0)) AS step4_points,
                        COUNT(*) AS processing_events,
                        COUNT(CASE WHEN points > 0 THEN 1 END) AS scoring_events,
                        COUNT(DISTINCT pbp_id) AS unique_pbp_ids
                    FROM step4_processed_events
                    WHERE off_team_id IS NOT NULL
                    GROUP BY off_team_id
                    ORDER BY team_abbrev
                """).df()
                
                for _, row in step4_df.iterrows():
                    audit_results["step4_processed_totals"][row["team_abbrev"]] = {
                        "points": int(row["step4_points"]),
                        "events": int(row["processing_events"]),
                        "scoring_events": int(row["scoring_events"]),
                        "unique_pbp_ids": int(row["unique_pbp_ids"])
                    }

            # 4) Lineup totals by method
            lineup_df = pd.DataFrame()
            if "final_dual_lineups" in existing_tables:
                lineup_df = self.conn.execute("""
                    SELECT 
                        method,
                        team_abbrev,
                        SUM(points_for) AS lineup_points_for,
                        SUM(off_possessions) AS total_off_poss,
                        COUNT(*) AS unique_lineups
                    FROM final_dual_lineups
                    GROUP BY method, team_abbrev
                    ORDER BY method, team_abbrev
                """).df()
                
                audit_results["lineup_totals"] = {}
                for _, row in lineup_df.iterrows():
                    method = row["method"]
                    team = row["team_abbrev"]
                    if method not in audit_results["lineup_totals"]:
                        audit_results["lineup_totals"][method] = {}
                    audit_results["lineup_totals"][method][team] = {
                        "points": int(row["lineup_points_for"]),
                        "possessions": int(row["total_off_poss"]),
                        "lineups": int(row["unique_lineups"])
                    }

            # 5) Calculate discrepancies at each stage
            teams = ["DAL", "HOU"]
            for team in teams:
                box_pts = audit_results["box_score_totals"].get(team, {}).get("points", 0)
                raw_pts = audit_results["raw_pbp_totals"].get(team, {}).get("points", 0)
                step4_pts = audit_results["step4_processed_totals"].get(team, {}).get("points", 0)
                
                trad_pts = audit_results["lineup_totals"].get("traditional", {}).get(team, {}).get("points", 0)
                enh_pts = audit_results["lineup_totals"].get("enhanced", {}).get(team, {}).get("points", 0)
                
                audit_results["discrepancies"][team] = {
                    "box_score": box_pts,
                    "raw_pbp": raw_pts,
                    "step4_processed": step4_pts,
                    "traditional_lineups": trad_pts,
                    "enhanced_lineups": enh_pts,
                    "raw_to_step4_diff": step4_pts - raw_pts,
                    "step4_to_traditional_diff": trad_pts - step4_pts,
                    "step4_to_enhanced_diff": enh_pts - step4_pts,
                    "box_to_traditional_diff": trad_pts - box_pts,
                    "box_to_enhanced_diff": enh_pts - box_pts
                }

            # 6) Export comprehensive summary
            summary_rows = []
            for team in teams:
                disc = audit_results["discrepancies"][team]
                summary_rows.append({
                    "team": team,
                    "box_score_points": disc["box_score"],
                    "raw_pbp_points": disc["raw_pbp"],
                    "step4_processed_points": disc["step4_processed"],
                    "traditional_lineup_points": disc["traditional_lineups"],
                    "enhanced_lineup_points": disc["enhanced_lineups"],
                    "raw_to_step4_diff": disc["raw_to_step4_diff"],
                    "step4_to_traditional_diff": disc["step4_to_traditional_diff"],
                    "step4_to_enhanced_diff": disc["step4_to_enhanced_diff"],
                    "box_to_traditional_diff": disc["box_to_traditional_diff"],
                    "box_to_enhanced_diff": disc["box_to_enhanced_diff"]
                })
            
            pd.DataFrame(summary_rows).to_csv(out_dir / "001_comprehensive_points_flow.csv", index=False)

            # 7) Detailed HOU analysis (the problematic team)
            if "step4_processed_events" in existing_tables:
                hou_events = self.conn.execute("""
                    SELECT 
                        pbp_id, period, pbp_order, wall_clock_int,
                        description, points, msg_type, action_type,
                        player_id_1, player_id_2, player_id_3,
                        traditional_off_lineup, enhanced_off_lineup,
                        CASE 
                            WHEN traditional_off_lineup IS NULL 
                            OR TRIM(CAST(traditional_off_lineup AS VARCHAR)) IN ('', '[]')
                            THEN 'NO_LINEUP'
                            ELSE 'HAS_LINEUP'
                        END as trad_lineup_status,
                        CASE 
                            WHEN enhanced_off_lineup IS NULL 
                            OR TRIM(CAST(enhanced_off_lineup AS VARCHAR)) IN ('', '[]')
                            THEN 'NO_LINEUP'
                            ELSE 'HAS_LINEUP'
                        END as enh_lineup_status
                    FROM step4_processed_events 
                    WHERE off_team_id = 1610612745 AND points > 0
                    ORDER BY period, pbp_order, wall_clock_int
                """).df()
                
                hou_events.to_csv(out_dir / "002_hou_scoring_events_detailed.csv", index=False)
                
                # HOU lineup attribution analysis
                hou_attribution = hou_events.groupby(['trad_lineup_status', 'enh_lineup_status']).agg({
                    'points': ['count', 'sum'],
                    'pbp_id': 'count'
                }).round(2)
                hou_attribution.to_csv(out_dir / "003_hou_lineup_attribution_breakdown.csv")

            # 8) Cross-reference with original PBP
            if "pbp" in existing_tables and "step4_processed_events" in existing_tables:
                cross_ref = self.conn.execute("""
                    SELECT 
                        pbp.pbp_id,
                        pbp.points as original_points,
                        pbp.description as original_description,
                        se.points as processed_points,
                        se.description as processed_description,
                        CASE 
                            WHEN pbp.team_id_off = 1610612742 THEN 'DAL'
                            WHEN pbp.team_id_off = 1610612745 THEN 'HOU'
                            ELSE CAST(pbp.team_id_off AS VARCHAR)
                        END as team,
                        (se.points - COALESCE(pbp.points, 0)) as points_diff
                    FROM pbp 
                    LEFT JOIN step4_processed_events se ON pbp.pbp_id = se.pbp_id
                    WHERE pbp.points > 0 OR se.points > 0
                    ORDER BY team, pbp.pbp_id
                """).df()
                
                cross_ref.to_csv(out_dir / "004_pbp_to_step4_cross_reference.csv", index=False)
                
                # Points difference summary
                diff_summary = cross_ref.groupby('team')['points_diff'].agg(['sum', 'count', 'mean']).round(3)
                diff_summary.to_csv(out_dir / "005_points_transformation_summary.csv")

            # 9) Log findings
            logger.info("=== ENHANCED AUDIT FINDINGS ===")
            for team in teams:
                disc = audit_results["discrepancies"][team]
                logger.info(f"{team} POINTS FLOW:")
                logger.info(f"  Box Score (Ground Truth): {disc['box_score']}")
                logger.info(f"  Raw PBP: {disc['raw_pbp']} (diff: {disc['raw_pbp'] - disc['box_score']:+})")
                logger.info(f"  Step4 Processed: {disc['step4_processed']} (diff: {disc['raw_to_step4_diff']:+})")
                logger.info(f"  Traditional Lineups: {disc['traditional_lineups']} (diff: {disc['step4_to_traditional_diff']:+})")
                logger.info(f"  Enhanced Lineups: {disc['enhanced_lineups']} (diff: {disc['step4_to_enhanced_diff']:+})")
                logger.info(f"  FINAL DISCREPANCY vs Box: Traditional={disc['box_to_traditional_diff']:+}, Enhanced={disc['box_to_enhanced_diff']:+}")

            return ValidationResult(
                step_name="Enhanced Points Attribution Audit",
                passed=True,
                details=f"Comprehensive audit completed. Key finding: HOU has {audit_results['discrepancies']['HOU']['box_to_enhanced_diff']:+} point discrepancy. Check {out_dir} for detailed analysis.",
                processing_time=time.time() - start_time
            )
            
        except Exception as e:
            return ValidationResult(
                step_name="Enhanced Points Attribution Audit",
                passed=False,
                details=f"Error creating enhanced audit: {e}",
                processing_time=time.time() - start_time
            )
            # Build a temp teams map if needed
            teams_df = pd.DataFrame()
            if "teams" in existing_tables:
                teams_df = self.conn.execute("SELECT DISTINCT team_id, team_abbrev FROM teams").df()
            elif "box_score" in existing_tables:
                teams_df = self.conn.execute("SELECT DISTINCT team_id, team_abbrev FROM box_score").df()

            # Register map if we had to synthesize
            if not teams_df.empty:
                self.conn.register("teams_temp_map", teams_df)
                try:
                    step4_df = self.conn.execute("""
                        SELECT m.team_abbrev, SUM(se.points) AS step4_points
                        FROM step4_processed_events se
                        JOIN teams_temp_map m ON m.team_id = se.off_team_id
                        WHERE se.points > 0
                        GROUP BY m.team_abbrev
                        ORDER BY m.team_abbrev
                    """).df()
                finally:
                    self.conn.unregister("teams_temp_map")
            else:
                # fallback: just sum by team id if we cannot resolve abbrev
                step4_df = self.conn.execute("""
                    SELECT CAST(off_team_id AS VARCHAR) AS team_abbrev, SUM(points) AS step4_points
                        FROM step4_processed_events
                        WHERE points > 0
                        GROUP BY off_team_id
                    """).df()

            # 4) Merge the three sources into one side-by-side table
            merged = None
            try:
                merged = box_df.copy()
                if not step4_df.empty:
                    merged = merged.merge(step4_df, on="team_abbrev", how="outer")
                if not lineup_df.empty:
                    for method in lineup_df['method'].unique():
                        sub = lineup_df[lineup_df['method'] == method][["team_abbrev", "lineup_points_for"]].rename(
                            columns={"lineup_points_for": f"lineup_points_for_{method}"}
                        )
                        merged = merged.merge(sub, on="team_abbrev", how="outer")
                # diffs
                if "box_points" in merged.columns and "lineup_points_for_traditional" in merged.columns:
                    merged["diff_traditional_vs_box"] = merged["lineup_points_for_traditional"] - merged["box_points"]
                if "box_points" in merged.columns and "lineup_points_for_enhanced" in merged.columns:
                    merged["diff_enhanced_vs_box"] = merged["lineup_points_for_enhanced"] - merged["box_points"]
            except Exception:
                pass

            if merged is not None:
                merged.to_csv(out_dir / "001_dual_vs_box_points.csv", index=False)

            # 5) Identify unattributed scoring events per method (needs step4)
            if "step4_processed_events" in existing_tables:
                # detect a time column to help downstream inspection
                cols = [r[1] for r in self.conn.execute("PRAGMA table_info('step4_processed_events')").fetchall()]
                time_col = next((c for c in ["order_ts","abs_time","wall_clock_int","pbp_order","eventnum"] if c in cols), None)

                def dump_unattr(method: str, lineup_col: str):
                    sql = f"""
                        SELECT se.*
                        FROM step4_processed_events se
                        WHERE se.points > 0
                        AND (
                            {lineup_col} IS NULL
                            OR TRIM(CAST({lineup_col} AS VARCHAR)) = ''
                            OR TRIM(CAST({lineup_col} AS VARCHAR)) = '[]'
                        )
                    """
                    df = self.conn.execute(sql).df()
                    if time_col and time_col in df.columns:
                        df = df.sort_values(time_col)
                    df.to_csv(out_dir / f"020_unattributed_scoring_events_{method}.csv", index=False)
                    return df

                trad_df = dump_unattr("traditional", "traditional_off_lineup") if "traditional_off_lineup" in cols else pd.DataFrame()
                enh_df  = dump_unattr("enhanced", "enhanced_off_lineup")       if "enhanced_off_lineup" in cols       else pd.DataFrame()

                # 6) Coverage table: how many scoring events had a valid lineup?
                coverage_rows = []
                for method, lineup_col in [("traditional","traditional_off_lineup"), ("enhanced","enhanced_off_lineup")]:
                    if lineup_col not in cols:
                        continue
                    tot = int(self.conn.execute("SELECT COUNT(*) FROM step4_processed_events WHERE points > 0").fetchone()[0])
                    ok = int(self.conn.execute(f"""
                        SELECT COUNT(*)
                        FROM step4_processed_events
                        WHERE points > 0
                        AND {lineup_col} IS NOT NULL
                        AND TRIM(CAST({lineup_col} AS VARCHAR)) NOT IN ('','[]')
                    """).fetchone()[0])
                    coverage_rows.append({
                        "method": method,
                        "scoring_event_count": tot,
                        "with_valid_off_lineup": ok,
                        "coverage_pct": (ok / tot * 100.0) if tot > 0 else 0.0
                    })
                pd.DataFrame(coverage_rows).to_csv(out_dir / "030_scoring_event_lineup_coverage.csv", index=False)

                # 7) Team-level unattributed points (quick lens on HOU -4)
                team_unattr_rows = []
                for method, lineup_col in [("traditional","traditional_off_lineup"), ("enhanced","enhanced_off_lineup")]:
                    if lineup_col not in cols:
                        continue
                    team_map_df = pd.DataFrame()
                    if "teams" in existing_tables:
                        team_map_df = self.conn.execute("SELECT DISTINCT team_id, team_abbrev FROM teams").df()
                    elif "box_score" in existing_tables:
                        team_map_df = self.conn.execute("SELECT DISTINCT team_id, team_abbrev FROM box_score").df()
                    if not team_map_df.empty:
                        self.conn.register("teams_temp_map2", team_map_df)
                        try:
                            unattributed = self.conn.execute(f"""
                                SELECT m.team_abbrev, SUM(se.points) AS unattributed_points
                                FROM step4_processed_events se
                                JOIN teams_temp_map2 m ON m.team_id = se.off_team_id
                                WHERE se.points > 0
                                AND ({lineup_col} IS NULL
                                    OR TRIM(CAST({lineup_col} AS VARCHAR)) IN ('','[]'))
                                GROUP BY m.team_abbrev
                                ORDER BY m.team_abbrev
                            """).df()
                        finally:
                            self.conn.unregister("teams_temp_map2")
                        unattributed.to_csv(out_dir / f"040_unattributed_points_by_team_{method}.csv", index=False)
                        team_unattr_rows.append((method, unattributed))

            return ValidationResult(
                step_name="Points Attribution Audit",
                passed=True,
                details=f"Wrote audit CSVs to {out_dir}",
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return ValidationResult(
                step_name="Points Attribution Audit",
                passed=False,
                details=f"Error creating audit: {e}",
                processing_time=time.time() - start_time
            )




def run_dual_method_final_export(db_path: str = "mavs_enhanced.duckdb") -> Tuple[bool, DualMethodFinalValidator]:
    """Run dual-method final validation and export for both traditional and enhanced approaches"""
    print("NBA Pipeline - Step 6: Dual-Method Final Validation & Export")
    print("="*70)

    with DualMethodFinalValidator(db_path) as validator:
        results = []

        # Step 6a
        logger.info("Step 6a: Validating dual-method tables...")
        table_result = validator.validate_dual_method_tables_exist()
        results.append(table_result)
        if not table_result.passed:
            logger.error("Dual-method tables validation failed - stopping")
            return False, validator

        # Step 6b
        logger.info("Step 6b: Validating against box score (dual-method)...")
        box_score_result = validator.validate_against_box_score_dual()
        results.append(box_score_result)
        if not box_score_result.passed:
            logger.warning("Box score validation issues detected.")
            logger.warning(box_score_result.details)   # richer per-team info
            for w in box_score_result.warnings:
                logger.warning(w)

        # >>> NEW: DEBUG AUDIT right after the mismatch appears <<<
        logger.info("Step 6b.1: Running points attribution audit (debug-only)...")
        audit_result = validator.export_points_attribution_audit()
        results.append(audit_result)
        if not audit_result.passed:
            logger.warning(f"Audit issues: {audit_result.details}")
        # <<< END NEW >>>

        # Step 6c
        logger.info("Step 6c: Validating data completeness (dual-method)...")
        completeness_result = validator.validate_data_completeness()
        results.append(completeness_result)
        if not completeness_result.passed:
            logger.warning(f"Data completeness issues: {completeness_result.details}")

        # Step 6d
        logger.info("Step 6d: Exporting project deliverables (dual-method)...")
        export_result = validator.export_project_deliverables()
        results.append(export_result)
        if not export_result.passed:
            logger.error(f"Export failed: {export_result.details}")
            return False, validator

        # Step 6e
        logger.info("Step 6e: Exporting violation reports...")
        violation_result = validator.export_violation_reports()
        results.append(violation_result)
        if not violation_result.passed:
            logger.warning(f"Violation export had issues: {violation_result.details}")

        # Step 6f
        logger.info("Step 6f: Exporting method comparison reports...")
        comparison_result = validator.export_method_comparison_reports()
        results.append(comparison_result)
        if not comparison_result.passed:
            logger.warning(f"Comparison export had issues: {comparison_result.details}")

        # Step 6g
        logger.info("Step 6g: Generating comprehensive quality report...")
        quality_result = validator.generate_quality_report()
        results.append(quality_result)
        if not quality_result.passed:
            logger.warning(f"Quality report issues: {quality_result.details}")

        # Summary
        validator.print_final_summary()

        critical_failures = [r for r in results if not r.passed and r.step_name in 
                           ["Validate Dual-Method Tables", "Export Project Deliverables"]]
        success = len(critical_failures) == 0
        return success, validator



# Example usage
if __name__ == "__main__":
    from eda.data.nba_possession_engine import run_dual_method_possession_engine

    database_path = "mavs_enhanced.duckdb"

    # Run Step 5 first to ensure dual-method data is available
    print("Running Step 5: Dual-Method Possession Engine...")
    step5_success = run_dual_method_possession_engine(database_path)
    if not step5_success:
        print("❌ Step 5 failed - dual-method data not available")
        exit(1)

    # Run Step 6: Dual-Method Final Export
    print("\nRunning Step 6: Dual-Method Final Export...")
    success, validator = run_dual_method_final_export(database_path)

    if success:
        print("\n✅ Step 6 Complete: Dual-method results validated and exported")
        print("📊 Exported files:")
        print("   - project1_lineups_traditional.csv")
        print("   - project1_lineups_enhanced.csv") 
        print("   - project2_players_traditional.csv")
        print("   - project2_players_enhanced.csv")
        print("   - traditional_lineup_violations.csv")
        print("   - method_comparison_summary.csv")
        print("   - method_effectiveness_report.txt")
        print("   - base_dataset_violations.txt")
        print("🎯 Both traditional and enhanced methods ready for project submission")
    else:
        print("\n❌ Step 6 Failed: Dual-method validation errors")
        print("🔧 Review validation messages above and fix export issues")



Overwriting api/src/airflow_project/eda/data/nba_final_export.py


Step 7: Complete Pipeline Integration

In [38]:
# %%writefile api/src/airflow_project/run_complete_pipeline.py
#!/usr/bin/env python3
"""
FIXED Complete NBA Pipeline Runner
==================================

This module runs the complete NBA pipeline from start to finish and outputs
the 3 required datasets with FIXED file path handling and error management.

FIXED ISSUES:
1. Proper database path construction and validation
2. Enhanced error handling and debugging
3. Corrected argument parsing to avoid kernel file path confusion
4. Added comprehensive validation at each step

Usage:
    python run_complete_pipeline.py [database_path]

If no database path is provided, defaults to 'mavs_enhanced.duckdb'
"""

import sys
import time
import logging
import os
from pathlib import Path
from typing import Tuple, Optional

# FIXED: Ensure proper working directory and path setup
def setup_pipeline_environment():
    """FIXED: Setup the pipeline environment with proper paths"""
    # Get current working directory
    current_dir = Path.cwd()

    # Check if we're in the right directory structure
    if current_dir.name != "airflow_project":
        # Look for airflow_project directory
        airflow_project_paths = [
            current_dir / "api" / "src" / "airflow_project",
            current_dir / "airflow_project", 
            Path("api/src/airflow_project")
        ]

        for path in airflow_project_paths:
            if path.exists() and path.is_dir():
                os.chdir(str(path))
                print(f"Changed working directory to: {path.absolute()}")
                break
        else:
            print(f"Warning: airflow_project directory not found. Current dir: {current_dir}")

    # Add current directory to Python path
    sys.path.insert(0, str(Path.cwd()))

    return Path.cwd()

# Setup environment before imports
working_dir = setup_pipeline_environment()

# Import all pipeline modules with error handling
try:
    from eda.data.nba_data_loader import load_all_data_enhanced
    from eda.data.nba_pbp_processor import process_pbp_with_step2_integration
    from eda.data.nba_entities_extractor import extract_all_entities_robust
    from eda.data.nba_possession_engine import run_dual_method_possession_engine
    from eda.data.nba_final_export import run_dual_method_final_export
    print("[SUCCESS] All pipeline modules imported successfully")
except ImportError as e:
    print(f"[ERROR] Error importing pipeline modules: {e}")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Python path: {sys.path[:3]}")  # Show first 3 entries
    sys.exit(1)

# Configure logging with FIXED paths
def setup_logging(working_dir: Path):
    """Setup logging with proper file paths"""
    logs_dir = working_dir / "logs"
    logs_dir.mkdir(exist_ok=True)

    log_file = logs_dir / "complete_pipeline.log"

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(),
            logging.FileHandler(str(log_file))
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging(working_dir)

def validate_database_path(db_path: str) -> str:
    """FIXED: Validate and construct proper database path"""
    # Convert to Path object for better handling
    path = Path(db_path)

    # Check if it looks like a kernel file (FIXED: detect and reject kernel paths)
    if "kernel" in str(path).lower() or "jupyter" in str(path).lower():
        logger.error(f"Invalid database path detected (kernel file): {path}")
        logger.info("Using default database path instead")
        return "mavs_enhanced.duckdb"

    # If relative path, make it relative to working directory
    if not path.is_absolute():
        path = working_dir / path

    # Ensure .duckdb extension
    if not str(path).endswith('.duckdb'):
        path = path.with_suffix('.duckdb')

    logger.info(f"Using database path: {path.absolute()}")
    return str(path)

def run_complete_pipeline(database_path: str = "mavs_enhanced.duckdb") -> Tuple[bool, dict]:
    """
    FIXED: Run the complete NBA pipeline from start to finish.

    Args:
        database_path: Path to the database file

    Returns:
        Tuple of (success: bool, results: dict)
    """
    start_time = time.time()

    # FIXED: Validate and construct proper database path
    database_path = validate_database_path(database_path)

    results = {
        "database_path": database_path,
        "working_directory": str(working_dir),
        "start_time": start_time,
        "steps_completed": [],
        "errors": [],
        "warnings": [],
        "outputs": {},
        "total_time": 0,
        "step_details": {}
    }

    try:
        logger.info("=" * 80)
        logger.info(" NBA COMPLETE PIPELINE RUNNER")
        logger.info("=" * 80)
        logger.info(f"Database: {database_path}")
        logger.info(f"Working Directory: {working_dir}")
        logger.info(f"Start time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time))}")
        logger.info("")

        # FIXED: Step 1 with enhanced error handling
        logger.info("STEP 1: Loading NBA Data...")
        step1_start = time.time()
        try:
            # Check if data directory exists
            data_dir = working_dir / "data" / "mavs_data_engineer_2025"
            if not data_dir.exists():
                raise Exception(f"Data directory not found: {data_dir}")

            success, loader = load_all_data_enhanced(data_dir=None, db_path=database_path)
            # FIXED: Be more tolerant of validation failures - check if core functionality works
            if not success:
                # Check if the database has the essential tables
                import duckdb
                conn = duckdb.connect(database_path)
                essential_tables = ['pbp', 'box_score', 'pbp_event_msg_types']
                missing_tables = []
                for table in essential_tables:
                    count = conn.execute(f"SELECT COUNT(*) FROM information_schema.tables WHERE table_name='{table}'").fetchone()[0]
                    if count == 0:
                        missing_tables.append(table)
                conn.close()

                if missing_tables:
                    raise Exception(f"Step 1 failed: Essential tables missing: {missing_tables}")
                else:
                    logger.warning("Step 1 completed with validation warnings but core data loaded successfully")
                    results["warnings"].append("Step 1 had validation warnings but core functionality works")

            step1_time = time.time() - step1_start
            results["steps_completed"].append("Step 1: Data Loading")
            results["step_details"]["step1"] = {"time": step1_time, "status": "success" if success else "warning"}
            logger.info(f"✅ Step 1 completed in {step1_time:.2f}s")

        except Exception as e:
            error_msg = f"Step 1 failed: {str(e)}"
            logger.error(error_msg)
            results["errors"].append(error_msg)
            results["step_details"]["step1"] = {"time": time.time() - step1_start, "status": "failed", "error": str(e)}
            return False, results

        # FIXED: Step 2 with validation
        logger.info("STEP 2: Extracting Entities...")
        step2_start = time.time()
        try:
            success, entities = extract_all_entities_robust(database_path)
            if not success:
                raise Exception("Step 2 failed: Entity extraction returned False")
            if entities is None:
                raise Exception("Step 2 failed: No entities returned")

            step2_time = time.time() - step2_start
            results["steps_completed"].append("Step 2: Entity Extraction")
            results["step_details"]["step2"] = {"time": step2_time, "status": "success"}
            logger.info(f"✅ Step 2 completed in {step2_time:.2f}s")

        except Exception as e:
            error_msg = f"Step 2 failed: {str(e)}"
            logger.error(error_msg)
            results["errors"].append(error_msg)
            results["step_details"]["step2"] = {"time": time.time() - step2_start, "status": "failed", "error": str(e)}
            return False, results

        # FIXED: Step 3 with proper entity passing
        logger.info("STEP 3: Processing PBP Data...")
        step3_start = time.time()
        try:
            success, processor = process_pbp_with_step2_integration(db_path=database_path, entities=entities)
            if not success:
                raise Exception("Step 3 failed: PBP processing returned False")

            step3_time = time.time() - step3_start
            results["steps_completed"].append("Step 3: PBP Processing")
            results["step_details"]["step3"] = {"time": step3_time, "status": "success"}
            logger.info(f"✅ Step 3 completed in {step3_time:.2f}s")

        except Exception as e:
            error_msg = f"Step 3 failed: {str(e)}"
            logger.error(error_msg)
            results["errors"].append(error_msg)
            results["step_details"]["step3"] = {"time": time.time() - step3_start, "status": "failed", "error": str(e)}
            return False, results

        # FIXED: Step 4 with comprehensive validation
        logger.info("STEP 4: Running Dual-Method Possession Engine...")
        step4_start = time.time()
        try:
            success, possession_engine = run_dual_method_possession_engine(db_path=database_path, entities=entities)
            if not success:
                raise Exception("Step 4 failed: Possession engine returned False")
            if possession_engine is None:
                raise Exception("Step 4 failed: No possession engine returned")

            step4_time = time.time() - step4_start
            results["steps_completed"].append("Step 4: Possession Engine")
            results["step_details"]["step4"] = {"time": step4_time, "status": "success"}
            logger.info(f"✅ Step 4 completed in {step4_time:.2f}s")

        except Exception as e:
            error_msg = f"Step 4 failed: {str(e)}"
            logger.error(error_msg)
            results["errors"].append(error_msg)
            results["step_details"]["step4"] = {"time": time.time() - step4_start, "status": "failed", "error": str(e)}
            return False, results

        # FIXED: Step 5 with output validation
        logger.info("STEP 5: Running Final Export and Validation...")
        step5_start = time.time()
        try:
            success, final_validator = run_dual_method_final_export(db_path=database_path)
            if not success:
                # Don't fail completely if exports have warnings but core functionality works
                results["warnings"].append("Step 5 had validation warnings but core exports succeeded")
                logger.warning("Step 5 completed with warnings")

            step5_time = time.time() - step5_start
            results["steps_completed"].append("Step 5: Final Export")
            results["step_details"]["step5"] = {"time": step5_time, "status": "success" if success else "warning"}
            logger.info(f"✅ Step 5 completed in {step5_time:.2f}s")

        except Exception as e:
            error_msg = f"Step 5 failed: {str(e)}"
            logger.error(error_msg)
            results["errors"].append(error_msg)
            results["step_details"]["step5"] = {"time": time.time() - step5_start, "status": "failed", "error": str(e)}
            return False, results

        # FIXED: Collect outputs with proper path validation
        exports_dir = working_dir / "data" / "mavs_data_engineer_2025" / "exports"
        if not exports_dir.exists():
            exports_dir = working_dir / "exports"

        logger.info(f"Looking for exports in: {exports_dir}")

        potential_outputs = {
            "project1_lineups_traditional": "project1_lineups_traditional.csv",
            "project1_lineups_enhanced": "project1_lineups_enhanced.csv", 
            "project2_players_traditional": "project2_players_traditional.csv",
            "project2_players_enhanced": "project2_players_enhanced.csv",
            "violation_reports": "traditional_lineup_violations.csv",
            "method_comparison": "method_comparison_summary.csv",
            "quality_report": "quality_report.txt"
        }

        # Check which files actually exist
        for name, filename in potential_outputs.items():
            file_path = exports_dir / filename
            if file_path.exists():
                results["outputs"][name] = str(file_path)
                logger.info(f"Found output: {filename}")
            else:
                results["warnings"].append(f"Expected output file not found: {filename}")

        total_time = time.time() - start_time
        results["total_time"] = total_time

        logger.info("")
        logger.info("=" * 80)
        logger.info("PIPELINE COMPLETED!")
        logger.info("=" * 80)
        logger.info(f"Total execution time: {total_time:.2f} seconds")
        logger.info(f"Steps completed: {len(results['steps_completed'])}")
        logger.info(f"Warnings: {len(results['warnings'])}")
        logger.info("")
        logger.info("OUTPUTS GENERATED:")
        for name, path in results["outputs"].items():
            file_size = Path(path).stat().st_size / 1024 if Path(path).exists() else 0
            logger.info(f"  • {name}: {Path(path).name} ({file_size:.1f} KB)")

        if results["warnings"]:
            logger.info("")
            logger.info("WARNINGS:")
            for warning in results["warnings"]:
                logger.warning(f"  • {warning}")

        logger.info("")
        logger.info("READY FOR PROJECT SUBMISSION!")

        return True, results

    except Exception as e:
        error_msg = f"Pipeline failed: {str(e)}"
        logger.error(error_msg)
        logger.error(f"Error occurred in working directory: {working_dir}")
        results["errors"].append(error_msg)
        results["total_time"] = time.time() - start_time
        return False, results

def main():
    """FIXED: Main entry point with proper argument handling."""
    # FIXED: Properly handle command line arguments
    if len(sys.argv) > 1:
        arg = sys.argv[1]
        # FIXED: Filter out any jupyter/kernel related arguments
        if "--f=" in arg or "kernel" in arg.lower() or "jupyter" in arg.lower():
            logger.warning(f"Ignoring invalid argument (appears to be kernel file): {arg}")
            database_path = "mavs_enhanced.duckdb"
        else:
            database_path = arg
    else:
        database_path = "mavs_enhanced.duckdb"

    logger.info(f"Starting complete pipeline with database: {database_path}")

    success, results = run_complete_pipeline(database_path)

    print("\n" + "="*80)
    if success:
        print("NBA Pipeline - Enhanced Data Loading & Validation")
        print("="*60)
        print("")
        print("✅ Pipeline completed successfully!")
        print(f"Total time: {results['total_time']:.2f}s")
        print(f"Generated {len(results['outputs'])} output files.")
        print("")
        print("Check the exports/ directory for output files:")
        for name, path in results['outputs'].items():
            print(f"  📄 {Path(path).name}")

        if results['warnings']:
            print(f"\n⚠️  {len(results['warnings'])} warnings (check logs for details)")

    else:
        print("NBA Pipeline - Enhanced Data Loading & Validation")
        print("="*60)
        print("")
        print("❌ Pipeline failed!")
        print(f"  Error: {results['errors'][-1] if results['errors'] else 'Unknown error'}")
        print(f"  Total steps completed: {len(results['steps_completed'])}")
        print(f"  Total time: {results['total_time']:.2f}s")
        print("\nAn exception has occurred, use %tb to see the full traceback.")

if __name__ == "__main__":
    main()


Changed working directory to: c:\docker_projects\interview_hackathon\api\src\airflow_project


2025-09-05 18:45:29,304 - WARNING - Ignoring invalid argument (appears to be kernel file): --f=c:\Users\ghadf\AppData\Roaming\jupyter\runtime\kernel-v3313d9cfac109dbeb75fe9c92c45b8d0cacfe4956.json
2025-09-05 18:45:29,306 - INFO - Starting complete pipeline with database: mavs_enhanced.duckdb
2025-09-05 18:45:29,306 - INFO - Using database path: c:\docker_projects\interview_hackathon\api\src\airflow_project\mavs_enhanced.duckdb
2025-09-05 18:45:29,307 - INFO - ================================================================================
2025-09-05 18:45:29,307 - INFO -  NBA COMPLETE PIPELINE RUNNER
2025-09-05 18:45:29,307 - INFO - ================================================================================
2025-09-05 18:45:29,308 - INFO - Database: c:\docker_projects\interview_hackathon\api\src\airflow_project\mavs_enhanced.duckdb
2025-09-05 18:45:29,308 - INFO - Working Directory: c:\docker_projects\interview_hackathon\api\src\airflow_project
2025-09-05 18:45:29,308 - INFO - Sta

[SUCCESS] All pipeline modules imported successfully
NBA Pipeline - Enhanced Data Loading & Validation


2025-09-05 18:45:29,493 - INFO - [PASS] Create PBP Enriched View: Created view pbp_enriched with 506 rows (matches pbp)
2025-09-05 18:45:29,498 - INFO - Processing 506 events with TRADITIONAL DATA-DRIVEN approach...
2025-09-05 18:45:29,500 - INFO - [TRADITIONAL SUB-IN] Daniel Gafford to DAL
2025-09-05 18:45:29,501 - INFO - [TRADITIONAL SUB-IN] Klay Thompson to DAL
2025-09-05 18:45:29,502 - INFO - [TRADITIONAL SUB-IN] Dillon Brooks to HOU
2025-09-05 18:45:29,503 - INFO - [TRADITIONAL SUB-IN] Kyrie Irving to DAL
2025-09-05 18:45:29,504 - INFO - [TRADITIONAL SUB-IN] P.J. Washington to DAL
2025-09-05 18:45:29,505 - INFO - [TRADITIONAL SUB-OUT] Dillon Brooks from HOU
2025-09-05 18:45:29,505 - INFO - [TRADITIONAL SUB-IN] Tari Eason to HOU
2025-09-05 18:45:29,506 - INFO - [TRADITIONAL SUB-IN] Alperen Sengun to HOU
2025-09-05 18:45:29,507 - INFO - [TRADITIONAL SUB-IN] Amen Thompson to HOU
2025-09-05 18:45:29,508 - INFO - [TRADITIONAL SUB-OUT] Daniel Gafford from DAL
2025-09-05 18:45:29,509 - I


ENHANCED NBA PIPELINE - DATA LOADING SUMMARY
BOX SCORE:
   Original rows: 35
   Active players: 28
   Final rows: 21
   Teams: HOU, DAL
   Starters per team: {'HOU': 5, 'DAL': 5}

PLAY-BY-PLAY:
   Original rows: 507
   Game events: 506
   Final rows: 506
   Total shots: 183
   Shots with coordinates: 183
   Rim attempts: 52
   Average distance: 12.1 ft

LINEUP ENGINE:
   Substitutions: 46
   First-actions auto-IN: 15
   Inactivity auto-OUTs: 16
   5-on-floor fixes: 1
   Minutes tolerance: ±120s
   Minutes offenders: 4/21

NBA PIPELINE VALIDATION SUMMARY
OVERALL STATUS: 14/15 tests passed
TOTAL VALIDATION TIME: 0.66 seconds
TOTAL WARNINGS: 7

[PASS] Load Box Score
   Details: Processed box score: 35 → 28 active → 21 final rows. Teams: ['HOU', 'DAL'], Starters: {'HOU': 5, 'DAL': 5}
   Data Count: 21
   Time: 0.022s
   [WARN] Removed 7 players with no playing time

[PASS] Load PBP
   Details: Processed PBP: 507 → 506 game events → 506 final rows. Shots: 183, Rim attempts: 52
   Data Coun

...

NBA Pipeline - Step 4: Integrated PBP Processing (Updated with Step 2)
output tables results=============== ValidationResult(step_name='Create Step 4 Output Tables', passed=True, details='Created Step 4 output tables: processed_events (506 rows), traditional_flags (68 rows), enhanced_flags (236 rows), method_comparison', data_count=506, processing_time=0.029732942581176758, warnings=[])

NBA PIPELINE - STEP 4 SUMMARY (INTEGRATED WITH STEP 2)
TRADITIONAL DATA-DRIVEN METHOD:
  Substitutions Processed: 46
  Flags Generated: 68
  Lineup Size Deviations: 31
  Current Lineup Sizes:
    DAL: 6 players
    HOU: 5 players

ENHANCED ESTIMATION METHOD:
  Substitutions Processed: 46
  First-Action Injections: 91
  Auto-Out Corrections: 98
  Flags Generated: 236
  Current Lineup Sizes:
    DAL: 5 players
    HOU: 5 players

TOTAL EVENTS PROCESSED: 506
LINEUP SIZE ACCURACY:
  Traditional: 1/2 teams have 5-man lineups (50.0%)
  Enhanced: 2/2 teams have 5-man lineups (100.0%)

NBA PIPELINE VALIDATION 

2025-09-05 18:45:30,394 - INFO - Diagnostic results: {'all_tables': ['action_types', 'box_score', 'canonical_pbp', 'canonical_players', 'canonical_starters', 'canonical_teams', 'dim_officials', 'dim_players', 'dim_teams', 'enhanced_flags', 'enhanced_lineup_flags', 'enhanced_lineup_state', 'enhanced_violation_report', 'event_types', 'final_dual_lineups', 'final_dual_players', 'final_lineups', 'final_players', 'final_players_rim', 'method_comparison_summary', 'minutes_basic', 'minutes_compare', 'minutes_enhanced', 'minutes_offenders', 'minutes_traditional', 'minutes_validation_full', 'missing_player_report', 'option_types', 'pbp', 'pbp_action_types', 'pbp_enriched', 'pbp_event_msg_types', 'pbp_only_players', 'pbp_option_types', 'pipeline_contract', 'processed_events', 'project1_lineups', 'project2_players', 'step4_enhanced_flags', 'step4_method_comparison', 'step4_processed_events', 'step4_traditional_flags', 'team_summary', 'traditional_lineup_flags', 'traditional_lineup_state', 'tradit


NBA PIPELINE - STEP 5 DUAL-METHOD SUMMARY
POSSESSION ANALYSIS:
  Total Dual Possessions: 274
  Total Points: 221
  Periods: 4

TRADITIONAL METHOD RESULTS:
  Unique Lineups: 20
  5-Man Lineups: 6 (30.0%)
  Violation Flags: 873

ENHANCED METHOD RESULTS:
  Unique Lineups: 42
  5-Man Lineups: 42 (100.0%)
  Violation Flags: 67

PLAYER RIM DEFENSE:
  Traditional Players with Rim Data: 18
  Enhanced Players with Rim Data: 19

METHOD EFFECTIVENESS:
  Lineup Count Change: -22 (Enhanced has fewer unique lineups)
  5-Man Accuracy: Traditional 30.0% vs Enhanced 100.0%

NBA PIPELINE VALIDATION SUMMARY
OVERALL STATUS: 6/6 tests passed
TOTAL VALIDATION TIME: 0.10 seconds
TOTAL WARNINGS: 0

[PASS] Load Dual Method Data
   Details: Loaded Step 5 inputs. Flags: traditional=873, enhanced=67. Optional sources missing (non-blocking): [].
   Data Count: 940
   Time: 0.021s

[PASS] Identify Dual Possessions
   Details: FIXED: Identified 274 dual-method possessions, 221 possession points vs 221 event points.

2025-09-05 18:45:30,700 - INFO - Generating quality report...
2025-09-05 18:45:30,707 - INFO - ✅ Step 5 completed in 0.14s
2025-09-05 18:45:30,707 - INFO - Looking for exports in: c:\docker_projects\interview_hackathon\api\src\airflow_project\data\mavs_data_engineer_2025\exports
2025-09-05 18:45:30,708 - INFO - Found output: project1_lineups_traditional.csv
2025-09-05 18:45:30,708 - INFO - Found output: project1_lineups_enhanced.csv
2025-09-05 18:45:30,709 - INFO - Found output: project2_players_traditional.csv
2025-09-05 18:45:30,709 - INFO - Found output: project2_players_enhanced.csv
2025-09-05 18:45:30,710 - INFO - Found output: traditional_lineup_violations.csv
2025-09-05 18:45:30,710 - INFO - Found output: method_comparison_summary.csv
2025-09-05 18:45:30,711 - INFO - Found output: quality_report.txt
2025-09-05 18:45:30,711 - INFO - 
2025-09-05 18:45:30,711 - INFO - ================================================================================
2025-09-05 18:45:30,711 - INFO - P


NBA PIPELINE - FINAL EXPORT & VALIDATION SUMMARY
EXPORTED FILES (12):
   - base_dataset_violations.txt (0.9 KB)
   - enhanced_method_flags.csv (5.9 KB)
   - method_comparison_summary.csv (0.3 KB)
   - method_effectiveness_report.txt (1.0 KB)
   - project1_lineups_enhanced.csv (4.0 KB)
   - project1_lineups_traditional.csv (1.9 KB)
   - project1_lineups_traditional_with_6th_player.csv (2.0 KB)
   - project2_players_enhanced.csv (1.2 KB)
   - project2_players_traditional.csv (1.1 KB)
   - quality_report.txt (0.9 KB)
   - traditional_lineup_violations.csv (218.3 KB)
   - violation_summary.txt (0.3 KB)

FINAL RESULTS:
   Unique Lineups: 28
   Active Players: 21
   Unable to retrieve final metrics

NBA Pipeline - Enhanced Data Loading & Validation

✅ Pipeline completed successfully!
Total time: 1.40s
Generated 7 output files.

Check the exports/ directory for output files:
  📄 project1_lineups_traditional.csv
  📄 project1_lineups_enhanced.csv
  📄 project2_players_traditional.csv
  📄 projec

# Real Pull

In [35]:
%%writefile api/src/airflow_project/eda/pipeline/__init__.py
# Pipeline module for data processing workflows


Overwriting api/src/airflow_project/eda/pipeline/__init__.py


# Dags

In [36]:
%%writefile api/src/airflow_project/dags/mavs_lineups_assetaware_dag.py
from __future__ import annotations

import sys
import inspect
from pathlib import Path

# Ensure project modules are importable without changing global CWD
THIS = Path(__file__).resolve()
AIRFLOW_PROJECT_ROOT = THIS.parents[1]  # dags -> airflow_project
if str(AIRFLOW_PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(AIRFLOW_PROJECT_ROOT))

from utils import config as CFG

from airflow.decorators import dag, task
from airflow.utils.task_group import TaskGroup
from airflow.exceptions import AirflowException


def _supports_deferrable_filesensor() -> bool:
    """Return True if FileSensor accepts 'deferrable' (Airflow >=2.9)."""
    try:
        from airflow.sensors.filesystem import FileSensor  # type: ignore
        return "deferrable" in inspect.signature(FileSensor.__init__).parameters
    except Exception:
        return False


def _file_sensor_partial():
    """Return a FileSensor.partial(...) using config-defined knobs."""
    from airflow.sensors.filesystem import FileSensor  # late import
    kwargs = dict(
        task_id="wait_for_file",
        fs_conn_id=CFG.AIRFLOW_FS_CONN_ID,
        poke_interval=CFG.FILE_SENSOR_POKE_SEC,
        timeout=CFG.FILE_SENSOR_TIMEOUT_SEC,
        soft_fail=False,
    )
    if _supports_deferrable_filesensor():
        kwargs["deferrable"] = True
    return FileSensor.partial(**kwargs)


@dag(
    dag_id="nba_lineups_assetaware_v3",
    # IMPORTANT: cross-version compatible — use 'schedule', not 'timetable'
    schedule=CFG.build_combined_schedule(),
    default_args=CFG.airflow_default_args(),
    catchup=False,
    max_active_runs=1,
    tags=["nba", "lineups", "duckdb", "asset-aware"],
    doc_md="""
### NBA Lineups DAG (Cron + optional Asset/Dataset triggers)
- **Scheduling**: Cron and, when supported by this Airflow, Assets/Datasets.
- **Inputs**: waits on required CSVs via (deferrable) FileSensor mapping.
- **Pipeline**: calls `run_complete_pipeline.run_complete_pipeline()`.
- **Validation**: asserts required CSV exports exist and are non-empty.
""",
)
def lineup_dag():

    required_files = [str(p) for p in CFG.required_input_files()]

    # Wait for all inputs (mapped sensor, deferrable when supported)
    with TaskGroup(group_id="wait_for_inputs", tooltip="Wait for required input CSVs") as wait_for_inputs:
        _file_sensor_partial().expand(filepath=required_files)

    @task
    def print_config() -> None:
        """Emit configuration summary and write column_usage_report.md."""
        CFG.print_configuration_summary()

    @task
    def preflight_checks() -> None:
        """Fail fast if required files are missing or perf config is unsafe."""
        files_ok = CFG.validate_data_files()
        perf_ok = CFG.validate_performance_config()
        if not files_ok:
            raise AirflowException("Preflight failed: required input files missing or empty.")
        if not perf_ok:
            raise AirflowException("Preflight failed: performance configuration invalid.")

    @task
    def run_pipeline() -> dict:
        """Run the complete pipeline; return compact results dict."""
        from run_complete_pipeline import run_complete_pipeline  # late import
        ok, results = run_complete_pipeline(database_path=str(CFG.DUCKDB_PATH))
        if not ok:
            err = (results.get("errors") or ["unknown"])[-1]
            raise AirflowException(f"Pipeline failed: {err}")
        return {
            "outputs": results.get("outputs", {}),
            "total_time": results.get("total_time", 0.0),
            "warnings": results.get("warnings", []),
        }

    @task
    def validate_outputs(results: dict) -> dict:
        """Check required CSVs exist and are non-empty."""
        from pathlib import Path
        outputs = results.get("outputs", {})
        required = [
            "project1_lineups_traditional",
            "project2_players_traditional",
        ]
        missing = [k for k in required if k not in outputs]
        if missing:
            raise AirflowException(f"Missing expected outputs: {missing}")
        zero = []
        for k in required:
            p = Path(outputs[k])
            if not p.exists() or p.stat().st_size == 0:
                zero.append(p.name)
        if zero:
            raise AirflowException(f"Zero-sized outputs: {zero}")
        return {
            "validated": True,
            "export_dir": str(CFG.EXPORTS_DIR),
            "count_required": len(required),
            "warnings": results.get("warnings", []),
            "total_time": float(results.get("total_time", 0.0)),
        }

    @task
    def write_run_report(summary: dict) -> str:
        """Persist a JSON run report and return its path."""
        import json
        from datetime import datetime
        CFG.EXPORTS_DIR.mkdir(parents=True, exist_ok=True)
        report = {
            "validated": summary["validated"],
            "count_required": summary["count_required"],
            "total_time_sec": round(summary["total_time"], 2),
            "warnings": summary.get("warnings", []),
            "generated_at": datetime.utcnow().isoformat() + "Z",
        }
        out = CFG.EXPORTS_DIR / "run_summary.json"
        out.write_text(json.dumps(report, indent=2))
        return str(out)

    wfi = wait_for_inputs
    pc = print_config()
    pf = preflight_checks()
    rp = run_pipeline()
    vo = validate_outputs(rp)
    wr = write_run_report(vo)

    wfi >> pc >> pf >> rp >> vo >> wr


dag = lineup_dag()


Overwriting api/src/airflow_project/dags/mavs_lineups_assetaware_dag.py


# Plugins
- plugins: Add custom or community plugins for your project to this file. It is empty by default.

In [37]:
%%writefile api/src/airflow_project/plugins/custom_operator.py



Overwriting api/src/airflow_project/plugins/custom_operator.py
